# Filtering Synthetic Data with LLMs

In [1]:
from claudette import *
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from fastcore.utils import *


In [2]:
models

['claude-3-opus-20240229',
 'claude-3-5-sonnet-20240620',
 'claude-3-haiku-20240307']

In [3]:
model = models[1]

In [4]:
cli = Client(model)

In [5]:
FP = Path.cwd() ; print(FP)

/home/zardar/Desktop/synthetic-medical-datatsets


In [6]:
load_dotenv(FP/'.env')

True

In [7]:
class TranslationCritique(BasicRepr):
    "A critique of the translation."
    def __init__(
        self,
        critique: str, # A brief 1-line critique of the translation.
        score: int # A score of the translation from 1 to 5. 
    ): store_attr()
    
    __repr__ = basic_repr('critique, score')


In [8]:
sp = "You will help critique synthetic radiology reports of patients' with and without microcalfications."

In [9]:
df = pd.read_csv(FP/'data/claude3-opus-microcalcs-n610.csv')
df.head()
df.label.value_counts()

label
Negative      210
Not Stated    210
Positive      202
Name: count, dtype: int64

In [10]:
class Report():
    "Radiology Report + Label"
    def __init__(self, report: str, label: str): store_attr()
    def __repr__(self): return f"{self.report} ➡ *{self.label}*"

In [11]:
example_reports = [Report(report, label) for report, label in zip(df.report_text, df.label)]

In [12]:
example_reports[0]

MAMMOGRAM REPORT:
HISTORY: Annual screening mammogram. Patient denies any breast complaints.
FINDINGS:
Bilateral breast ultrasound performed.
Mammogram reveals heterogeneously dense breast tissue.
There are no dominant masses, architectural distortion or suspicious calcifications.
Stable postoperative changes noted in the upper outer quadrant of the right breast consistent with prior lumpectomy.
ULTRASOUND:
Ultrasound was performed targeted to the site of prior right breast lumpectomy. No sonographic abnormality or recurrence is identified.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening.
2. Stable postoperative changes right breast, no sonographic evidence of recurrence.
BIRADS 2, benign. ➡ *Negative*

In [13]:
from IPython.display import Markdown

In [14]:
clps_fmt = '- {s}\n\n<details>\n<summary> Click to show the rest </summary>\n{ls}\n</details>'
def to_md(ss, collapsible=False):
    ls = '\n'.join(f'- {s}' for s in ss) 
    return clps_fmt.format(s=str(ss[0]), ls=ls.replace(f'- {ss[0]}', '')) if collapsible else ls
def show(ss, collapsible=False): return Markdown(to_md(ss, collapsible=collapsible))

In [15]:
show(example_reports, collapsible=True)

- MAMMOGRAM REPORT:
HISTORY: Annual screening mammogram. Patient denies any breast complaints.
FINDINGS:
Bilateral breast ultrasound performed.
Mammogram reveals heterogeneously dense breast tissue.
There are no dominant masses, architectural distortion or suspicious calcifications.
Stable postoperative changes noted in the upper outer quadrant of the right breast consistent with prior lumpectomy.
ULTRASOUND:
Ultrasound was performed targeted to the site of prior right breast lumpectomy. No sonographic abnormality or recurrence is identified.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening.
2. Stable postoperative changes right breast, no sonographic evidence of recurrence.
BIRADS 2, benign. ➡ *Negative*

<details>
<summary> Click to show the rest </summary>

- MAMMOGRAM.
INDICATION: Postoperative screening. Left new lump.

COMPARISON: 2025-06-15.

RIGHT BREAST: 
Right mastectomy.  

LEFT BREAST: 
There are scattered areas of fibroglandular density.
There is no focal mass or suspicious calcifications.  No new significant changes compared to prior exams.

IMPRESSION:
1. Negative screening mammogram of the left breast. No suspicious masses or calcifications.  No significant interval change.
2. Status post right mastectomy.
BIRADS 1.

ULTRASOUND LEFT BREAST

CLINICAL HISTORY: New left breast lump.

TECHNIQUE: Targeted ultrasound was performed using a 12 MHz linear transducer.  

COMPARISON: None.

FINDINGS: 
Targeted ultrasound of the left breast at 12:00, 6 cm from the nipple, demonstrates normal fibroglandular tissue with no focal solid lesion, complex cystic lesion or suspicious mass identified.

IMPRESSION: 
No sonographic evidence of malignancy at the site of the palpable abnormality. Recommend clinical follow up and correlation with physical exam findings. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. Patient denies breast pain, skin changes or nipple discharge.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: Comparison is made to outside screening mammogram from 1 year prior.
FINDINGS:
Right breast: The breast tissue is heterogeneously dense (type C). This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications are identified. 
Left breast: The breast tissue is heterogeneously dense (type C). This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications are identified.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Heterogeneously dense breast tissue which could obscure detection of small masses. 
RECOMMENDATIONS:
1. Routine annual screening mammography is recommended in 1 year. Clinical breast examination is also recommended.
2. Please inform the patient that her breast tissue is dense. Dense breast tissue is very common and is not abnormal. However, dense breast tissue can make it harder to find cancer on a mammogram and may also be associated with an increased risk of breast cancer. This information is given to raise awareness and to inform her medical care decisions. A report of the results was sent to the patient and referring provider.
ASSESSMENT CATEGORY: BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS:
The right breast is status post mastectomy with expected postoperative changes. The left breast is heterogeneously dense with scattered fibroglandular tissue. No dominant mass, architectural distortion, or microcalcifications are identified. Stable intramammary lymph nodes are noted. There are no new or enlarging masses or areas of architectural distortion. No skin changes or nipple retraction.

Bilateral axillae appear clear. Stable surgical clips overlie the right chest wall.

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Annual screening mammography is recommended.
2. Stable postmastectomy changes of the right breast. 

BI-RADS category 1: Negative. ➡ *Negative*
- MAMMOGRAM REPORT:

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: Compared with prior studies from 10/15/2021.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.

Left Breast: 
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:

Negative bilateral screening mammogram. 

BIRADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION - BILATERAL

HISTORY: Annual screening exam. No palpable findings. No family history of breast cancer.

FINDINGS:
RIGHT BREAST: The breast is predominantly fatty replaced with minimal scattered fibroglandular densities. No focal mass, significant calcifications or architectural distortion.

LEFT BREAST: The breast parenchyma is almost completely fatty replaced. No focal mass, significant calcifications or architectural distortion.

IMPRESSION:
Bilateral mammogram, negative. No significant changes from prior study.

BI-RADS Category 1 - Normal mammogram. Recommend annual screening mammography. ➡ *Negative*
- COMPARISON:  Digital screening mammogram from 2 years ago.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

In the left breast, there are no dominant masses, areas of architectural distortion or suspicious calcifications.  

In the right breast, there are no dominant masses, areas of architectural distortion or suspicious calcifications.  

IMPRESSION:

1. No mammographic evidence of malignancy in either breast.  
2. Heterogeneously dense breast tissue which can lower the sensitivity of mammography.

RECOMMENDATIONS:

Annual screening mammography is recommended.   

ASSESSMENT CATEGORY:

BI-RADS Category 1: Negative.

ADDENDUM:  Spoke to ordering provider Dr. Smith who confirms no palpable lumps on clinical breast examination. 

Per ACR BI-RADS Atlas, BI-RADS Category 1 assessment is given when there are no mammographic findings (benign or suspicious) that require further evaluation, even if palpable finding is present on clinical exam.  Thus, final BIRADS assessment remains BIRADS 1. ➡ *Negative*
- MAMMOGRAM.
INDICATION: Family history of breast cancer.
FINDINGS: Scattered fibroglandular densities. No significant masses, calcifications or architectural distortion.
IMPRESSION: Benign/negative mammogram. Recommend routine screening in 1 year. BI-RADS 1. ➡ *Negative*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Annual screening mammogram. No current breast complaints.
COMPARISON: Prior study of 10/15/2020.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Targeted ultrasound of the left breast at 3 o'clock position was also performed.
FINDINGS:
The left breast is composed of scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified.
Targeted ultrasound of the left breast at 3 o'clock position demonstrates normal fibroglandular tissue without a discrete solid or cystic lesion.
IMPRESSION:
1. Negative left screening mammogram. No mammographic evidence of malignancy.
2. Negative targeted left breast ultrasound.
ASSESSMENT: BI-RADS 1 - Negative.
RECOMMENDATION: Routine screening mammogram in 1 year. ➡ *Negative*
- CLINICAL HISTORY: Annual screening mammogram. No new complaints. Prior history of right breast cancer treated with lumpectomy and radiation.

FINDINGS:
RIGHT BREAST: 
There are scattered areas of fibroglandular density. No suspicious masses, architectural distortion or suspicious calcifications. 

LEFT BREAST:
There are scattered areas of fibroglandular density. No suspicious masses, architectural distortion or suspicious calcifications.  

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Annual screening mammogram is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- RIGHT BREAST SCREENING MAMMOGRAM. 
CLINICAL HISTORY: Annual screening mammogram.
TECHNIQUE: Standard screening CC and MLO views of the right breast were obtained.
COMPARISON: 10/23/2021.
FINDINGS:  
The right breast is heterogeneously dense. This may lower the sensitivity of mammography. 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
There is no significant interval change from the prior exam.
IMPRESSION:
BI-RADS 1: Negative. 
Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAM, BILATERAL.
HISTORY: Ms. XYZ presents for her annual screening mammogram. She reports no complaints or concerns at this time. LMP: Unknown. Prior surgical history: None.
FINDINGS:
Right breast:
Breast composition: The breast tissue is predominantly fibroglandular.
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion. 
Left breast:
Breast composition: The breast tissue is predominantly fibroglandular.
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion.
IMPRESSION:
1. Negative bilateral screening mammogram.  
2. No mammographic evidence of malignancy.
3. Recommend routine screening mammography in 1 year.
BIRADS 1 - Negative. ➡ *Negative*
- LEFT MAMMOGRAM AND ULTRASOUND
HISTORY: 54-year-old female with a family history of breast cancer, maternal aunt and grandmother, presents for routine screening mammogram. No palpable masses or breast complaints.
FINDINGS: The breast tissue is heterogeneously dense, which could obscure detection of small masses. No dominant masses, architectural distortion, or suspicious calcifications are seen in either breast. 
Targeted ultrasound was performed in the left breast at the 2 o'clock position. No sonographic abnormality or suspicious lesions were identified.
Stable post-surgical changes are noted in the right breast from prior lumpectomy.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography and clinical breast exams.
2. Heterogeneously dense breast tissue, which lowers the sensitivity of mammography. Consider supplemental screening with breast MRI or ultrasound as clinically indicated.
3. No suspicious lesions seen on targeted left breast ultrasound at 2 o'clock position. 
4. Stable right breast post-surgical changes.
BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense parenchyma. No dominant mass, architectural distortion or suspicious calcifications.
Left breast: The breast is almost entirely fatty. No dominant mass, architectural distortion or suspicious calcifications. Stable postoperative changes from prior lumpectomy are noted in the upper outer quadrant. Surgical clips are seen in expected location.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography.
2. No suspicious microcalcifications on either side.
BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No focal dominant masses, architectural distortion, or suspicious calcifications are identified. Stable bilateral benign-appearing calcifications are noted. The skin appears normal with no evidence for thickening or retraction. 

IMPRESSION:

1. No mammographic evidence of malignancy. There are no microcalcifications present.

2. Recommend continued routine screening mammography due to heterogeneously dense breast composition. Ultrasonography may be a useful adjunct screening tool.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior study of 6/15/2022.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally, stable compared to prior exam.
Benign-appearing calcifications are noted bilaterally, also stable.
IMPRESSION:
1. No mammographic evidence of malignancy. No suspicious microcalcifications.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- RADIOLOGY REPORT
CLINICAL HISTORY: Annual screening mammogram. No clinical concerns reported by patient.
COMPARISON: Comparison is made to prior mammograms from 05/18/2022 at this facility. 
FINDINGS: 
Both breasts are composed of scattered fibroglandular densities. 
There are no dominant masses, architectural distortions, or suspicious calcifications in either breast. The skin is unremarkable. 
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Scattered fibroglandular densities are noted bilaterally, consistent with ACR density category C.
2. Recommend continued annual screening mammography in conjunction with monthly breast self-exams and annual clinical breast exams.
BIRADS Category 1: Negative. ➡ *Negative*
- HISTORY: Screening mammogram. Post-menopausal with no current breast complaints. Maternal aunt and grandmother with history of breast cancer.

FINDINGS:
Both breasts are heterogeneously dense, which could obscure detection of small masses. 

Right breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 

IMPRESSION:
1. Negative digital bilateral screening mammogram.  
2. Heterogeneously dense breast tissue which could lower the sensitivity of mammography. 

RECOMMENDATION:  
Recommend routine screening mammogram in 1 year.

BIRADS 1 - Negative.

ADDENDUM: Reviewed images with 3D tomosynthesis, no significant change to above findings or BIRADS category. ➡ *Negative*
- BILATERAL MAMMOGRAM:
INDICATION: Screening
COMPARISON: Prior exams from 3/22/2021
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Both breasts are almost entirely fatty. No dominant masses, suspicious calcifications, or other findings.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend continued annual screening mammography in 12 months.
BI-RADS CATEGORY 1 - NEGATIVE. ➡ *Negative*
- TECHNIQUE: Bilateral digital mammography.

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammogram from 12/15/2021.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

IMPRESSION:
1. No evidence of malignancy in either breast. 
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
3. Stable mammographic appearance compared to prior exam.

RECOMMENDATION:
Annual screening mammogram is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.

CLINICAL HISTORY: Annual screening mammogram. Patient denies breast symptoms. No family history of breast cancer. Last mammogram 1 year ago.

COMPARISON: Prior mammogram from 1 year ago.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 
3. Routine annual screening mammography is recommended. Supplemental screening with breast ultrasound or MRI is not indicated based on breast density alone but may be considered based on individual risk factors.

BIRADS Category 1: Negative. ➡ *Negative*
- ULTRASOUND OF LEFT BREAST
CLINICAL HISTORY: 55 year old female with palpable lump in the left breast upper outer quadrant, family history includes mother with breast cancer.
COMPARISON: Compared to screening mammogram from 2 years prior.
FINDINGS:
Left breast ultrasound was performed. The breast is almost entirely fatty replaced. No discrete mass, architectural distortion, or suspicious calcifications are identified. Specifically, no abnormality is seen in the area of palpable concern in the left upper outer quadrant. Benign appearing lymph nodes are noted in the left axilla.
IMPRESSION:
No sonographic evidence of malignancy in the left breast. Recommend continued routine screening mammography. 
BIRADS 1 - Negative.
ADDENDUM: Patient contacted and informed of results. She will continue with yearly screening mammography. Clinical follow up of palpable area in 2 months is advised to document stability or resolution. ➡ *Negative*
- FINDINGS: The right breast has scattered fibroglandular densities. There is no focal mass, suspicious calcification or architectural distortion. The skin is normal in thickness. The nipple is normal in configuration. No axillary adenopathy. Benign calcifications consistent with arterial calcifications are noted. No significant interval change.

The left breast is predominantly fatty replaced. Benign scattered vascular calcifications are present. No focal mass or architectural distortion. The nipple and skin are unremarkable.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No evidence of malignancy or suspicious abnormality.
3. Benign bilateral arterial calcifications.

BIRADS 1 - Negative. ➡ *Negative*
- REPORT:
HISTORY: 64 y/o female patient presenting for screening mammogram. No palpable lumps, no nipple discharge, no breast pain. Patient has a personal history of breast cancer with right mastectomy and reconstruction performed 5 years back. No family history of breast cancer. 
COMPARISON: Prior mammogram 2 years back.
FINDINGS:
Right reconstructed breast is unremarkable.
The left breast is composed predominantly of fatty tissue. There is no mammographic evidence of malignancy. No suspicious masses, architectural distortion or calcifications are identified. No significant change since prior mammogram.
IMPRESSION:
BI-RADS Category 1: Negative. No mammographic evidence of malignancy in the left breast. Recommend continued annual screening mammogram in 12 months. Follow up as clinically indicated.
ADDENDUM: Retrospective review of the studies do not reveal any suspicious microcalcifications in the left breast. ➡ *Negative*
- TECHNIQUE: Bilateral digital mammogram, including craniocaudal and mediolateral oblique views.

COMPARISON: Comparison is made with prior outside mammogram from December 2022.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses. The breasts are stable in appearance compared to prior mammograms. There are no dominant masses, architectural distortions, or suspicious calcifications.

Stable bilateral scattered benign-appearing calcifications are again noted. No new or concerning grouped, pleomorphic, or fine linear microcalcifications. 

IMPRESSION:
Bilateral heterogeneously dense but stable mammogram without evidence of malignancy. No suspicious microcalcifications. 
BI-RADS Category 1 - Negative.

Recommend continued annual screening mammography in 12 months. Clinical breast exam also advised.

BIRADS 1. ➡ *Negative*
- LEFT SCREENING MAMMOGRAM
CLINICAL HISTORY: Routine screening. Negative prior mammogram at this facility on 09/12/2021.
TECHNIQUE: Standard CC and MLO views of the left breast.
COMPARISON: Prior left screening mammogram from 09/12/2021.
FINDINGS:
The left breast is composed of scattered fibroglandular densities.
There are no dominant masses, architectural distortions, or suspicious calcifications.
No significant interval change is observed.
IMPRESSION:
1. Negative left screening mammogram.
2. No mammographic evidence of malignancy. Annual screening mammography is recommended.
BI-RADS Category 1 - Negative. ➡ *Negative*
- LEFT BREAST MAMMOGRAPHY AND TARGETED ULTRASOUND
INDICATION: New palpable lump left breast at 3 o'clock. Personal history of right breast cancer.
TECHNIQUE: Standard CC and MLO views of the left breast. Targeted ultrasound of the left breast 3 o'clock position using a 12 MHz linear transducer. 
COMPARISON: Prior mammogram from June 2021.
FINDINGS:
The left breast is heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications. The palpable area of concern corresponds to an area of normal fibroglandular tissue on targeted ultrasound, with no discrete mass.
Stable benign-appearing calcifications are scattered throughout the breast. 
There is no abnormality in the left axilla.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. Stable benign calcifications. 
2. Palpable finding at 3 o'clock on the left corresponds to normal fibroglandular tissue, with no sonographic correlate. 
3. Recommend clinical follow up of palpable finding, as indicated. Routine screening mammography in 1 year.
BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
HISTORY: Follow up for 45 year old female with maternal history of breast cancer and left breast lump that is palpable by patient but not physician. Last mammogram 2 years ago was normal.

COMPARISON: Compared to previous mammography from 2 years ago.

FINDINGS:
Right breast: Architectural distortion at 9:00 position upper outer quadrant, 5 cm from the nipple. There are no associated calcifications or skin changes. 
Left breast: Scattered fibroglandular tissue. No abnormal masses, calcifications, or architectural distortion.

IMPRESSION: 
1. New architectural distortion in the right breast at 9:00, 5 cm from the nipple, highly suspicious for malignancy. Biopsy recommended. BIRADS 4C. 
2. Stable appearance of the left breast with no mammographic evidence of malignancy. However, given the new finding in the right breast, short interval follow up is recommended in 6 months. Clinical correlation recommended for the palpable lump. BIRADS 3 for the left breast.

ULTRASOUND ADDENDUM:
Targeted ultrasound performed to the left breast lump palpated by patient at 5:00 position. No sonographic correlate identified. 

FINAL ASSESSMENT:
1. Right breast: BIRADS 4C 
2. Left breast: BIRADS 3 ➡ *Negative*
- FINDINGS: Left breast tissue contains scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications. Stable appearance compared to prior mammograms.

Targeted ultrasound of the left breast at 7 o'clock position demonstrates normal fibroglandular tissue with no discrete solid or cystic lesion.

IMPRESSION:

1. No mammographic evidence of malignancy in the left breast. Recommend continued annual screening mammography in 12 months.

2. No sonographic correlate for the palpable abnormality in the left breast 7 o'clock position. Recommend clinical follow up and correlation with physical exam findings as indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM AND ULTRASOUND.
HISTORY: F/u from screening mammogram at which time pt had called back for additional views. Reported  area of new focal density. Prior benign biopsies on right.
FINDINGS:
Right breast tissue contains scattered fibroglandular densities. There is no focal mass, suspicious calcification, or architectural distortion. Stable benign-appearing calcifications in the upper outer quadrant from prior biopsy. There is no significant change from previous.
Left breast tissue is heterogeneously dense, which could obscure detection of small masses. There is no focal mass, architectural distortion, or suspicious calcifications. Specifically, no abnormality is seen in the area of palpable concern in the upper outer left breast. There is no significant change from previous.
Bilateral breast ultrasound was performed. There are no suspicious solid or cystic masses. No suspicious calcifications are identified.
IMPRESSION: 
1. No mammographic or sonographic evidence of malignancy. 
2. Stable benign right breast calcifications. 
3. Heterogeneously dense breast tissue.
BI-RADS 2: Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- CLINICAL HISTORY: 53-year-old female presents for bilateral diagnostic mammogram and ultrasound. Patient reports a new palpable lump in the left breast at 10 o'clock position. Last mammogram was 18 months ago at an outside facility and was reported as normal. Patient has a family history of breast cancer in her mother diagnosed at age 65.

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified in either breast.

Targeted ultrasound of the left breast at 10 o'clock position, 3 cm from the nipple, demonstrates a 0.8 x 0.6 x 0.5 cm oval, circumscribed, hypoechoic mass, parallel in orientation. This corresponds to the patient's palpable abnormality. No suspicious features are identified.

IMPRESSION:
1. Bilateral mammogram shows no evidence of malignancy. Scattered fibroglandular densities are noted. 
2. Left breast ultrasound shows a 0.8 cm circumscribed oval mass at 10 o'clock, corresponding to the palpable finding. This has benign features and likely represents a fibroadenoma. 
3. No microcalcifications are identified in either breast.

RECOMMENDATION: 
Routine annual screening mammography is recommended. Short-term follow up ultrasound of the left breast mass in 6 months to document stability. 

BI-RADS Category 2: Benign finding. Recommend routine screening. ➡ *Negative*
- IMPRESSION:
1. Left craniocaudal and mediolateral oblique views and targeted ultrasound demonstrate scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications.
2. Ultrasound demonstrates no focal lesion in the area of palpable concern at 7 o'clock position left breast middle depth. Recommend clinical follow-up and diagnostic mammogram if clinically indicated.
3. Interval increase in fibroglandular density in the upper outer left breast, likely reflecting variable compression. No sonographic correlate.
BIRADS 2: Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious microcalcifications.

IMPRESSION: 
Negative bilateral screening mammogram.

BI-RADS Category 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL MAMMOGRAM.
HISTORY: Patient presents for annual screening mammogram. Last prior mammogram 9/2021 at this facility.
FINDINGS:
Parenchymal pattern is heterogeneously dense (C). This decreases sensitivity of mammography. Consider supplemental screening in women with dense breast tissue.
There are no dominant masses, suspicious calcifications or areas of architectural distortion bilaterally.
There are stable bilateral scattered benign-appearing calcifications.
There is no significant interval change from the prior exam.
IMPRESSION:
1. No mammographic evidence of malignancy. BIRADS 2.
2. Recommend routine screening in one year.
ADDENDUM: 
Patient was notified of BIRADS 2 results via MyChart. Routine screening is recommended in one year. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL INFORMATION: Annual screening.
COMPARISON: Mammograms from 05/2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
There are no significant changes compared to prior mammograms.
IMPRESSION:
BIRADS 1 - There are no mammographic findings to suggest malignancy. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL MAMMOGRAM
HISTORY: Annual screening mammogram.
FINDINGS:
Right breast tissue is predominantly fatty. There are no concerning masses, calcifications, or areas of architectural distortion.
Left breast tissue is extremely dense, which may lower the sensitivity of mammography. There are no concerning masses, calcifications, or areas of architectural distortion.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Breast density is extremely dense in the left breast, which may lower the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI as clinically appropriate.
BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND REPORT.
HISTORY: Left breast pain, 7 o'clock position, history of right breast cancer, previous right mastectomy. No other significant history.
FINDINGS:
Right mastectomy changes noted. 
Left breast: Scattered fibroglandular densities. No mammographic evidence of malignancy or suspicious microcalcifications. Stable compared to previous exams.
Ultrasound of the left breast at 7 o'clock: No sonographic evidence of a discrete mass or cyst. 
IMPRESSION:
1. Stable left breast, no evidence of malignancy on mammogram or ultrasound. BIRADS 1.
2. Right mastectomy changes.
Recommend annual screening mammograms.
ADDENDUM: 
Patient called and expressed concerns regarding a palpable lump in the left breast at 7 o'clock, corresponding to the area of pain. Clinical exam by the breast surgeon revealed no palpable abnormalities in this region. No further imaging is recommended at this time based on the negative mammogram and ultrasound findings, as well as the negative clinical exam. ➡ *Negative*
- LEFT DIAGNOSTIC MAMMOGRAM:
INDICATION: Further evaluation of palpable abnormality in the upper outer quadrant, left breast.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
COMPARISON: With prior study of 12/17/2019.
FINDINGS:
The breasts are predominantly fatty. There are no mammographic findings to indicate malignancy. No dominant mass, area of architectural distortion or suspicious calcifications.  
There is stable post-surgical scarring in the upper outer left breast.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Stable post-surgical changes left breast.
3. Breasts are predominantly fatty, limiting sensitivity of mammography. Recommend ultrasound for further evaluation of palpable finding.
BIRADS 1 - Negative mammogram. Recommend annual screening in 1 year. ➡ *Negative*
- HISTORY: Annual screening mammogram. Last mammogram performed 1 year prior was normal. No prior history of breast cancer or biopsies. No family history of breast cancer. No palpable lumps.

FINDINGS: Bilateral breast tissue contains scattered fibroglandular densities. There is no focal mass, suspicious calcification, or architectural distortion in either breast. There are no significant changes compared to prior mammograms. 

Targeted ultrasound was performed to the left breast at 2 o'clock position, corresponding to a palpable area of concern. No sonographic abnormality was identified. 

IMPRESSION:
1. There is no mammographic evidence for malignancy in either breast. Annual screening mammography is recommended.
2. No sonographic correlate for the palpable area of concern in the left breast at 2 o'clock position. Clinical follow up and correlation is recommended as indicated.

BI-RADS Category 1: Negative.

ADDENDUM: Spoke with ordering physician, Dr. Smith, and informed of the results. No further imaging is recommended at this time. Follow up as clinically indicated. ➡ *Negative*
- FINDINGS: The right breast demonstrates a mastectomy scar with no residual breast tissue identified.

The left breast is composed of heterogeneously dense fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications are identified. The skin, nipple, and areola are unremarkable. No significant interval change compared to the prior exam.

Left breast ultrasound was performed with attention to the palpable concern at 7 o'clock. Targeted scanning of the area demonstrates normal fibroglandular tissue with no discrete solid or cystic lesion identified.

IMPRESSION:
1. Negative bilateral mammogram. No suspicious findings or microcalcifications.
2. Negative targeted left breast ultrasound. No sonographic correlate for the palpable finding at 7 o'clock.

BI-RADS category 1: Negative.

RECOMMENDATIONS:
Routine annual screening mammography is recommended. Clinical follow up of the palpable finding as needed.

ADDENDUM: Comparison was made to an outside mammogram from December 2024, which was reported as normal. The current study is concordant with the prior exam. ➡ *Negative*
- BILATERAL MAMMOGRAM
HISTORY: Annual screening mammogram. No symptoms or complaints.
Comparison made to prior imaging.
FINDINGS:
Both breasts are predominantly fatty.
There are no masses, architectural distortions, or suspicious calcifications in either breast.
There is no significant interval change since the prior exam.
IMPRESSION:
Negative bilateral screening mammogram.
BIRADS 1. Annual screening recommended. ➡ *Negative*
- FINDINGS: The left breast parenchyma is composed of scattered fibroglandular densities. No dominant masses, architectural distortion, or suspicious calcifications are identified. Stable appearance compared to the most recent prior examination.

Targeted ultrasound of the left breast 7 o'clock position was performed. No correlating abnormality is seen in this region. The tissue is unremarkable.

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend routine screening mammography in 1 year.
2. Targeted left breast ultrasound is negative for sonographic abnormality at the area of clinical concern. Recommend clinical follow up as indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- REPORT: "LEFT MAMMOGRAM AND ULTRASOUND.
HISTORY: Screening mammogram. 54-year-old female patient. No known breast issues. Family history of breast cancer in mother.
Comparison made to prior imaging from 2 years ago.
FINDINGS: The left breast tissue is predominantly fatty with scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious microcalcifications are identified. There are no significant changes compared to prior mammogram.
Targeted left breast ultrasound was also performed. No sonographic abnormality is identified. The visualized left axilla appears unremarkable.
IMPRESSION: No mammographic or sonographic evidence of malignancy in the left breast. Recommend continued annual screening mammography.
BIRADS 1 - Negative." ➡ *Negative*
- MAMMOGRAM - BILATERAL DIAGNOSTIC.
CLINICAL INDICATION: Intermittent bilateral breast pain.
COMPARISON: Mammogram from 9/18/2023.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Targeted ultrasound was performed on the right breast at the 3 o'clock position.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. 
Stable, benign-appearing calcifications are noted bilaterally.
Ultrasound of the right breast at 3 o'clock, 4 cm from the nipple, demonstrates a 5 mm simple cyst. No solid component or suspicious features.
IMPRESSION: 
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. 5 mm simple cyst in the right breast at 3 o'clock position, benign. No further evaluation needed.
BIRADS 2: Benign findings. ➡ *Negative*
- FINDINGS: There are scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. Entirely fatty replaced breasts.

BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS: Bilateral mammographic views show predominantly fatty replaced breast tissue. There are no focal masses, architectural distortion, or suspicious calcifications in either breast. Stable benign calcifications are noted in the right breast. No significant change from prior mammogram.

Targeted ultrasound was performed in the left breast at the site of palpable abnormality reported by the patient at 2 o'clock position, 3 cm from the nipple. No correlating sonographic mass or cystic lesion identified.

IMPRESSION:
1. Negative bilateral screening mammogram. Fatty replaced breast tissue. 
2. No sonographic correlate for the palpable finding in the left breast.

BIRADS 1 - Negative. Recommend annual screening mammography in 1 year.

ADDENDUM: Comparison was made to prior outside mammogram from 10/15/2021, which was also negative. No prior ultrasound is available for comparison. ➡ *Negative*
- MAMMOGRAPHIC FINDINGS:
Bilateral digital screening mammogram, comparison made to prior exam dated 11/15/2022.
RIGHT BREAST:
There is scattered fibroglandular tissue. No dominant mass or architectural distortion. No suspicious calcifications.
LEFT BREAST:
There is heterogeneously dense parenchyma. No dominant mass or architectural distortion. No suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy, both breasts. Recommend continued routine screening mammography in 1 year.
2. Mammographic breast density: Heterogeneously dense, which may obscure detection of small masses. For moderate to very dense breast tissue, additional screening with breast ultrasound or breast MRI may be considered, as determined by individual risk factors and in consultation with the referring provider.
BIRADS 1 - NEGATIVE.
ADDENDUM:
Comparison is made to prior study dated 5/12/2022 at outside facility, DiagnosticImaging Partners. The breasts appear stable with no interval change or development of suspicious findings. Heterogeneously dense breast tissue is again noted, unchanged. No microcalcifications are identified in either breast. ➡ *Negative*
- MAMMOGRAM REPORT
CLINICAL HISTORY: 52-year-old female presenting for routine screening mammogram. No current breast complaints. 
COMPARISON: Comparison made to prior mammogram from 04/15/2022. 
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications. 
LEFT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography in 12 months.
BIRADS 1- Negative. ➡ *Negative*
- MAMMOGRAM, LEFT BREAST.
CLINICAL INDICATION: 34 year old female with large breast presenting for routine screening.
FINDINGS: There are scattered areas of benign appearing fibroglandular density. No suspicious masses, calcifications, architectural distortion or other mammographic evidence of malignancy is seen. No significant change from prior examinations.
IMPRESSION: Left breast negative mammogram. No radiographic evidence of malignancy.
BIRADS 1. Recommend annual screening mammography in 1 year. ➡ *Negative*
- FINDINGS: The left breast appears heterogeneously dense. No suspicious masses, architectural distortions, or dominant masses. Specifically, no abnormalities in the area of palpable concern at the 6 o'clock position of the left breast. There are no suspicious microcalcifications. Benign calcifications are noted.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. No sonographic correlate for the palpable abnormality at 6 o'clock position of the left breast. 

BI-RADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: 55 year old female, screening mammogram.
COMPARISON: Previous mammograms from 2021 and 2022.
FINDINGS: 
Right breast: The breast is heterogeneously dense (type C). There are no dominant masses, suspicious calcifications or areas of architectural distortion.  
Left breast: The breast is heterogeneously dense (type C). There are no dominant masses, suspicious calcifications or areas of architectural distortion.  
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography.
BIRADS Category 1 - Negative. ➡ *Negative*
- FINDINGS: The right breast is absent, consistent with prior mastectomy. The left breast is heterogeneously dense. No dominant mass, architectural distortion, or suspicious calcifications are identified. Stable appearance compared to prior mammograms. 

Targeted ultrasound was performed at the area of palpable concern, 7 o'clock position of the left breast. No correlating sonographic mass or abnormality was identified.

IMPRESSION:
1. Negative left screening mammogram. No mammographic evidence of malignancy. Recommend annual screening mammography. 
2. Targeted left breast ultrasound is negative for correlating mass at site of palpable abnormality in the 7 o'clock position. Recommend clinical follow-up as indicated. 

BI-RADS Category 1: Negative.

ADDENDUM: This examination was reviewed in detail with Dr. Jane Smith following the interpretation. No changes were made to the above report. ➡ *Negative*
- SCREENING MAMMOGRAM.
CLINICAL INDICATION: Annual screening mammography.
COMPARISON: Screening mammogram from May 2023.
TECHNIQUE: Standard CC and MLO views obtained bilaterally.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no suspicious masses, architectural distortion, or suspicious calcifications.
There are stable scattered fibroglandular densities and areas of benign-appearing calcifications bilaterally, likely representing fibrocystic changes.
Stable postoperative changes noted in the right breast, consistent with history of prior lumpectomy.
No significant interval change compared to prior examination.
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI as clinically appropriate.
BIRADS 1 - Negative.
FOLLOW-UP RECOMMENDATIONS: Recommend continued annual screening mammography and clinical breast exams. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Screening. High risk patient.
COMPARISON: 6/15/2019
TECHNIQUE: Bilateral digital mammography was performed including 2D synthesized and 3D tomosynthesis images.
FINDINGS:
Both breasts show scattered areas of fibroglandular density. There are no dominant masses, suspicious calcifications, or areas of architectural distortion bilaterally. No significant interval change compared to prior mammogram.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy.
3. Recommend continued annual screening mammography in one year given high risk status.
ASSESSMENT: Negative. BI-RADS Category 1.
ADDENDUM:
Whole breast ultrasound of both breasts was also performed today. No suspicious masses or findings were identified sonographically in either breast. Specifically, no abnormalities were seen in the 6 o'clock position of the left breast, corresponding to the patient's palpable finding on self-exam. Clinical correlation recommended for palpable abnormality. ➡ *Negative*
- DIAGNOSTIC MAMMOGRAPHY LEFT
HISTORY: Annual screening. 56 years old. No palpable lumps or complaints.
COMPARISON: 2022/01/04
FINDINGS: The breast tissue is heterogeneously dense, which could obscure detection of small masses. There is no architectural distortion or new masses. There are no clusters of suspicious microcalcifications. 
No significant changes compared to prior study.
IMPRESSION: 1. No evidence of malignancy, BIRADS 1. Annual screening recommended. ➡ *Negative*
- MAMMOGRAPHY REPORT
CLINICAL INDICATION: Annual screening mammogram. Patient reports no breast problems.
COMPARISON: Prior screening mammogram from 1 year ago.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
FINDINGS:  
Right breast: The breast is comprised of scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications are noted.
Left breast: The breast is comprised of scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications are noted.  
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continued annual screening mammography.
BIRADS: 1 Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST

INDICATION: 41 year old female presenting with left breast upper outer quadrant palpable mass.

TECHNIQUE: Targeted ultrasound of the left breast was performed using a 12 MHz linear transducer.

COMPARISON: None.

FINDINGS:
Skin: Normal. No skin thickening or abnormal vascularity.
Subcutaneous fat and premammary zone: Normal. No abnormality.  
Parenchyma: Heterogeneously echogenic fibroglandular tissue. No abnormal masses, architectural distortion or suspicious calcifications. 
Ducts: Non-dilated. No intraductal masses or debris.
Vascularity: No increased vascularity on color Doppler interrogation.
Other findings: No axillary, infraclavicular or supraclavicular adenopathy.

IMPRESSION:
No sonographic abnormality in the area of palpable concern in the left upper outer breast. Negative targeted ultrasound of the left breast. 
Recommend clinical follow up and diagnostic mammography for further evaluation. 
BI-RADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAPHIC REPORT
INDICATION: Annual screening mammogram. No clinical concerns.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to outside screening mammogram from April 2022.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. Specifically, no microcalcifications are identified.
Stable benign calcifications are noted in the right breast, likely representing oil cysts or fibrocystic changes. These are unchanged from prior exam.
No skin thickening or retraction. No lymphadenopathy.
IMPRESSION:
1. No mammographic evidence of malignancy. Specifically, no suspicious microcalcifications.
2. Breasts are heterogeneously dense, which lowers the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI for increased breast cancer detection.
BIRADS 1 - Negative. Recommend continued annual screening mammography in 12 months.
ADDENDUM: 
Screening automated whole breast ultrasound performed today as adjunct screening exam for dense breast tissue. No sonographic abnormalities were identified in either breast. BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. The breasts are symmetric. There are no new findings compared to prior exams.

Right breast ultrasound was performed targeted to the area of palpable concern at 10 o'clock. The breast tissue in this region appears normal, containing scattered fibroglandular tissue. No sonographic mass, cyst, or other abnormality is identified. 

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. No sonographic correlate for the palpable area of concern in the right breast at 10 o'clock. 

ASSESSMENT: BI-RADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening

COMPARISON: Mammogram from 1 year ago

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications are identified in either breast.

Stable benign-appearing calcifications are present in the left breast, likely representing fibrocystic change.

There are no significant changes compared to prior mammogram.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography.
2. Stable benign-appearing calcifications in the left breast, likely representing fibrocystic change.

BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- FINDINGS: Bilateral mammogram and ultrasound performed.

Right breast: Predominantly fatty breast tissue. No mass, architectural distortion, or suspicious calcifications.

Left breast: Scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications. 

Targeted ultrasound of the left breast at 10 o'clock position, correlating with area of clinical concern, demonstrates normal fibroglandular tissue. No sonographic correlate for palpable finding.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No sonographic abnormality in the left breast at site of palpable abnormality. 

ASSESSMENT: BI-RADS 1 - Negative. Recommend annual screening mammography in 1 year.

ADDENDUM: Comparison is made to outside mammograms from June 2024, which are similar. No prior ultrasound for comparison. ➡ *Negative*
- FINDINGS:
Right mastectomy. 
Left breast tissue is composed of scattered fibroglandular densities. There are no dominant masses, architectural distortions, or suspicious microcalcifications. The breast tissue appears stable compared to prior examinations.

Targeted left breast ultrasound was performed at the site of palpable abnormality at 6 o'clock position, 3 cm from the nipple. No correlating sonographic mass or cystic lesion was identified. 

IMPRESSION:
1. Negative left screening mammogram. No mammographic evidence of malignancy. Recommend annual screening mammography. 
2. Negative targeted left breast ultrasound. No sonographic correlate for palpable finding. Recommend clinical follow up and correlation in 2-3 months. If persists, short term follow up diagnostic evaluation may be considered.

BI-RADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.

INDICATION: Annual screening mammogram. 

COMPARISON: 10/15/2021

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Both breasts demonstrate heterogeneously dense parenchyma. The amount of dense tissue may lower the sensitivity of mammography.

There are no suspicious masses, architectural distortions, or suspicious calcifications in either breast. 

Stable benign appearing calcifications are noted in the right breast.

No significant interval change compared to prior mammogram.

IMPRESSION: 
BI-RADS Category 1: Negative bilaterally. Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- BILATERAL MAMMOGRAM
INDICATION: Annual routine screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior mammograms from 4/12/2022.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. Specifically, no microcalcifications are identified.
There are no significant changes compared to prior mammograms. 
IMPRESSION: 
BI-RADS 1 - Negative bilateral screening mammogram. No evidence of malignancy. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- FINDINGS: The left breast tissue is heterogeneously dense with scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications. The skin appears normal. 

At the 7 o'clock position in the left breast there is a new 8 mm asymmetry on the mammographic views, corresponding to the area of palpable abnormality. Targeted ultrasound of this region demonstrates an 8 x 6 x 5 mm circumscribed oval parallel hypoechoic mass with no posterior acoustic features. This likely represents a fibroadenoma. No other suspicious sonographic findings.

The right breast is surgically absent, consistent with history of right mastectomy.

IMPRESSION:
1. Asymmetry in the left breast at 7 o'clock, corresponding to palpable finding. Targeted ultrasound suggests this is a fibroadenoma. 6 month follow up ultrasound is recommended to document stability. 
2. No mammographic or sonographic evidence of malignancy. 
3. Right mastectomy changes, stable.

BI-RADS 3 - Probably benign finding. Short interval follow up suggested. ➡ *Negative*
- MAMMOGRAM, BILATERAL.
HISTORY: Screening mammography. Patient is a 45-year-old woman with no significant past medical history. Reports annual screening mammograms have been previously unremarkable. No palpable lumps, skin changes, or nipple discharge.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
Right breast: There are no masses, architectural distortion, or suspicious calcifications.
Left breast: There are no masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Both breasts are heterogeneously dense, which could obscure small masses. Consider supplemental screening with breast ultrasound or MRI for increased breast cancer detection.
BIRADS 1 - Negative. Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAM REPORT
INDICATION: Annual screening mammogram
COMPARISON: None available
FINDINGS:
Right breast: The breast is composed of scattered fibroglandular densities. There are no suspicious masses, calcifications or areas of architectural distortion. No significant change from previous exam.
Left breast: The breast is composed of scattered fibroglandular densities. There are no suspicious masses or calcifications. No significant change from previous exam.
IMPRESSION:
1. No mammographic evidence of malignancy. Routine annual screening recommended.
2. No evidence of microcalcifications.
BI-RADS Category 1: Negative ➡ *Negative*
- MAMMOGRAPHIC REPORT:

INDICATION: Screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts.

FINDINGS:
The breasts are predominantly fatty replaced with scattered areas of fibroglandular density. No focal mass, architectural distortion, or suspicious calcifications are identified.

Bilateral benign lymph nodes are noted in the axillary regions.

IMPRESSION:
Negative bilateral screening mammogram.

ADDENDUM:
Examination is limited due to patient motion. Repeat mammogram is suggested if clinically indicated.

BIRADS CATEGORY 1 - Negative. ➡ *Negative*
- BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Annual screening. No current breast complaints.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
COMPARISON: 10/15/2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. Stable benign appearing bilateral breast tissue. No significant interval change compared to prior mammograms.
Ultrasound of the right breast 10:00 position demonstrates normal fibroglandular tissue without a discrete mass, cyst, or other finding.
Ultrasound of the left breast 2:00 position shows normal ducts and stromal tissue without a discrete mass, cyst, or other finding. 
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months. 
2. No sonographic abnormality in the areas scanned. 
BIRADS 1- Negative. ➡ *Negative*
- FINDINGS:
Both breasts show scattered fibroglandular tissue. No suspicious mass, architectural distortion or calcifications are identified.

IMPRESSION:
Negative bilateral screening mammogram. Recommend continued annual screening in 12 months. 

BIRADS 1 - Negative. ➡ *Negative*
- REPORT: "BILATERAL MAMMOGRAM AND ULTRASOUND.
HISTORY: 55-year-old female with occasional bilateral breast tenderness. No palpable lumps. Last mammogram 1 year ago at StLukes - reported as normal.
FINDINGS: Bilateral mammograms demonstrate scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast. Stable appearance compared to prior mammogram.  
Bilateral breast ultrasound was performed. No solid or cystic lesions are seen in either breast. No abnormal axillary lymph nodes are identified.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend annual screening mammography.
2. No sonographic abnormalities identified. Bilateral breast tenderness is likely due to hormonal changes.
BIRADS 1.
ADDENDUM: Upon further review and discussion with the referring clinician, no additional imaging is required at this time. Patient is advised to return for routine annual screening mammography." ➡ *Negative*
- FINDINGS:
Both breasts are composed of scattered fibroglandular densities. The breasts are symmetric. 
No dominant masses, architectural distortion or suspicious microcalcifications.
Stable benign calcifications are noted in the right breast.
No mammographic evidence of malignancy. 
IMPRESSION:
No mammographic evidence of malignancy. 
Continue routine screening mammography in one year.
BIRADS 1 - Negative. ➡ *Negative*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
HISTORY: Screening mammogram. No prior mammograms available for comparison. No reported breast complaints. 
FINDINGS:
Mammogram: The breasts are composed of scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Ultrasound: Targeted ultrasound was performed in the upper outer quadrant of the left breast corresponding to the patient's area of concern. No sonographic abnormality was identified. 
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No sonographic correlate for palpable abnormality in the left breast.
MANAGEMENT: Routine screening mammography is recommended in 1 year. Clinical follow up for left breast palpable abnormality as needed.
BIRADS 1 - Negative. ➡ *Negative*
- RADIOLOGY REPORT
HISTORY: Left breast lump for 2 weeks.
CLINICAL INFORMATION: No previous mammograms. Breast cancer in maternal grandmother. Patient complains of a tender, moving mass in the left breast upper outer quadrant, noticed in the shower.
EXAMINATION: Digital 2D left diagnostic mammogram views and left ultrasound performed.
FINDINGS:
2D Left Mammogram:
Left breast composed of scattered fibroglandular densities. The breast is heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious microcalcifications.
Left Ultrasound:
Targeted ultrasound to the left upper outer breast reveals no discrete mass. Normal fibroglandular tissue and fat lobules present. No abnormal vascularity or fluid collection. The area of palpable concern corresponds to normal glandular tissue.
IMPRESSION:
1. Left breast heterogeneously dense. No suspicious mammographic findings.
2. Targeted left ultrasound is negative. Clinically palpated area corresponds to normal glandular tissue.
BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHY AND ULTRASOUND OF THE LEFT BREAST

PATIENT HISTORY:
Patient presents with a palpable lump in the upper outer quadrant of the left breast, noted on self-examination. No prior history of breast cancer or breast surgery. Maternal aunt with history of breast cancer.

FINDINGS:
Left breast is composed of heterogeneously dense breast tissue. No dominant mass, suspicious calcifications, or architectural distortion identified.

Targeted ultrasound was performed over the palpable area of concern in the left upper outer breast. A 0.8 x 0.6 x 0.5 cm oval, circumscribed, hypoechoic mass was identified at the 10 o'clock position, 6 cm from the nipple, corresponding to the palpable finding. The mass demonstrates no posterior acoustic features and no internal vascularity. 

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend annual screening mammography.
2. Benign-appearing left breast mass at 10 o'clock, corresponding to the palpable finding, most consistent with a fibroadenoma. Short-term follow-up ultrasound in 6 months is recommended to confirm stability. 

BI-RADS CATEGORY 2 - Benign finding. Recommend routine screening. ➡ *Negative*
- MAMMOGRAM REPORT - LEFT BREAST
HISTORY: Patient reports new lump in the upper outer quadrant of the left breast, first noticed 2 weeks ago. No prior mammograms available for comparison. Family history of breast cancer in maternal grandmother.
FINDINGS:
Left breast composed of scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications. Skin and nipple are unremarkable.
Targeted ultrasound of the palpable area in the left upper outer quadrant demonstrates a 0.8 cm oval circumscribed hypoechoic mass, likely representing a benign fibroadenoma. No suspicious features identified.
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast.
2. 0.8 cm benign-appearing mass in the left upper outer quadrant on ultrasound, corresponding to the palpable finding. Recommend clinical follow up in 3-6 months to document stability. 
BIRADS 2 - Benign findings. Recommend routine screening in 1 year unless clinically indicated sooner based on the palpable finding. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 
There are no new or enlarging masses in comparison with prior exams.
No sonographic abnormality is seen.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 
3. Recommend annual screening mammography. 

BI-RADS Category 1: Negative. ➡ *Negative*
- REPORT:
"RIGHT MAMMOGRAM.
HISTORY: Annual screening mammogram. Patient has history of left breast cancer treated with lumpectomy and radiation therapy in 2020.
FINDINGS: Left mastectomy. Right breast tissue is heterogeneously dense, which may obscure detection of small masses. There is no mammographic evidence of malignancy. There are no suspicious masses, areas of architectural distortion, or suspicious calcifications. Stable benign calcifications are noted in the upper outer quadrant. No significant change compared to prior mammograms.
IMPRESSION: 
1. Negative digital screening mammogram of the right breast. 
2. Left mastectomy.
3. Recommend annual screening mammography of the right breast.
BIRADS 1 - Negative." ➡ *Negative*
- MAMMOGRAM REPORT:

PROCEDURE: Screening Bilateral Mammogram.

HISTORY: 51 year old female with family history of breast cancer.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

COMPARISON: None available.

FINDINGS:
Right Breast:
The breast is almost entirely fatty. There are no masses, architectural distortion, or suspicious calcifications.

Left Breast:
The breast is heterogeneously dense, which could obscure detection of small masses. There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts.

RECOMMENDATION:
Routine yearly screening mammography is recommended.

BIRADS CATEGORY 1: Negative. ➡ *Negative*
- HISTORY: Patient presents for her annual screening mammogram. No prior history of breast cancer.

FINDINGS: Both breasts are composed of scattered fibroglandular densities. The parenchyma is relatively similar in density and distribution bilaterally. There is no mammographic evidence of a suspicious mass, architectural distortion, or abnormal calcifications.

IMPRESSION: 
1. Bilateral benign appearing mammograms. No suspicious mass, calcifications, or other findings to suggest malignancy.  
2. No significant interval change compared to prior exam from 1/25/2022.

ASSESSMENT: BI-RADS Category 1 - Negative. Recommend routine screening in 1 year.

ADDENDUM: Final assessment category remains 1 after additional evaluation. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses. 
No dominant mass, suspicious calcifications or architectural distortion.
There are no new masses, calcifications, or other findings.
Stable postsurgical changes noted in the upper outer quadrant of the right breast. Surgical clips are visible.
There are bilateral implants, unchanged in appearance.
No axillary lymphadenopathy.

IMPRESSION:
No mammographic evidence of malignancy.
BI-RADS category 1: Negative.
Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: None available.

FINDINGS:
Both breasts show scattered areas of heterogeneously dense fibroglandular tissue.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast: 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Heterogeneously dense breasts. 
2. No mammographic evidence of malignancy. 
3. Recommend routine screening mammography in 1 year, sooner if clinically indicated.

BIRADS 1 - Negative

ADDENDUM: Bilateral mammogram. No suspicious calcifications. Routine screening recommended. ➡ *Negative*
- FINDINGS: The right breast is absent, status post mastectomy. The left breast consists of scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified. Stable mammographic appearance compared to prior exams.

Targeted ultrasound was performed over the palpable area of concern at the 4 o'clock position of the left breast. No sonographic correlate was identified. The area consists of normal appearing fibroglandular tissue.

IMPRESSION:
1. Negative left breast mammogram. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Negative targeted left breast ultrasound at the site of palpable abnormality in the 4 o'clock position. No sonographic findings to explain the palpable concern. Recommend clinical follow up as needed.

BIRADS 1 - Negative. ➡ *Negative*
- SCREENING MAMMOGRAM.
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints. No personal history of breast cancer. 
COMPARISON: Screening mammogram of March 27, 2021.
FINDINGS:
The tissue pattern is predominantly fatty.
There are no masses, architectural distortion or suspicious calcifications.
There are no significant changes compared to prior mammogram.  
IMPRESSION:
Negative bilateral screening mammogram. 
BIRADS Category 1 - Negative. Routine screening in 1 year. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, architectural distortions or suspicious calcifications in either breast. Specifically, no abnormalities are noted in the left subareolar region corresponding to the area of palpable concern.

IMPRESSION:
No mammographic evidence of malignancy, BI-RADS 1. Recommend continued routine screening mammography in 1 year.

ADDENDUM:
Bilateral targeted ultrasound was performed.
Findings: At the site of palpable abnormality in the left breast 2 o'clock position 2 cm from the nipple there is a 0.5 x 0.3 x 0.4 cm oval circumscribed hypoechoic mass, parallel to the skin surface. This is consistent with a benign finding such as fibroadenoma.
Impression:
Benign left breast mass, BI-RADS 2. Short term follow up ultrasound can be considered to document stability. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL, DIAGNOSTIC.
INDICATION: Right breast lump, palpated by physician and patient.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
The breast tissue is heterogeneously dense which could obscure detection of small masses. No dominant mass, architectural distortion or suspicious microcalcifications are seen in either breast. Benign bilateral vascular calcifications are seen. Benign calcifications are seen in the right breast, likely representing dermal calcifications.
Supplemental screening ultrasound is recommended to evaluate for right breast mass.
IMPRESSION:
1. Negative bilateral mammogram. No suspicious microcalcifications, masses or architectural distortion.
2. Dense breasts with possible decreased sensitivity of mammography.
3. Recommend targeted right breast ultrasound to evaluate palpable abnormality. 
BIRADS 1 - Negative mammogram. Recommend supplemental screening with ultrasound. ➡ *Negative*
- ULTRASOUND AND MAMMOGRAM OF RIGHT BREAST 

CLINICAL HISTORY: Patient noted palpable lump in right breast, upper outer quadrant. Negative family history for breast cancer.

COMPARISON: Comparison made to outside mammogram from 1 year ago.

FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense. No suspicious mass, architectural distortion or suspicious calcifications. 

Targeted ultrasound was performed at the site of the palpable abnormality in the right upper outer quadrant at 10 o'clock, 8 cm from the nipple. This corresponds to a 0.8 x 0.5 x 0.5 cm oval circumscribed hypoechoic solid mass, parallel orientation. This likely represents a fibroadenoma. 

LEFT BREAST:
The breast tissue is heterogeneously dense. No suspicious mass, architectural distortion or calcifications.  

IMPRESSION:

1. Likely benign right breast mass at site of palpable abnormality, BI-RADS 3. Short term follow up is recommended to confirm stability. 

2. No suspicious mammographic findings bilaterally. No microcalcifications. Recommend continued annual screening mammography. ➡ *Negative*
- MAMMOGRAPHY, RIGHT BREAST.
INDICATION: Follow up after right breast lumpectomy.
TECHNIQUE: Standard CC and MLO views of the right breast were obtained. Targeted ultrasound of the right breast upper outer quadrant was also performed.
COMPARISON: Prior exam from 10/15/2022.
FINDINGS:
The right breast is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, architectural distortions, or suspicious calcifications. The surgical scar in the right upper outer quadrant is unchanged. Targeted ultrasound of this area shows no residual or recurrent lesion. The remainder of the right breast is unremarkable.
IMPRESSION:
1. No evidence of malignancy in the right breast. 
2. Stable postoperative changes in the right breast, as described above. 
3. Recommend annual screening mammography and continued clinical follow up.
BIRADS 1 - Negative.
ADDENDUM:
Radiology/Pathology Correlation: The patient's prior right breast lumpectomy pathology from 10/01/2022 showed invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative, with negative margins. The current exam shows no evidence of residual or recurrent disease. ➡ *Negative*
- BILATERAL MAMMOGRAM AND LEFT BREAST ULTRASOUND.
HISTORY: 65 year old female, routine screening. Prior benign biopsy of right breast at 10 o'clock position. No other significant history. No current complaints.
Comparison made to prior imaging from May 2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right breast: There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign biopsy clip noted in the upper outer quadrant at 10 o'clock position, corresponding to the patient's history. No significant change from prior exam.
Left breast: There are no dominant masses, architectural distortion, or suspicious calcifications. No significant change from prior exam.
Bilateral axillae are unremarkable.
Targeted ultrasound of the right breast at 10 o'clock position demonstrates a stable benign-appearing biopsy site, corresponding to the mammographic biopsy clip. No suspicious lesions seen.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend continued annual screening mammography.
2. Stable benign right breast biopsy site at 10 o'clock position, unchanged from prior exams.
BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- FINDINGS:

Bilateral breast tissue contains scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications.

Targeted ultrasound was performed in the right breast at the site of palpable abnormality at 10 o'clock, 4 cm from the nipple. No sonographic correlate was identified. The palpable area corresponds to normal fibroglandular tissue.

IMPRESSION:

1. No mammographic or sonographic evidence of malignancy. Recommend continued routine screening mammography. 

2. Palpable abnormality right breast 10 o'clock represents normal fibroglandular tissue, no sonographic correlate.

BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAPHY - BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammograms from 1/15/2022.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Bilateral axillae are unremarkable.

IMPRESSION:
BI-RADS Category 1: Negative bilaterally. No evidence of malignancy.

RECOMMENDATIONS:
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Screening. Patient with family history of breast cancer. 
COMPARISON: Prior exam 2 years ago.
FINDINGS:
The left breast is heterogeneously dense. No suspicious masses, architectural distortion, or microcalcifications.  
Targeted ultrasound to the left breast upper outer quadrant: No suspicious mass or cystic lesion corresponding to the palpable area of concern. 
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend continued screening mammography. 
2. No sonographic correlate for the palpable finding in the left upper outer breast. 
ASSESSMENT: 
Left breast, BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Right breast: There is a right mastectomy with no mammographic abnormality.
Left breast: The breast is composed of scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications are identified.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Right mastectomy.
ASSESSMENT: BI-RADS 1 - Negative. Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Annual screening mammogram
COMPARISON: Screening mammogram from 08/20/2024 at St. Mary's Hospital
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Bilateral axillary regions are unremarkable.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammograms and clinical breast exams. 
BIRADS 1 - Negative ➡ *Negative*
- CLINICAL HISTORY: 55-year-old female presenting for a routine screening mammography. No known palpable masses or other breast concerns. Last mammogram performed 1 year prior was unremarkable.

FINDINGS:
Bilateral digital 2D mammographic views were obtained including craniocaudal and mediolateral oblique projections.

Right breast: The breast is predominantly fatty replaced. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable.

Left breast: The breast consists of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy. 
3. Recommend continued annual screening mammography per guidelines. 

BIRADS Category 1: Negative. ➡ *Negative*
- REPORT:
"SCREENING MAMMOGRAM.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views obtained bilaterally.
FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS 1: Negative screening mammogram. Recommend continued annual screening in 12 months.
ADDENDUM:
The interpreting radiologist reviewed the study independently and agrees with the above findings. No suspicious calcifications are identified. There are no masses, asymmetries, or areas of architectural distortion in either breast. Benign-appearing lymph nodes are noted in both axillae."
LABEL: Negative for microcalfications in the current report. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
NAME: Jane Smith
EXAMINATION: Bilateral digital mammography, followed by targeted left breast ultrasound
CLINICAL HISTORY: 56-year-old female presents for high risk screening. Mother had breast cancer at age 59. No palpable complaints.
COMPARISON: 10/15/2022
TECHNIQUE: Standard CC and MLO views of bilateral breasts.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure small masses.
Right breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Left breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Targeted ultrasound of the left breast upper outer quadrant was performed. No sonographic correlate for the palpable complaint was identified.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in the setting of dense breast tissue.
2. Targeted left breast ultrasound is negative.
ASSESSMENT: BI-RADS Category 1: Negative
MANAGEMENT: Recommend continued annual screening mammography. ➡ *Negative*
- MAMMOGRAM REPORT:
PROCEDURE: Bilateral digital mammogram with tomosynthesis.
HISTORY: 45 year old female presents for screening mammogram. No current breast complaints.
TECHNIQUE: Bilateral CC and MLO views were obtained using tomosynthesis. 
COMPARISON: Comparison is made to outside screening mammogram from 6/15/2021.
FINDINGS:
Right breast: The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign appearing lymph node in the right axilla.
Left breast: The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, architectural distortion, or suspicious calcifications.  
IMPRESSION:
1. No evidence of malignancy in either breast. Recommend continued annual screening in 12 months with consideration for supplemental screening given patient's dense breast tissue. 
2. BI-RADS Category 1: Negative.
ADDENDUM: 
This examination was interpreted using digital mammography with tomosynthesis which increases lesion conspicuity. The breasts are heterogeneously dense which can lower the sensitivity of mammography. Given negative mammogram, no additional imaging is recommended at this time. The patient is encouraged to maintain routine clinical breast exams and follow up with annual screening mammography. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
INDICATION: Screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior exams dated 5/15/2021 and 5/20/2020.
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally, which are stable compared to prior exams.
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Breast tissue is heterogeneously dense, which may lower the sensitivity of mammography. 
BIRADS Category 1: Negative.
RECOMMENDATIONS:
Recommend continued annual screening mammography in 12 months.
ADDENDUM:
Comparison to prior ultrasounds dated 5/15/2021 and 5/20/2020 reveals no abnormality.
No breast ultrasound is indicated at this time based on mammographic findings. ➡ *Negative*
- FINDINGS:
Right breast: Postmastectomy changes noted. No suspicious masses or abnormalities appreciated.
Left breast: Predominantly fatty breast tissue with scattered fibroglandular densities. No focal masses, architectural distortion, or suspicious calcifications identified. 
IMPRESSION:
1. Postmastectomy changes of the right breast. No evidence of recurrence.
2. No suspicious findings in the left breast. Scattered fibroglandular tissue present.
BIRADS 1 - Negative bilateral screening mammogram. ➡ *Negative*
- REPORT:"BILATERAL SCREENING MAMMOGRAM.
CLINICAL INDICATION: Screening mammogram.
COMPARISON: Comparison with prior studies from June 2020.
TECHNIQUE: Bilateral digital 2D mammogram with tomosynthesis views in CC and MLO projections. 
FINDINGS: 
RIGHT BREAST: The breast is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, areas of architectural distortion or suspicious calcifications. No significant interval change since the prior exam.
LEFT BREAST: The breast is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, areas of architectural distortion or suspicious calcifications. No significant interval change since the prior exam.
IMPRESSION:
1. No evidence of malignancy in either breast. 
2. Recommend continued annual screening mammography.
BIRADS 1 - Negative.
ADDENDUM: Reviewed with Dr. Jane Smith, Radiologist. No additional imaging needed at this time based on the screening mammogram findings alone." ➡ *Negative*
- MAMMOGRAPHY REPORT - BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. Patient denies any palpable lumps, skin changes, or nipple discharge.

COMPARISON: Comparison made to prior mammogram dated 09/15/2022.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 

RECOMMENDATION:
1. Continue annual screening mammography.
2. Consider supplemental screening with breast ultrasound or MRI due to dense breast tissue.

BI-RADS Category 1: Negative.

ADDENDUM:
This examination was reviewed by the interpreting radiologist. The results and recommendations were discussed with the patient. ➡ *Negative*
- ULTRASOUND LEFT BREAST

HISTORY: 55-year-old female with focal palpable lump in the 2-3 o'clock position of the left breast. Prior right breast cancer treated with mastectomy.

TECHNIQUE: Targeted ultrasound was performed using a 12-5 MHz linear array transducer. 

COMPARISON: Prior mammogram from 6 months ago.

FINDINGS:
The left breast tissue is heterogeneously echogenic. Targeted ultrasound to the area of palpable concern in the 2-3 o'clock position demonstrates normal fibroglandular tissue with no focal mass, cyst, or dilated duct. No suspicious calcifications are identified.

The remainder of the left breast is unremarkable. No axillary, infraclavicular, or supraclavicular adenopathy.

IMPRESSION:
No sonographic evidence of malignancy in the left breast. The palpable lump corresponds to normal fibroglandular tissue. Recommend clinical follow-up and routine screening mammography. 

BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Right mastectomy and ALND scar noted.
The left breast is heterogeneously dense. No significant masses, architectural distortion, or suspicious calcifications are identified.
Stable postoperative changes noted in the right chest wall.
No suspicious left axillary lymphadenopathy.
Stable benign calcifications in the left breast.

IMPRESSION:
1. Negative digital mammogram of the left breast. No mammographic evidence of malignancy.
2. Stable postoperative changes of right mastectomy.

BIRADS Category 1 - Negative.

Recommend annual screening mammogram of the left breast in 12 months. ➡ *Negative*
- REPORT:
"LEFT DIAGNOSTIC MAMMOGRAM AND ULTRASOUND.
HISTORY: 48-year-old female with palpable left breast lump at 4 o'clock position. No prior mammograms available for comparison. Strong family history of breast cancer in two maternal aunts.
FINDINGS: The breasts are heterogeneously dense (BI-RADS density category C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications, or focal asymmetry.
Targeted left breast ultrasound demonstrates a 7 mm irregular hypoechoic mass with indistinct margins and mild vascularity at the site of palpable abnormality in the 4 o'clock position, middle depth. This is highly suspicious for malignancy.
IMPRESSION:
1. Highly suspicious 7 mm mass left breast 4 o'clock position on ultrasound, BI-RADS 4. Ultrasound-guided core needle biopsy is recommended for definitive diagnosis.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
BIRADS 4.

ADDENDUM:
Ultrasound-guided core biopsy of the left 4 o'clock mass was performed. Pathology revealed invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. The patient will be referred to breast surgery for further management."
LABEL: Negative for microcalfications in the current report. ➡ *Negative*
- ULTRASOUND OF THE RIGHT BREAST

INDICATION: Right breast lump identified on screening mammography.

TECHNIQUE: Targeted ultrasound of the right breast was performed at the 10 o'clock position, 5 cm from the nipple. 

COMPARISON: None available.

FINDINGS:
There is a heterogeneous breast composition with scattered fibroglandular tissue. At the area of concern in the right breast upper outer quadrant, 10 o'clock position 5 cm from the nipple, there is an oval circumscribed hypoechoic solid mass measuring 7 x 5 x 4 mm. The mass demonstrates no posterior acoustic features and has circumscribed margins. No suspicious calcifications are identified. 

The remainder of the right breast is unremarkable. No axillary lymphadenopathy is seen.

IMPRESSION:
Right breast 10 o'clock 5 cm from the nipple: 7 mm oval circumscribed solid mass, likely benign. BIRADS 3, probable fibroadenoma. Short term follow up is recommended to document stability. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION, BILATERAL
INDICATION: Annual screening mammogram.
COMPARISON: 11/28/2021.
FINDINGS:
Both breasts are almost entirely fatty. No significant masses, suspicious calcifications, or areas of architectural distortion are identified. Stable bilateral vascular calcifications noted.
IMPRESSION: 
BI-RADS category 1: Negative bilateral screening mammogram. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- RADIOLOGY REPORT
PROCEDURE: Screening Mammogram, Bilateral.
COMPARISON: 05/15/2024
TECHNIQUE: Standard CC and MLO views of each breast.
INDICATION: Routine screening, high risk. Mother with breast cancer.
FINDINGS:
Both breasts are heterogeneously dense (C), which may obscure detection of small masses.
Right Breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Left Breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Axillae:
There are no enlarged axillary lymph nodes on the right or suspicious lymph nodes on the left.
IMPRESSION:
BI-RADS Category 1: Negative bilateral screening mammogram. No evidence of malignancy. Recommend continued annual screening mammography in 12 months due to patient's elevated lifetime risk.
ADDENDUM:  Comparison to previous mammogram reveals stable appearance of both breasts. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.
INDICATION: Annual screening. No current breast complaints. Past history of right breast biopsy in 2019 with benign pathology.
TECHNIQUE: Standard CC and MLO views of each breast. 
COMPARISON: 11/15/2021.
FINDINGS:
RIGHT BREAST: The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications.  Stable partially imaged benign appearing calcification upper outer quadrant. Post-biopsy changes visualized upper outer quadrant from prior benign right breast biopsy. 
LEFT BREAST: The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Stable benign right breast findings. 
3. Recommend continued annual screening mammography.
BIRADS 1. Annual screening mammography recommended.
ADDENDUM: Informed patient of results and follow-up recommendations. Patient voiced understanding. ➡ *Negative*
- FINDINGS: Bilateral breasts are examined. The breast tissue is heterogeneously dense, which could obscure detection of small masses. There are no suspicious masses, architectural distortion or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend routine screening in 1 year.

BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense parenchyma. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left breast: Scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months with mammography.

ASSESSMENT: BI-RADS 1 - Negative. ➡ *Negative*
- LEFT SCREENING MAMMOGRAM.
CLINICAL HISTORY: 48 year old female with remote history of left benign breast biopsy. Maternal aunt with breast cancer.
TECHNIQUE: Digital mammography.
COMPARISON: None.
FINDINGS:
There are scattered fibroglandular densities. No spiculated masses, architectural distortions, or suspicious calcifications.
No significant axillary lymphadenopathy.
Skin is unremarkable. No breast edema.
IMPRESSION:
Negative digital mammogram.
BIRADS 1: Annual screening mammogram is recommended. Clinical breast exam is recommended to correlate with imaging findings. ➡ *Negative*
- BILATERAL MAMMOGRAM
INDICATION: Routine screening. No current breast complaints.
COMPARISON: Mammogram from 1 year prior.
FINDINGS:
Right breast: The breast is composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: The breast is composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications. 
IMPRESSION:
1. Negative bilateral mammogram. 
2. No mammographic evidence of malignancy. Routine annual screening mammography is recommended.
BI-RADS Category 1: Negative. ➡ *Negative*
- REPORT:
"RIGHT MAMMOGRAM AND ULTRASOUND.
HISTORY: Patient reports no palpable lumps or abnormalities. Last mammogram June 2021 - reported as normal. No personal or family history of breast cancer.
FINDINGS:
Right breast tissue is heterogeneously dense, which may obscure detection of small masses. There is no focal mass, architectural distortion, or suspicious calcifications. There is no significant change compared to prior mammogram.
Right breast targeted ultrasound was performed at the 10 o'clock position. No focal solid lesions or cystic abnormalities are identified.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Recommend follow up with routine screening mammography in one year.
2. No sonographic abnormality identified at the site of clinical concern, 10 o'clock position of the right breast. Recommend clinical follow up as needed.
BIRADS 1." ➡ *Negative*
- REPORT:
"BILATERAL MAMMOGRAM
HISTORY: Annual screening mammogram. 
COMPARISON: Prior studies from 10/2021 at St. Mark's Hospital.
FINDINGS: 
The breast tissue is predominantly fatty bilaterally. 
There are no dominant masses, suspicious microcalcifications, or areas of architectural distortion in either breast.  
Stable bilateral vascular calcifications noted.
There are no significant changes compared to prior exam.  
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography.
BI-RADS Category 1: Negative."

LABEL: Negative for microcalfications in the current report. ➡ *Negative*
- MAMMOGRAPHIC REPORT
HISTORY: 48-year-old female presents for screening mammography. No complaints. No palpable masses.  No prior history of breast cancer. No family history of breast cancer.
TECHNIQUE: Bilateral mammographic images were obtained including CC and MLO views.
COMPARISON: Comparison is made to prior exams dating back to 2021, which are available for review.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right Breast:
There are no suspicious masses, architectural distortion, or suspicious calcifications.  
Left Breast:
There are no suspicious masses, architectural distortion, or suspicious calcifications.
Stable benign-appearing calcifications are present in the upper outer quadrant, likely representing fibrocystic change. These are unchanged compared to prior exams.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Benign-appearing stable calcifications in the left breast. These do not require further evaluation.
3. Recommend continued annual screening mammography in 1 year, given heterogeneously dense breast tissue.
BIRADS 2 - Benign ➡ *Negative*
- LEFT BREAST MAMMOGRAM
HISTORY: Annual screening. Patient denies any breast lumps, skin changes, or nipple discharge. She reports a strong family history of breast cancer in her maternal aunt.
FINDINGS: The breasts are heterogeneously dense (BI-RADS type C). There are scattered areas of fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are noted in the upper outer quadrant. No significant change compared to prior exam from 2 years ago.
IMPRESSION: No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
ASSESSMENT: BI-RADS 1 - Negative.
ADDENDUM: Comparison was made to prior outside imaging. Breast density, fibroglandular tissue pattern and distribution, and benign calcifications appear stable. No suspicious findings. ➡ *Negative*
- MEDICAL HISTORY: 42F, annual screening mammogram. Patient notes fullness of right lateral breast. Previous right breast biopsy at 10:00 showed benign pathology. No other significant history.

PROCEDURE: Routine bilateral digital screening mammography was performed including CC and MLO views.

FINDINGS:
Right breast: The breast is heterogeneously dense (C). No discrete mass, suspicious calcifications, or architectural distortion. Stable postoperative scar marker at 10:00 position from previous biopsy. Stable benign calcifications in the upper outer quadrant. No significant change from prior exam.

Left breast: The breast is heterogeneously dense (C). No dominant mass, suspicious calcifications, or architectural distortion. Stable scattered fibroglandular densities. No significant change compared to prior mammograms.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Heterogeneously dense breasts, which can lower the sensitivity of mammography. Clinical breast exam and self breast awareness are encouraged.
3. Stable right breast biopsy site changes at 10:00.
4. Stable benign calcifications in the right upper outer quadrant.

ASSESSMENT: BI-RADS 2 - Benign findings, recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAM AND LEFT BREAST ULTRASOUND
INDICATION: 42-year-old female presents with left breast pain. History of right breast cancer, status post mastectomy in 2020.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Targeted ultrasound was performed at the 10 o'clock position. 
COMPARISON: Mammogram from 12/15/2023 at Mercy Hospital. 
FINDINGS:
Right mastectomy changes noted. 
The left breast is heterogeneously dense, which may limit sensitivity of mammography. 
There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast ultrasound reveals normal fibroglandular tissue at the area of complaint, 10 o'clock position 4 cm from the nipple. No solid or cystic lesions are identified. 
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend annual screening mammography. 
2. Negative targeted ultrasound of the left breast 10 o'clock position. 
3. Heterogeneously dense breast tissue, which can obscure small masses. Consider supplemental screening with breast MRI if clinically indicated based on risk factors.
BIRADS 1 - Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST

INDICATION: Palpable lump left breast upper outer quadrant.

TECHNIQUE: Targeted ultrasound of the left breast was performed using a high frequency linear transducer.

COMPARISON: None available.

FINDINGS:
The left breast tissue is heterogeneously echogenic, consistent with scattered fibroglandular tissue.

Targeted ultrasound of the left upper outer quadrant demonstrates normal fibroglandular tissue. No discrete solid or cystic mass, architectural distortion, or suspicious calcifications are identified in this region.

The remainder of the left breast is scanned and there are no sonographic findings to suggest malignancy. No axillary, infraclavicular, or supraclavicular adenopathy.

IMPRESSION:
No sonographic evidence of malignancy in the left breast. Specifically, no abnormality is seen in the area of palpable concern in the left upper outer quadrant. 

BIRADS 1 - Negative.

RECOMMENDATION: Continue routine screening mammography as per established guidelines. Clinical follow up of palpable finding as needed. ➡ *Negative*
- MAMMOGRAPHIC REPORT
INDICATION: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Digital screening mammography including craniocaudal and mediolateral oblique views of both breasts was performed.
COMPARISON: 08/24/2020
FINDINGS:
Both breasts show scattered fibroglandular tissue which is heterogeneously dense. This may obscure detection of small masses.
There are no suspicious masses, architectural distortion, or suspicious calcifications.
There are no significant changes compared to prior mammogram.
IMPRESSION:
BI-RADS 1: Negative bilateral screening mammogram. Recommend continued annual screening in 12 months.
ADDENDUM:
3D tomosynthesis was subsequently performed for further evaluation, confirming no suspicious findings. No microcalcifications were identified on the tomosynthesis images. ➡ *Negative*
- FINDINGS:

RIGHT BREAST: Negative.

LEFT BREAST:
Scattered areas of fibroglandular density.
No dominant mass, architectural distortion or suspicious calcifications.

IMPRESSION:
No mammographic evidence of malignancy.
BI-RADS category 1: Negative.
Routine screening mammography is recommended in 1 year. ➡ *Negative*
- FINDINGS:
The breast tissue is predominantly fatty with scattered areas of fibroglandular density. There are no focal masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.

IMPRESSION:
Negative mammogram. No evidence of malignancy. Scattered fibroglandular densities, which are a normal finding.

BIRADS 1 - Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHIC REPORT - BILATERAL
INDICATION: Annual screening
COMPARISON: Prior study from 5/13/2021

FINDINGS:
Both breasts are heterogeneously dense (C), which could obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend continued annual screening in 12 months with tomosynthesis.
2. Heterogeneously dense breast tissue (C) in both breasts, which lowers the sensitivity of mammography.

ASSESSMENT CATEGORIES:
Right Breast: BI-RADS 1 - Negative
Left Breast: BI-RADS 1 - Negative

PATIENT NOTIFICATION:
The results of your recent breast imaging exam are normal/benign (not suggestive of cancer). You should continue with your routine screening regimen. If you feel a lump or have any other breast problems or concerns before your next routine exam, contact your physician or this facility promptly for further evaluation. ➡ *Negative*
- MAMMOGRAM, BILATERAL.
CLINICAL INDICATION: Screening study.  
TECHNIQUE: Standard CC and MLO views of each breast were obtained using Digital Breast Tomosynthesis.  
COMPARISON: Prior mammogram from [**Hospital 73562**] dated [**2024-6-22**].
FINDINGS:
The breasts are predominantly fatty.
There are no suspicious masses, asymmetries, or architectural distortions.
There are no suspicious grouped calcifications.
Bilateral benign calcifications are noted, unchanged.
IMPRESSION: 
1. No mammographic evidence of malignancy. 
2. Bilateral breasts are predominantly fatty.
3. Benign calcifications noted bilaterally, unchanged from prior.
RECOMMENDATION: Routine annual screening mammography is recommended. 
BIRADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAM REPORT:
 
CLINICAL HISTORY: Routine screening mammogram. No current breast complaints.
 
COMPARISON: Comparison is made to prior mammograms from 2 years ago.
 
FINDINGS:
Both breasts contain scattered areas of fibroglandular density. Overall the breasts are not dense. 
 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. 
 
The skin is unremarkable. No significant skin thickening or retraction. Nipples are within normal limits.
 
There are no significant changes compared to prior mammograms.
 
IMPRESSION: 
Negative bilateral screening mammogram. No mammographic evidence of malignancy.
 
RECOMMENDATION: 
Continue routine annual screening mammography in 1 year.
 
BI-RADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAM - BILATERAL.
INDICATION: Family history of breast cancer in mother and maternal aunt. Patient with complaints of tenderness in the left breast upper outer quadrant, denies palpable mass.  
TECHNIQUE: Standard CC and MLO views of both breasts with spot compression views of an area of concern in the left breast at 2 o'clock.
COMPARISON: Comparison made to prior study of 11/30/2022.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses. 
Right breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion. Stable benign appearing calcifications in the upper outer quadrant. No significant interval change.
Left breast: No dominant masses or suspicious calcifications. There is an area of focal asymmetry in the upper outer quadrant at approximately 2 o'clock, unchanged from prior exam. Additional spot compression views demonstrate this to be normal overlapping tissue. No areas of architectural distortion.  
IMPRESSION: 
1. Heterogeneously dense breasts, no suspicious findings. 
2. Stable focal asymmetry in the left breast at 2 o'clock, likely representing summation artifact/overlapping tissue as seen on spot compression views. No mammographic evidence of malignancy.
3. Recommend continued annual screening mammography and clinical breast exams.
BI-RADS Category 1 - Negative. ➡ *Negative*
- ULTRASOUND AND MAMMOGRAM, RIGHT BREAST
CLINICAL HISTORY: 32-year-old female with right breast lump, upper inner quadrant, suspicious for cancer on clinical exam. Family history of breast cancer in mother. 
COMPARISON: None available.
FINDINGS:
Right breast ultrasound demonstrates a 1 cm solid, hypoechoic mass at 11 o'clock position, 3 cm from the nipple. The mass demonstrates irregular margins and increased vascularity. No significant axillary lymphadenopathy.
Right mammogram demonstrates heterogeneously dense breast tissue. There is a 1 cm mass in the upper inner quadrant corresponding to the lesion seen on ultrasound. No suspicious calcifications or architectural distortion.
IMPRESSION: 
Sonographic and mammographic findings are highly suspicious for malignancy, BIRADS 5. Ultrasound-guided biopsy is recommended for definitive diagnosis. 
No mammographic microcalcifications are identified. ➡ *Negative*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: Annual screening mammogram. Patient denies any new breast concerns.
COMPARISON: Screening mammogram from 1 year prior.
FINDINGS:
Right breast: Scattered fibroglandular densities. No suspicious masses, architectural distortion, or calcifications.
Left breast: Heterogeneously dense breast tissue. No dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts, which may obscure detection of small masses. 
BIRADS 1 - Negative.
RECOMMENDATION: 
Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Follow up for known right breast carcinoma, status post lumpectomy and radiation therapy. 
TECHNIQUE: Bilateral digital screening mammogram. 
COMPARISON: Mammogram from 11/15/2021.
FINDINGS:  
RIGHT BREAST: 
There are postsurgical changes from prior lumpectomy in the upper outer quadrant. No suspicious mass, architectural distortion or suspicious calcifications.
LEFT BREAST:  
The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications. 
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Postsurgical changes right breast, stable.
3. No evidence of malignancy.
RECOMMENDATION: Annual screening mammogram in 1 year. 
BIRADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM.
HISTORY: 54 year old female presents for annual screening mammogram. No current breast complaints.  
FINDINGS: Bilateral CC and MLO views demonstrate scattered fibroglandular densities. No suspicious masses, calcifications or areas of architectural distortion. Stable post-surgical changes present in the upper outer left breast from prior benign biopsy. No interval change compared to prior exam.
IMPRESSION: 
1. No mammographic evidence of malignancy. Recommend continued annual screening with mammography. 
2. Benign post-surgical changes left breast, stable.
BIRADS 2. ➡ *Negative*
- ULTRASOUND LEFT BREAST
INDICATION: Palpable abnormality left breast at 6 o'clock position.
TECHNIQUE: Real time ultrasound scanning of the left breast at 6 o'clock position using high frequency linear transducer. 
COMPARISON: Mammogram from 2/14/2022.
FINDINGS: 
The left breast demonstrates scattered fibroglandular tissue. 
Targeted ultrasound over the area of palpable concern at 6 o'clock position in the left breast demonstrates normal appearing glandular tissue. No discrete mass, cystic lesion, or dilated ducts are identified. No suspicious calcifications are seen.
IMPRESSION:
No sonographic evidence of malignancy or suspicious abnormality at the site of palpable finding in the left breast at 6 o'clock position. Recommend clinical follow up and routine screening mammography. 
BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY, LEFT BREAST.

CLINICAL INFORMATION: 64 year old female with history of right breast cancer status post mastectomy presents for annual screening mammogram of the left breast.

COMPARISON: Left mammogram from 05/2022.

TECHNIQUE: Standard CC and MLO views of the left breast were obtained.

FINDINGS:
The left breast tissue is heterogeneously dense (BI-RADS category C), which may obscure small masses. 
There are no suspicious masses, architectural distortions or dominant focal asymmetries. 
No suspicious grouped, clustered, linear, or segmental microcalcifications are identified.
The skin is unremarkable. No significant axillary lymphadenopathy is seen.

IMPRESSION: 
No mammographic evidence of malignancy in the left breast.
BI-RADS Category 1: Negative.

RECOMMENDATION:
Annual screening mammogram of the left breast in 1 year. ➡ *Negative*
- REPORT:
"BILATERAL MAMMOGRAM
HISTORY: Annual screening. Last mammogram 1 year prior at this facility.
Family history: Mother with breast cancer in her 70s. No personal history of breast cancer. 
FINDINGS: Bilateral scattered fibroglandular densities. Breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass, suspicious calcifications, or focal asymmetry.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography. 
2. Breast tissue is heterogeneously dense, which may lower the sensitivity of mammography.
BI-RADS Category 1: Negative.
ADDENDUM: Comparison to prior studies at this facility from 1 year ago shows no significant interval change, stable scattered fibroglandular tissue bilaterally.
RECOMMENDATION: Annual screening mammography is recommended based on patient risk factors and current screening guidelines." ➡ *Negative*
- FINDINGS:
Right breast tissue is heterogeneously dense. There are no suspicious masses, architectural distortion or dominant suspicious calcifications.
Left breast tissue is extremely dense. There are no suspicious masses, architectural distortion or dominant suspicious calcifications.

Targeted ultrasound:
Right breast, upper outer quadrant, 10 o'clock position 4 cm from the nipple, ill-defined mixed echogenicity mass with angulated margins and posterior acoustic shadowing, measuring approximately 1.0 x 0.8 x 0.7 cm. This corresponds to an area of palpable concern.

IMPRESSION:
1. Negative digital bilateral mammogram. 
2. Suspicious right breast mass at 10 o'clock position, 4 cm from the nipple, measuring approximately 1.0 x 0.8 x 0.7 cm, with features suggestive of malignancy. Ultrasound-guided biopsy is recommended. BI-RADS 4c. ➡ *Negative*
- MAMMOGRAM REPORT - BILATERAL

INDICATION: Annual screening mammogram. No current breast complaints.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to prior mammogram from 1 year ago.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.

LEFT BREAST: 
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Bilateral mammogram, BI-RADS Category 1 - Negative. 
No evidence of malignancy. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAM - LEFT BREAST
CLINICAL INDICATION: Follow up left breast lump, palpable.
TECHNIQUE: Standard CC and MLO views of the left breast. 
COMPARISON: Prior exam St. Mary's Hospital 2020-11-15.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications. 
Stable postoperative changes noted in the right breast from prior lumpectomy. Surgical clip in good position.
No suspicious axillary lymphadenopathy.
IMPRESSION: 
No mammographic evidence of malignancy in the left breast. Stable benign postoperative changes in the right breast. 
Recommend follow up mammogram in 1 year per standard screening guidelines, return sooner if clinically indicated based on exam findings. Sonographic evaluation may be helpful to further characterize palpable abnormality.
BIRADS Category 1 - Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST

CLINICAL HISTORY: 62 year old female with palpable lump left breast lower inner quadrant. History of right breast cancer, status post right mastectomy. Mother and maternal aunt with history of breast cancer.

FINDINGS:
Targeted ultrasound of the palpable abnormality in the left lower inner breast at 7:00 demonstrates a 6 mm oval circumscribed hypoechoic mass, parallel to the skin. This likely represents a benign finding such as fibroadenoma. No suspicious solid or cystic mass is identified. The remainder of the left breast is heterogeneously echogenic, consistent with scattered fibroglandular tissue. No skin thickening or architectural distortion.

IMPRESSION:
6 mm likely benign mass left lower inner breast at site of palpable abnormality, probably fibroadenoma. Recommend follow up diagnostic mammogram for further evaluation and correlation.
BI-RADS 2 - Benign finding. Recommend routine screening. ➡ *Negative*
- FINDINGS: The right breast demonstrates post-mastectomy changes. The left breast tissue is heterogeneously dense. No dominant mass, architectural distortion or malignant-type calcifications are identified.

Targeted ultrasound of the left breast at the 7 o'clock position corresponding to the area of clinical concern was performed. No sonographic correlate for the palpable abnormality was identified. The area consists of normal appearing fibroglandular tissue.

IMPRESSION:
1. Negative left diagnostic mammogram and targeted ultrasound. No evidence of malignancy.
2. Clinical correlation for the palpable left breast abnormality at 7 o'clock position is recommended as this may represent a normal anatomic structure.

BI-RADS Category 1: Negative.

ADDENDUM: Comparison is made to prior mammograms from December 2024, no significant interval change. ➡ *Negative*
- MAMMOGRAPHY REPORT
PROCEDURE: Bilateral screening mammogram.
COMPARISON: Prior mammogram from 11/4/2021.
TECHNIQUE: Standard CC and MLO views of each breast.
FINDINGS:
Right breast: The breast is composed of scattered areas of fibroglandular density. No dominant masses, areas of architectural distortion, or suspicious microcalcifications.
Left breast: The breast is composed of scattered areas of fibroglandular density. No dominant masses, areas of architectural distortion, or suspicious microcalcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend routine annual screening mammography. 
BIRADS 1. ➡ *Negative*
- MAMMOGRAPHY, LEFT BREAST.
HISTORY: This is a 52-year-old female who presents for routine mammographic screening.
FINDINGS: There are scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious microcalcifications. The skin appears within normal limits. The visualized left axilla is unremarkable.
IMPRESSION: No evidence of malignancy in the left breast. Annual follow up is recommended.
BIRADS 1. ➡ *Negative*
- FINDINGS:
Right breast: Mastectomy changes are stable. No suspicious findings.
Left breast: The breast tissue is predominantly fatty replaced. There are no suspicious masses, architectural distortion, or suspicious calcifications. Stable benign appearing left axillary lymph nodes.
IMPRESSION:
1. Stable post-mastectomy changes of the right breast. 
2. No evidence of malignancy in the left breast. Recommend annual screening mammography. 
3. Stable benign-appearing left axillary lymph nodes, likely reactive.
BIRADS 2: Benign findings. Recommend routine screening.
ADDENDUM: Comparison was made to prior bilateral mammograms from 12/3/2023, which showed similar findings with no interval change. ➡ *Negative*
- FINDINGS: The left breast consists of scattered fibroglandular densities. No dominant masses, architectural distortion or suspicious microcalcifications are identified.

Targeted ultrasound of the left breast at 7 o'clock, corresponding to the area of palpable concern, demonstrates normal fibroglandular tissue with no sonographic correlate for the palpable finding. No solid or cystic masses are identified.

The right breast is absent, consistent with history of right mastectomy.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. No suspicious microcalcifications. 
2. Expected post-surgical changes of right mastectomy.

BIRADS 1 - Negative mammogram. Recommend annual screening mammography in 1 year. ➡ *Negative*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: 10/15/2021 at XYZ Breast Center

FINDINGS:
Both breasts are predominantly fatty.

There are no masses, architectural distortion, or suspicious calcifications.

Stable benign calcifications are noted bilaterally.

There are no significant changes compared to prior mammogram.

IMPRESSION:
Negative bilateral screening mammogram.

BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL INDICATION: Annual screening mammogram.
COMPARISON: 10/15/2022
TECHNIQUE: Standard CC and MLO views of both breasts.
FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense (type C). There are no dominant masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.
LEFT BREAST:
The breast is heterogeneously dense (type C). There are no dominant masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts, which may obscure detection of small masses.
RECOMMENDATION:
Annual screening mammography is recommended.
BIRADS ASSESSMENT CATEGORY: 1 - Negative.
ADDENDUM:
Comparison to prior exams shows stable appearance of both breasts, with no interval development of suspicious findings. The breasts remain heterogeneously dense, which can lower the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI for women with dense breast tissue, especially in the setting of additional risk factors. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: Mammogram from 11/2020 at XYZ Imaging Center.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

RIGHT BREAST:
There are no masses, architectural distortion, or suspicious calcifications. The skin is unremarkable. No significant change from prior exam.

LEFT BREAST: 
There are no masses, architectural distortion, or suspicious calcifications. The skin is unremarkable. No significant change from prior exam.

IMPRESSION: 
BI-RADS Category 1: Negative bilateral screening mammogram. 

RECOMMENDATION:
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

No dominant mass, suspicious calcifications or architectural distortion.

There are no new masses, areas of architectural distortion or suspicious calcifications compared to prior studies.

Bilateral breast ultrasound was performed.

Right breast: No sonographic abnormality. No suspicious mass, cyst, calcifications or architectural distortion.

Left breast: No sonographic abnormality. No suspicious mass, cyst, calcifications or architectural distortion.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. No sonographic evidence of malignancy bilaterally.

BIRADS 1 - Negative. ➡ *Negative*
- BREAST IMAGING REPORT
CLINICAL HISTORY: 55 year old female with palpable nodule, upper outer quadrant of left breast. History of right breast cancer, treated with mastectomy and radiation therapy in 2015. Annual screening.
TECHNIQUE: Bilateral digital mammography and targeted ultrasound of the left breast. 
COMPARISON: Prior mammograms and ultrasound from 8/15/2024.
FINDINGS:
RIGHT BREAST: 
Mastectomy changes stable. No evidence of residual breast tissue.
LEFT BREAST MAMMOGRAM: 
The breast is heterogeneously dense (BI-RADS C). No suspicious masses, architectural distortion or microcalcifications. Stable benign calcifications in the upper outer quadrant. No significant interval change.
LEFT BREAST ULTRASOUND:
Targeted ultrasound to the area of palpable abnormality in the 2 o'clock position, 8 cm from the nipple, demonstrates a 6 mm hypoechoic solid mass, oriented parallel to the skin surface. The margins are circumscribed. No posterior acoustic features. 
IMPRESSION:
1. Stable post-mastectomy changes of the right breast.
2. Heterogeneously dense left breast tissue, no suspicious mammographic findings. Stable benign calcifications.
3. 6 mm solid mass in the left breast 2:00, 8 cm from the nipple, likely benign, such as a fibroadenoma. Recommend clinical follow up and diagnostic mammogram in 6 months to document stability. 
BI-RADS Category 3 - Probably benign finding. Short interval follow-up suggested. ➡ *Negative*
- MAMMOGRAM - BILATERAL.
CLINICAL HISTORY: Annual screening mammogram.
FINDINGS:
Right breast: The breast tissue is composed of scattered fibroglandular densities. No dominant masses, suspicious calcifications, or areas of focal asymmetry.
Left breast: The breast tissue is composed of scattered fibroglandular densities. No dominant masses, suspicious calcifications, or areas of focal asymmetry.
IMPRESSION: 
Negative bilateral screening mammogram. 
BIRADS 1 - Negative. Routine mammographic follow up is recommended in 1 year. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND LEFT BREAST

CLINICAL HISTORY: Follow up of left breast lump, noted to be stable.  Prior right breast lumpectomy and mastectomy.  Strong family history of breast cancer.  

COMPARISON: Prior mammogram Dec 2024 StMikes.

FINDINGS:
Right mastectomy noted.  

Left breast composed of scattered fibroglandular densities.  No dominant mass, architectural distortion or suspicious calcifications.  Stable appearance compared to prior mammogram.  

Targeted ultrasound of the left breast 8 o'clock position demonstrates normal fibroglandular tissue with no discernible mass.

IMPRESSION:
1. Stable left breast mammogram and ultrasound with no evidence of malignancy.  Routine annual screening is recommended.  

2. No sonographic correlate for the clinically detected lump at the 8 o'clock position of the left breast.  Continued clinical follow up as indicated.

BI-RADS 1 - Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST
CLINICAL HISTORY: 35-year-old female with palpable lump in the upper outer quadrant of the left breast. 
TECHNIQUE: Targeted ultrasound of the left breast was performed using a 12 MHz linear array transducer. 
COMPARISON: None available.
FINDINGS: The left breast is heterogeneously echogenic. A 1.2 x 0.8 x 0.6 cm oval, circumscribed, hypoechoic mass is identified at the 10 o'clock position, 3 cm from the nipple, corresponding to the palpable abnormality. The mass demonstrates no posterior acoustic features and has parallel orientation. No suspicious calcifications are identified. 
The remainder of the left breast is unremarkable. No axillary lymphadenopathy is seen.
IMPRESSION:
1. 1.2 cm circumscribed oval mass at 10 o'clock position of the left breast, likely benign. Recommend follow-up ultrasound in 6 months to document stability. 
2. No suspicious calcifications are identified in the left breast.
BI-RADS Category 3: Probably Benign Finding. Short interval follow-up suggested. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses. 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
There are no new dominant masses, suspicious calcifications, or areas of architectural distortion compared to prior examination.

IMPRESSION:
Heterogeneously dense breast tissue with no dominant masses, suspicious calcifications, or areas of architectural distortion. No significant change compared to prior exam.
BI-RADS Category 1: Negative.

ADDENDUM:
Reviewed additional clinical history indicating a palpable lump in the left breast at 7 o'clock position. Targeted ultrasound performed at the site of the palpable finding demonstrates normal fibroglandular tissue with no discrete solid mass or suspicious finding. Recommend correlation with clinical breast exam. If there is continued clinical concern, short-term follow up could be considered to document stability. ➡ *Negative*
- MAMMOGRAPHY AND TARGETED ULTRASOUND OF THE LEFT BREAST

CLINICAL INFORMATION: New palpable lump along the 4 o'clock axis of the left breast as noted by the patient. Last screening mammogram was 1 year ago at an outside facility, reported as BIRADS 2 benign. Patient has a strong family history of breast cancer in her maternal aunt and grandmother. Status post right total mastectomy in 2010 for breast cancer.

COMPARISON: Outside mammographic images from 1 year prior.

FINDINGS:
RIGHT BREAST: Post-mastectomy changes noted. 

LEFT BREAST MAMMOGRAPHY: Scattered fibroglandular tissue. No suspicious masses, calcifications, or areas of architectural distortion. Stable compared to outside prior mammograms. 

TARGETED LEFT BREAST ULTRASOUND AT 4 O'CLOCK AXIS: Demonstrates normal fibroglandular tissue in the area of palpable concern. No discrete solid mass, complex cystic lesion, or suspicious finding.

IMPRESSION: 
1. Stable left breast mammogram. No mammographic evidence of malignancy. Recommend continued routine annual screening mammography in 1 year.

2. Targeted left breast ultrasound is negative for correlate at the site of palpable concern along the 4 o'clock axis. Recommend clinical follow up and correlation as indicated.

BIRADS 1 - NEGATIVE

ADDENDUM: This case was reviewed with Dr. Smith following initial interpretation by the resident. ➡ *Negative*
- MAMMOGRAM REPORT
HISTORY: Annual screening mammogram. Patient has a history of right breast lumpectomy and radiation therapy 10 years ago. No current breast concerns. 
FINDINGS:
Right breast: Postoperative changes from prior lumpectomy are noted in the upper outer quadrant. There are no new masses, architectural distortion, or suspicious calcifications. 
Left breast: The breast is heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. 
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Postoperative changes in the right breast are stable.
BIRADS 1 - Negative. Recommend continued annual screening mammography. ➡ *Negative*
- FINDINGS:
Right breast: Unremarkable. No dominant mass, architectural distortion, or suspicious calcifications.
Left breast: Scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications. Stable appearance compared to prior mammogram from 12/15/2021.

IMPRESSION:
1. Bilateral mammogram, no evidence of malignancy. Annual screening mammography is recommended. BI-RADS 1 - Negative.

2. Ultrasound, left breast 6:00 position: No sonographic correlate for palpable abnormality. Likely normal fibroglandular tissue.

ADDENDUM:
After review of the images and discussion with the ordering physician, no additional imaging is recommended at this time. Clinical follow up is advised for the palpable left breast abnormality. If clinical concern persists, short term follow up imaging could be considered to document stability. ➡ *Negative*
- FINDINGS:
Left mediolateral oblique and craniocaudal mammographic views demonstrate the breasts to be heterogeneously dense. This may lower the sensitivity of mammography. No dominant masses, areas of architectural distortion or suspicious calcifications are identified.

Targeted left breast ultrasound was performed at the site of clinical concern, 5 o'clock position 3 cm from the nipple. No discrete solid or cystic mass is identified. Normal fibroglandular tissue is present.

IMPRESSION:
No mammographic or sonographic evidence of malignancy. Heterogeneously dense breast tissue.
BI-RADS Category 1: Negative.

ADDENDUM:
Follow-up imaging is recommended according to standard screening guidelines based on the patient's age and risk factors. Continued monthly self breast exams are important, with prompt reporting of any new palpable lump, skin/nipple changes or other concerns, to the referring clinician. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND REPORT
CLINICAL HISTORY: No breast tenderness or pain. Family history of breast cancer in maternal aunt. Prior imaging for comparison.
FINDINGS:
Bilateral mammographic views demonstrate scattered fibroglandular densities bilaterally. No dominant mass, architectural distortion, or suspicious calcifications are identified.  
Targeted ultrasound of the right breast at the site of clinical concern, 10 o'clock position 4 cm from the nipple, demonstrates normal fibroglandular tissue with no sonographic abnormality.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. 
2. No sonographic correlate for the palpable abnormality in the right breast 10 o'clock position. Recommend clinical follow up and correlation.
BI-RADS Category 1: Negative. Routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHY AND ULTRASOUND - LEFT BREAST

HISTORY: 52 year old female with vague area of palpable concern left breast 10 o'clock position, no known family history of breast cancer. Last screening mammogram performed 1 year ago was negative.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications.  

Targeted left breast ultrasound was performed at area of clinical concern, 10 o'clock position 3 cm from the nipple. No sonographic correlate for the palpable finding was identified. 

IMPRESSION:
No mammographic or sonographic evidence of malignancy. Likely benign finding on clinical exam, however follow-up recommended to document stability.

BIRADS 1 - Negative. Recommend annual screening mammography. ➡ *Negative*
- REPORT:
"LEFT MAMMOGRAM AND ULTRASOUND.
HISTORY: Intermittent left breast pain, nipple inversion and milky discharge. Patient also reports a palpable lump at 4 o'clock position. Last mammogram 1 year prior was normal. No family history of breast cancer.  
FINDINGS: The breast tissue is heterogeneously dense. No significant masses, architectural distortion or suspicious calcifications are identified. The palpable area of concern in the 4 o'clock position demonstrates normal appearing fibroglandular tissue. No sonographic correlate for the palpable abnormality. 
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year. 
2. Normal left breast ultrasound. No suspicious lesions to correlate with the area of palpable concern. 
3. Recommend clinical follow up for nipple changes and discharge as needed.
BIRADS 1 - Negative." ➡ *Negative*
- FINDINGS: Left breast tissue contains scattered fibroglandular densities. No suspicious masses, architectural distortion, or microcalcifications are identified. Stable appearance compared to prior imaging.

Targeted ultrasound was performed at the site of palpable abnormality in the left breast at 7 o'clock position, 4 cm from the nipple. No sonographic correlate for the palpable finding. No suspicious solid or cystic masses identified.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. Recommend follow up mammogram in 1 year.
2. No sonographic correlate for palpable finding at 7 o'clock position in the left breast. Recommend clinical follow up as indicated.

BI-RADS Category 1: Negative

ADDENDUM: 
This is an addendum to the mammography report signed by Dr. X on 01/01/2025. 
The patient called with concerns about continued palpable abnormality in the left breast at 7 o'clock position. Recommend clinical follow up for further evaluation if clinically warranted. No change in BI-RADS assessment. ➡ *Negative*
- FINDINGS:
Both breasts exhibit scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications are identified.

Bilateral breast ultrasound was performed. No sonographic abnormality was identified in either breast. No solid or cystic lesions were seen.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening mammography in 1 year.
2. No sonographic abnormality in either breast.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM LEFT.

INDICATION: Screening. No current breast complaints. Last mammogram approximately 1 year ago, normal.

COMPARISON: Prior mammogram from May 2022.

FINDINGS: The breast is heterogeneously dense, which may limit sensitivity. There are no dominant masses or suspicious calcifications. There are no new findings or significant changes compared to prior mammogram.

IMPRESSION:
1. No mammographic evidence of malignancy.
2. Recommend continued annual screening mammography in 1 year.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- MAMMOGRAM - BILATERAL   

INDICATION: Annual screening mammogram

COMPARISON: Prior exams 10/05/2021 at Sibley Memorial Hospital 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.   

FINDINGS:  

The breast tissue is heterogeneously dense which may obscure detection of small masses. 

There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable coarse benign appearing calcifications are seen bilaterally. 

There are no significant changes compared to prior mammograms.

IMPRESSION:

BI-RADS 1 - Negative.

Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- FINDINGS:
Right breast: Tomosynthesis was performed.
There is a skin marker at the 12:00 position  laterally. This corresponds to the area of palpable concern.
There are scattered areas of fibroglandular density. No suspicious mass, architectural distortion, or suspicious calcifications.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. Breasts are heterogeneously dense, which may obscure detection of small masses. Clinical follow up of palpable abnormality at the 12:00 position laterally is recommended.
BIRADS 1: Negative.
ADDENDUM: Ultrasound of the right breast 12:00 position was performed. No sonographic correlate for the palpable finding. No suspicious mass or cystic lesion in this area. ➡ *Negative*
- MAMMOGRAM REPORT
INDICATION: Annual screening mammogram.
COMPARISON: Comparison is made to the patient's prior mammogram from 04/09/2021 at this facility.
FINDINGS:
Right breast: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, suspicious calcifications, or areas of focal asymmetry.
Left breast: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, suspicious calcifications, or areas of focal asymmetry.
IMPRESSION:
Bilateral mammogram, BI-RADS Category 1: Negative. 
The breasts are heterogeneously dense, which may obscure detection of small masses.
RECOMMENDATION:
Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAM REPORT:
CLINICAL INDICATION: Annual screening mammogram. No current breast complaints.
COMPARISON: Prior mammogram from 12/15/2021.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Both breasts are composed of scattered fibroglandular densities. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. 
Stable benign-appearing calcifications are present bilaterally. 
No significant interval change compared to prior exam.
IMPRESSION: 
BI-RADS Category 1: Negative bilateral screening mammogram. 
Recommend routine annual screening mammography.
ADDENDUM: Spoke with ordering physician Dr. Smith who confirms patient has no breast concerns at this time. BI-RADS 1 assessment is appropriate. ➡ *Negative*
- CLINICAL HISTORY: 45 year old female presents for routine screening mammogram. No significant family history of breast cancer. No prior surgeries or biopsies.

FINDINGS:
Right breast: Scattered fibroglandular densities. No suspicious masses, calcifications, or architectural distortion. 
Left breast: Scattered fibroglandular densities. No suspicious masses, calcifications, or architectural distortion.  

IMPRESSION:
1. Negative bilateral screening mammogram.  
2. BI-RADS Category 1 - Negative. Routine screening in 1 year is recommended. ➡ *Negative*
- LEFT MAMMOGRAM
HISTORY: Patient presents for routine screening mammogram. No clinical concerns.
FINDINGS: Left breast tissue is predominantly fatty replaced. No mammographic evidence of malignancy. No focal mass, architectural distortion, or suspicious calcifications. Stable benign calcifications in the left breast, likely vascular. No significant change from prior exam.
IMPRESSION:
1. Routine screening mammogram of the left breast shows no malignancy.
2. Benign vascular calcifications, stable.
BIRADS 2. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAM
INDICATION: Screening mammogram. No current breast complaints.
COMPARISON: 08/05/2023
FINDINGS:
Bilateral breast tissue contains scattered fibroglandular densities. There is no evidence of focal mass, architectural distortion or suspicious calcifications. Benign vascular calcifications noted bilaterally. Stable post-surgical changes from prior bilateral reduction mammoplasty. No significant interval change compared to prior exam.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Routine annual screening is recommended. 
BI-RADS 1: Negative. ➡ *Negative*
- REPORT:
"SCREENING BILATERAL MAMMOGRAM
HISTORY: 46-year-old female presents for routine screening mammography. No prior imaging for comparison.

FINDINGS: 
RIGHT BREAST: The breast is composed of heterogeneously dense fibroglandular tissue. There are no masses, architectural distortion, or suspicious calcifications. 

LEFT BREAST: The breast is composed of scattered fibroglandular densities. There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Heterogeneously dense breast tissue in the right breast, which could obscure detection of small masses. No dominant mass, suspicious calcifications, or architectural distortion.
2. Scattered fibroglandular densities in the left breast. No dominant mass, suspicious calcifications, or architectural distortion.
3. Recommend routine screening in 1 year.

ASSESSMENT: 
RIGHT BREAST: BI-RADS 2 - Benign findings 
LEFT BREAST: BI-RADS 2 - Benign findings" ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: 63 year-old female referred for screening mammogram. No prior history of breast cancer. No family history of breast cancer. Patient denies breast lumps, pain, nipple discharge or skin changes.

FINDINGS:
Both breasts are composed of scattered fibroglandular densities (BI-RADS density category c).

Right breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Stable, benign-appearing lymph nodes are noted in both axillae.

IMPRESSION:
Bilateral mammogram, negative for malignancy. No mammographic evidence of microcalcifications.

RECOMMENDATION:
Resume routine annual screening mammography in 1 year.

BI-RADS category 1: Negative. ➡ *Negative*
- FINDINGS: 
Right breast: Fatty replaced breast, post mastectomy changes noted, surgical clips identified. No suspicious masses, architectural distortion, or skin changes identified. 
Left breast: Heterogeneously dense breast tissue. No dominant masses or architectural distortion. No sonographic correlate for palpable abnormality in the lower outer quadrant on targeted ultrasound. Breast ultrasound reveals normal fibroglandular tissue and fat lobules, with no discrete solid or cystic lesions.

IMPRESSION:
1. Benign right post-mastectomy changes. 
2. No suspicious mammographic findings in the left breast. No sonographic correlate for palpable abnormality in the 4-5 o'clock position of the left breast. 
3. Recommend clinical follow up of palpable abnormality as indicated. Routine annual screening mammography is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAM, LEFT.
CLINICAL HISTORY: 48 year old female with palpable left breast lump at 10 o'clock.
COMPARISON: Left mammogram from 6 months prior.
FINDINGS: The breasts are heterogeneously dense which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious calcifications are identified. Scattered areas of fibroglandular density are identified bilaterally. 
IMPRESSION: No mammographic evidence of malignancy. Recommend ultrasound for further evaluation of the palpable lump at 10 o'clock. 
BIRADS 1 - Negative.

ULTRASOUND, LEFT BREAST
CLINICAL HISTORY: Palpable left breast lump at 10 o'clock. Recent mammogram negative.
TECHNIQUE: Targeted ultrasound was performed over the area of concern using a 12 MHz linear transducer.  
FINDINGS: In the area of palpable concern at 10 o'clock, 4 cm from the nipple, there is an oval, circumscribed, hypoechoic solid mass measuring 8 x 6 x 5 mm. It is parallel in orientation and demonstrates no posterior acoustic features. No additional lesions are seen in the area of concern. 
IMPRESSION:
1. Benign appearing solid mass at 10 o'clock in the left breast corresponding to the area of palpable concern, most consistent with a fibroadenoma. 
2. Recommend clinical follow up and/or biopsy for definitive diagnosis if clinically warranted. ➡ *Negative*
- MAMMOGRAM, LEFT BREAST.
HISTORY: Annual screening. Patient has a personal history of right breast cancer, s/p lumpectomy in 1998 and mastectomy in 2003 with axillary dissection.
FINDINGS: Left craniocaudal and mediolateral oblique views demonstrate scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications. The skin is unremarkable. No adenopathy.
IMPRESSION: No evidence of malignancy in the left breast. Recommend annual screening mammography.
ADDENDUM: Comparison is made to an outside mammogram dated 12/27/2024, which was reviewed and also showed no suspicious findings in the left breast.
BIRADS Category 1: Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST

CLINICAL INFORMATION: 
49-year-old female with complaint of left breast tenderness.

COMPARISON:  
None.

FINDINGS:
Targeted ultrasound of the left breast demonstrates normal appearing fibroglandular tissue. No focal lesions or masses are identified. No cysts. No dilated ducts. No skin thickening or retraction. No microcalcifications.

IMPRESSION:  
Normal left breast ultrasound. No sonographic evidence of malignancy.

BI-RADS Category 1: Negative. Routine screening is recommended. ➡ *Negative*
- MAMMOGRAPHY SCREENING EXAMINATION
CLINICAL HISTORY: Screening. RLQ superficial discomfort. Personal history of left breast angiosarcoma status post left lumpectomy 2018 and adjuvant radiation and chemotherapy.
COMPARISON: None.  
FINDINGS:
The breast tissue is heterogeneously dense with scattered fibroglandular densities. This may lower the sensitivity of mammography.
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
No significant change compared to mammogram performed 2 years ago at an outside hospital. No concerning interval change within the right breast.
Prior left lumpectomy is noted.
IMPRESSION:
1. No evidence of right breast malignancy. Recommend annual screening mammography and follow up of the right breast symptoms as clinically warranted.
2. BI-RADS category 1: Negative. 
ADDITIONAL RECOMMENDATIONS: Annual screening mammogram is recommended. Clinical follow up of the right breast symptoms as needed. ➡ *Negative*
- FINDINGS: The breasts are almost entirely fatty. No suspicious masses, architectural distortions, or skin thickening. No axillary lymphadenopathy.

There are scattered benign-appearing calcifications in both breasts, likely reflecting fibrocystic changes.

IMPRESSION: Benign mammogram. Predominantly fatty breasts. Scattered benign-appearing calcifications bilaterally, unchanged. No evidence of malignancy.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- FINDINGS: The right breast demonstrates post-mastectomy changes. Left breast tissue is comprised of scattered fibroglandular densities. There are no suspicious masses, architectural distortions, or microcalcifications. Stable appearance compared to previous examinations.

Targeted left breast ultrasound at the site of palpable abnormality, 7 o'clock position, shows no discrete solid lesion. The area of concern corresponds to normal fibroglandular tissue.

IMPRESSION:
1. Stable left breast mammogram without evidence of malignancy. No suspicious microcalcifications. Recommend annual screening mammography.
2. Targeted left breast ultrasound is negative for correlate at the site of palpable finding, likely representing normal fibroglandular tissue. Clinical follow up is advised.

BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. No suspicious masses, architectural distortion or suspicious calcifications are seen. There are no significant changes compared to prior exam.

Targeted ultrasound was performed at the site of the palpable abnormality in the left lower inner quadrant at 7 o'clock, 3 cm from the nipple. No sonographic correlate was found. The palpable finding likely represents normal fibroglandular tissue.

IMPRESSION:
1. Negative digital breast tomosynthesis examination. No evidence of malignancy.
2. No sonographic correlate for the palpable finding in the left breast at 7 o'clock. This likely represents normal fibroglandular tissue.

BI-RADS Category 1: Negative. Recommend annual screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAM, BILATERAL.
INDICATION: Annual screening. No current breast complaints. Personal history of left breast invasive ductal carcinoma treated with lumpectomy and radiation 7 years ago.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. Stable post-surgical changes are noted in the upper outer quadrant of the left breast. 
IMPRESSION: 
No mammographic evidence of malignancy. 
BI-RADS 1: Negative.
RECOMMENDATION: 
Continue annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAM, LEFT.
HISTORY: Routine screening mammogram. No current breast complaints.
FINDINGS:
Left breast tissue is predominantly fatty replaced. No dominant mass, suspicious calcifications, or concerning asymmetries. 
Stable, benign-appearing calcifications are seen in the upper outer quadrant, likely vascular in nature. These are unchanged from prior exams. No new calcifications are identified. 
The skin appears normal in thickness. No significant axillary lymphadenopathy.
IMPRESSION: 
Left breast, BIRADS 2 - Benign finding. Recommend continued routine screening in 1 year.
ADDENDUM: Comparison was made to outside screening mammograms from 1 and 2 years prior, which were retrieved and show no significant interval change. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density and scattered fat lobules noted in both breasts. These are stable compared to prior mammograms.
No skin thickening or retraction is identified.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued routine screening mammography in 1 year.
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. Please inform patient that her breast tissue density falls under the category of "Dense Breast Tissue" as per CT Public Act 09-41. Recommend clinical breast examination.

ASSESSMENT CATEGORY: 1 - Negative

ADDENDUM:
Comparison was made to the patient's last mammogram from 8/24/2021, which is similar. No previous mammograms are available at this facility for comparison.
This examination was reviewed by the interpreting radiologist. If there are any questions regarding this report, please contact the Radiology department. ➡ *Negative*
- MAMMOGRAM REPORT:
CLINICAL HISTORY: Patient reports a palpable lump in the right breast at the 10 o'clock position. Last mammogram was 2 years ago and was normal. Patient has no personal or family history of breast cancer.
FINDINGS: 
Both breasts show scattered fibroglandular density. 
Right breast: There is a 1.2 cm oval circumscribed mass at the 10 o'clock position, 3 cm from the nipple, corresponding to the palpable abnormality. No suspicious calcifications, architectural distortion or skin thickening.
Left breast: No dominant mass, suspicious calcifications, or architectural distortion. 
IMPRESSION:
Right breast: 1.2 cm circumscribed mass at 10 o'clock, 3 cm from the nipple, corresponding to the palpable finding. This is most likely a benign finding such as a fibroadenoma. However, given the new palpable lump, a targeted ultrasound is recommended for further evaluation. 
Left breast: Negative.
BI-RADS Category 0: Additional imaging evaluation needed. Right breast ultrasound is recommended for the palpable lump at 10 o'clock.

ADDENDUM - RIGHT BREAST ULTRASOUND:
Targeted ultrasound of the right breast 10 o'clock lump shows a 1.2 x 1.0 x 0.8 cm oval, circumscribed, parallel, hypoechoic mass. This corresponds to the mammographic finding and is consistent with a benign fibroadenoma. 
IMPRESSION:
Benign right breast mass most consistent with a fibroadenoma. Recommend routine screening mammography in 1 year.
BI-RADS Category 2: Benign finding. ➡ *Negative*
- FINDINGS:
Left mammogram shows a moderate amount of scattered fibroglandular tissue. No focal asymmetry, questionable lesion, or architectural distortion identified. No suspicious calcifications seen. Skin is unremarkable. Stable post-surgical changes from prior right mastectomy noted.

Targeted left breast ultrasound at the site of the palpable abnormality shows normal fibroglandular tissue, with no discrete solid mass, complex cystic lesion or other sonographic abnormality in the area of concern. 

IMPRESSION:
1. No evidence of malignancy in the left breast. Recommend routine annual screening mammography. 
2. Targeted left breast ultrasound is negative for correlate to the palpable finding. If remains of clinical concern, short term follow up or biopsy could be considered.

BIRADS 1 - Negative.

ADDENDUM: 
After review of additional clinical history provided, the final assessment category remains BIRADS 1 - Negative. No mammographic or sonographic findings to explain the palpable abnormality in the left breast. ➡ *Negative*
- FINDINGS: Left breast mammogram shows a fatty replaced breast with scattered areas of benign-appearing fibroglandular density. No dominant mass, suspicious calcification, or architectural distortion. Stable appearance compared to prior mammograms. 

Right breast ultrasound demonstrates normal appearing breast tissue with no sonographic abnormality in the area of palpable concern at the 9 o'clock position. 

IMPRESSION:
1. Negative bilateral screening mammogram. No mammographic evidence of malignancy. Continue annual screening mammography.

2. Negative targeted right breast ultrasound of the 9 o'clock palpable abnormality. No sonographic correlate. Clinical correlation recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- SCREENING MAMMOGRAM.
CLINICAL HISTORY: Screening.  
COMPARISON: 5/15/2022.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  Additional spot compression views performed.  
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.  
There are no masses, asymmetries, suspicious calcifications or areas of architectural distortion bilaterally.  Stable benign appearing calcifications noted in both breasts. 
Stable post-surgical changes from prior left breast lumpectomy are again noted.
IMPRESSION:
1. No mammographic evidence of malignancy.  Recommend continued screening mammography in 1 year.
2. Heterogeneously dense breast tissue.  
ASSESSMENT CATEGORY: BI-RADS 1, Negative.  
ADDENDUM: Reviewed and agreed with by Dr. XXX. ➡ *Negative*
- BREAST IMAGING REPORT

INDICATION: Annual screening mammogram. 

TECHNIQUE: Bilateral digital mammography including craniocaudal and mediolateral oblique views.

COMPARISON: 10/15/2021

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.  

LEFT BREAST: 
There are no dominant masses, architectural distortion, or suspicious calcifications.  

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography in 12 months. 

ASSESSMENT CATEGORIES:
Right Breast: BI-RADS 1 - Negative
Left Breast: BI-RADS 1 - Negative

ADDENDUM: Clinical breast exam and review of risk factors performed. Patient is asymptomatic. No palpable abnormalities on clinical exam bilaterally. Patient is average risk and has no significant family history of breast cancer. Updated patient with results and recommendation for continued annual screening. ➡ *Negative*
- BREAST IMAGING REPORT

CLINICAL INFORMATION: 65-year-old female with family history of breast cancer presents with right breast lump.

COMPARISON: Prior mammogram and ultrasound from 08/15/2022 at ABC Imaging Center.

FINDINGS:
Right CC and MLO views demonstrate heterogeneously dense breast tissue. There is a 1.2 cm irregular mass at the 10 o'clock position of the right breast, corresponding to the palpable abnormality. No suspicious calcifications are identified. The left breast is unremarkable.

Targeted right breast ultrasound demonstrates an irregular, hypoechoic mass with angular margins and posterior shadowing at the 10 o'clock position, 4 cm from the nipple, measuring 1.2 x 1.0 x 0.9 cm. This corresponds to the mammographic finding and the palpable mass. No abnormal axillary lymph nodes are seen.

IMPRESSION:
Highly suspicious irregular mass right breast 10 o'clock position, BI-RADS 5. Ultrasound-guided core needle biopsy is recommended. No suspicious microcalcifications are present in either breast.

ADDENDUM:
Ultrasound-guided core needle biopsy of the right 10 o'clock mass was performed. Pathology results: Invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. The patient will be referred to oncology for further management. ➡ *Negative*
- FINDINGS: The left breast is composed of scattered fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications are identified.

Right breast ultrasound was performed at the site of palpable abnormality in the upper outer quadrant. No correlating mass or other suspicious finding was identified in this region. 

IMPRESSION:
1. Negative bilateral screening mammograms. No mammographic evidence of malignancy.
2. Targeted right breast ultrasound is negative at the site of palpable finding. 

BIRADS 1 - Negative.

1 year routine follow up is recommended in the absence of new signs or symptoms. ➡ *Negative*
- BILATERAL MAMMOGRAM
INDICATION: Screening mammogram.
COMPARISON: None available.
FINDINGS:
Both breasts are predominantly fatty replaced. Scattered areas of benign-appearing fibroglandular tissue are present bilaterally. No dominant masses, suspicious calcifications, or areas of architectural distortion are identified in either breast. The skin appears unremarkable. No axillary, supraclavicular or internal mammary adenopathy is seen.
IMPRESSION:
1. There is no mammographic evidence of malignancy in either breast.
2. Predominantly fatty replaced breast tissue bilaterally.
BIRADS 1 - Negative bilateral screening mammogram.
RECOMMENDATION: Routine screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of the breasts were performed.
COMPARISON: Comparison is made to prior mammogram from 12/15/2021, available in PACS.
FINDINGS:
Right breast: The breast is heterogeneously dense (BI-RADS category C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable coarse benign calcifications are noted in the upper outer quadrant.
Left breast: The breast is heterogeneously dense (BI-RADS category C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable coarse benign calcifications are noted in the upper inner quadrant.
IMPRESSION:
1. No mammographic evidence of malignancy. No suspicious calcifications.
2. Stable bilateral benign calcifications.
3. Heterogeneously dense breasts, which may obscure detection of small masses.
RECOMMENDATIONS:
Continue annual screening mammography in 12 months. Please inform us if you notice any new breast lumps, skin changes, nipple discharge or other concerns before your next mammogram.
BI-RADS ASSESSMENT CATEGORY: 2 - Benign findings. Recommend routine screening in 12 months. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL INDICATION: Annual screening mammogram. Patient complains of intermittent left breast tenderness, but no palpable mass.
COMPARISON: Prior mammograms from 4/15/2022.  
FINDINGS: 
The right breast demonstrates scattered fibroglandular densities. No dominant mass, suspicious calcifications or architectural distortion.
The left breast contains heterogeneously dense parenchyma. No dominant mass, suspicious calcifications or architectural distortion. There are no significant changes from prior exam.  
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Specifically no suspicious microcalcifications. 
2. Heterogeneously dense breasts which may obscure detection of small masses. 
RECOMMENDATIONS:
Recommend annual screening mammography. Advise patient to report any new breast symptoms or changes promptly.
ASSESSMENT CATEGORY: BI-RADS 1 - Negative. ➡ *Negative*
- FINDINGS:
Right breast tissue predominantly fatty replaced. There are scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications.  
Left breast tissue heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, suspicious calcifications or architectural distortion. No significant change compared to prior mammograms.

IMPRESSION:
1. Negative bilateral screening mammogram.  
2. Heterogeneously dense breasts, which lowers the sensitivity of mammography. 

BIRADS 1 - Negative. Recommend annual screening mammography. ➡ *Negative*
- CLINICAL HISTORY: 47-year-old female presents for screening mammogram. No complaints or palpable abnormalities. Has family history of breast cancer in maternal aunt.

FINDINGS:
Right breast:
The breast appears heterogeneously dense on mammography, which may obscure detection of small masses. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 

Left breast:
The left breast also appears heterogeneously dense. No dominant masses, suspicious calcifications, or areas of architectural distortion are identified.

Comparison is made with prior studies from 9/15/2022 and no significant interval change is seen.

Targeted ultrasound was performed on the left breast at the 10 o'clock position in the region of patient reported tenderness. No sonographic correlate or suspicious finding was identified.

IMPRESSION: 
1. No mammographic evidence of malignancy in either breast. Recommend continued annual screening in 12 months.
2. Heterogeneously dense breast tissue bilaterally, which can lower the sensitivity of mammography. 
3. No sonographic abnormality in area of clinical concern in the left breast.

BIRADS Category 1: Negative
Recommended follow up: Routine screening in 12 months

ADDENDUM: Patient was notified of results via patient portal. Given family history, patient may benefit from supplemental screening with MRI if lifetime risk exceeds 20%. Will generate risk score and notify referring provider with recommendations. ➡ *Negative*
- FINDINGS:
- Bilateral mammography and ultrasound were performed. 
- The breast tissue is heterogeneously dense, which may obscure small masses.
- There are no mammographic or sonographic findings to suggest malignancy. Specifically, no dominant mass, suspicious calcifications, or architectural distortion are identified in either breast.
- Bilateral axillary regions are unremarkable.
- There is no significant interval change compared to prior studies.

IMPRESSION: 
Negative bilateral screening mammogram and ultrasound. No evidence of malignancy.
BI-RADS Category 1 - Negative. Routine screening in 1 year is recommended. ➡ *Negative*
- FINDINGS:
Bilateral mammogram images demonstrate scattered fibroglandular densities. There are no suspicious masses, architectural distortion, or malignant-type calcifications.

Targeted ultrasound was performed at the site of palpable abnormality in the right breast at 10 o'clock position, 5 cm from the nipple. No corresponding sonographic mass or suspicious finding was identified. 

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. No sonographic correlate for the palpable finding in the right breast. 

BI-RADS Category 1: Negative. Recommend annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
NAME: Smith, Jane 
EXAMINATION: Bilateral Screening Mammogram 
CLINICAL HISTORY: Annual screening mammogram.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: 03/15/2021
FINDINGS:
Right Breast:
Tissue composition is heterogeneously dense. This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications in the upper outer quadrant. 
Left Breast: 
Tissue composition is heterogeneously dense. This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications.  
Stable benign calcifications scattered throughout the breast. 
IMPRESSION:
BI-RADS Category 1: Negative bilateral screening mammogram. 
Heterogeneously dense breast tissue bilaterally. 
RECOMMENDATION:  
Recommend routine yearly screening mammography.
Continue monthly self breast exams. Return promptly if palpable abnormality discovered. ➡ *Negative*
- MAMMOGRAPHY REPORT
PROCEDURE: Bilateral screening mammogram.
HISTORY: 49-year-old female, no palpable abnormalities. Patient reports no breast symptoms.
TECHNIQUE: Standard CC and MLO views of each breast.
COMPARISON: Prior study of June 12, 2022.
FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortion, or suspicious calcifications.
The skin and nipple are unremarkable.
No significant interval change since the prior study.
IMPRESSION:
1. BI-RADS Category 1: Negative.
2. Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAM REPORT
INDICATION: Annual screening mammogram, patient reports possible lump in left breast at 3 o'clock.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Right mastectomy noted.
COMPARISON: None available.
FINDINGS:
Right breast shows post-mastectomy changes. No concerning abnormalities in the right chest wall.
Left breast shows scattered fibroglandular densities. No suspicious masses, architectural distortions or abnormal calcifications are identified. The breast is heterogeneously dense which may limit the sensitivity of mammography. 
A possible palpable abnormality is reported in the left breast at 3 o'clock position, this is included on the ultrasound requisition for further assessment.
IMPRESSION:
1. Negative left screening mammogram, no evidence of malignancy. Recommend annual screening with mammography.
2. Heterogeneously dense breast tissue which lowers the sensitivity of mammography.
3. Possible palpable abnormality left breast at 3 o'clock position, ultrasound is recommended for further assessment.
BI-RADS CATEGORY 1 - Negative. Routine screening recommended in 1 year. ➡ *Negative*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: Annual screening mammogram. Patient denies any new breast symptoms or palpable lumps. Patient's mother had breast cancer at age 55.
TECHNIQUE: Bilateral digital screening mammography including CC and MLO views.
COMPARISON: Comparison is made to prior exam dated 10/15/2020 at this facility.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
Right Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.
Left Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.
Bilateral benign-appearing lymph nodes are noted in the axillary regions.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Heterogeneously dense breast tissue, which lowers the sensitivity of mammography. Clinical breast examination is recommended.
BI-RADS Category 1: Negative. ➡ *Negative*
- ULTRASOUND AND MAMMOGRAM OF THE RIGHT BREAST

CLINICAL HISTORY: 57-year-old female with right breast lump at 10 o'clock position, painful. No prior mammograms available for comparison.

FINDINGS:  
Mammogram: The breast is heterogeneously dense. No dominant mass, architectural distortion, or suspicious calcifications are identified. 

Ultrasound: Targeted ultrasound of the right breast at 10 o'clock position 3 cm from the nipple demonstrates a 0.8 x 0.6 x 0.7 cm oval, circumscribed, hypoechoic solid mass. This corresponds to the palpable abnormality. No suspicious axillary lymph nodes are seen.

IMPRESSION:
1. Oval circumscribed hypoechoic solid mass at 10 o'clock position of the right breast corresponding to the palpable lump. This is most likely a fibroadenoma. 6-month follow-up ultrasound is recommended to document stability.

2. No suspicious mammographic findings in the right breast. Recommend annual screening mammography.

BIRADS 2 - Benign findings. Recommend 6-month follow-up ultrasound of the right breast mass.

ADDENDUM: Pathology from ultrasound-guided core biopsy of the right breast mass confirms a fibroadenoma. Recommend clinical follow-up as needed. Annual screening mammography is recommended. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
INDICATION: Left upper outer quadrant palpable lump. 
PREVIOUS STUDIES: 2 years prior.
FINDINGS:  
Both breasts are composed of heterogeneously dense fibroglandular tissue. There is no evidence of dominant mass, suspicious calcifications, or architectural distortion bilaterally. No significant interval change from prior exam. 
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Stable heterogeneously dense breast tissue. 
3. Clinical breast exam finding of palpable left upper outer quadrant lump, not definitively seen on mammogram. Recommend targeted ultrasound for further evaluation. 
BIRADS 1 - Negative mammogram. 
ULTRASOUND OF THE LEFT BREAST
INDICATION: Palpable abnormality left upper outer quadrant. Negative mammogram.
TECHNIQUE: High resolution linear transducer.
FINDINGS: Targeted ultrasound of the left upper outer breast quadrant demonstrates normal fibroglandular tissue. No discrete solid or cystic mass lesion identified in the area of clinical concern. The remainder of the left breast demonstrates normal echotexture and vascularity without suspicious findings. 
IMPRESSION: 
1. No sonographic evidence of malignancy. 
2. Negative targeted ultrasound of the left upper outer quadrant. No definite cause for palpable finding identified. Recommend clinical follow up and correlation with physical exam findings. Consider biopsy if clinically warranted.
BIRADS 1 - Negative ultrasound. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. There are no dominant masses, architectural distortions, or suspicious calcifications.

There are scattered areas of fibroglandular density bilaterally. No mammographic evidence of malignancy.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend continued annual screening mammography and clinical breast exams. 
3. Breast density is heterogeneously dense, which may lower the sensitivity of mammography. Inform the patient that dense breast tissue is a common finding that is noted in about 40 percent of mammograms and is a normal variant. While dense breast tissue may make it more difficult to detect breast cancer on a mammogram, it does not increase the risk for developing breast cancer. Consider supplemental screening with breast ultrasound or MRI if clinically indicated based on additional risk factors.

BIRADS 1 - NEGATIVE.

ADDENDUM: Spoke with ordering physician Dr. Smith who agrees with recommendations. Patient also notified of results. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense parenchyma. Stable postsurgical changes of right mastectomy are noted. No suspicious mass, architectural distortion, or malignant type calcifications.
Left breast: Heterogeneously dense breast tissue. No mammographic evidence of malignancy. No spiculated mass, suspicious calcifications, or architectural distortion. Stable appearance compared to prior exams.

IMPRESSION:
1. Negative bilateral mammogram. No evidence of malignancy.
2. Postoperative changes of right mastectomy appear stable.

BIRADS Category 1 - Negative. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAM AND BREAST ULTRASOUND
INDICATION: Left breast pain and heaviness. History of right breast cancer status post mastectomy.
TECHNIQUE: Standard bilateral mammogram views are obtained. Targeted ultrasound of the left breast was also performed.  
COMPARISON: Prior mammograms and ultrasound from 5/12/2022.
FINDINGS:
Right breast: Mastectomy changes are noted. No concerning findings.
Left breast: The breast is predominantly fatty replaced. No masses, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications in the upper outer quadrant. No mammographic evidence of malignancy. 
Ultrasound of the left breast demonstrates scattered fibroglandular tissue. No solid or cystic masses. No sonographic correlate for the patient's symptoms.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. 
2. Stable right mastectomy site.
3. Recommend annual screening mammography. 
BI-RADS 1- Negative. ➡ *Negative*
- MAMMOGRAM
INDICATION: Annual screening. No palpable complaints.
TECHNIQUE: Standard CC and MLO views of each breast.
COMPARISON: Prior study of 5/22/2021.
FINDINGS:
Breast composition is scattered fibroglandular density (C). The amount of dense tissue could obscure detection of small masses.
No significant masses, calcifications, or other findings. Stable appearance compared to prior mammograms.
IMPRESSION:
BI-RADS Category 1: Negative. Recommend continued annual screening with mammography. ➡ *Negative*
- BILATERAL MAMMOGRAM.
HISTORY: Screening mammogram. No current breast complaints.
Comparison made to prior imaging.
FINDINGS: There are scattered fibroglandular densities in both breasts. There are no suspicious masses, architectural distortions, or skin thickening. No sonographic correlate is seen in either breast. 
IMPRESSION: 
1. No mammographic evidence of malignancy. 
2. Scattered fibroglandular densities. Recommend annual screening mammography.
BIRADS 1. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No dominant masses, architectural distortion or suspicious calcifications are identified. 

There are no significant changes compared to prior mammogram from 1/15/2021.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend annual screening mammography in 12 months.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- REPORT:
"RIGHT MAMMOGRAM AND ULTRASOUND
HISTORY: Patient with palpable mass in right breast at 2 o'clock. Last normal mammogram 1 year prior.
FINDINGS:
The right breast is heterogeneously dense. No suspicious masses, architectural distortion, or calcifications. Stable post-surgical changes in the right breast from prior lumpectomy. Targeted right breast ultrasound demonstrates normal fibroglandular tissue with no discrete mass at the palpable area of concern.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Recommend annual screening mammography.
2. Targeted right breast ultrasound is negative for focal abnormality at the area of palpable concern. Likely represents normal fibroglandular tissue.
BI-RADS 1 - Negative.
ADDENDUM: Comparison made to prior outside mammogram. No significant interval change." ➡ *Negative*
- TECHNIQUE: Bilateral digital mammography with tomosynthesis. COMPARISON: Compared to outside mammogram from 10/15/2022.

CLINICAL HISTORY: 54-year-old female presents for routine screening mammogram.

FINDINGS:
RIGHT BREAST:
The breast is almost entirely fat. There are no masses, architectural distortions, or suspicious calcifications. 

LEFT BREAST: 
The breast is almost entirely fat. There are no masses, architectural distortions, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Fatty replaced breasts bilaterally. 
3. No suspicious masses, calcifications, or architectural distortion in either breast.

BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAPHY
INDICATION: Routine screening mammogram.
COMPARISON: Mammogram from 1 year ago.
TECHNIQUE: Standard CC and MLO views of both breasts.
FINDINGS:
The breasts are heterogeneously dense. No suspicious masses, architectural distortion, or asymmetries are identified. No microcalcifications are seen. Stable post-surgical changes noted in the right breast. No skin thickening or retraction. No axillary, internal mammary, or supraclavicular adenopathy.
IMPRESSION:
No mammographic evidence of malignancy. Routine annual screening mammogram is recommended.
BIRADS Category 2: Benign findings.
ADDENDUM:
No microcalcifications are identified in either breast on current examination. ➡ *Not Stated*
- FINDINGS:
Both breasts are composed of scattered areas of fibroglandular density.  There are no dominant masses, suspicious calcifications or areas of architectural distortion.

IMPRESSION:
BIRADS Category 1: Negative.  Routine screening mammography is recommended in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAPHY REPORT

INDICATION: Routine screening mammogram.

TECHNIQUE:
Bilateral screening full field digital mammography. CC and MLO views obtained.

COMPARISON:
None available.

FINDINGS:
The breasts are predominantly fatty.
The breasts are symmetric. No significant masses, architectural distortion or suspicious calcifications.

IMPRESSION:
BIRADS Category 1: Negative. Routine screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Comparison Exam: Screening Mammogram from [DATE], available in PACS.
FINDINGS:
The breasts are heterogeneously dense (51-75% fibroglandular tissue), which may obscure detection of small masses.
There are no dominant masses, suspicious microcalcifications, or focal architectural distortion.
Stable bilateral scattered fibroglandular densities.
IMPRESSION:
No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
BIRADS Category 2: Benign Finding.
ADDENDUM: 
Reviewed final report with interpreting radiologist. No suspicious microcalcifications are identified in either breast. Report finalized without modification. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: No previous mammograms available for comparison.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breast tissue is heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2.

Addendum: 
The client called to report a palpable lump in the right breast upper outer quadrant. Targeted ultrasound was performed which demonstrated a 1.2 cm oval circumscribed hypoechoic mass, likely representing a benign fibroadenoma. Short term follow up ultrasound is recommended in 6 months to document stability. 
BIRADS-3 for the palpable right breast mass. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
No significant mammographic abnormalities. The breasts are heterogeneously dense. No suspicious masses, architectural distortions or suspicious calcifications.
BIRADS-2.
ADDENDUM: There are no concerning microcalcifications. ➡ *Not Stated*
- MAMMOGRAPHIC BREAST DENSITY: Extremely dense (>75% glandular tissue).
IMPRESSION: BIRADS category 1/2.
Both breasts appear almost entirely fatty replaced, with minimal residual glandular tissue in the upper outer quadrants. No suspicious focal masses, architectural distortions or new dominant clusters of microcalcifications.
No significant change compared to previous screening mammogram of 2021-05-03.
Normal screening mammogram.
Routine annual screening is recommended.
FINAL ASSESSMENT: Negative. BI-RADS Category 1/2 - Routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Technique: Standard CC and MLO views of both breasts.
Comparison: Previous, 2 years prior.
Breast Composition: The breasts are almost entirely fatty. 
Findings: There are no masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS 2- Benign.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
NEGATIVE mammogram.

BI-RADS Category 1: Negative.
Routine screening is recommended in 1 year.

PROVIDER ADDENDUM: 
Breast density notification has been sent to the patient and her referring provider as per state regulations. ➡ *Not Stated*
- DIAGNOSTIC MAMMOGRAM/ULTRASOUND
CLINICAL HISTORY: 52-year-old female for diagnostic workup of recently identified left upper outer quadrant mass 2 cm away from the nipple.
COMPARISON: None available.
FINDINGS:
The breasts are scattered fibroglandular density.
There is a palpable mass in the left upper outer quadrant 2 cm away from the nipple at 2:00 corresponding with an approximately 15 x 14 x 12 mm lobulated hypoechoic solid mass on ultrasound with irregular margins and internal vascularity. An ultrasound-guided core needle biopsy was performed.
There are no suspicious microcalcifications or architectural distortion.
There are no dominant masses in the right breast.
IMPRESSION:
BIRADS 4B: Suspicious left breast mass. Biopsy performed. Recommend surgical consultation and treatment pending pathology results. ➡ *Not Stated*
- SCREENING MAMMOGRAM   
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular), which may limit detection of small masses.  
No suspicious masses, calcifications, asymmetries, or areas of architectural distortion are present in either breast.  
No significant interval change since the prior examination.  
IMPRESSION:
BIRADS Category 1: Negative examination. Routine screening is recommended in 1 year.  
No suspicious mammographic findings. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
INDICATION: Annual screening mammogram
TECHNIQUE: Standard CC and MLO views of both breasts obtained. 
COMPARISON: Previous exam from April 2022
FINDINGS:
Breasts are heterogeneously dense, which may obscure small masses. 
No significant masses, architectural distortion, or suspicious calcifications are seen in either breast.
There are no mammographically evident malignancies.
IMPRESSION: 
1. Negative screening mammogram.
2. Breast Imaging Reporting and Data System (BI-RADS) Category 1: Negative.
MANAGEMENT: Recommend routine screening in 1 year.

ADDENDUM REPORT:
Addendum Reason: Clarification of report
The mammogram images have been reviewed again after initial report was finalized.  
This serves to clarify that there are NO microcalcifications described in either breast in this screening exam.
All other findings and management recommendations remain the same. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  
CLINICAL INDICATION:  Screening.  

COMPARISON:  None.  

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.  

FINDINGS:  
The breasts are heterogeneously dense which may obscure detection of small masses.  
There are no dominant masses, areas of architectural distortion or suspicious calcifications.  
Stable benign appearing calcifications are noted in the right breast.  

IMPRESSION:  
No mammographic evidence of malignancy. Recommend routine screening in 1 year.  
BIRADS 2- Benign Finding.  

ADDENDUM:  
This screening mammogram was double read by two MQSA certified radiologists in accordance with our institution's policy. No discrepancies were identified between reads. ➡ *Not Stated*
- FFDM TMIST OBSP MAMMOGRAM
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. BIRADS-1.
Compared to Prior: Yes. There are no suspicious findings or masses. The breast tissue is primarily fibroglandular.
Breast Density >= 75%: No.  
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No. 
No changes are identified compared to prior studies. There are no concerning findings for malignancy. 
ADDENDUM: 
Upon further review and discussion with the ordering physician, no mention of microcalcifications was made in the original report. This addendum serves to clarify that no suspicious microcalcifications are identified. The final assessment category remains BIRADS-1. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM

INDICATION: Routine screening mammogram.

COMPARISON: None available. 

TECHNIQUE: Standard CC and MLO views of the right and left breasts were obtained. 

FINDINGS:
The breasts are nearly entirely fatty replaced. No dominant mass, suspicious calcification, or other finding suspicious for malignancy.

IMPRESSION:
Negative bilateral screening mammogram.
BIRADS Category 1 - Negative. Routine screening is recommended in 1 year.

ADDENDUM:
Breast Density < 25%.
Not Stated for microcalfications in the current report. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
No mammographic findings concerning for malignancy. No suspicious masses, architectural distortion or definite abnormal calcifications. 
BIRADS-1. 
Annual screening mammography is recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
BIRADS CATEGORY 1 - NEGATIVE
BREAST COMPOSITION: The breasts are almost entirely fatty.
FINDINGS:
Mammographic views demonstrate bilateral scattered fibroglandular densities.
There are no dominant masses, suspicious microcalcifications or focal architectural distortion.
IMPRESSION:
1. There is no mammographic evidence of malignancy. Routine screening is recommended. 
ADDENDUM: Additional clinical history was obtained following the finalization of this report by discussion with the referring clinician. No change to the above impressions or recommendations is needed based on this additional clinical history. ➡ *Not Stated*
- SCREENING DIGITAL BREAST TOMOSYNTHESIS (DBT) BILATERAL

INDICATION: Screening.

TECHNIQUE: Bilateral digital breast tomography (DBT) views were obtained.

COMPARISON: None.

FINDINGS:
The breasts are predominantly fibroglandular in composition.

No suspicious masses, architectural distortion or dominant suspicious calcifications are identified. No abnormal axillary lymph nodes are seen.

IMPRESSION:
Negative bilateral screening mammogram. 

BIRADS Category 1: Negative. Routine screening mammography is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. No significant interval change.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breast tissue is heterogeneously dense. No new masses, architectural distortion or suspicious calcifications.
BIRADS-2.
ADDENDUM: Comparison with prior outside films dated Feb 12, 2019 shows stable appearance of the breasts with no concerning interval changes. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM
This client participated in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST). The client was randomly assigned to the full field digital mammography only arm and informed consent was obtained. 
Comparison to previous mammogram available.
Breast density: Scattered fibroglandular densities.
No mammographic findings concerning for malignancy identified. No suspicious microcalcifications, architectural distortions, dominant masses or abnormal lymph nodes.  
IMPRESSION:
BIRADS Category 1: Negative. 
Routine screening mammogram recommended in 2 years as per OBSP guidelines. ➡ *Not Stated*
- "ROUTINE SCREENING MAMMOGRAM.
Bilateral diagnostic full field digital mammography obtained on <<today's date>>. The Hologic system was utilized.
Mammographic views of the breasts reveal scattered areas of fibroglandular density bilaterally. The overall density pattern is ACR BI-RADS b. 
There are no dominant masses or architectural distortion. No suspicious microcalcifications are seen. No significant skin thickening or retraction.
IMPRESSION:
BI-RADS Category 2: Benign mammographic findings. Recommend continued routine screening.
ADDENDUM:
Comparison is made to prior mammogram dated 3 years ago at an outside facility. No significant interval change. Stable scattered fibroglandular densities bilaterally.
" ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breasts are predominantly fatty replaced. No significant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2.
ADDENDUM: The breast calcifications seen are typical benign vascular calcifications. No suspicious microcalcifications are noted. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: CC and MLO views of each breast were obtained.

COMPARISON: 08/14/2020.

FINDINGS:
There is fatty and scattered fibroglandular tissue in both breasts. No focal asymmetry, architectural distortion, or suspicious mass.

No clustered pleomorphic microcalcifications are identified.

Stable benign-appearing calcifications are noted bilaterally.

IMPRESSION:
No mammographic evidence of malignancy. Benign mammogram.

BI-RADS Category 2: Benign. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to prior study from 1 year ago.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
No dominant masses, architectural distortion or suspicious calcifications are identified.
There are no significant changes compared to prior mammogram.
IMPRESSION:
Negative screening mammogram. 
BIRADS Category 1: Normal.
Recommend routine screening in 1 year. ➡ *Not Stated*
- NORMAL OBSP MAMMOGRAM - TMIST FFDM ARM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: No.
Breast Density: Heterogeneously dense.
OBSP to recall client in 2 years: Yes.
There are no suspicious masses, architectural distortions or mammographically significant lymph nodes.
BIRADS-2. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
Bilateral breasts are almost entirely fatty.
No suspicious masses, architectural distortion or suspicious calcifications.
BIRADS-2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. 
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breasts are predominantly fatty. No dominant masses, suspicious calcifications or architectural distortion.  
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAPHY

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of bilateral breasts were obtained.

COMPARISON: None.

FINDINGS:
The breasts are symmetrical and composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Negative digital screening mammogram, BIRADS Category 1. 

RECOMMENDATION:
Routine screening as per guidelines.

Breast density: C. Heterogeneously dense, which may obscure detection of small masses.

ADDENDUM:
No microcalcifications are identified in either breast. ➡ *Not Stated*
- MAMMOGRAM:

FINDINGS:
Breast Density: C. The breasts are heterogeneously dense (51-75% fibroglandular), which may obscure detection of small masses.

There is no suspicious mass, architectural distortion or concerning asymmetry identified in either breast.

No suspicious clustered micro-calcifications are identified. 
  
There are stable scattered bilateral benign-appearing calcifications.

IMPRESSION:
Negative bilateral screening mammogram.
BIRADS Category 2: Benign findings. 

The patient is recommended to continue routine annual screening mammography in 12 months. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. No suspicious masses, calcifications, or architectural distortion identified.
BIRADS-2. ➡ *Not Stated*
- NORMAL RIGHT SCREENING MAMMOGRAM. BENIGN LEFT.
Compared to previous exam 2 years ago, the breasts are heterogeneously dense (51-75% fibroglandular). No suspicious masses, architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are scattered vague calcifications in the central left breast, unchanged for several years. Benign.

RIGHT BREAST: 
Negative.

IMPRESSION:
BIRADS 2: Benign findings. Routine screening in 1 year recommended. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM.

Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.

Scattered areas of fibroglandular density.

There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Negative digital screening mammogram.
No mammographic evidence of malignancy.

BIRADS: 1 - Negative.

1-year OBSP recall. ➡ *Not Stated*
- TMIST FFDM ARM OBSP RECALL MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
No specific findings. Recommendation for annual recall as part of OBSP trial protocol.
BIRADS-2. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.  
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
Normal mammographic screening exam. No suspicious masses, distortions or calcifications.
BIRADS-2. Normal mammogram.

ADDENDUM: 
Previous screening mammograms from June 2021 and May 2020 were reviewed and showed stable scattered fibroglandular densities. No significant interval change. ➡ *Not Stated*
- SCREENING BILATERAL 2D DIGITAL MAMMOGRAM.

INDICATION: Routine screening mammogram.  

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

COMPARISON: Compared to prior mammogram from 1 year ago.  

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.

There are no suspicious masses, architectural distortions, or suspicious calcifications.  

IMPRESSION:
BIRADS 2- Benign/Negative. No significant change from prior exam. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM. COMPARISON: None.

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

FINDINGS:
Breast composition: The breasts are heterogeneously dense, which could obscure detection of small masses. 

There are no suspicious masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
BIRADS 1, Negative.

1-year OBSP screening mammogram is recommended. ➡ *Not Stated*
- NORMAL OBSP FFDM SCREENING MAMMOGRAM

Compared to Previous: Yes.
Date of previous study: 10 Jan 2022
Finding: Stable, scattered fibroglandular densities.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses. 
Breast Density >= 75%: No.
Breast Density < 75%: Yes.

FINDINGS: 
There are no concerning masses, architectural distortions or suspicious calcifications.

IMPRESSION:
1. Negative study. 
2. Heterogeneously dense breasts.
3. Recommend annual screening mammogram as per OBSP guidelines.

BIRADS Category: 1 - Negative.

OBSP to recall client in 1 year: Yes. ➡ *Not Stated*
- MAMMOGRAPHY, BILATERAL SCREENING  

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammogram from 1 year ago.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications.  

No significant change compared to prior mammogram.

IMPRESSION:
No mammographic evidence of malignancy. 
BIRADS Category 1, Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breasts are composed of scattered areas of fibroglandular density. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM.  
Scattered areas of fibroglandular density.
No new masses, architectural distortions or suspicious calcifications. 
Breast density Grade B; 40-50% fibroglandular.
BIRADS Category 2: Benign.
Annual screening recommended.

ADDENDUM: For completeness, microcalcifications are not stated to be present on today's examination.  ➡ *Not Stated*
- SCREENING MAMMOGRAM
TMIST FULL-FIELD DIGITAL MAMMOGRAPHY (FFDM) ARM
INFORMED CONSENT OBTAINED
CLINICAL INDICATION:
Annual screening mammogram through the provincial OBSP program. No concerning breast symptoms. No family history of breast cancer.
COMPARISON:
Prior screening mammograms are available and were compared, most recently from 1 year ago.  
FINDINGS:
Right and left breasts are heterogeneously dense (51-75% fibroglandular tissue). No dominant mass, architectural distortion or definite asymmetry. No malignant-type calcifications. The breasts are symmetrical.
No mammographic evidence of malignancy. 
IMPRESSION:
BIRADS Category 1 - Negative. Routine annual screening is recommended.
ADDENDUM:
Patient participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomized to the full-field digital mammography arm of the study. Informed consent was obtained. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
INDICATION: Annual screening mammogram
COMPARISON: Previous exam dated 11/12/2021

FINDINGS:
There is scattered fibroglandular density. Breast tissue is heterogeneously dense, which may obscure detection of small masses. 
No mammographic evidence of malignancy or concerning findings.
No suspicious microcalcifications, masses, or architectural distortion.
Stable benign calcifications noted bilaterally.

IMPRESSION:
BIRADS Category 2: Benign findings. Routine screening recommended in 1 year.

ADDENDUM:
This study was read by Dr. Jane Smith, MD.
The patient's lifetime risk of breast cancer is estimated to be 12% based on the Tyrer-Cuzick model.
The patient is currently taking no medications known to increase breast density. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of each breast. 
COMPARISON: Compared to prior exam from 2 years ago.
FINDINGS:
The breasts are predominantly fatty with scattered fibroglandular densities. 
There are no suspicious masses, architectural distortions, or asymmetries. 
The skin is normal. No significant axillary lymphadenopathy.
IMPRESSION: 
Negative screening mammogram, BI-RADS Category 1. 
Recommend routine screening in 1 year.
BREAST DENSITY: The breasts are almost entirely fatty, which may allow detection of small masses.  

ADDENDUM: No mention of microcalcifications in the current report. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 25%: Yes.
Client to return for routine annual screening in 1 year: Yes.

FINDINGS:
The breasts are predominantly fatty.
No masses, architectural distortion, or suspicious calcifications.
Stable benign calcifications noted bilaterally.

IMPRESSION:
BIRADS 1 - Negative.
No mammographic evidence of malignancy.

MANAGEMENT:
Routine annual screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
CLINICAL INDICATION: Routine screening.
COMPARISON: 2 year prior screening mammogram.
TECHNIQUE: Standard CC and MLO views obtained bilaterally. 
FINDINGS:
There are no concerning masses, dominant focal asymmetries, architectural distortions, or suspicious calcifications.
Breast tissue is heterogeneously dense, which could obscure detection of small masses.
IMPRESSION:
BIRADS Category 1: Negative.
Continue annual screening mammography. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
INDICATION: Screening, female, age 53 years.
TECHNIQUE: Tomosynthesis bilateral MLO and CC views obtained.
COMPARISON: Initial screening exam.
FINDINGS: Left breast heterogeneously dense (ACR B); right breast extremely dense (ACR D).  
No mammographic findings concerning for malignancy. No suspicious masses, calcifications, architectural distortion or skin thickening identified in either breast.
IMPRESSION:
BIRADS Category 1-Negative. Recommend continued annual screening.
ADDENDUM: Breast density is a strong, independent risk factor for breast cancer. Supplemental screening, such as with ultrasound, MRI or molecular breast imaging might be beneficial in detecting early breast cancer not seen on mammography. Patient is encouraged to discuss her breast cancer risk factors and potential benefits of supplemental screening with her provider. ➡ *Not Stated*
- SCREENING FFDM OBSP ROUTINE MAMMOGRAM. 
This client participated in OBSP screening. Consent was obtained.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
The breast parenchyma is heterogeneously dense. No significant masses, architectural distortion or suspicious calcifications are identified.  
BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No significant abnormality or suspicious finding. The breasts are almost entirely fatty.
BIRADS-2. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM SHOWS SCATTERED FIBROGLANDULAR DENSITIES. BENIGN VASCULAR CALCIFICATIONS ARE SEEN BILATERALLY. MILD BENIGN ASYMMETRICAL DENSITY IN THE LEFT UPPER OUTER QUADRANT IS LIKELY SUMMATION ARTEFACT. NO SKIN THICKENING OR SUSPICIOUS MAMMOGRAPHIC FINDINGS THAT MAY INDICATE MALIGNANCY ARE SEEN.

BREAST ULTRASOUND ADDENDUM - LEFT BREAST:
A focused sonographic evaluation of the left breast at 2 o'clock position 5 cm from the nipple reveals a hypoechoic mass with circumscribed margins measuring 5 x 3 mm, consistent with a simple cyst. No other suspicious sonographic findings are present.

BIRADS-2. BENIGN / PROBABLY BENIGN FINDINGS.
OBSP TO RECALL CLIENT IN 1 YEAR FOR ROUTINE SCREENING MAMMOGRAPHY. ➡ *Not Stated*
- FFDM TMIST OBSP SCREENING MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Comparison: 02/08/2019. No prior mammograms at this facility.
Both breasts are heterogeneously dense (51-75% fibroglandular tissue), which may obscure detection of small masses.
No mammographic evidence of malignancy. There are a few scattered benign-appearing calcifications bilaterally. No microcalcification clusters, dominant masses, or architectural distortion.
BIRADS Assessment Category: 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM X-RAY SCREENING MAMMOGRAM.
The client is a research participant in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomized to the full field digital mammography arm after providing informed consent. REB approval for the study was obtained.  
Comparison to previous mammogram: Yes.
Breast tissue density: <75%.
Routine annual screening recommended by OBSP.
No suspicious mammographic findings. 
Final assessment: BIRADS-2 Benign. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
No suspicious masses, architectural distortion, or suspicious calcifications are identified.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
BIRADS-2.

Addendum: Compared to previous study 2 years ago, no interval change. Fatty replaced involuting breasts. No microcalcifications or concerning features identified. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Screening.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
There is scattered fibroglandular tissue bilaterally, consistent with heterogeneously dense breasts which could obscure small masses.
No suspicious masses, architectural distortion, or suspicious calcifications are identified in either breast.
No significant skin thickening or retraction.
The visualized axillary regions appear unremarkable.
IMPRESSION:
BIRADS Category 1: Negative.
There are no mammographic features suspicious for malignancy.
Routine screening mammography is recommended in one year. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Procedure and Technique: Digital bilateral two view screening mammography was performed including craniocaudal and mediolateral oblique views of each breast. 
Comparison: None available.
Findings:
The breasts are predominantly fatty.
No masses, architectural distortion or suspicious calcifications are identified.
Impression:
BIRADS Category 1: Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy.
BIRADS-1/2. ➡ *Not Stated*
- SCREENING DIGITAL MAMMOGRAPHY BILATERAL

INDICATION: Screening mammogram.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

COMPARISON: None available.

FINDINGS:
The breasts are predominantly fatty.
No suspicious mass, asymmetry, architectural distortion or microcalcifications are seen.
There are no significant changes compared to prior studies.

IMPRESSION:
BIRADS 1 - Negative mammogram.

Routine follow-up is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No significant mammographic findings. Scattered benign-appearing calcifications noted bilaterally.
BIRADS-2.

Addendum: Upon further review, no suspicious microcalcifications are identified. Benign calcifications as described above. Continue routine screening. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM
INDICATION: Routine screening. No current breast complaints.
COMPARISON: Comparison is made to prior exam dated July 18, 2018. 
FINDINGS: 
Right breast: Scattered areas of fibroglandular density. No suspicious masses, calcifications or architectural distortions.
Left breast: Scattered fibroglandular densities. No dominant masses, suspicious calcifications or architectural distortions.
IMPRESSION: 
Negative screening mammogram. The breasts are heterogeneously dense (50-75% fibroglandular) which may obscure detection of small masses.
BI-RADS Category 1: Negative. Recommend continued annual screening in conjunction with monthly self breast exams. ➡ *Not Stated*
- DIAGNOSTIC MAMMOGRAM WITH TOMOSYNTHESIS.
Clinical Indication: Screening recall.

FINDINGS:
The breast tissue is heterogeneously dense. When compared to the prior mammogram from [DATE], there are no concerning changes.

There are scattered areas of fibroglandular density. No dominant masses, architectural distortion or suspicious calcifications are identified.

IMPRESSION:
BIRADS 2-Benign finding. Recommend routine screening in 1 year.

ADDENDUM: 
While heterogeneously dense breast tissue can limit sensitivity, there are no mammographic findings suspicious for malignancy. No microcalcifications are identified. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
BIRADS-2.
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, suspicious calcifications, or other findings.
Radiologic stability with prior exams.
Lumpy fibroglandular and nodular breast architecture bilaterally, stable.
Recommendation:
Continue routine screening mammography. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM.  
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of each breast. 
COMPARISON: 2 years prior.
FINDINGS:
Both breasts are predominantly fatty.
There are no masses, architectural distortions, or suspicious calcifications.
There are no significant changes compared to prior exam.
IMPRESSION: 
NEGATIVE BILATERAL SCREENING MAMMOGRAM. BIRADS 1. 
Patient to return to routine screening in 1 year. ➡ *Not Stated*
- NORMAL TMIST DBT ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the digital breast tomosynthesis arm.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breast tissue is heterogeneously dense. 
No dominant masses, architectural distortions or suspicious calcifications are identified.
BIRADS-2. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM 

SCREENING TYPE: 
High-risk.

CLINICAL INDICATION: 
Family history of breast cancer - aunt.

COMPARISON:
None available.

TECHNIQUE:
TOMOSYNTHESIS AND 2D MAMMOGRAPHIC VIEWS. 
MLO and CC views bilaterally.  

FINDINGS:
Scattered areas of fibroglandular density.  
No concerning masses, architectural distortion, or suspicious calcifications. 
No significant interval change compared to outside screening mammogram from February 2021.

IMPRESSION:
BI-RADS 1 - NEGATIVE. 
Routine annual screening mammography is recommended in 1 year.

BREAST DENSITY:
The breasts are heterogeneously dense (C), which may obscure detection of small masses. 

ADDENDUM:
This examination was double read by Computer Aided Detection Software with no additional findings. ➡ *Not Stated*
- TMIST - Digital Mammogram
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Comparison: Films available and reviewed from December 7, 2019.
Breast Composition: The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
FINDINGS:
Both breasts are symmetric in size and density, with no skin thickening, distortion, or concerning dominant masses.
There are no new microcalcifications, architectural distortions, or suspicious abnormalities, compared to prior study. Stable scattered benign-appearing calcifications are noted in both breasts.
IMPRESSION: 
Negative digital mammogram. No mammographic evidence of malignancy.
BIRADS Category 1: Negative.
Routine screening mammography is recommended in 1 year as per Organized Breast Screening Program (OBSP) guidelines.
ADDENDUM:
Correlation with clinical findings is recommended if clinically indicated. Any new signs or symptoms should be investigated with appropriate imaging. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: No.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
There are no suspicious masses, architectural distortions or dominant abnormal calcifications. 
The breasts are almost entirely fatty.
BIRADS-2. Routine screening mammogram is recommended in 1 year. ➡ *Not Stated*
- TMIST FULL FIELD DIGITAL SCREENING MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. No significant interval change.
Breast Density >= 75%: No.  
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
Benign-appearing calcifications are present in the right breast. No other findings suspicious for malignancy. 
BIRADS-2.
Addendum:
The calcifications described above in the right breast are coarse and diffusely scattered. No suspicious microcalcification clusters are identified. Routine screening recommended. ➡ *Not Stated*
- TMIST FFDM ARM MAMMOGRAM.

Compared to Previous: No previous mammogram for comparison.

Breast Density <= 25%: Yes.
Breast Density > 25%: No.

No suspicious mass, significant asymmetry or concerning calcifications are seen.  

IMPRESSION:
BIRADS category 1 - Negative mammogram.  

RECOMMENDATION:
Routine screening mammogram in 1-2 years. ➡ *Not Stated*
- MAMMOGRAM REPORT:
Clinical History: Annual screening mammogram. No concerning symptoms reported.
Technique: Standard CC and MLO views of both breasts.
Findings:
The breasts are predominantly fatty replaced.
No dominant masses, architectural distortions, or suspicious microcalcifications are identified.
Stable, benign-appearing calcifications are noted bilaterally.
No skin thickening or retraction.
No mammographic evidence of malignancy.
Impression:
1. Negative bilateral screening mammogram.
2. Predominantly fatty replaced breasts.
3. Stable, benign-appearing calcifications bilaterally. These do not require further evaluation.
BI-RADS Category 2: Benign findings. Routine screening recommended in 1 year. ➡ *Not Stated*
- SCREENING DIGITAL BREAST TOMOSYNTHESIS (DBT).
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
There are no suspicious masses, architectural distortions, or asymmetries.
No radiographic evidence of malignancy is identified. The breast tissue is primarily fatty.
BIRADS-1. Routine screening mammography is recommended in 1 year.
ADDENDUM: Comparison with prior mammograms does not reveal any developing abnormalities. No microcalcifications are described. ➡ *Not Stated*
- SCREENING MAMMOGRAPHY
BILATERAL ROUTINE SCREENING MAMMOGRAPHY
TMIST Full Field Digital Mammography Arm
FINDINGS:
- This patient is a participant in the Research Ethics Board-approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST).
- Informed consent was obtained.
- The patient was randomly assigned to the full field digital mammography (FFDM) arm of the trial.
- Both breasts demonstrate scattered fibroglandular densities.
- No suspicious masses, architectural distortions, or suspicious calcifications.
- Breast density: Heterogeneously dense, which may obscure small masses (>25% and <75% glandular).
- Negative study.
BIRADS 1 - Normal.
1 year screening recommended per OBSP. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram demonstrates symmetric firm breasts without any focal abnormality.
No mammographic evidence of malignancy, suspicious masses or suspicious calcifications.
No mammographic evidence of a suspicious focal asymmetry or architectural distortion.
No significant interval change from previous mammograms.

IMPRESSION:
BIRADS 2: Negative mammogram.
Routine screening with mammography is recommended in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM REPORT 
Clinical information provided: Patient enrolled in TMIST, FFDM arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The parenchymal density is heterogeneously dense, slightly asymmetric being more pronounced in the left breast. No discrete mass, suspicious calcification, or concerning asymmetry. 
IMPRESSION: 
Negative screening mammogram, benign findings. Routine screening recommended.
BIRADS Category 2 - Benign.
ADDENDUM:
Results discussed with patient. Due to the presence of heterogeneously dense tissue, supplemental screening with ultrasound may provide additional information. Patient advised to discuss risks and benefits with referring physician. ➡ *Not Stated*
- REPORT:
"SCREENING MAMMOGRAM
HISTORY: Annual screening mammogram. 
PROCEDURE: Bilateral full field digital mammography (FFDM).
COMPARISON: Previous mammogram from 1 year ago.
FINDINGS:
Breast composition is heterogeneously dense (C), which may obscure small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are no significant interval changes compared to the prior mammogram.
IMPRESSION: 
Negative screening mammogram. No mammographic evidence of malignancy.
BIRADS Category 2: Benign. 
Routine screening is recommended in 1 year.
ADDENDUM: No microcalcifications are identified in either breast.
" ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
Bilateral mammogram demonstrates no suspicious masses, architectural distortion, or asymmetries. No malignant type calcifications are identified. There are no mammographic findings to suggest malignancy.
IMPRESSION: BIRADS-1, negative screening mammogram. Routine annual screening is recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Comparison: None.
Calcifications: None.
Skin: Within normal limits.
Nipple: Within normal limits. 
Axilla: Within normal limits.
Pectoralis Muscle: Within normal limits.
Parenchymal Density: Scattered areas of fibroglandular density. 
Masses: None.
Architecture: Within normal limits. 
Dominant Lesion: None.

IMPRESSIONS: 
BIRADS Category 1 - Negative. Routine screening is recommended.

ADDENDUM:  
Breast Density: The breasts are heterogeneously dense, which may obscure small masses. 

ADDENDUM: 
Comparison to prior studies would increase diagnostic sensitivity, however no prior studies are available at this time. 

RECOMMENDATION:
Routine screening is recommended. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM AT AGE 63.
Clinical history: Patient presents for routine screening mammography. No concerning breast symptoms.
Compared to Previous: Yes, from 12 months ago.
Technique: Full field digital mammography.
Findings:
The breast tissue is heterogeneously dense. No significant masses, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
BIRADS-1, Negative. The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No mammographic findings that are concerning for malignancy. Recommend continued routine screening mammography per institutional guidelines.
ADDENDUM:
No mention of microcalcifications is made in the current report. ➡ *Not Stated*
- NORMAL SCREEN-FILM MAMMOGRAM.
This client participated in the TMIST study and was randomized to the digital mammography screening arm.
Findings were compared to prior full-field digital mammograms dated March 2021, which were available for review.
Parenchymal pattern consistent with scattered fibroglandular density. 
There are no suspicious masses, suspicious calcifications, architectural distortion or new areas of asymmetry in either breast.
No lymphadenopathy.
BIRADS Category 1 - Normal.
Follow-up mammogram in 1 year as part of the TMIST trial is recommended. The patient is aware.
ADDENDUM: This report was generated using computer-aided detection (CAD) software. The radiologist has reviewed and agrees with the findings described above. ➡ *Not Stated*
- FINDINGS:
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to previous mammogram from July 2019, there are no significant changes. The breast tissue is heterogeneously dense. There are no dominant masses, architectural distortions or suspicious calcifications. 
Skin is within normal limits. No lymphadenopathy.

IMPRESSION:
BIRADS 2 - Benign finding. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Technique: Full field digital mammography including left craniocaudal (LCC), left mediolateral oblique (LMLO), right craniocaudal (RCC), and right mediolateral oblique (RMLO) views.
Comparison: Yes.
Breast Composition: The breasts are predominantly fatty.
Findings: There are no suspicious masses, architectural distortions, or asymmetries. No suspicious calcifications are identified.
Impression:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year.

Addendum:
This patient is participating in an IRB-approved research study on abbreviated MRI for supplemental screening in women with dense breasts. Today's study is the baseline examination. Research images were obtained and archived under the research protocol. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Annual screening mammogram.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to previous exam dated July 4, 2020.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
The skin is unremarkable.  
IMPRESSION:
Normal screening mammogram. No concerning findings.
BIRADS Category 1 - Negative. Routine screening is recommended in one year. ➡ *Not Stated*
- TMIST FULL FIELD DIGITAL MAMMOGRAPHY SCREENING.
The patient is a participant in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), for which Research Ethics Board approval and informed consent were obtained. The patient was randomized to the full field digital mammography only arm.
Comparison to Prior Exams: Available.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
Recall: Routine screening is recommended in 1 year.
FINDINGS:
There are no suspicious masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
Negative mammogram.
BI-RADS Category 1: Negative. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to prior study of 3/15/2021.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Stable benign-appearing calcifications are noted in the right breast.
IMPRESSION:
BIRADS 2- Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- TASK RESPONSE:
REPORT: "NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No suspicious dominant masses, areas of architectural distortion or suspicious clustered microcalcifications are identified. The breast tissue is predominantly fibroglandular.
BIRADS-2: Benign findings. 
Addendum: Breast ultrasound recommended to further characterize nodule in upper outer quadrant of left breast. Final assessment pending additional imaging.
" ➡ *Not Stated*
- TMIST FFDM ARM MAMMOGRAPHY EXAM
The patient is enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomized to the full field digital mammography arm. Informed consent was obtained prior to the study.
Comparison: Previous mammograms available.
Breast composition: The breasts are heterogeneously dense, which may obscure small masses.
Findings: There are no suspicious masses, architectural distortions, or calcifications. Specifically, no clustered microcalcifications are identified.
IMPRESSION:
BIRADS Category 1: Negative.
Routine screening mammography is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
No significant masses, architectural distortion, or suspicious calcifications.
BIRADS-2.

Addendum: No microcalcifications are identified. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM. 
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
Bilateral mammography demonstrates scattered areas of fibroglandular density. No significant masses, architectural distortion or suspicious calcifications.
IMPRESSION: Negative screening mammogram. 
BIRADS-1. Routine screening recommended. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breasts are heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  
INDICATION:  Screening.  
TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.  
COMPARISON:  Comparison is made to prior exam dated July 15, 2020.  
FINDINGS:  
The breast tissue is heterogeneously dense which may obscure small masses.  
There are no dominant masses, architectural distortion or suspicious calcifications.  
Stable bilateral benign appearing calcifications are present.  
IMPRESSION:  
NEGATIVE SCREENING MAMMOGRAM.  BIRADS 2.  
1 YEAR ROUTINE FOLLOW UP RECOMMENDED.  
ADDENDUM:  No mention of microcalcifications for this patient in this report. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. The breasts are symmetrical, with no focal asymmetry, architectural distortion, or suspicious masses. No suspicious calcifications are identified.
BIRADS-2.
ADDENDUM: The patient's previous mammogram from 2 years ago was reviewed and shows stable breast tissue with no significant interval change. ➡ *Not Stated*
- OBSP TMIST FFDM MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.  
Compared to previous exam, there are no significant interval changes.
Breast tissue is heterogeneously dense. The breasts are not composed of 75% or greater of glandular tissue.
No suspicious masses, architectural distortions or suspicious calcifications are seen. 
BIRADS 2 - Benign findings. Routine screening recommended.
OBSP to recall client in 1 year. ➡ *Not Stated*
- TMIST FFDM - BIRADS Mammogram Report 

This woman is participating in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) at our facility. Informed consent was obtained and she was randomly assigned to the 2D full field digital mammography arm. 

Indication: Routine screening mammogram.

Technique: Standard CC and MLO views of both breasts were obtained using Hologic Selenia Dimensions 2D FFDM.

Comparison: Comparison is made to previous exam dated 12 months ago.

Findings:
The breasts are predominantly fatty, consistent with BI-RADS density category b. 

The mammograms show stable scattered areas of fibroglandular density bilaterally. No dominant mass, architectural distortion or suspicious calcifications are identified. 

There are no significant changes compared to the prior exam.

Impression: 
BI-RADS Category 1: Negative. 
Routine screening mammography is recommended in 1 year per the TMIST protocol. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
Breasts are predominantly fatty. No significant masses, architectural distortion or suspicious calcifications.
IMPRESSION: 
Negative screening mammogram. 
BI-RADS Category 1: Negative.
1 year screening recommended. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM

INDICATION: Routine screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are predominantly fatty, corresponding to a density of less than 25%. 
No suspicious masses, architectural distortion, or skin thickening.
No enlarged axillary lymph nodes.

IMPRESSION:
Negative screening mammogram. 
BIRADS Category 1 - Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- REPORT: "RESEARCH STUDY MAMMOGRAM  .
This study was performed as part of an REB approved research study: Digital Breast Tomosynthesis vs Standard Mammography, randomly assigned to standard mammography arm.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
Normal mammogram. No suspicious findings. 
Routine screening recommended in 1 year.
BIRADS 1.
Addendum: Details of microcalcifications are not stated as none are identified.
" ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

COMPARISON: 4/25/2014

INDICATION: Screening.

TECHNIQUE: Full field digital mammography including the craniocaudal and mediolateral oblique views of the breasts has been performed.

FINDINGS:
There is scattered fibroglandular tissue present.

There are no dominant masses, architectural distortions, or suspicious clustered microcalcifications.

There are no significant changes since the prior examination.

IMPRESSION:
BIRADS Category 1: Negative mammogram. Routine screening is recommended in one year.

BREAST DENSITY: The breasts are heterogeneously dense, which may obscure small masses. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
No suspicious masses, architectural distortion or asymmetries.
BIRADS-1/2. ➡ *Not Stated*
- TMIST FULL FIELD DIGITAL MAMMOGRAM (FFDM) OBS ARM.
This patient is enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomized to the full field digital mammography arm. Informed consent was obtained.
Compared to previous mammogram: No.
Breast density < 75%: Yes.
No suspicious mammographic findings.
BI-RADS Category 1 - Negative.
Routine annual OBSP screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM 
INDICATION: Annual screening mammography.
COMPARISON: Screening mammogram from 1 year ago.
BREAST DENSITY: The breasts are heterogeneously dense, which may obscure small masses (C).
FINDINGS:
The breast tissue is symmetrical. There are no dominant masses, suspicious calcifications, or other findings. 
No microcalcifications are present. No significant interval change compared to the prior study.
IMPRESSION:
NEGATIVE MAMMOGRAM. BIRADS 1 - NORMAL.
Routine screening mammography recommended in 1 year per institutional guidelines. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
This is a TMIST screening mammogram. Consent was obtained and the client was randomly assigned to undergo screening with full-field digital mammography only.
When compared to the previous exam dated **, no concerning interval changes are identified.
Breast tissue is heterogeneously dense. The breasts are not composed of more than 75% glandular tissue.
No suspicious masses, architectural distortions, or areas of abnormal parenchymal enhancement are seen.
No microcalcifications are described in this report.
BIRADS Category 1 - Negative. Routine screening is recommended. This is in keeping with the OBSP screening guidelines of 1 year for this client. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
There are no suspicious masses, architectural distortions, or areas of asymmetry. The breasts are symmetrical.
BIRADS-1/2.

ADDENDUM: No microcalcifications are identified. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: No.
Breast Density < 75%: Yes.
No concerning mammographic findings. The breasts are composed of scattered areas of fibroglandular density. There are no dominant masses, suspicious calcifications, or concerning asymmetries.
IMPRESSION:
BIRADS 1- Normal mammogram. Routine screening is recommended. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM.
Clinical history: Routine screening mammogram.  
Technique: Standard CC and MLO views of both breasts were obtained. 
Comparison: None available.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses. 
Findings:
There are no masses, architectural distortion, or suspicious calcifications.  
Impression:
Negative screening mammogram. 
BIRADS Category 1: Negative. Routine screening is recommended.  ➡ *Not Stated*
- NORMAL TMIST FFDM MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No mammographic findings that are concerning for malignancy. The breasts are composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion or suspicious calcifications.
BIRADS-2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- MAMMOGRAPHIC VIEWS OBTAINED:
Bilateral Screening Digital Mammogram:
Right CC; Left CC; Right MLO; Left MLO.
COMPARISON: 
Screening Mammogram 2 years prior.
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
No suspicious mass, architectural distortion, or asymmetry is identified.
No concerning calcifications are seen.
Bilateral benign-appearing lymph nodes are noted in the axillary regions.
Stable bilateral vascular calcifications.
No skin thickening or retraction.
IMPRESSION:
BIRADS Category 2: Benign.
Recommend routine screening in 1 year. ➡ *Not Stated*
- TMIST FFDM OBSP SCREENING MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
Scattered areas of fibroglandular density. No dominant masses, architectural distortion or suspicious calcifications.
BIRADS-2.

ADDENDUM:
Further review reveals no significant interval change compared to prior studies. The mammogram is within normal limits. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Bilateral CC and MLO views performed.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No suspicious masses, architectural distortions or focal asymmetries.
No mammographic findings that are concerning for malignancy.
BIRADS-2. ➡ *Not Stated*
- BILATERAL MAMMOGRAM TMIST BIRADS-1 REPORT FFDM ARM

This client was recruited to participate in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomly assigned to the full field digital mammography (FFDM) only arm of the study after providing informed consent as approved by the Research Ethics Board.

In comparison to previous mammograms, there are no concerning findings for malignancy identified. The breast parenchymal density is estimated to be less than 75% fibroglandular tissue.

No suspicious microcalcifications, masses, or other findings are identified in either breast.

BIRADS Category: 1 - Negative.

Routine annual screening with FFDM is recommended as per the OBSP guidelines. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breast tissue is heterogeneously dense.  
No dominant masses, architectural distortion or suspicious calcifications.
BIRADS-2.
ADDENDUM: Patient also had an MRI which was reviewed. There are no suspicious enhancing masses or non-mass enhancement to suggest malignancy. 
MRI BIRADS-2. 
OVERALL BIRADS-2. Annual screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  
INDICATION:  Routine screening.  

TECHNIQUE:  
Full field digital mammography, bilateral 2 view mammogram performed.
Compared to Previous: Yes.

FINDINGS:  
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 
No suspicious masses, architectural distortion or suspicious calcifications are seen. 

IMPRESSION:  
1. Heterogeneously dense breasts.
2. No mammographic evidence of malignancy. Recommend routine screening in 1 year.

BIRADS Category: 2 - Benign findings. 

ADDENDUM: No microcalcifications are identified in either breast. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  
INDICATION:  Screening.

TECHNIQUE:  Breast tomosynthesis and 2D synthesized views.  

COMPARISON:  None.

FINDINGS:  
The breasts are heterogeneously dense.  This may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion or suspicious calcifications.  

IMPRESSION:  
BIRADS 1 - Negative.  

1-year routine screening mammogram is recommended.  

ADDENDUM: 
The breasts are heterogeneously dense, which may obscure detection of small masses.  There are no suspicious calcifications.  
Clinical breast examination and breast self-examination are recommended. ➡ *Not Stated*
- MAMMOGRAPHIC FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.
No dominant masses, areas of abnormal architectural distortion or suspicious calcifications are identified.
There are no mammographic abnormalities to suggest malignancy. Bilateral benign-appearing calcifications are noted, likely representing vascular calcifications.
IMPRESSION:
Negative bilateral screening mammogram.
BIRADS Category 1: Negative. Routine screening recommended. ➡ *Not Stated*
- TMIST FFDM MAMMOGRAPHY REPORT.
This patient was enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and randomly allocated to the full field digital mammography arm.
Screening mammogram obtained with informed consent.
Breast density visually estimated at approximately 55-60% fibroglandular tissue.
Comparison with prior study: Yes.
No significant masses, suspicious calcifications, or other findings concerning for malignancy.
BI-RADS 2 Benign Findings. Recommend routine screening mammography.
Radiologist signature: Dr. Jane Smith ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density: Mixed fibroglandular and fatty, approximately 50%.
No suspicious masses, calcifications or architectural distortion.  
Stable benign calcifications noted in the left breast.
BIRADS-2.

Addendum: Scattered vascular calcifications noted bilaterally. No suspicious clustered microcalcifications identified. ➡ *Not Stated*
- SCREENING FULL FIELD DIGITAL MAMMOGRAM.
Compared to Previous: Yes.
Clinical Information: Routine screening.  
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
Dominant Pattern: scattered areas of fibroglandular density
OBSP to recall client in 1 year: No.
There are no mammographic findings to indicate malignancy. Specifically no suspicious microcalcifications, masses, architectural distortion or abnormal lymph nodes are identified.
BIRADS-1.  
Recommend routine screening with OBSP in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
INDICATION: Screening.
TECHNIQUE: Bilateral standard CC and MLO views obtained.
COMPARISON: None.
FINDINGS:
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
No suspicious masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS Category 1: Negative.
Routine screening recommended in 1 year. ➡ *Not Stated*
- OBSP TMIST FFDM ARM SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breasts are predominantly fatty replaced. 
No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No suspicious masses, architectural distortions or abnormal axillary lymph nodes.
The breasts are mildly heterogeneously dense. This may lower the sensitivity of mammography.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM REPORT.
Compared to Previous: No prior exams available for comparison.
Indication: Screening mammogram.
Views: Standard CC and MLO views of both breasts.
Breast Composition: The breasts are predominantly fatty.

Findings:
There are no dominant masses or suspicious calcifications. 
The breast tissue is symmetric bilaterally with no architectural distortion or skin thickening.
No axillary lymphadenopathy is appreciated.

IMPRESSION: 
Normal screening mammogram.
BIRADS Category 1 - Negative.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breasts are almost entirely fatty. No suspicious masses, architectural distortion or suspicious calcifications are seen.
BIRADS-2.
No previous exams available for comparison. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM

This client participated in screening mammography. 
Compared to Previous: Yes. 
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
No suspicious abnormalities or findings are identified. The breasts are heterogeneously dense. 
BIRADS 2: Benign findings. Routine screening recommended.

Addendum:
The technologist noted that this patient was anxious and had difficulty holding still during the exam. However, the images are of diagnostic quality. No mammographic evidence of malignancy. ➡ *Not Stated*
- MAMMOGRAPHY BREAST IMAGING REPORT 
FINAL IMPRESSION:
BIRADS 2 Benign
NORMAL EXAM
There are no suspicious masses, calcifications, or other findings. The breasts are almost entirely fatty.

Annual screening mammography is recommended in 1 year.

COMPARISON:  Comparison is made to prior examination of January 15, 2019.
TECHNIQUE:
A. Bilateral digital 2D mammogram including craniocaudal (CC) and mediolateral oblique (MLO) views. 
B. Tomosynthesis was not performed.
FINDINGS:
The breasts are almost entirely fatty (BI-RADS density category A). 
No suspicious masses, calcifications, asymmetries, or other abnormalities.
Benign calcifications are noted bilaterally.
No significant interval change compared to prior examination. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
The breasts are heterogeneously dense. No focal abnormality or architectural distortion. No significant interval change compared to previous study.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to prior: No.
Technique: Full field digital mammography, bilateral.
FINDINGS:
Breast Density: Heterogeneously dense, which may obscure small masses.
No suspicious masses, architectural distortion or definite calcifications.
IMPRESSION:
BIRADS Category 1: Negative.
Routine screening is recommended in 1 year.
***ADDENDUM***
Multiple groups of microcalcifications are stable compared to prior mammograms dating back to 2019. These likely represent benign calcifications. No suspicious morphology or pleomorphic calcifications are present to suggest malignancy. Recommend continued routine screening. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to prior exam dated 01/01/2021.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Stable bilateral scattered fibroglandular densities.
No significant interval change.
IMPRESSION:
BIRADS 2-Benign findings. Recommend routine screening in 1 year.
ADDENDUM: 
No new suspicious clustered microcalcifications are identified in either breast. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No dominant masses, architectural distortion or suspicious calcifications.
BIRADS-2. ➡ *Not Stated*
- DIAGNOSTIC TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: No. 
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breast parenchyma is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications are identified.
BIRADS-2. Annual screening recommended. ➡ *Not Stated*
- TMIST MAMMOGRAPHY REPORT
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography arm.  
Compared to previous examination: Yes
Breast composition: The breasts are heterogeneously dense, which may obscure small masses. 
There are no dominant masses, suspicious calcifications or architectural distortion. 

IMPRESSION: 
BIRADS Category 1: Negative.
Recommend routine screening in 1 year. ➡ *Not Stated*
- OBSP SCREENING MAMMOGRAM
Left and right craniocaudal and mediolateral oblique views.
Compared to previous: Yes.
Breast density >= 75%: No.
Breast density < 75%: Yes.
The breasts are heterogeneously dense.
IMPRESSION: 
No concerning masses, architectural distortions or areas of focal asymmetry. 
No mammographic features suggestive of malignancy are identified.
BI-RADS category 1 - Negative. Annual screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM
Indication: Screening.
Technique: Standard TMIST full field digital mammogram views only of both breasts.
Comparison: 2 years prior.
Findings:
The breasts are almost entirely fatty.
There are no dominant masses, architectural distortion or suspicious calcifications.
There are stable scattered vascular calcifications bilaterally.
Impression:
Negative bilateral screening mammogram.
BIRADS 2 - benign findings. Recommend routine screening in 2 years. ➡ *Not Stated*
- SCREEN-DETECTED ABNORMALITY IN RIGHT BREAST. BENIGN.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
There is an oval circumscribed mass in the upper outer quadrant of the right breast, likely representing a benign fibroadenoma. No suspicious calcifications, architectural distortion or other findings of concern.
BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.

This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.

Compared to Previous: Yes.  
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
OBSP to recall client in 1 year: No.

There are no suspicious masses, architectural distortions, or focal asymmetries. The breast tissue is predominantly fatty.

BIRADS-2.

ADDENDUM:
Upon further review, no clusters of microcalcifications are identified. The screening mammogram is negative. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density < 75%: Yes.
No significant interval change. Scattered areas of fibroglandular density. No suspicious masses, architectural distortion or mammographic findings concerning for malignancy are identified.  
BIRADS 2 - Benign. Routine screening is recommended. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
There are no suspicious masses, architectural distortions or microcalcifications.
The breasts are almost entirely fatty.
BIRADS-1.  
Routine screening mammogram is recommended in 1 year. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. The breasts are heterogeneously dense. No dominant masses, architectural distortion, or suspicious calcifications are seen.
BIRADS 2 - Benign findings. Routine screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Comparison: Previous studies.
INDICATION:  Screening.
COMPOSITION: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 
FINDINGS:
There are no suspicious masses, calcifications, or other findings.
IMPRESSION:
BIRADS Category 2: Benign.
Annual screening mammography is recommended. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: No prior.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No malignant-appearing masses, architectural distortion, or suspicious calcifications.  
No microcalcifications that are concerning for malignancy.  
The breasts are heterogeneously dense, which may obscure detection of small masses.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to previous screening mammogram from last year, there are no significant changes.
The breasts are almost entirely fatty.
There are no dominant masses, architectural distortion or suspicious calcifications.
A benign-appearing lymph node is noted in the right axilla.
BIRADS Category 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAPHY.
Compared to Previous: Yes. Most recent prior examination from 2 years ago.
Breast composition: The breasts are heterogeneously dense, which may obscure small masses.  
FINDINGS:
There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
Negative mammogram.
BIRADS-2: Benign findings. Recommend routine screening in 1 year.

ADDENDUM: 
This examination was double read as per screening protocol. There is agreement with the above interpretation and BIRADS category assignment. 
The interpreting radiologists agree there are no suspicious microcalcifications requiring further evaluation at this time. ➡ *Not Stated*
- TMIST FFDM ARM OBSP NORMAL SCREENING MAMMOGRAM.
The client participated in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) with Research Ethics Board approval. After providing informed consent, she was randomly assigned to the full field digital mammography only arm. 
Comparison to previous exam: Yes.
Breast density: Scattered fibroglandular densities (25-50%).
No suspicious mammographic findings. Specifically, no concerning masses, architectural distortion, or suspicious calcifications.
BI-RADS Category 1 - Negative. 
Routine annual screening mammogram recommended per OBSP guidelines. ➡ *Not Stated*
- SCREENING FULL FIELD DIGITAL MAMMOGRAM WITH LEFT CC VIEW.
No evidence of previous imaging for comparison.
Breast density: Heterogeneously dense. 
No abnormalities detected. No masses, architectural distortion or suspicious calcifications.  
Normal lymph nodes in the left axilla.
IMPRESSION:
BIRADS 1 - Negative. Routine screening is recommended. 
This is a screening exam only, additional diagnostic views would be required for a complete assessment. ➡ *Not Stated*
- SCREENING MAMMOGRAM

CLINICAL HISTORY: Routine screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were performed.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.

There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
BIRADS Category 2: Benign finding. Negative screening mammogram.

RECOMMENDED FOLLOW UP: Routine screening mammogram in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  
INDICATION:  Routine screening.

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.

COMPARISON:  Comparison is made to prior exam dated March 15, 2021.

FINDINGS:  
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Stable bilateral vascular calcifications are noted.

IMPRESSION:
No mammographic evidence of malignancy.  

BIRADS Category 2: Benign findings. Routine screening in 1 year is recommended. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No suspicious masses, architectural distortion or suspicious calcifications are identified. The breasts are symmetrical.
BIRADS-2. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
No suspicious masses or architectural distortions are seen. The breasts are symmetric with no skin thickening or retraction. No axillary lymphadenopathy is present.
BIRADS-2.
ADDENDUM: Parenchyma is primarily fibroglandular. No microcalcifications are identified. ➡ *Not Stated*
- OBSP REGIONAL AFFILIATE STUDY MAMMOGRAM
This client has voluntarily participated in an REB approved research study investigating the association between volumetric breast density and clinical outcomes using Volpara breast density software.  The study has been explained to the client and informed consent has been obtained.
Comparison: Yes.
Composition: The breasts are heterogeneously dense (51-75% glandular tissue).
Findings: Normal.
Management: The client should return for screening mammography in 1 year.
BIRADS 2
ADDENDUM
Please note that assessments are based on mammographic findings only. Clinical correlation and further investigations may be required based on clinical signs and symptoms. ➡ *Not Stated*
- SCREENING MAMMO BILATERAL.
HISTORY: 55 year old female.
TECHNIQUE: Hologic full field digital mammography. Standard bilateral mediolateral oblique and craniocaudal views obtained.
COMPARISON: None.
MAMMOGRAPHIC FINDINGS:
Breast parenchyma is heterogeneously dense, which may obscure small masses.
Right breast: No dominant masses or architectural distortion. No suspicious calcifications.
Left breast: No dominant masses or architectural distortion. No suspicious calcifications.
IMPRESSION: 
BIRADS category 1 - Negative. Recommend routine screening in 1 year.
ADDENDUM: No microcalcifications are seen in the current study. ➡ *Not Stated*
- SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
TECHNIQUE: Full field digital mammography including CC and MLO views of both breasts.
COMPARISON: Previous screening mammogram 1 year prior.
FINDINGS:
The breast tissue is predominantly fatty.
There are no suspicious masses, architectural distortions, or areas of abnormal calcifications.
No significant interval change compared to prior exam.
IMPRESSION:
ACR BI-RADS Category 1: Negative.
Routine annual screening mammogram is recommended. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breast tissue is heterogeneously dense. No suspicious masses, architectural distortion or suspicious calcifications are seen.
BIRADS-2. ➡ *Not Stated*
- FFDM BRCA SCREENING MAMMOGRAM.
The study is being performed as part of lifetime high risk screening program due to family history of breast cancer.
Compared to Previous: No.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
Recommend follow up in 1 year. 
No suspicious masses, architectural distortions, or microcalcifications are identified. 
BIRADS-1 Negative. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
No suspicious masses, architectural distortions, or mammographic findings that are concerning for malignancy.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of the bilateral breasts were obtained. 
FINDINGS: 
The breasts are almost entirely fatty.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are no mammographic features suspicious for malignancy.
IMPRESSION:
BIRADS Category 1: Negative mammogram.
Recommend routine screening in 1 year. ➡ *Not Stated*
- TMIST OFD Mammogram:
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.

Comparison: Prior routine screening mammography from approximately one year ago.

Findings: 
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 
No significant masses, architectural distortion, or suspicious calcifications.

IMPRESSION: 
Negative mammogram.
Recommend continued annual screening mammography in 12 months.

BI-RADS Category 1: Negative. ➡ *Not Stated*
- TMIST FFDM RIGHT AND LEFT BREASTS ARM SCREENING MAMMOGRAM

This patient was enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) with approval from the Institutional Review Board. She provided informed consent and was randomly allocated to the full field digital mammography (FFDM) arm of the study.

FFDM views of the right and left breasts demonstrate scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications are identified. The breast tissue is heterogeneously dense, which may obscure small masses. 

Comparison with prior mammograms from August 15, 2019 shows stable appearance of the breast tissue with no interval change.

IMPRESSION:
RIGHT AND LEFT BREASTS, BIRADS 2: Benign mammographic findings. Recommend routine screening in 12 months.

ADDENDUM:
Breast Density: The breasts are heterogeneously dense, which could obscure detection of small masses. For the purposes of the TMIST study, the breast density is categorized as less than 75%. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM. BILATERAL. 
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
Compared to previous exam dated 5/15/2019, there are no concerning interval changes.
ASSESSMENT:
BIRADS 2- benign finding. Heterogeneously dense breast tissue. 
Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Indication: Screening.
Technique: Standard CC and MLO views of both breasts were obtained.
Comparison: Compared to prior study from 1 year ago.
Findings:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
Impression:
BIRADS 2-BENIGN. Recommend continued routine screening in 1 year.
Addendum:
This screening mammogram was also reviewed by the senior radiologist and discussed at our group consensus conference. We are in agreement with the above interpretation. ➡ *Not Stated*
- SCREENING MAMMOGRAM
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to prior study from 2 years ago.
FINDINGS:
The breasts are almost entirely fatty.
No mass, architectural distortion, or asymmetry.
No suspicious microcalcifications.
Stable benign calcifications are noted in the right breast.
No significant interval change compared to prior mammogram.
IMPRESSION:
BIRADS 2- Benign findings. Recommend routine screening in 1 year.
BREAST DENSITY:
The breasts are almost entirely fatty, which may allow small masses to be more readily detected. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM
ADDITIONAL CLINICAL INDICATION: None

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

FINDINGS:
The breast tissue is predominantly fatty.
There are no suspicious masses, calcifications, or other findings.

IMPRESSION:
BI-RADS Category 1: Negative.
Recommend routine screening mammography in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Screening.  
TECHNIQUE: Tomosynthesis images were obtained bilaterally.
COMPARISON: Screening mammogram from [**2022-05-18**].

FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 
There are no concerning masses, architectural distortions, or suspicious calcifications.

IMPRESSION: 
NEGATIVE SCREENING MAMMOGRAM.  
BIRADS 1: Normal. Recommend routine screening in 1 year.

ADDENDUM:
Patient advised to continue monthly breast self-exams.
Breast density: C. Heterogeneously dense.
Lifetime risk of breast cancer: Average.
No personal history of breast cancer.
No family history of breast cancer in a first degree relative.
No known genetic predisposition to breast cancer. ➡ *Not Stated*
- SCREENING MAMMOGRAM IMPRESSIONS
Compared to Previous: Yes. Last screening mammogram was 2 years ago.
Breast Density: Heterogeneously dense breasts. Moderate fibroglandular densities are noted.
No findings suspicious for malignancy are identified. There are no concerning masses, areas of architectural distortion, or suspicious microcalcifications.
BIRADS Category 2: Benign findings. Routine screening is recommended.
Addendum: No grouped, fine linear, branching, pleomorphic or fine punctate microcalcifications are noted in either breast. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. The breasts are heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: No prior studies available for comparison.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.

BIRADS ASSESSMENT CATEGORY:  1.
FINDINGS: The breasts are heterogeneously dense. No significant masses, architectural distortion or suspicious calcifications are seen in either breast.  

IMPRESSION: Negative screening mammogram. Routine screening is recommended. BIRADS 1. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  FULL FIELD DIGITAL MAMMOGRAPHY.  
COMPARISON: Screening mammogram from May 15, 2021.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
FINDINGS: 
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, suspicious calcifications or architectural distortion.
Stable calcifications are noted in the left breast, unchanged from prior exams and likely representing stromal calcifications.  
IMPRESSION:
BIRADS 2: Benign findings. 
Annual screening mammography is recommended in 1 year.
ADDENDUM: 
The calcifications in the left breast do not have features suspicious for microcalcifications. No additional work-up is required at this time based on the mammographic appearance. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breasts are symmetrical. No dominant masses, architectural distortion or suspicious calcifications. 
BIRADS-1: Negative. ➡ *Not Stated*
- NORMAL MAMMOGRAM.
Technique: Bilateral digital diagnostic mammogram. 
Comparison: Outside prior mammograms from November 2017.
Findings: 
The breasts are heterogeneously dense. 
There are no new or enlarging masses, architectural distortion, or suspicious calcifications. 
Calcifications in both breasts are unchanged and benign in appearance.
No suspicious clusters of microcalcifications are present.
Impression:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
BI-RADS Category 1: Negative.
Addendum: No microcalcifications mentioned. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to screening mammogram dated 1/1/2020, the breast tissue is stable in appearance.  There are no suspicious masses, asymmetries, architectural distortions or suspicious calcifications.
Breast composition: The breasts are heterogeneously dense, which may obscure small masses (C).
BIRADS Category 2: Benign - routine screening recommended in 1 year. ➡ *Not Stated*
- BIRADS 1 SCREENING BILATERAL MAMMOGRAM
Compared with the patient's prior study dated  January 25, 2021, the breast tissue is mildly heterogenous and non dense but unchanged over the 2 years.  There are no suspicious microcalcifications, masses, asymmetries, distortions or significant skin thickening. No significant axillary lymph nodes are seen.
 
RIGHT BREAST:
No suspicious microcalcifications, masses, asymmetries or distortions are seen.  

LEFT BREAST:  
No suspicious microcalcifications, masses, asymmetries or distortions are seen.  
  
IMPRESSION:
Benign/negative bilateral screening mammogram. BI-RADS category 1.  

*Addendum: The remaining breast tissue imaged was unremarkable with no concerning microcalcifications or other suspicious mammographic findings. ➡ *Not Stated*
- TMIST Full Field Digital Mammogram:

This patient consented to the TMIST trial and was assigned to the full field digital mammography arm. Informed consent was obtained.

Compared to prior mammograms, there are no significant changes.

Breast tissue is heterogeneously dense, which may obscure small masses.

No suspicious masses, architectural distortion, or suspicious calcifications are seen in either breast.

There are no microcalcifications described in this report.

IMPRESSION:
BIRADS 2- Benign. Recommend continued routine screening mammography in one year. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Comparison Mammograms: Screening mammograms from 1 year ago.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
There are no dominant masses, suspicious calcifications, or other findings of concern.
BIRADS Category 1: Negative.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

TECHNIQUE: 
Standard full-field digital mammography was performed in the craniocaudal and mediolateral oblique projections. 

FINDINGS:
Breast density is predominantly fatty. There is no architectural distortion, suspicious calcifications, or concerning masses.

IMPRESSION:
No mammographic evidence of malignancy. Routine screening is recommended. BI-RADS Category 1.

ADDENDUM: This patient is enrolled in the TMIST trial and was assigned to the FFDM only study arm. Informed consent has been obtained. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
Stable, scattered fibroglandular densities. No suspicious masses, architectural distortions or dominant nodules. No significant interval change from the prior study.
BIRADS-2. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
No mammographic findings that are concerning for malignancy. There are no suspicious microcalcifications identified.
BIRADS-2.
ADDENDUM:
The recent mammogram is overall normal with no significant abnormalities detected. No clusters of microcalcifications are seen. Routine yearly screening is recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Indication: Routine screening.
Technique: Full field digital mammography was performed, including craniocaudal and mediolateral oblique images of both breasts.
Comparison: Previous outside screening mammograms from 2 years ago.
Findings:
The breasts are heterogeneously dense, which could obscure detection of small masses.
There are no suspicious masses, architectural distortions, or areas of asymmetry.
No mammographically suspicious axillary lymphadenopathy is identified.
The skin is within normal limits.
IMPRESSION:
BIRADS Category 2: Benign.
Recommendation: Routine screening mammography in 1 year. ➡ *Not Stated*
- ROUTINE SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Comparison made with screening mammogram dated June 8, 2019. No interval change. 
Density: The breast tissue is heterogeneously dense, with scattered areas of fibroglandular density. 
Calcifications: None.
Masses: No dominant mass, area of architectural distortion or suspicious calcifications are identified in either breast.
Lymph nodes: Normal.
Skin: Normal. 
Implants: None.
IMPRESSION:
1. No mammographic evidence of malignancy. Breast Imaging Reporting and Data System (BI-RADS) Category 1, Negative. Recommend routine screening in 1 year.
2. Heterogeneously dense breasts (scattered fibroglandular densities), which may obscure detection of small masses.
ADDENDUM:
The breast parenchymal density is heterogeneously dense, with scattered areas of fibroglandular density. This limits the sensitivity of mammography. Digital breast tomosynthesis was not performed. Consider supplemental screening with whole breast ultrasound or breast MRI. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Screening.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortions or suspicious calcifications.

No significant interval change is identified.

IMPRESSION:
BIRADS Category 2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- TMIST FFDM MAMMOGRAM - OBSP NORMAL.
This individual was randomly assigned to the full field digital mammography only arm as part of the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), an REB approved study for which informed consent was obtained. 
Comparison to prior mammograms: Yes.
Breast tissue composition visually estimated as consisting of scattered areas of fibroglandular density.
No dominant masses, architectural distortion or suspicious calcifications are identified.  
Routine annual OBSP screening recommended.
BIRADS 1 - Negative. ➡ *Not Stated*
- SCREENING MAMMOGRAM
INDICATION: Screening.
TECHNIQUE: Full field digital mammography (FFDM) – 2D Bilateral
COMPARISON: None available.
FINDINGS: 
Breast composition is heterogeneously dense, which may obscure small masses.
No masses, architectural distortion or suspicious calcifications.
No significant changes compared to prior exam.
IMPRESSION:
Negative screening mammogram. 
BIRADS-2 benign findings. Recommend routine screening in 1 year.
ADDENDUM:  This patient participated in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) after informed consent. She was randomized to the FFDM only arm. ➡ *Not Stated*
- FINDINGS:
There are no suspicious masses, architectural distortions, or asymmetries.
No mammographic findings concerning for malignancy.
No microcalcifications are identified.
There are scattered areas of fibroglandular density.
BIRADS: 2. ➡ *Not Stated*
- NORMAL TMIST FFDM MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
There are no suspicious masses, architectural distortion, or skin changes.  
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
BIRADS-1.
Addendum: Spoke with client regarding normal screening mammogram results. She will return for routine screening in 1 year per OBSP guidelines. ➡ *Not Stated*
- TMIST STUDY - MLO AND CC FULL FIELD DIGITAL MAMMOGRAPHY. OBSP.
Participant in the Tomographic Mammographic Imaging Screening Trial (TMIST), as per protocol randomized to the FFDM only imaging arm. Informed consent obtained.
FINDINGS:
Bilateral heterogeneously dense breast tissue.
No mammographic evidence of a suspicious dominant mass, architectural distortion, definite suspicious clustered microcalcifications or skin thickening.
Stable scattered calcifications bilaterally likely reflecting benign involution change.
Findings are benign. BIRADS 2.
IMPRESSION:
Benign mammography.
Recommend annual screening mammography in 1 year. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. No suspicious microcalcifications are identified.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM.  
Compared to previous mammogram from July 2019, no significant change.
Breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.  
No dominant masses, suspicious calcifications or architectural distortion.  
There are no suspicious mammographic findings.
IMPRESSION: 
BIRADS 2- Benign finding. Normal interval follow-up recommended in 1 year.
ADDENDUM:  Full field digital screening mammogram. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No suspicious masses, architectural distortion or asymmetries are seen. The skin and nipple are normal.  
BIRADS-2. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density: Predominantly fatty.
OBSP to recall client in 2 years: Yes.
No significant mammographic findings. No suspicious masses, calcifications or architectural distortion.  
BIRADS-2.

Addendum: Scattered vascular calcifications noted bilaterally. These are benign. ➡ *Not Stated*
- FINDINGS:
Bilateral digital 2D mammographic views show the breasts to be heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, architectural distortions or suspicious calcifications.
Benign vascular calcifications are noted bilaterally.
No significant interval change compared to prior exam.

IMPRESSION:
BIRADS 2 - Benign/Negative.
Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
COMPARISON: Prior screening mammogram obtained 1 year ago.
CLINICAL HISTORY: 55-year-old female, asymptomatic, here for annual screening mammogram.
FINDINGS:
Breast density: The breasts are heterogeneously dense (51-75% glandular), which may obscure small masses.
No dominant mass, suspicious calcifications, or other findings suspicious for malignancy are identified.
Calcifications: There are scattered benign appearing calcifications bilaterally. No suspicious morphology or distribution identified. Microcalcifications are not specifically commented on within the report narrative.  
IMPRESSION:
Negative screening mammogram, BI-RADS Category 1.
The patient is recommended to continue with annual screening mammography per guidelines.
ADDENDUM:
This is an addendum to the original report to explicitly state that no microcalcifications were mentioned in the initial report narrative. The benign scattered calcifications were noted, but there was no discussion of microcalcifications or their absence. ➡ *Not Stated*
- TMIST RIGHT DIGITAL SCREENING MAMMOGRAM.  
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to previous exam of March 15, 2021, no significant interval change. Both breasts are heterogeneously dense, which may obscure small masses.
No suspicious masses, architectural distortion or definite calcifications are seen in either breast.  
Benign calcifications are present bilaterally.
BIRADS 2- Benign findings. 
Routine OBSP screening recommended in one year. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: No previous comparison available.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
The breasts are heterogeneously dense. No suspicious masses, architectural distortion or suspicious calcifications are identified.  
IMPRESSION: 
BIRADS-2, Benign.

Addendum: There are no microcalcifications described in this report. Annual screening mammogram is recommended. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM.  

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available.

FINDINGS: 
The breasts are heterogeneously dense, which may obscure detection of small masses. 

There are no masses, architectural distortions, or suspicious calcifications.

IMPRESSION:
BIRADS 1 - Negative mammogram. Routine screening as per provincial guidelines is recommended. 

No suspicious findings. Microcalcifications are not mentioned in this report. ➡ *Not Stated*
- FINDINGS:
Bilateral screening mammogram was obtained. The breast tissue is heterogeneously dense, which may obscure detection of small masses.
No dominant mass, suspicious calcifications or architectural distortion.
There are no significant changes compared to prior study.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 12 months.
2. Heterogeneously dense breasts.

BIRADS Category 2 - Benign Finding. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION:  Screening. No reported symptoms.
COMPARISON:  None.
TECHNIQUE:  Standard CC and MLO views of both breasts.
FINDINGS:
The breasts are heterogeneously dense. No dominant mass, architectural distortion or definite calcifications are identified on either side. No apparent significant interval change since the prior study.
The pattern of fibroglandular tissue is symmetric. There are no new dominant masses. The skin appears within normal limits.
IMPRESSION:  
Bilateral heterogeneously dense breasts. Negative digital screening mammogram.
BI-RADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Not Stated*
- FINDINGS:
Screening mammographic examination consists of bilateral MLO and CC views only.
There has been no interval change compared to prior study in 2019.
Breast tissue is heterogeneously dense.  
No suspicious mammographic findings are identified. Specifically, no concerning masses, architectural distortions, or microcalcifications are seen.
BIRADS Assessment Category: 2 - Benign
IMPRESSION:
Benign/negative screening mammogram. Routine annual screening is recommended. ➡ *Not Stated*
- TMIST FFDM ARM TOMOSYNTHESIS MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography with 3D tomosynthesis arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
There are no concerning mammographic findings. The breast tissue is heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM
Normal mammogram. This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
There are no suspicious masses, architectural distortions, or asymmetries. No suspicious calcifications are identified.  
Compared to previous mammogram from October 17, 2020, the breasts remain stable in appearance with no interval development of suspicious findings.
Breast tissue is heterogeneously dense, which may obscure detection of small masses. 
BIRADS Category 1 - Negative. Routine screening is recommended. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breast tissue is heterogeneously dense. No suspicious masses, architectural distortions or asymmetries are identified.  
BIRADS-2. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Comparison was made to the previous mammogram of June 2020. 
The breast tissue is heterogeneously dense.
No dominant masses, architectural distortions or suspicious calcifications.
Benign calcifications are present bilaterally.
BIRADS Category 2 - Benign Findings.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- BIRADS-1 SCREENING MAMMOGRAM RIGHT AND LEFT.
Clinical Indication: Annual high risk screening.
Technique: Standard CC and MLO views of the right and left breasts were obtained.
Comparison: None.
Breast Composition: The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
Right Breast: No dominant mass, architectural distortion or suspicious calcifications.  
Left Breast: No dominant mass, architectural distortion or suspicious calcifications.
Impression:
BIRADS-1: Negative mammogram.
1 year routine screening mammography is recommended.
Post-Procedure Addendum: No additional cancers or findings were identified on this study. ➡ *Not Stated*
- SCREENING MAMMOGRAM 

INDICATION: Screening. 

COMPARISON: None.

TECHNIQUE: Full field digital mammography.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses. 

There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative screening mammogram. 
2. Recommend routine screening in 1 year.

BI-RADS Category 1: Negative. ➡ *Not Stated*
- "TMIST FFDM ARM OBSP MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. No significant interval change noted.
Breast Density > 75%: No. 
Breast Density < 75%: Yes. Scattered areas of fibroglandular density.
OBSP to recall client in 1 year: No. Routine screening recommended.
There are no suspicious masses, architectural distortions or mammographic findings concerning for malignancy. 
BIRADS-2: Benign findings. Recommend routine screening.

ADDENDUM: 
This is a screening mammogram. Diagnostic workup is not indicated based on the screening findings. If the patient has a palpable abnormality or other clinical concerns, further diagnostic imaging may be warranted for complete evaluation.
" ➡ *Not Stated*
- SITE: Sunnybrook Hospital
REASON FOR EXAM: Annual screening mammogram 
TECHNIQUE: Full field digital mammography, bilateral, craniocaudal and mediolateral oblique views.
COMPARISON: Prior mammograms from June 12, 2020.
BREAST DENSITY: The breasts are heterogeneously dense. This may lower the sensitivity of mammography.  
FINDINGS:
There are no suspicious masses, architectural distortion, or suspicious calcifications. 
Postoperative changes are noted in the left breast, stable.
IMPRESSION:
BIRADS 2 - Benign findings. Recommend continued routine screening mammography in 1 year.
ADDENDUM: No mammographic evidence of malignancy. No microcalcifications are identified. ➡ *Not Stated*
- NORMAL MAMMOGRAM.
RIGHT CC/MLO: There are scattered fibroglandular densities. There are no suspicious masses, calcifications or architectural distortion.
LEFT CC/MLO: There are scattered fibroglandular densities. There are no suspicious masses, calcifications or architectural distortion.
IMPRESSION:
BIRADS 2 - Benign. Recommend routine screening in 1 year.
Compared to previous: Yes.
Breast Density < 75%: Yes.
Addendum: Reviewed with Dr. Smith. No change to report. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Screening.
TECHNIQUE: Tomosynthesis and 2D synthesized mammographic images were obtained on GE 3Dimensions Digital Mammography System.
COMPARISON: None available.
FINDINGS:
Right breast:
The breast appears heterogeneously dense. No abnormalities are seen.
Left breast:
The breast appears heterogeneously dense. No abnormalities are identified.
Intramammary lymph nodes are present.
IMPRESSION:
1. No evidence of malignancy.
2. Heterogeneously dense breasts. The mammogram sensitivity is somewhat limited by the breast density. Annual screening is advised.
BIRADS Category 1: Negative.
RADIOLOGIST: Dr. Jane Smith
ADDENDUM: The patient elected to participate in the TMIST trial and was assigned to the full field digital mammography only arm. Informed consent was obtained as part of the Research Ethics Board approved study protocol. ➡ *Not Stated*
- FULL FIELD DIGITAL SCREENING MAMMOGRAM
Indication: Annual screening mammogram.
Patient demographics: 67 years old, female.
Technique: Standard CC and MLO views of both breasts.
Comparison: Prior mammogram from 1 year ago.
Breast Composition: The breasts are heterogeneously dense, which could obscure detection of small masses. Breast density < 75%.
Findings:
There are no dominant masses, architectural distortions or suspicious calcifications.
Scattered bilateral fibroglandular densities, consistent with patient's age and heterogeneous parenchymal pattern.  
Impression:
Negative screening mammogram. There are no mammographic findings that are concerning for malignancy.  
Breast Imaging Reporting and Data System (BI-RADS) Assessment Category 1: Negative. Recommend routine screening in 12 months. ➡ *Not Stated*
- BIRADS 2 ROUTINE SCREENING.
Bilateral digital 2D screening mammograms were obtained.
Compared to previous images from 08/20/2019. No suspicious interval change.
Breast tissue is heterogeneously dense, which may obscure detection of small masses. Scattered areas of fibroglandular density.
No radiographic evidence of malignancy. No suspicious microcalcifications, architectural distortion or abnormal asymmetry. Stable bilateral benign-appearing calcifications, likely vascular.
IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- FINDINGS:
Bilateral diagnostic screening mammography consisting of MLO and CC views was obtained.
The breast tissue is of scattered fibroglandular density.
There are no suspicious masses, architectural distortions or definite malignant-type calcifications in either breast.

IMPRESSION:
Bilateral BIRADS Category 1: Negative. 

RECOMMENDATION:
Recommend routine screening in 1 year. ➡ *Not Stated*
- Here is a generated radiology report that also returns the label "Not Stated for microcalcifications in the current report":

SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram. 
COMPARISON: Previous exam dated 5/15/2020.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained using Hologic Dimensions 3D mammography.
FINDINGS: 
The breasts are predominantly fatty.
No dominant masses, architectural distortion, or suspicious calcifications.  
Stable benign-appearing calcifications scattered in both breasts.
Stable post-surgical changes from prior right breast biopsy in the upper outer quadrant. Surgical clip in place.
IMPRESSION:
1. Negative mammogram. No evidence of malignancy. 
2. Predominantly fatty breast tissue.
RECOMMENDATIONS:
Recommend routine yearly screening mammography in 12 months. 
Continue monthly breast self-exams and inform clinician of any new breast symptoms or changes.

BI-RADS Category 1: Negative. ➡ *Not Stated*
- FINDINGS:
Bilateral digital 2D screening mammogram consisting of craniocaudal and mediolateral oblique views.
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable benign appearing intramammary lymph nodes are noted bilaterally.
Skin and nipple are unremarkable.

IMPRESSION:
1. Negative digital screening mammogram.  
2. Heterogeneously dense breasts (BI-RADS category C).

RECOMMENDATION:
Continued annual screening mammography is recommended.

ADDENDUM:
1. Breast density is subjectively determined to be heterogeneously dense (50-75% glandular tissue).
2. This examination is a screening examination. Additional diagnostic views are not included.
3. No prior mammograms are available for comparison.

ASSESSMENT CATEGORY:
BI-RADS Category 1: Negative. ➡ *Not Stated*
- MAMMOGRAM SCREENING, BILATERAL.
Compared to Previous: Yes.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses. 
FINDINGS: Scattered areas of fibroglandular density. There are no suspicious masses, calcifications, or other findings.
IMPRESSION:
NEGATIVE SCREENING MAMMOGRAM. BIRADS 1. Routine screening recommended in 1 year. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No mammographic findings that are concerning for malignancy. There are no concerning masses, architectural distortions, or calcifications.  
The breasts are almost entirely fatty.
BI-RADS Category 1 - Negative. 
Patient to return for annual screening mammogram. ➡ *Not Stated*
- NORMAL RIGHT SCREENING MAMMOGRAM.
Compared to Previous: No comparison.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
FINDINGS: The breasts are composed of scattered areas of fibroglandular density. No dominant masses, architectural distortion or suspicious calcifications are identified.  
IMPRESSION:
BIRADS-2: Benign.
Routine screening in 1 year is recommended. ➡ *Not Stated*
- "NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. The breasts are symmetric with no focal asymmetry, architectural distortion or suspicious masses. 
BIRADS-2.

ADDENDUM: 
Comparison was made to previous mammogram dated May 15, 2021 which showed no suspicious findings. The breast tissue is heterogeneously dense which could obscure detection of small masses. The patient is advised to continue routine screening mammography.
" ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram from April 2019.

FINDINGS:
The breasts are almost entirely fatty.

There are no dominant masses, areas of architectural distortion or asymmetries in either breast.

In the left breast at the 6 o'clock position, 8 cm from the nipple, there is a new 6 mm cluster of pleomorphic microcalcifications. This is a suspicious finding and warrants further evaluation.

No other significant interval change.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Biopsy should be considered.

RECOMMENDATIONS:
Stereotactic core biopsy is recommended for the new cluster of microcalcifications in the left breast at 6 o'clock, 8 cm from the nipple. The patient should call to schedule this procedure.

A 6 month follow up mammogram is recommended for the left breast following biopsy to evaluate for any additional changes. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening

TECHNIQUE: Standard CC and MLO views obtained bilaterally.

COMPARISON: None available

FINDINGS:

RIGHT BREAST:
The right breast is heterogeneously dense (BI-RADS category c).
There are no dominant masses, areas of architectural distortion or suspicious calcifications.

LEFT BREAST:
The left breast is heterogeneously dense (BI-RADS category c).
There are no dominant masses or areas of architectural distortion.
In the upper outer quadrant of the left breast, there is a small cluster of indeterminate microcalcifications at approximately 2 o'clock, 5 cm from the nipple. The cluster measures approximately 5 mm in greatest dimension. No other suspicious calcifications are identified.

IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. BI-RADS 1 - Negative.
2. Small cluster of indeterminate microcalcifications in the upper outer quadrant of the left breast, BI-RADS 0 - Incomplete. Additional magnification views recommended for further evaluation.

ADDENDUM:
Left magnification views demonstrate a cluster of pleomorphic microcalcifications in the upper outer quadrant, which are suspicious for malignancy. Stereotactic biopsy is recommended.

FINAL ASSESSMENT:
RIGHT: BI-RADS 1 - Negative
LEFT: BI-RADS 4 - Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- BILATERAL MAMMOGRAMS AND ULTRASOUND LEFT BREAST.   
INDICATION: Left bloody nipple discharge. 
FINDINGS: Scattered fibroglandular densities.
In the right breast there is a spiculated oval mass in the superomedial quadrant 6 cm from the nipple measuring 12 x 10 mm. Focally associated pleomorphic microcalcifications are noted. The mass appears to abut the pectoral muscle. Skin thickening is noted overlying the lesion. On ultrasound the mass is solid but with some cystic spaces, measures 13 x 11 x 14 mm and demonstrates increased peripheral vascularity.
There is an additional 8 mm well circumscribed oval mass at the 11:00 position right breast seen on ultrasound only.
In the left breast there are multiple round to oval smooth masses at 2:00 5cm from the nipple, the largest 10 x 8 mm. These likely represent fibroadenomas. No suspicious masses or calcifications are seen in the left breast. No significant axillary lymphadenopathy.
IMPRESSION:
Right breast: Highly suspicious spiculated mass with associated pleomorphic microcalcifications and skin thickening at 12:00 right breast. Additional probably benign mass at 11:00. BI-RADS 5. Right breast ultrasound guided biopsies recommended. 
Left breast: Negative. Likely fibroadenomas at 2:00 left breast. BI-RADS 2. 
BIOPSY ADDENDUM: Right breast 12:00 mass, core biopsy: Invasive ductal carcinoma, grade 2. ER+, PR-, HER2 negative. Right 11:00 mass FNA: Benign cyst contents. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. There are scattered areas of fibroglandular density.
In the left breast at 12:00 there is a 0.5 cm cluster of pleomorphic microcalcifications, appearing new compared to previous studies.
At 6:00 posterior depth left breast there is a 1.2 cm irregular hypoechoic mass with indistinct margins and internal vascularity. This corresponds to an area of architectural distortion on mammography.
In the left axilla there are two abnormal appearing lymph nodes with eccentric cortical thickening, measuring up to 1.5 cm in short axis diameter.
The remainder of the left breast is unremarkable. 
The right breast is unremarkable, with no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. New cluster of pleomorphic microcalcifications at 12:00 left breast, suspicious for DCIS. Biopsy recommended. BI-RADS 4B.
2. 1.2 cm irregular mass at 6:00 left breast with associated architectural distortion, highly suspicious for malignancy. BI-RADS 5. Biopsy recommended.
3. Abnormal left axillary lymph nodes suspicious for regional nodal metastases.  
4. Benign right breast. BI-RADS 2. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM:

INDICATION: Annual screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 

There are scattered areas of fibroglandular density bilaterally. 

No dominant masses, areas of architectural distortion or suspicious calcifications are appreciated in the right breast.

In the left breast, there is a group of amorphous microcalcifications in the upper outer quadrant anteriorly. The calcifications span an area of approximately 10 mm. No associated mass or distortion.

IMPRESSION:
1. Heterogeneously dense breasts.
2. New group of amorphous microcalcifications spanning 10 mm in the left upper outer breast, suspicious for ductal carcinoma in situ (DCIS). Additional evaluation with magnification views recommended. BI-RADS 4B.
3. Remainder of both breasts unremarkable. BI-RADS 2.

RECOMMENDATION:  
Left diagnostic mammogram with magnification views.

ADDENDUM - LEFT DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS:
The group of amorphous microcalcifications in the left upper outer quadrant span 12 mm on magnification views. Stereotactic biopsy is recommended. BI-RADS 4C. ➡ *Positive*
- SCREENING BILATERAL MAMMOGRAM.
Indication: Patient is 54 years old with family history of breast cancer.
FINDINGS: 
The breast tissue consists of scattered areas of fibroglandular density. Scattered benign appearing masses and cysts appreciated on left breast. No dominant mass or architectural distortion identified in the right breast.
There is a cluster of fine polymorphic calcifications in an anterior distribution in the left upper outer quadrant that appear somewhat round and scattered. This measures 14 x 7mm and is approximately 7 cm from the nipple. This appears suspicious for malignancy.  Otherwise there are no suspicious calcifications.
IMPRESSION:
Suspicious calcifications in the left breast in the upper outer quadrant  7 cm from the nipple in an anterior depth. BIRADS 4. Stereotactic biopsy recommended.
Right breast benign. BIRADS 2. ➡ *Positive*
- FINDINGS: The right breast is heterogeneously dense. No suspicious right breast masses, calcifications or other findings.

The left breast is heterogeneously dense. There are multiple regions of architectural distortion in the anterior depth left breast at the 12:00 position 5 cm from the nipple, 1:00 position 4 cm from the nipple and 2:00 position 3 cm from the nipple. Associated with these are small groups of suspicious pleomorphic microcalcifications. In the posterior depth at 6:00 10 cm from the nipple there is an irregular hypoechoic mass with indistinct margins measuring 18 x 12 mm with internal vascularity.

Ultrasound of the left axilla demonstrates 3 morphologically abnormal lymph nodes, the largest measuring 20 mm in maximum dimension with loss of the fatty hilum.

IMPRESSION:
BI-RADS 5: Highly suggestive of malignancy. 
- Multiple areas of architectural distortion with associated suspicious microcalcifications left breast, biopsy recommended. 
- Irregular hypoechoic mass left breast 6:00 position posterior depth.
- Abnormal left axillary lymph nodes suspicious for regional nodal metastases. 

BI-RADS 1: Negative right breast.

ADDENDUM: Core needle biopsies of the left breast 12:00 and 1:00 lesions performed under ultrasound guidance using a 14g needle. The 6:00 mass was also biopsied. Core biopsies of the left axillary lymph node performed. Specimens submitted to pathology. The patient tolerated the procedure well. Post-procedure mammograms demonstrate biopsy clip markers in good position with no immediate complications. ➡ *Positive*
- ULTRASOUND GUIDED CORE BIOPSY LEFT BREAST
CLINICAL HISTORY: 49-year-old female for biopsy of suspicious mass and calcifications left breast 12:00, 5 cm from nipple. 
PROCEDURE: Informed consent was obtained. The skin was prepped and draped in the usual sterile fashion. 1% lidocaine was used for local anesthesia. Using ultrasound guidance, an 18-gauge spring loaded core biopsy device was used to obtain 5 core specimens from the mass at 12:00 5 cm from the nipple. Specimen radiograph showed calcifications. The specimens were placed in formalin and sent to pathology. A tissue marker clip was deployed at the biopsy site. Post procedure mammograms showed the marker clip to be in good position. The patient tolerated the procedure well. There were no immediate complications. The patient was discharged in stable condition with post-biopsy instructions.  
IMPRESSION:
Ultrasound guided core biopsy of the suspicious mass with calcifications in the 12:00 region of the left breast 5 cm from the nipple was performed. Calcifications were identified on the specimen radiograph. A marker clip was placed at the biopsy site. The pathology results are pending.

ADDENDUM: 
PATHOLOGY DIAGNOSIS: LEFT BREAST - 12:00, 5 cm FN (US GUIDED CORE BIOPSY):
A. INVASIVE DUCTAL CARCINOMA, GRADE 2, FOCUS OF GRADE 3, MEASURING AT LEAST 0.9 CM (SEE COMMENT).  
B. FIBROCYSTIC CHANGES WITH CALCIFICATIONS.
COMMENT: The tumor size is a microscopic estimate only and may not reflect the actual cancer size. 
BIRADS Category: 6 Known biopsy-proven malignancy. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or skin thickening in either breast.
At the 12:00 position of the right breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. No other suspicious calcifications are seen in either breast.
IMPRESSION:
BI-RADS 0: Incomplete. Additional imaging evaluation with magnification views is needed to further assess the group of amorphous calcifications in the right breast at 12:00, 6 cm from the nipple. 
RECOMMENDATIONS:
Magnification views of the right breast are recommended for further evaluation of the microcalcifications. Please contact the patient to schedule this additional imaging. A final assessment will be made following completion of the diagnostic evaluation. ➡ *Positive*
- FINDINGS:
Bilateral digital mammography was performed including tomosynthesis.
Breast composition is heterogeneously dense. This may lower the sensitivity of mammography.
There is an irregular spiculated mass measuring approximately 15 mm seen in the left breast at the 2:00 position, 6 cm from the nipple. Associated with this mass is a cluster of fine pleomorphic microcalcifications extending over an area of approximately 8 mm.
Several additional groups of calcifications are seen in the upper outer left breast. At 1:00, 8 cm from the nipple, there is a group of amorphous calcifications. At 2:00, 10 cm from the nipple, there is a cluster of fine linear branching calcifications extending over an area of 7 mm. These are a cause for concern.
The right breast is unremarkable. No concerning masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
Highly suspicious findings left breast with irregular spiculated mass at 2:00 with associated pleomorphic calcifications, and additional suspicious calcifications upper outer left breast. Findings are highly suggestive of multifocal malignancy. Recommend tissue diagnosis. BIRADS 5.
Benign right breast. BIRADS 1.
ADDENDUM - ULTRASOUND LEFT BREAST:
Targeted left breast ultrasound was performed. 
The irregular mass at 2:00 in the left breast corresponds to an irregular hypoechoic mass with angulated margins and posterior acoustic shadowing, measuring 16 x 15 x 14 mm. 
The clusters of calcifications in the upper outer left breast are not associated with a mass. Ultrasound guided core biopsy is recommended for the 2:00 mass. Stereotactic biopsy is recommended for the calcifications. BI-RADS 5. ➡ *Positive*
- FINDINGS:
Bilateral diagnostic mammogram demonstrates heterogeneous dense parenchyma. There is no suspicious mass, architectural distortion, or asymmetry in either breast.

In the left breast at the 9:00 position, 8 cm from the nipple, there is an 8 mm cluster of fine pleomorphic microcalcifications. Additional magnification views in the mediolateral and craniocaudal projections confirm the presence of the microcalcifications. No other suspicious microcalcifications are identified in either breast.

Spot compression views of the left breast at 9:00 were performed, which demonstrate the cluster of microcalcifications more clearly. The microcalcifications are indeterminate in morphology and distribution, and further evaluation with stereotactic biopsy is recommended.

The skin is normal in thickness and appearance. No suspicious skin lesions are identified. The nipples are in profile and are normal in appearance. No suspicious calcifications are identified in the right breast.

IMPRESSION:
1. 8 mm cluster of indeterminate microcalcifications in the left breast at 9:00, 8 cm from the nipple. Stereotactic biopsy is recommended for further evaluation. BI-RADS 4.
2. Heterogeneously dense breasts, otherwise negative right breast. BI-RADS 1. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
No dominant masses, architectural distortion or definite asymmetries.
At the 12 o'clock position of the right breast, approximately 4 cm from the nipple, there is a new group of amorphous microcalcifications spanning an area of about 8 mm. This is a suspicious finding.
No concerning calcifications in the left breast.
There are stable bilateral axillary lymph nodes, likely benign.
Skin is unremarkable. No evidence of prior surgery.
IMPRESSION:
BIRADS 4: Suspicious grouped amorphous microcalcifications in the right breast at 12 o'clock position, 4 cm from the nipple. Biopsy is recommended for further evaluation. Pathology results should be correlated with imaging findings for appropriate management.
BIRADS 2: Benign findings in the left breast.
ADDENDUM: Stereotactic biopsy of the right breast microcalcifications at 12 o'clock was performed. Specimen radiograph confirmed calcifications were sampled. Final pathology results showed ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended for complete removal and definitive treatment of the DCIS. 6 month follow-up mammogram is advised to ensure stability and evaluate for any residual calcifications. ➡ *Positive*
- FINDINGS: Bilateral digital screening mammography was performed.

The breast tissue is heterogeneously dense, which may obscure small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant at the 2:00 position, approximately 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated mass or architectural distortion is identified.

The remainder of the left breast is unremarkable, with no dominant masses, areas of architectural distortion, or additional suspicious calcifications.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
Indeterminate cluster of microcalcifications in the left breast at 2:00, 6 cm from the nipple, as described above. Additional magnification views are recommended for further evaluation.

ADDENDUM:
Magnification views of the left breast were performed, confirming the presence of a cluster of fine pleomorphic calcifications at the 2:00 position of the left breast, approximately 6 cm from the nipple. The calcifications are more conspicuous on the magnification views and are concerning for ductal carcinoma in situ (DCIS).

REVISED IMPRESSION:
BI-RADS Category 4: Suspicious
Suspicious cluster of microcalcifications in the left breast at 2:00, 6 cm from the nipple, as described above. Stereotactic core biopsy is recommended for definitive diagnosis. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms were obtained. The breasts are heterogeneously dense, which may obscure detection of small masses.
In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.
In the left breast at the 10:00 position 6 cm from the nipple, there is a new group of fine pleomorphic calcifications spanning approximately 8 mm. Additional magnification views confirm these are suspicious and a stereotactic core biopsy is recommended for further evaluation. The remainder of the left breast is unremarkable.

IMPRESSION:
LEFT BREAST, NEW GROUP OF SUSPICIOUS CALCIFICATIONS AT THE 10:00 POSITION 6 CM FROM THE NIPPLE FOR WHICH A STEREOTACTIC CORE BIOPSY IS RECOMMENDED. BIRADS 4B.
RIGHT BREAST, NEGATIVE. BIRADS 1. ➡ *Positive*
- REPORT:
"DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS AND TARGETED RIGHT BREAST ULTRASOUND.

INDICATION: Callback for grouped calcifications upper outer quadrant right breast.

PREVIOUS OUTSIDE EXAMINATIONS: 2018 screening bilateral digital mammograms from the Womens College Hospital in Toronto were sent for comparison.

FINDINGS: The breasts are heterogeneously dense. 

A dominant asymmetry is noted on the craniocaudal view of the right breast which corresponds to a small area of relatively grouped and pleomorphic calcifications in the upper outer quadrant right breast 10 cm from the nipple. On the prior from 1 year ago there is a smaller area of pleomorphic calcifications in the same area suggesting some interim change. Additional magnification views confirm these findings. 

On targeted ultrasound at the 10:00 position 10 cm from the nipple there is a 7 x 4 x 5 mm irregular hypoechoic mass with indistinct margins and no posterior acoustic shadowing. There is no increased vascularity associated.  No other sonographic abnormalities or enlarged axillary lymph nodes.

In the left breast there are no suspicious mammographic findings or palpable sonographic correlates. BIRADS 2.

IMPRESSION: 
BI-RADS 4B: Suspicious pleomorphic calcifications upper outer quadrant right breast with an associated small spiculated mass on targeted ultrasound. Recommend image guided core biopsy. 

ADDENDUM - ULTRASOUND GUIDED CORE BIOPSY RIGHT BREAST:
Under ultrasound guidance using a 14 gauge spring loaded core biopsy device, 5 tissue cores were obtained from the sonographic lesion at the 10:00 position right breast. The cores were submitted in formalin. The post biopsy mammogram shows the clip to be in good position. The patient tolerated the procedure well. There were no immediate complications.

ADDENDUM - PATHOLOGY RIGHT BREAST BIOPSY:
The core biopsy samples demonstrate invasive ductal carcinoma, grade 2, with associated high grade DCIS. Calcifications are present. Estrogen receptor positive, Progesterone receptor negative, Her2/neu equivocal (2+) by immunohistochemistry. Additional FISH testing for Her2 gene amplification will be performed. The patient will be notified and referral made to surgical oncology.
"
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAPHY WITH TARGETED LEFT MEDIOLATERAL OBLIQUE MAGNIFICATION VIEWS.

CLINICAL: Left breast lump, palpable at 3:00.

FINDINGS:
The breasts are heterogeneously dense (Type C) which could obscure small masses. 

There are no suspicious findings or dominant masses in the right breast. BI-RADS 1 right.

In the left breast, there is an indistinct asymmetry in the upper outer quadrant on the CC view only, corresponding to the palpable abnormality at 3:00. Targeted magnification views in the mediolateral oblique projection show pleomorphic microcalcifications spanning 12 mm at the site of the asymmetry. No dominant mass, architectural distortion, or skin thickening.

IMPRESSION:
BI-RADS 1 right breast. Negative.
BI-RADS 4B left breast. Suspicious pleomorphic microcalcifications spanning 12 mm at site of palpable abnormality, 3:00, 6 cm from nipple. Biopsy recommended for definitive diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Patient is a 45-year-old female presenting for routine screening mammogram.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

LEFT BREAST:
There is no evidence of a mass, architectural distortion, or abnormal calcifications.

RIGHT BREAST: 
There are scattered groups of amorphous microcalcifications in the upper outer quadrant, ranging in size from 0.2-0.4 mm. The largest cluster measures approximately 5 mm. No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:
RIGHT BREAST: Scattered groups of amorphous microcalcifications in the upper outer quadrant. Additional magnification views recommended for further evaluation. BI-RADS 0, incomplete. 

LEFT BREAST: No mammographic evidence of malignancy. BI-RADS 1, negative.

ADDENDUM: Additional magnification views of the right breast were performed. The amorphous microcalcifications in the upper outer quadrant are more apparent, with the largest cluster measuring 7 mm. Stereotactic biopsy is recommended for tissue diagnosis. BI-RADS 4, suspicious. ➡ *Positive*
- FINDINGS:

Bilateral digital mammograms demonstrate scattered fibroglandular densities. 

In the left breast at 10:00, 3 cm from the nipple, there is a small group of amorphous microcalcifications spanning 6 mm. No associated mass or architectural distortion is identified.

The right breast demonstrates no suspicious masses, architectural distortion, or calcifications.

There are no appreciable skin thickening or nipple retraction bilaterally.  No axillary lymphadenopathy is appreciated.

IMPRESSION:
1. Small group of amorphous microcalcifications left breast 10:00, 3 cm from the nipple, spanning 6 mm. Additional magnification views recommended for further evaluation. BI-RADS 0.
2. No suspicious findings right breast. BI-RADS 2.

ADDENDUM: Magnification views were performed on the left breast in the mediolateral and craniocaudal projections following administration of a skin marker over the area of interest. The marker is seen to overlie a small group of indistinct microcalcifications measuring approximately 6 mm in size at 10 o'clock, 3 cm from the nipple. Stereotactic biopsy is recommended for tissue diagnosis. Final BI-RADS assessment left breast is BI-RADS 4. BI-RADS assessment right breast remains BI-RADS 2. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Bilateral CC and MLO views.

FINDINGS:
The breasts are predominantly fibroglandular in composition.

There is no evidence of a mass, architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant on the MLO view, spanning approximately 1 cm. This cluster was not present on the CC view. No associated mass or distortion is identified.

The remainder of the left breast is unremarkable.

IMPRESSION: 
RIGHT BREAST: BI-RADS 1 - Negative.

LEFT BREAST: BI-RADS 0 - Incomplete assessment. A cluster of pleomorphic microcalcifications is identified in the left upper outer breast on the MLO view, not seen on the CC view. Additional mammographic views are recommended for further evaluation. 

RECOMMENDATION: Spot compression magnification views of the left upper outer breast are recommended to further characterize the cluster of microcalcifications. ➡ *Positive*
- FINDINGS:
There are scattered areas of fibroglandular density. 
No suspicious masses, architectural distortion or suspicious calcifications are seen in either breast.
At the 10 o'clock position of the right breast there is a group of amorphous calcifications spanning approximately 3 mm. 

IMPRESSION:
BI-RADS Category 0: Incomplete. 
Additional imaging evaluation with magnification views is recommended for further characterization of the group of amorphous calcifications at the 10 o'clock position of the right breast.

ADDENDUM:
Right breast magnification views demonstrate a cluster of amorphous calcifications at the 10 o'clock position.

IMPRESSION (ADDENDUM):
BI-RADS Category 4B: Suspicious - Moderate suspicion for malignancy. Biopsy should be considered. Small cluster of amorphous calcifications right breast requires tissue diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (BI-RADS C). 

There are scattered benign calcifications bilaterally. Additional magnification views in the left CC view demonstrate a new cluster of indeterminate microcalcifications in the upper outer quadrant, approximately 8 cm from the nipple. The calcifications are irregular in shape and distribution, spanning an area of about 1 cm. No associated mass, distortion or skin changes are identified.

The right breast is unremarkable, with no suspicious masses, calcifications, asymmetries or skin changes. 

IMPRESSION:
BI-RADS 0: Indeterminate cluster of microcalcifications in the left breast at the 2:00 position, 8 cm from the nipple. Additional imaging evaluation with magnification views is recommended for further characterization. If the microcalcifications remain suspicious on additional imaging, a stereotactic biopsy should be considered.

BI-RADS 2: Benign findings in the right breast. Recommend continued routine screening in 1 year.

ADDENDUM:
Magnification views of the left breast demonstrate a cluster of pleomorphic microcalcifications spanning approximately 1 cm at the 2:00 position, 8 cm from the nipple. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS). 

REVISED IMPRESSION:
BI-RADS 4: Suspicious cluster of microcalcifications in the left breast. Stereotactic core biopsy is recommended for definitive diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening. 

COMPARISON: None.

FINDINGS:
Both breasts show scattered fibroglandular densities. No significant masses, architectural distortion, or suspicious calcifications are seen in the right breast.

In the left breast, there is a new cluster of pleomorphic microcalcifications in the upper outer quadrant, measuring approximately 8 mm in extent. No associated mass, architectural distortion or skin changes are identified.

IMPRESSION:
RIGHT BREAST: NEGATIVE. BI-RADS 1.
LEFT BREAST: Cluster of pleomorphic microcalcifications in the upper outer left breast, suspicious for malignancy. BI-RADS 4B. 

RECOMMENDATION:
Stereotactic core biopsy of the microcalcifications in the upper outer left breast is recommended for further evaluation. ➡ *Positive*
- SCREENING MAMMOGRAM AND SPOT COMPRESSION VIEWS LEFT BREAST
HISTORY: Annual screening mammogram
TECHNIQUE:  Bilateral screening mammogram performed with additional spot compression views of left breast.
FINDINGS: The overall density of the breasts is heterogeneously dense which can obscure small masses.
There are no suspicious findings in the right breast.
In the left upper outer breast at 2:00 11 cm from the nipple there are clustered amorphous calcifications. On spot compression mag views this measures 4 mm. There is associated ill-defined density. An ultrasound was performed and did not show an underlying mass. A marker clip was placed at this site for localization.
IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous microcalcifications in the left upper breast at 2:00 with no ultrasound correlate. Stereotactic biopsy advised. Marker placed at site of concern.
BI-RADS 2. Benign. Right breast.
ADDENDUM: Left stereotactic core biopsy showed ductal carcinoma in situ, intermediate grade, with comedonecrosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Bilateral digital mammography was performed including CC and MLO views of each breast. 

COMPARISON: Previous outside screening mammogram from 11/16/2020.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

Magnification views of both breasts taken to further evaluate vague asymmetries seen on screening views. In the left breast at 3 o'clock, 7 cm from the nipple, there is a new cluster of indeterminate microcalcifications spanning approximately 8 mm. This cluster was not present on the prior mammogram. 

In the right breast, the magnification views show no significant change in the previously noted benign appearing calcifications scattered throughout the breast.

No suspicious axillary lymphadenopathy.

IMPRESSION:
New cluster of indeterminate microcalcifications in the left breast at 3 o'clock, 7 cm from the nipple, spanning 8 mm. Recommend diagnostic mammographic workup of the left breast for further evaluation, to include spot compression and magnification views. BI-RADS 0.

Stable right breast. BI-RADS 2. 

Recommend routine annual screening mammography in 1 year, with the new cluster of microcalcifications in the left breast to be addressed in the interim with diagnostic evaluation. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available

FINDINGS:
Both breasts show a scattered fibroglandular tissue pattern.

At the 10:00 position of the RIGHT breast, approximately 6 cm from the nipple, there is a cluster of pleomorphic microcalcifications spanning an area of about 8 mm. No associated mass, architectural distortion or skin changes are seen. 

The LEFT breast shows no suspicious masses, calcifications, architectural distortion or skin thickening.  

No abnormal axillary lymph nodes are seen on either side.

IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality in the RIGHT breast - biopsy should be considered. 
BI-RADS Category 1: Negative in the LEFT breast.

RECOMMENDATION: 
Stereotactic core biopsy of the cluster of microcalcifications in the RIGHT breast at 10:00, 6 cm from nipple.
Routine screening of the LEFT breast with mammography in 1 year. ➡ *Positive*
- BILATERAL MAMMOGRAMS.
History: Annual screening. No palpable lumps palpable. No personal history of breast cancer.
FINDINGS: The breasts are heterogeneously dense.
In the right breast there are no suspicious masses, architectural distortion or suspicious calcifications. Skin and areola are unremarkable. 
There is a group of pleomorphic calcifications measuring approximately 6 mm in the left breast at 12:00 4 cm from the nipple. These were not present on prior imaging from 2018 and are new. No associated mass or distortion.
Bilateral magnification images were obtained. The calcifications in the left breast upper outer quadrant are confirmed. They are irregular in shape, varying in size from punctate to 0.5 mm. A biopsy should be considered.
Axillae, skin and nipples are normal bilaterally.  
SUMMARY.
New group of suspicious microcalcifications upper outer left breast. BI-RADS 4B. Stereotactic or tomosynthesis guided biopsy is recommended. 
Right breast is normal. BI-RADS 1.
ADDENDUM:
The patient returned for stereotactic biopsy of the group of calcifications at left breast 12:00. Specimen radiograph confirmed calcifications. A clip was placed at the biopsy site. There were no complications. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening. No current breast complaints.

COMPARISON: Screening exam dated 9/15/2019 at this facility.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

There is an oval equal density circumscribed mass in the right breast at 2 o'clock posterior depth measuring approximately 9 mm, most consistent with an isolated fibroadenoma.

A few scattered punctate and amorphous microcalcifications are noted in the upper outer quadrant of the left breast which are new since the prior study, distributed over an area spanning approximately 2 cm. These are indeterminate but likely benign. Follow-up magnification views are recommended for further evaluation.

The visualized axillary regions appear unremarkable bilaterally.

Unchanged benign-appearing vascular calcifications are noted bilaterally.

IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation needed. Magnification views are recommended to further characterize the scattered microcalcifications in the left breast upper outer quadrant.

ADDENDUM:
Magnification views of the left breast confirm a cluster of indeterminate microcalcifications in the upper outer quadrant. Stereotactic biopsy is recommended for tissue diagnosis. BI-RADS Category 4. ➡ *Positive*
- FINDINGS:
Bilateral diagnostic mammogram with coned compression magnification views of the left breast. 

Mammography shows the breasts are heterogeneously dense, which may obscure detection of small masses. 

No suspicious findings are seen in the right breast. There are no dominant masses, areas of architectural distortion or suspicious calcifications.

In the left breast at the 11:00 position, 7 cm from the nipple, there is an 11 mm irregular mass with indistinct margins. No other discrete masses or areas of architectural distortion are identified in the left breast. 

Magnification views of the left breast demonstrate a grouped distribution of pleomorphic microcalcifications extending over an area of approximately 2 cm, centered at the 10:00 position, 5 cm from the nipple. A few additional scattered calcifications are seen in the vicinity.

IMPRESSION:
1. Right breast: No mammographic evidence of malignancy. BI-RADS 1. Recommend routine screening in 1 year.

2. Left breast:
- 11 mm irregular mass at 11:00, 7 cm from the nipple, suspicious for malignancy. 
- Pleomorphic microcalcifications in a grouped distribution over 2 cm at 10:00, 5 cm from the nipple, suspicious for malignancy. BI-RADS 4C. 

RECOMMENDATIONS:
Ultrasound-guided core needle biopsy is recommended for the irregular mass and stereotactic core needle biopsy for the pleomorphic calcifications in the left breast, to exclude malignancy. 

This is a BI-RADS 4C assessment of the left breast, with a high suspicion of malignancy based on the imaging findings described above. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS
INDICATION: Follow up after lumpectomy and radiation therapy for right breast cancer.
COMPARISON: Mammograms from 1 year ago.
FINDINGS:
The breasts are heterogeneously dense. Scattered benign-appearing calcifications in both breasts.
The skin is normal with no significant thickening or retraction. No mammographic evidence of a discrete mass lesion in either breast. There are stable post-surgical changes in the upper outer right breast. Metallic surgical clips are noted at the lumpectomy site.
Additional magnification views in the craniocaudal projection demonstrate a new group of amorphous microcalcifications in the upper outer left breast at the 2 o'clock position. These span an area of approximately 1 cm and are located 6 cm from the nipple. This cluster was not present on the prior mammogram.
Remainder of the left breast is unremarkable.
IMPRESSION:
New group of suspicious microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple. BIRADS 4B. Stereotactic core biopsy is recommended for diagnosis. Stable post-lumpectomy changes and calcifications right breast. BIRADS 2. ➡ *Positive*
- IMPRESSION:
Bilateral diagnostic mammography demonstrates a new cluster of suspicious pleomorphic microcalcifications in the upper outer left breast at the 2:00 position, 6 cm from the nipple, spanning approximately 6 mm. This cluster is highly concerning for ductal carcinoma in situ (DCIS). Additionally, there is a 1.2 cm irregular mass with spiculated margins at the 10:00 position of the left breast, 3 cm from the nipple, highly suspicious for invasive ductal carcinoma.

Ultrasound of the left breast confirms an irregular hypoechoic mass with angular margins at 10:00, corresponding to the mammographic finding and measuring 11 x 9 x 8 mm. Ultrasound-guided core needle biopsy is recommended for definitive diagnosis. 

No suspicious masses, architectural distortion, or microcalcifications are identified in the right breast.

BIRADS ASSESSMENT:
Right breast: BI-RADS 1, Negative.
Left breast: BI-RADS 4C, Highly suspicious for malignancy. Biopsy recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Screening mammogram.  

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Tomosynthesis was performed bilaterally.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (BI-RADS density C).

In the right breast, there are no dominant masses, areas of architectural distortion or suspicious calcifications. 

In the left breast at the 10:00 position approximately 4 cm from the nipple, there is a new 0.4 cm cluster of fine pleomorphic calcifications. This is new compared to patient's most recent screening mammogram from 2 years ago at an outside facility. No dominant masses or areas of architectural distortion are identified in the left breast.

There are no abnormal axillary lymph nodes bilaterally.

IMPRESSION: 
Left breast: New group of suspicious microcalcifications in the 10:00 position of the left breast, BI-RADS 4B. Stereotactic biopsy is recommended for tissue diagnosis. 

Right breast: No mammographic evidence of malignancy, BI-RADS 1.

ADDENDUM: The patient returned for stereotactic biopsy of the cluster of calcifications in the left breast at the 10:00 position, using an 8G vacuum-assisted biopsy device. Specimens were obtained and sent to pathology. Post-biopsy mammograms show that the area of concern has been adequately sampled. Pathology results are pending. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. 
There are scattered areas of fibroglandular density bilaterally. No dominant mass, architectural distortion, or suspicious calcifications are seen in the right breast.
In the upper outer quadrant of the left breast, there is a new group of pleomorphic calcifications spanning approximately 8mm. The calcifications are irregular in size and shape, and some appear to be branching. No associated mass or architectural distortion is seen. 
The remainder of the left breast is unremarkable.

IMPRESSION:
New group of suspicious calcifications in the left upper outer breast, BIRADS 4B. Stereotactic biopsy is recommended for further evaluation. The remainder of the exam is benign, BIRADS 2.

ADDENDUM: 
The patient returned for stereotactic biopsy of the group of calcifications in the left upper outer breast. Specimen radiograph confirmed calcifications within the core samples.
Final pathology results: Ductal carcinoma in situ, intermediate grade, cribriform type. ➡ *Positive*
- ULTRASOUND GUIDED LEFT BREAST BIOPSY
 
REASON FOR EXAMINATION: 67-year-old female with suspicious 8 mm cluster of microcalcifications at 2:00, 6 cm from the nipple on screening mammogram. 

TECHNIQUE: With the patient in the supine position, the skin overlying the area of interest was prepped and draped in the usual sterile fashion. 1% lidocaine was used for local anesthesia. Using ultrasound guidance, an 18-gauge spring-loaded biopsy device was used to obtain 5 core biopsy specimens from the area of microcalcifications in the 2:00 axis of the left breast, 6 cm from the nipple. The specimens were placed in formalin and sent to pathology for analysis. A titanium clip was deployed at the biopsy site. The skin was then cleansed and Steri-strips were applied for closure. The patient tolerated the procedure well. No immediate complications.

IMPRESSION: 
Successful ultrasound guided core biopsy of left breast microcalcifications at 2:00, 6 cm from the nipple. Pathology results pending. Recommend mammographic follow up of biopsy clip in 6 months.

BIRADS 4B: Suspicious abnormality, biopsy should be considered.

ADDENDUM - PATHOLOGY RESULTS: 
Left breast, 2:00, 6 cm from nipple, core biopsy:
- Fibrocystic change with focal atypical ductal hyperplasia and microcalcifications.
- Invasive mammary carcinoma is not identified.

Final BIRADS Category 4B based on imaging and histologic findings.
Discordant pathology, recommend surgical consultation and excisional biopsy for definitive diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening mammogram. No prior mammograms available for comparison.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
There is a small cluster of amorphous microcalcifications at the 12 o'clock position in the left breast, approximately 6 cm from the nipple. This cluster measures approximately 5 mm in greatest dimension. There are no associated masses, architectural distortion or skin changes.
IMPRESSION:
BI-RADS 0: Incomplete. There is a small cluster of amorphous microcalcifications in the left breast, which is new since the prior exam and requires additional imaging evaluation with magnification views for complete characterization.
RECOMMENDATION: Recall for diagnostic evaluation with magnification views of the left breast at the 12 o'clock position.
ADDENDUM - DIAGNOSTIC MAMMOGRAM LEFT BREAST, MAGNIFICATION VIEWS:
Magnification views of the left breast at 12 o'clock position demonstrate a cluster of amorphous microcalcifications, which are more apparent compared to the screening views. This cluster measures approximately 8 mm. No associated mass or architectural distortion.
FINAL ASSESSMENT: BI-RADS 4B: Suspicious abnormality. Biopsy should be considered.
RECOMMENDATION: Stereotactic core biopsy is recommended for definitive characterization of this cluster of microcalcifications in the left breast. ➡ *Positive*
- BILATERAL MAMMOGRAM:
COMPARISON: None.
INDICATION: Annual screening. No concerns.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Both breasts are heterogeneously dense which may lower the sensitivity of mammography.
There are scattered areas of fibroglandular density bilaterally.
No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast.
At the 9 o'clock position of the right breast, approximately 6 cm from the nipple, there is a small group of amorphous calcifications spanning an area of about 6 mm. These are new since any prior mammograms. Additional magnification views in the CC and lateral projections confirm these are distinct calcifications and not vascular calcifications or artifact.
Skin is unremarkable. No evidence of significant axillary, intramammary, or internal mammary adenopathy. 
IMPRESSION:
BI-RADS Category 0: Incomplete. Small group of amorphous calcifications in the right breast as described above. Additional magnification views confirm these to be true calcifications. Recommend biopsy for definitive characterization as the presence of a grouped distribution of amorphous calcifications raises the possibility of early ductal carcinoma in situ (DCIS). 
Remainder of bilateral mammogram is negative.
ADDENDUM: Stereotactic guided vacuum assisted biopsy of the group of right breast calcifications was performed. Specimen radiograph confirmed calcifications were sampled. Final pathology showed benign changes with microcalcifications and no evidence of malignancy. 
With biopsy proven benign calcifications, the final BI-RADS category is 2: Benign findings. Recommend annual screening mammography. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM WITH MAGNIFICATION VIEWS

CLINICAL HISTORY: 52 year old woman, annual screening mammogram. No palpable lumps or nipple discharge. Maternal aunt with breast cancer at age 59.

TECHNIQUE: CC and MLO views of bilateral breasts. Magnification views of right breast.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

RIGHT BREAST:
There is a new cluster of pleomorphic calcifications in the upper outer quadrant on the CC and MLO views, with more than 10 calcifications in a linear distribution measuring approximately 8mm. Magnification CC and MLO views confirm these suspicious calcifications. No dominant masses, architectural distortion or skin thickening.

LEFT BREAST: 
No dominant masses, suspicious calcifications, architectural distortion or skin thickening.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for the cluster of suspicious calcifications in the right upper outer quadrant.
BI-RADS Category 1: Normal left breast.

ADDENDUM:
Stereotactic core biopsy of the right breast calcifications was performed. Pathology results showed ductal carcinoma in situ, high nuclear grade with comedonecrosis. ➡ *Positive*
- BILATERAL MAMMOGRAM

INDICATION: Screening mammogram

COMPARISON: None.

FINDINGS:

Both breasts show scattered areas of fibroglandular density. Breast parenchymal density is heterogeneously dense, which may obscure detection of small masses.

Right breast: Negative.

Left breast: 
There is a 6 mm cluster of amorphous microcalcifications in the upper outer quadrant, 8 cm from the nipple at the 1:00 position. No associated mass, distortion or asymmetry.

There are no dominant masses, areas of architectural distortion or other suspicious calcifications in either breast. 

Skin is normal bilaterally with no evidence of thickening or retraction. 

IMPRESSION:

BI-RADS Category 0: Incomplete. Need additional imaging evaluation. 
Stereotactic biopsy is recommended for the cluster of amorphous microcalcifications in the left breast upper outer quadrant as described above.

BI-RADS Category 1 for the right breast.

ADDENDUM: Patient was informed of the results and the need for additional evaluation of the left breast. She is agreeable to stereotactic biopsy. This will be scheduled for the earliest available appointment. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was performed.
The breast parenchymal density is heterogeneously dense, which may obscure detection of small masses.
There are scattered areas of fibroglandular density bilaterally.
No dominant masses, areas of architectural distortion or suspicious calcifications are seen in the right breast. 
In the left breast, there is a new cluster of pleomorphic microcalcifications in the upper outer quadrant at the 2:00 position, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. The morphology and distribution are suspicious for ductal carcinoma in situ.
No other suspicious calcifications, masses, or areas of architectural distortion are identified in the left breast.
No mammographic evidence of multicentric or multifocal disease.
No skin thickening or retraction. Benign calcifications are noted bilaterally.
Both nipples are in profile and appear unremarkable. No significant axillary lymphadenopathy.

IMPRESSION:
New cluster of pleomorphic microcalcifications upper outer left breast at 2:00, suspicious for ductal carcinoma in situ. Biopsy is recommended. BI-RADS 4B.
Remainder of left breast is BI-RADS 2. Benign/negative right breast, BI-RADS 1.

ADDENDUM:
Ultrasound guided core biopsy of the microcalcifications in the left breast at 2:00 was performed. Specimen radiograph confirms calcifications within the cores. Final pathology results are pending. Post-biopsy mammogram shows biopsy clip in good position at the site of prior calcifications. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Routine screening. No current breast concerns.
COMPARISON: None.
FINDINGS:
Both breasts are almost entirely fatty.
There are no dominant masses, architectural distortion, or abnormal calcifications in either breast.
In the right breast at the 10:00 position 6 cm from the nipple, there is a group of amorphous calcifications in a segmental distribution, spanning an area of approximately 2 cm. Additional magnification views confirm these are suspicious calcifications.
The left breast is unremarkable.
No axillary, supra- or infraclavicular adenopathy.
IMPRESSION:
Right breast: Suspicious segmental distribution of amorphous calcifications at 10:00, 6 cm from the nipple, spanning 2 cm. BIRADS 4. Stereotactic core biopsy recommended for tissue diagnosis.
Left breast: Negative. BIRADS 1. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAM WITH ULTRASOUND OF THE RIGHT BREAST. 

HISTORY: Annual screening. No breast complaints.

FINDINGS:
The breasts are heterogeneously dense (C). This may lower the sensitivity of mammography.

Left Breast:
No suspicious mammographic findings.

Right Breast:
There is a 6 mm cluster of indeterminate calcifications at the 10:00 position, 5 cm from the nipple. Spot compression magnification views show these to be pleomorphic. 

Targeted right breast ultrasound at site of the mammographic calcifications shows no correlate mass or dilated duct. 

Incidental finding of a 9 mm simple cyst at 2:00, 3 cm from the nipple. 

IMPRESSION:
1. New cluster of pleomorphic calcifications at 10:00 in the right breast. Stereotactic biopsy is recommended. BIRADS 4B. 
2. No suspicious findings left breast. BIRADS 2.

ADDENDUM - Stereotactic biopsy right breast calcifications:
Specimen radiograph confirms calcifications. Pathology demonstrates ductal carcinoma in-situ, intermediate grade, cribriform type.

BIRADS 6. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: 51 year-old female presenting for high risk screening. Mother had breast cancer at age 45.

COMPARISON: None.

FINDINGS:
Both breasts are heterogeneously dense (C), which may obscure detection of small masses.

Right breast:
There are no dominant masses, architectural distortions or suspicious calcifications.

Left breast: 
There are segmental fine linear calcifications in a ductal distribution at the 3 o'clock position of the left breast middle depth, spanning an area of about 2 cm. There are no associated masses, architectural distortions or skin changes.

IMPRESSION:
BIRADS 4B: Suspicious calcifications in the left breast requiring stereotactic biopsy for further evaluation. 

BIRADS 1: Negative right breast.

RECOMMENDATION:  
Stereotactic core biopsy of the segmental calcifications in the left breast at 3 o'clock position is recommended for further evaluation. The patient should return to the Breast Center for the biopsy with the Mammography Requisition Form. ➡ *Positive*
- FINDINGS:
Bilateral digital diagnostic mammography was performed, including CC and MLO views of both breasts. Additionally, spot compression magnification views were obtained of the left breast.

The breast tissue is heterogeneously dense, which may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a focal asymmetry in the upper outer quadrant on the MLO view, which persists on spot compression magnification views. Associated with this asymmetry, there are fine pleomorphic calcifications in a segmental distribution, extending over an area of approximately 2 cm. No associated mass or architectural distortion is identified.

The remainder of the left breast is unremarkable.

IMPRESSION:
1. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography.
2. Focal asymmetry with segmental fine pleomorphic calcifications in the left upper outer breast, suspicious for ductal carcinoma in situ (DCIS). Additional evaluation with ultrasound is recommended for further characterization and to guide biopsy. BI-RADS 4C.
3. No suspicious findings in the right breast. BI-RADS 1.

MANAGEMENT:
Biopsy recommended for 4C finding in the left breast. Follow-up recommended per BI-RADS category on the right. 

ADDENDUM - ULTRASOUND LEFT BREAST:
Targeted ultrasound was performed on the area of focal asymmetry and calcifications in the left upper outer quadrant. A correlating area of irregular ductal ectasia was identified, confirming the mammographic finding. Ultrasound-guided core biopsy was performed with pathology pending. ➡ *Positive*
- REPORT:
"MAMMOGRAPHIC VIEWS BILATERAL BREASTS AND ADDITIONAL VIEWS RIGHT BREAST.  
History: Recalled following screening mammogram for abnormality seen right breast. No palpable lumps.
FINDINGS: The breasts are almost entirely fatty.
There is a focal asymmetry noted in the upper inner quadrant of the right breast and this corresponds to a 15 mm solid nodule on ultrasound. Ultrasound also shows associated clustered microcalcifications. The left breast is unremarkable. BI-RADS 4B right and BI-RADS 1 left.
Impression: Suspicious finding right breast with associated microcalcifications. Ultrasound guided core biopsy recommended."
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.

There are scattered areas of fibroglandular density.

At the 6 o'clock position of the right breast, 6 cm from the nipple, there is a new 8 mm cluster of pleomorphic calcifications. Additional magnification views confirm the presence of this cluster. 

No dominant masses, architectural distortion, or suspicious calcifications are seen in the left breast.

There are no abnormalities of the skin, subcutaneous tissues, or nipple-areolar complexes. No axillary lymphadenopathy.

IMPRESSION:
New cluster of pleomorphic calcifications in the right breast as described above, suspicious for malignancy. Additional imaging evaluation with diagnostic views of the right breast is recommended. BI-RADS 4B.

Normal left breast. BI-RADS 1.

ADDENDUM: The patient was contacted and informed of the results. She will present for additional right breast diagnostic mammographic views and ultrasound. If the calcifications are confirmed, stereotactic core biopsy will be recommended. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS RIGHT BREAST.
History: Screening mammogram.
FINDINGS: The breasts are heterogeneously dense. There are no suspicious left mammographic findings.
In the right breast, there is a 7 mm cluster of round pleomorphic calcifications spanning 2 cm located at the 9:00 position 6 cm from the nipple.
Magnification views of the right breast demonstrate the pleomorphic calcifications as described above. No associated mass, asymmetry, or architectural distortion.
No skin thickening or nipple retraction.
SUMMARY:
7 mm cluster of pleomorphic calcifications at 9:00 position of right breast, suspicious for DCIS. Recommend image-guided core biopsy. BI-RADS 4B.
No suspicious findings left breast. BI-RADS 2.
ADDENDUM: The patient returned for stereotactic core biopsy of the right breast calcifications. Pathology revealed ductal carcinoma in situ, intermediate grade with comedonecrosis. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS BILATERAL.
History: Patient reports no changes. No nipple changes or discharge. No breast lumps or skin changes.
Comparison: None.
FINDINGS:
The breasts are heterogeneously dense which may obscure small masses. 
There are scattered areas of benign appearing calcifications in both breasts. 
No spiculated masses, suspicious calcifications or areas of architectural distortion are identified in either breast. 
There are no suspicious findings in the right breast. BIRADS 2.
In the upper outer left breast there is a 6 mm cluster of indeterminate microcalcifications at the 2:00 position, 6 cm from the nipple. Spot compression magnification views confirm these as pleomorphic calcifications in a segmental distribution. These are new since the prior study and suspicious for ductal carcinoma in situ (DCIS). A stereotactic biopsy is recommended for further assessment. BIRADS 4B. 
The visualized axillary regions appear unremarkable. 
IMPRESSION:
1. New 6 mm cluster of suspicious pleomorphic microcalcifications upper outer left breast as described, BIRADS 4B. Stereotactic biopsy recommended. 
2. Routine biennial screening for the right breast. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was obtained.
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in the right breast. 
In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant at the 2:00 position, 8 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No other suspicious calcifications are identified in the left breast. 
There are no dominant masses or architectural distortion in the left breast.
IMPRESSION:
Left breast: New cluster of suspicious microcalcifications in the upper outer quadrant (2:00 position, 8 cm from nipple), measuring 8 mm. BI-RADS 4B. Stereotactic biopsy is recommended for further evaluation.
Right breast: No suspicious mammographic findings. Recommend continued routine screening in the right breast. BI-RADS 2. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Screening mammogram. Patient has a family history of breast cancer.

FINDINGS:
The breasts are heterogeneously dense (type C). This may lower the sensitivity of mammography.

Left breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.

Right breast: 
At the 9 o'clock position, 6 cm from the nipple, there is a 6 mm cluster of pleomorphic microcalcifications. These were not present on the prior study. Additional magnification views confirm these are suspicious.
There are no dominant masses or architectural distortion.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Biopsy should be considered. 

RECOMMENDATION: 
Right breast biopsy is recommended for the cluster of pleomorphic microcalcifications at 9 o'clock, 6 cm from the nipple. The patient should follow up with the referring physician to discuss results and management options. ➡ *Positive*
- REPORT:
"RIGHT DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS AND RIGHT BREAST ULTRASOUND

History: Focal right breast pain. 
Compared to previous outside mammograms from 2019.

FINDINGS: The breasts are heterogeneously dense. 
There is a round 7 mm noncalcified circumscribed mass at 6:00 3 cm from the nipple in the right breast which appears stable compared to the previous mammogram and corresponds to a simple cyst sonographically. 
There is a cluster of finely pleomorphic calcifications across a 6 mm span at 2:00 6 cm from the nipple in the right breast which appear slightly larger in number than on the previous mammogram. Magnification views confirm these calcifications.
Sonographically this corresponds to an irregular 6 x 4 x 6 mm hypoechoic mass with mild peripheral vascularity.
No other suspicious mammographic or sonographic findings in the right breast. 
The left breast was not imaged.

IMPRESSION: 
New cluster of pleomorphic calcifications at 2:00 right breast with a correlating sonographic finding, suspicious for malignancy. BIRADS 4C. Recommend stereotactic core biopsy. 

ADDENDUM: 
Stereotactic core biopsy of the right breast calcifications was performed. Pathology showed ductal carcinoma in situ, intermediate nuclear grade with comedonecrosis."

LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammogram from 10/15/2020.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

Stable bilateral circumscribed masses, likely benign. Largest measures 8 mm in the right breast at 10:00, 6 cm from the nipple.

At the 3:00 position of the left breast, 6 cm from the nipple, there is a new group of amorphous calcifications spanning an area of approximately 6 mm. Additional magnification views in the craniocaudal and mediolateral projections confirm these calcifications.

Stable bilateral axillary lymph nodes, with no abnormal morphology.

IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation with magnification views is needed to better characterize the new group of amorphous calcifications in the left breast at 3:00, 6 cm from the nipple. These calcifications are suspicious and will require biopsy if confirmed on additional imaging.

ADDENDUM (Post Magnification Views):
Magnification views of the left breast at the 3:00 position, 6 cm from the nipple, demonstrate a cluster of amorphous calcifications spanning 7 mm. These calcifications are new and suspicious for malignancy. Stereotactic biopsy is recommended.

FINAL ASSESSMENT: BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS
HISTORY: Annual screening mammogram. No prior mammograms available for comparison.
FINDINGS:
Both breasts are primarily fatty replaced. 
There are no dominant masses, architectural distortion or skin thickening in either breast.
In the upper outer right breast, there is a new cluster of amorphous microcalcifications spanning an area of approximately 1.0 cm at the 10:00 position, 5 cm from the nipple. Additional magnification views in the CC and ML projections confirm the presence of the microcalcifications. 
The remainder of the right breast is unremarkable.
The left breast is unremarkable. No suspicious microcalcifications, masses, or areas of architectural distortion.
IMPRESSION:
BI-RADS Category 0: Incomplete: Need additional imaging evaluation. 
New cluster of amorphous microcalcifications in the upper outer right breast. Recommend diagnostic mammogram workup with spot compression magnification views for further evaluation.
ADDENDUM:
The patient returned for additional diagnostic imaging of the right breast, including spot compression magnification views of the area of microcalcifications in the upper outer right breast.
The magnification views demonstrate a cluster of amorphous microcalcifications spanning approximately 10 mm at the 10:00 position of the right breast, 5 cm from the nipple. The microcalcifications are new compared to the patient's prior mammograms.
IMPRESSION (ADDENDUM):
BI-RADS Category 4B: Suspicious abnormality, biopsy should be considered. Suspicious cluster of amorphous microcalcifications in the right breast. Stereotactic biopsy is recommended for further evaluation. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY
INDICATION: Annual screening mammogram.
COMPARISON: None.
FINDINGS:
RIGHT BREAST:
The breast tissue is heterogeneously dense.
There are no dominant masses, architectural distortions, or significant asymmetries.
There are no suspicious calcifications.
LEFT BREAST:
The breast tissue is heterogeneously dense.
At 12:00, 6 cm from the nipple, there is a suspicious 1.1 cm spiculated mass with associated pleomorphic microcalcifications spanning approximately 2 cm. 
At 6:00, 4 cm from the nipple, there is an indeterminate 7 mm hypoechoic mass.
There are no dominant masses elsewhere, architectural distortions, or additional significant asymmetries.
IMPRESSION:
1. Highly suspicious findings in the left breast, BI-RADS 5. Further evaluation with diagnostic mammography, spot compression views, ultrasound, and biopsy is recommended.   
2. Benign right breast, BI-RADS 1.
ADDENDUM:  
LEFT DIAGNOSTIC MAMMOGRAPHY AND ULTRASOUND
Additional magnification and spot compression views of the left breast demonstrate the spiculated mass with pleomorphic microcalcifications at 12:00. Ultrasound shows an irregular hypoechoic mass with indistinct margins and posterior shadowing, corresponding to the mammographic mass. The mass measures 11 x 8 x 9 mm.  
Ultrasound also shows the 7 mm hypoechoic mass at 6:00, which demonstrates circumscribed margins.
IMPRESSION:
1. Highly suspicious mass left breast 12:00, BI-RADS 5. Ultrasound-guided core needle biopsy is recommended.
2. Probably benign mass left breast 6:00, BI-RADS 3. Short term follow-up is recommended. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms were obtained.
The breast tissue is heterogeneously dense which may obscure detection of small masses.
No spiculated masses, areas of architectural distortion or suspicious areas of skin thickening.
There are scattered areas of benign-appearing calcifications in both breasts.
At the 12:00 position of the right breast, approximately 5 cm from the nipple, there is a new group of amorphous calcifications spanning an area of about 1 cm. This is a suspicious finding. Magnification views confirm the presence of a cluster of pleomorphic calcifications at this site. 
The remainder of the left breast is unremarkable.

IMPRESSION:
New cluster of pleomorphic microcalcifications at the 12:00 position of the right breast, about 5 cm from the nipple, spanning 1 cm, suspicious for ductal carcinoma in situ (DCIS). Stereotactic core needle biopsy is recommended. BIRADS 4B.
Benign calcifications and heterogeneously dense breast tissue, otherwise negative bilateral screening mammogram. BIRADS 2 for the left breast. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was performed.
The breasts are almost entirely fatty.
There are no suspicious masses, architectural distortion or dominant masses in either breast. 
In the right breast at the 12:00 position 4 cm from the nipple there is a small cluster of indeterminate microcalcifications measuring about 5 mm in size. No previous mammograms are available for comparison. 
The left breast is unremarkable. 
SUMMARY:
Small cluster of indeterminate microcalcifications right breast at 12:00 position 4 cm from the nipple. These are new since no previous mammograms are available. Recommend magnification views of this area and possibly biopsy depending on the appearance on the magnification views. 
BIRADS 4 right breast.
BIRADS 1 left breast. ➡ *Positive*
- MAMMOGRAM, BILATERAL WITH TOMOSYNTHESIS AND ULTRASOUND OF RIGHT BREAST
INDICATION: Right breast lump.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
In the left breast, there are no dominant masses, architectural distortion or suspicious calcifications. 
In the right breast at 10 o'clock, 4 cm from the nipple, there is an 8 mm equal density mass. Ultrasound shows this to be a complex cystic and solid mass with irregular margins, measuring 9 x 8 x 7 mm. It demonstrates internal vascularity.
In the right breast, a group of amorphous calcifications is identified at the 2 o'clock position, 6 cm from the nipple. This spans an area of approximately 7 mm. Additional magnification views confirm the presence of these suspicious calcifications.
The remainder of the right breast is unremarkable. No axillary, supra or infraclavicular adenopathy.
IMPRESSION:
RIGHT BREAST, 10 O'CLOCK, COMPLEX CYSTIC AND SOLID MASS WITH IRREGULAR MARGINS, HIGHLY SUSPICIOUS FOR MALIGNANCY. ULTRASOUND GUIDED BIOPSY IS RECOMMENDED. BI-RADS 4C.  
RIGHT BREAST, 2 O'CLOCK, 7 MM GROUP OF SUSPICIOUS AMORPHOUS CALCIFICATIONS. STEREOTACTIC CORE BIOPSY IS RECOMMENDED. BI-RADS 4B.  
LEFT BREAST, BI-RADS 1, NEGATIVE. ➡ *Positive*
- FINDINGS:

Bilateral diagnostic mammogram was obtained.

The breast tissue is heterogeneously dense which may obscure small masses.

There are scattered areas of fibroglandular density. 

At the 12:00 position of the left breast there is a small group of amorphous microcalcifications measuring approximately 4 mm, located about 4 cm from the nipple.

No dominant mass, architectural distortion or skin thickening.

Benign calcifications are noted in both breasts. 

IMPRESSION:

BIRADS 4: Suspicious abnormality in the left breast. Small group of amorphous microcalcifications in the left breast at 12:00, 4 cm from the nipple. This is a suspicious finding and warrants biopsy. Recommend stereotactic biopsy for tissue diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS

CLINICAL HISTORY: Annual screening mammogram.  

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.  

There are no suspicious masses, architectural distortion or suspicious calcifications in either breast.

In the right upper outer quadrant there is a new cluster of pleomorphic calcifications spanning 9 mm at 10:00, 5 cm from the nipple. Additional magnification views confirm these are suspicious and new since the prior mammogram. 

Stable bilateral circumscribed masses consistent with benign fibroadenomas, largest measuring 6 mm in the right breast at 9:00.  

IMPRESSION: 
BI-RADS 4: Suspicious cluster of microcalcifications in the right breast at 10:00. Stereotactic biopsy is recommended. The patient was informed and agrees with this plan. Please send a stereotactic biopsy requisition.

ADDENDUM: Stereotactic biopsy of the right breast calcifications at 10:00 demonstrated ductal carcinoma in situ, intermediate grade with necrosis.  

ADDENDUM 2:  Specimen radiograph confirmed the presence of calcifications. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Annual mammographic screening exam.

TECHNIQUE: Standard CC and MLO views obtained bilaterally.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure small masses.

There are grouped amorphous calcifications in a segmental distribution in the upper outer quadrant of the right breast at approximately the 10:00 position, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated masses, architectural distortions or skin thickening.

No significant masses, calcifications, architectural distortions or skin thickening in the left breast.

IMPRESSION:
New cluster of amorphous calcifications upper outer right breast, suspicious for malignancy. Additional magnification views recommended for further evaluation.

BIRADS 4: Suspicious abnormality in the right breast. Biopsy should be considered.

ADDENDUM:
The patient returned for diagnostic mammogram and ultrasound. Magnification views of the right breast at the 10:00 position confirm a 9 mm cluster of amorphous calcifications. Ultrasound of this area demonstrates a correlating 7 mm irregular hypoechoic mass with indistinct margins and internal vascularity.

BIRADS 4C: Highly suggestive of malignancy. Biopsy recommended. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS AND TARGETED ULTRASOUND OF THE RIGHT BREAST

CLINICAL HISTORY: 58-year-old female with new palpable lump in the right breast upper outer quadrant.

COMPARISON: Prior mammograms from 2 years ago.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There is a new 12 mm irregular mass at the 10 o'clock position of the right breast, approximately 6 cm from the nipple, corresponding to the palpable abnormality described by the patient. Targeted ultrasound demonstrates an irregular hypoechoic mass with angular margins and posterior acoustic shadowing, highly suspicious for malignancy.

Additionally, there is a new group of fine pleomorphic calcifications spanning approximately 8 mm at the 9 o'clock position of the right breast, 4 cm from the nipple. These calcifications are highly concerning for malignancy.

The left breast demonstrates stable fibroglandular tissue and a few scattered benign-appearing calcifications. No suspicious masses, architectural distortion, or suspicious calcifications are identified in the left breast.

IMPRESSION:
1. Highly suspicious 12 mm mass at 10 o'clock position of the right breast, corresponding to palpable finding. Recommend biopsy. BI-RADS 5.

2. Highly suspicious group of pleomorphic calcifications at 9 o'clock position of the right breast. Recommend stereotactic biopsy. BI-RADS 4C.

3. Stable left breast. BI-RADS 2. ➡ *Positive*
- BILATERAL MAMMOGRAM

COMPARISON: 2 years prior

INDICATION: 51-year-old female presents for routine screening mammogram. No known palpable masses, pain, or nipple discharge.

FINDINGS:
RIGHT BREAST:
Heterogeneously dense parenchyma. No mammographic evidence of malignancy. No significant interval change since prior exam. BI-RADS 1 - Negative.

LEFT BREAST:
Heterogeneously dense parenchyma. Cluster of amorphous microcalcifications spanning 6 mm is noted in the upper outer quadrant, 11 cm from the nipple. This is a new finding compared to prior exam. No spiculated masses, architectural distortion or skin thickening. BI-RADS 4 - Suspicious abnormality.

IMPRESSION:
1. Stable and benign appearing right breast. BI-RADS 1.
2. New cluster of suspicious microcalcifications in the left upper outer breast, BI-RADS 4. Stereotactic biopsy is recommended for further evaluation.

ADDENDUM:
The patient returned for left stereotactic biopsy of the microcalcifications in the 2:00 position. Specimen radiograph confirmed calcifications within the core samples. Final pathology revealed atypical ductal hyperplasia with associated calcifications. 6-month follow-up diagnostic mammogram is recommended to assess for stability. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED MAGNIFICATION LEFT BREAST.
History: Follow-up to assess response to neoadjuvant chemotherapy for known left breast cancer.
FINDINGS: There is scattered fibroglandular density in both breasts. 
The right breast mammogram is within normal limits. BI-RADS 1.
In the left breast, residual asymmetry is noted at 6:00, extending to the nipple. At the site of the known mass, architectural distortion persists however no discrete mass is identified.
Magnification views of the left breast demonstrate two small clusters of microcalcifications. At 12:00, 6 cm from the nipple there is a cluster of fine linear microcalcifications in a segmental distribution spanning 5 mm. At 3:00, 4 cm from the nipple there is a cluster of fine pleomorphic microcalcifications spanning 8 mm. These were not present on the prior study.
The left axilla is clear.
IMPRESSION:
Residual asymmetry and architectural distortion at the site of known left breast cancer, consistent with partial response to neoadjuvant chemotherapy. Recommend further mammographic and MR imaging follow-up to determine stability.
New clusters of suspicious microcalcifications at 12:00 and 3:00 left breast, highly suspicious for multifocal disease. Stereotactic core biopsy recommended. BI-RADS 4C.
Ultrasound correlation is advised to further assess extent of residual disease.
ADDENDUM: Stereotactic core biopsy of the 12:00 and 3:00 left microcalcifications performed. Pathology shows ductal carcinoma in situ, high nuclear grade with comedonecrosis. Ductal carcinoma in situ also demonstrated at the site of the 3:00 biopsy. BI-RADS 6. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of the right and left breast.

FINDINGS:
The breasts are heterogeneously dense which can obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast. 

In the left breast at the 12:00 position, 8 cm from the nipple, there is a small group of amorphous calcifications spanning approximately 6 mm. This is new compared to the patient's prior mammograms. Additional magnification views in the CC and MLO projection confirm the presence of these calcifications. 

No other mammographic abnormalities are identified. The skin is clear. No axillary, supraclavicular or internal mammary adenopathy is evident.

IMPRESSION:

BI-RADS Category 0: Incomplete - Need additional imaging evaluation and/or prior mammograms for comparison. 

Small group of amorphous calcifications in the left breast at 12:00, 8 cm from nipple. Additional magnification views confirm these are not artifactual. These are indeterminate and a stereotactic core needle biopsy should be recommended for definitive characterization. 

ADDENDUM: 
The patient returned for left breast stereotactic core needle biopsy of the group of amorphous calcifications at the 12:00 position of the left breast. Specimen radiograph confirmed that calcifications were sampled. Final pathology showed focal atypical ductal hyperplasia with associated calcifications, in a background of fibrocystic change. The patient is recommended for 6 month follow up diagnostic mammogram of the left breast given this result. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM.
CLINICAL HISTORY: 44 year old female, family history of breast cancer in sister at age 40.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense (Category C). No dominant mass, skin thickening or new architectural distortion. 
In the right breast at 10:00, 8 cm from the nipple, there is a group of  new amorphous microcalcifications spanning 15 mm. No other suspicious calcifications right breast.
In the left breast at 9:00 4 cm from the nipple there are few punctate calcifications unchanged from the previous exam. No dominant mass or suspicious calcifications left breast. 
IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous calcifications right breast at 10:00. Recommend stereotactic core biopsy.
BI-RADS 2: Benign calcifications left breast. 
ADDENDUM (Post Stereotactic Right Breast Biopsy): 
Stereotactic vacuum assisted core biopsy of the 10:00 right breast calcifications performed. Specimen radiograph demonstrates calcifications. Pathology pending. ➡ *Positive*
- FINDINGS:
Bilateral digital mammograms demonstrate that both breasts are heterogeneously dense. No dominant mass, architectural distortion, or abnormal calcifications are identified in the right breast. 

In the left breast, at the 10 o'clock position approximately 5 cm from the nipple, there is a cluster of pleomorphic microcalcifications. Additional magnification views in the craniocaudal and mediolateral projections confirm the presence of a 1 cm group of heterogeneous calcifications, varying in size and shape. The morphology is suspicious for malignancy. No other microcalcifications are seen elsewhere in the left breast.

Targeted ultrasound at the 10 o'clock position of the left breast corresponding to the area of microcalcifications demonstrates normal appearing fibroglandular tissue with no discrete mass or shadowing. The microcalcifications are not sonographically apparent.

The remainder of the left breast is unremarkable. No skin thickening or nipple retraction.

IMPRESSION:
New development of a 1 cm cluster of pleomorphic microcalcifications at 10 o'clock position of the left breast, suspicious for malignancy. BI-RADS 4C. Stereotactic biopsy is recommended for definitive diagnosis.

The right breast is negative. BI-RADS 1.

ADDENDUM: Stereotactic core biopsy of the group of microcalcifications in the left breast at 10:00, 5 cm from the nipple was performed. Specimen radiograph confirmed calcifications within the cores. Final pathology results demonstrated ductal carcinoma in situ, high nuclear grade with comedonecrosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
Both breasts are almost entirely fatty.
There are no masses, asymmetries, or areas of architectural distortion in either breast.
No suspicious calcifications are seen in the right breast.
In the left breast at the 12 o'clock position, 5 cm from the nipple, there is a 1 cm cluster of amorphous calcifications. No other suspicious calcifications in the left breast.
No skin thickening or retraction. No dilated ducts.

IMPRESSION:
1. BI-RADS 1: Negative in the right breast.
2. BI-RADS 0: Cluster of amorphous calcifications in the left breast. Additional magnification views recommended for further evaluation.

ADDENDUM:
Magnification views of the left breast demonstrate pleomorphic calcifications in a segmental distribution at the 12 o'clock position. Suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy recommended.

FINAL ASSESSMENT:
RIGHT BREAST: BI-RADS 1
LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- FINDINGS:
Bilateral digital diagnostic mammograms were obtained.
The breasts are heterogeneously dense, which may obscure detection of small masses.
At the 12 o'clock position of the right breast, there is a new 6 mm cluster of fine pleomorphic calcifications, located approximately 6 cm from the nipple. This was not present on the prior mammogram from 2 years ago. Additional magnification views confirm the presence of this cluster.
No other suspicious calcifications, masses, or areas of architectural distortion are identified in either breast.
There are no suspicious calcifications in the left breast. 
The skin is normal in thickness and appearance. 

IMPRESSION:
New cluster of suspicious microcalcifications in the right breast at 12 o'clock, 6 cm from the nipple, BI-RADS 4B. Stereotactic core biopsy is recommended for further evaluation. The remainder of the breast tissue is negative bilaterally, BI-RADS 2. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH MAGNIFICATION VIEWS:

CLINICAL HISTORY: Patient presents for routine screening mammogram. No palpable abnormalities noted.

FINDINGS:
The breasts are predominantly fatty replaced.

There are no suspicious masses, architectural distortion, or skin thickening in either breast.

In the right breast at approximately 11:00, 6 cm from the nipple, there is a new group of amorphous microcalcifications spanning an area of about 1 cm. Additional magnification views in the CC and MLO projections confirm these calcifications.

The left breast is unremarkable. No dominant masses, calcifications, architectural distortion or skin thickening.

IMPRESSION:
New group of amorphous microcalcifications right breast at 11:00, 6 cm from the nipple. Additional imaging evaluation with a focal compression view is recommended for further characterization. BI-RADS 0, incomplete. Recommend additional imaging.

Left breast, BI-RADS 1, negative.

ADDENDUM: 

RIGHT BREAST FOCAL COMPRESSION VIEWS:
Additional views of the right breast were performed using a focal compression device. 

The calcifications in question are better visualized on these views. They are amorphous in morphology and are seen in a segmental distribution raising concern for ductal carcinoma in situ (DCIS). 

REVISED IMPRESSION:
Suspicious segmental pleomorphic microcalcifications right breast, BI-RADS 4. Stereotactic biopsy is recommended for definitive diagnosis.

Left breast, BI-RADS 1, negative. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

FINDINGS:

Right Breast:
The right breast is heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.

Left Breast: 
The left breast is also heterogeneously dense. There is an approximately 1 cm irregular mass at the 10 o'clock position, 5 cm from the nipple. This corresponds to a palpable abnormality reported by the patient. Additionally, there is a new cluster of fine pleomorphic microcalcifications at the 12 o'clock position, 3 cm from the nipple. No other suspicious masses, calcifications or areas of architectural distortion are seen. 

IMPRESSION:
1. BI-RADS 4: Suspicious finding in the left breast. Recommend stereotactic biopsy of the microcalcifications at 12 o'clock and ultrasound correlation of the 10 o'clock mass. 
2. BI-RADS 1: Normal right breast.

ADDENDUM: The patient returned for stereotactic biopsy of the left breast microcalcifications. Pathology results showed ductal carcinoma in situ, high nuclear grade with comedo necrosis. The patient will be referred to surgery for further management. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram. 
The breasts are heterogeneously dense. 
In the right breast at the 12:00 position 6 cm from the nipple is a group of amorphous calcifications measuring about 8 mm. These are new compared to the previous mammogram from 1 year ago. A few punctate calcifications are seen at 6:00 in the left breast. Otherwise no suspicious masses, calcifications, architectural distortion or asymmetries in either breast.

IMPRESSION:  
BI-RADS 4: Suspicious finding right breast. Stereotactic core biopsy recommended for calcifications at 12:00 right breast. Likely benign finding left breast, 6 month follow up recommended for calcifications at 6:00 left breast. 

ADDENDUM: Stereotactic core biopsy right breast 12:00 position performed. Specimen radiograph confirms calcifications. Pathology pending. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAM, LEFT BREAST ULTRASOUND
INDICATION: Palpable abnormality left breast.
COMPARISON: None available.
FINDINGS:
The breasts are fibroglandular without gross asymmetry.
Left breast: There is an irregular 10 mm hypoechoic nodule at the 2:00 position, 5 cm from the nipple. This demonstrates posterior acoustic shadowing. Elsewhere in the left breast, there are 2-3 small simple-appearing cysts.
Right breast: There are no dominant masses or suspicious calcifications.
Associated findings: The left axilla demonstrates an enlarged lymph node measuring up to 8 mm in short axis with cortical thickening.
Impression:
1. 10 mm solid mass, left breast 2:00, with suspicious features on ultrasound. This could represent a primary breast malignancy. Strongly recommend tissue sampling.
2. Enlarged left axillary node with abnormal cortical thickening, suspicious for regional lymph node involvement.
3. Cluster of indeterminate microcalcifications in upper outer left breast. These require further evaluation with magnification views. If patient is to have biopsy of the 2:00 mass, suggest these calcifications also be targeted concomitantly.
BI-RADS: Left: 4C
BI-RADS: Right: 2 
OVERALL BI-RADS: 4C ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms demonstrate heterogeneously dense breast tissue. 
In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.
In the left breast, there is a new cluster of pleomorphic microcalcifications in the upper outer quadrant at the 1 o'clock position, approximately 7 cm from the nipple. The cluster measures 8 mm in greatest dimension. No associated mass or architectural distortion is identified. 
Magnification views of the left breast in the craniocaudal and mediolateral projections confirm the presence of the cluster of pleomorphic microcalcifications. The distribution is segmental.
There are no suspicious findings in the left axilla.

IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
Left breast: New cluster of pleomorphic microcalcifications in the upper outer quadrant, suspicious for malignancy. BI-RADS 4C - Suspicious abnormality. Stereotactic biopsy recommended for definitive diagnosis.

ADDENDUM:
The patient proceeded to left breast stereotactic biopsy under ultrasound guidance. Specimen radiograph confirmed calcifications within the core samples. Final pathology results demonstrated ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended for complete evaluation of extent of disease and to rule out invasion. ➡ *Positive*
- ULTRASOUND GUIDED BIOPSY PROCEDURE LEFT BREAST
Clinical history: 51 year old female with new cluster of calcifications left breast.
Technique: Under ultrasound guidance using a 14 gauge needle an ultrasound guided core biopsy was performed at 3:00 left breast 4 cm from the nipple corresponding to the new cluster of calcifications on mammography. Specimen radiograph shows calcifications.
BILATERAL DIGITAL MAMMOGRAPHY WITH SPOT COMPRESSION MAG VIEWS LEFT BREAST:
Comparison: Outside mammograms from August 2021.
FINDINGS: The breasts are heterogeneously dense. Stable benign appearing calcifications are seen bilaterally.
There is a new cluster of fine pleomorphic calcifications measuring 6 mm at 3:00 left breast middle depth on the CC view. This was targeted on today's ultrasound guided biopsy.
The breasts are otherwise stable.
IMPRESSION:
New cluster of suspicious calcifications at 3:00 left breast middle depth with a 6 mm extent. Targeted on ultrasound and biopsied. BIRADS 4B. Specimen radiograph shows calcifications.
Remainder of bilateral mammogram is benign. BIRADS 2. ➡ *Positive*
- BILATERAL SCREENING DIGITAL MAMMOGRAM
HISTORY: 46-year-old female with no family history of breast cancer. Screening mammogram.
TECHNIQUE: Standard bilateral CC and MLO views were performed. 
COMPARISON: None available.  
FINDINGS:
RIGHT BREAST:
The breast tissue is heterogeneously dense (BI-RADS composition category C). There are no suspicious masses, architectural distortions, or areas of abnormal calcifications. Skin is normal in thickness. Benign-appearing lymph nodes are present in the right axilla.
LEFT BREAST:
The breast tissue is heterogeneously dense (BI-RADS composition category C). There are no suspicious masses or architectural distortions. In the upper outer quadrant, there is a small cluster of pleomorphic microcalcifications spanning an area of approximately 7 mm. Skin is normal in thickness. Benign-appearing lymph nodes are present in the left axilla.
IMPRESSION:
RIGHT BREAST: No suspicious findings. BI-RADS 1 - Negative. Routine screening in 1 year recommended.
LEFT BREAST: Small cluster of pleomorphic microcalcifications in the upper outer quadrant, suspicious for malignancy. BI-RADS 4 - Suspicious abnormality. Stereotactic-guided core needle biopsy recommended for definitive diagnosis.
ADDENDUM: 
The patient was recalled for magnification views of the left breast, which confirmed the presence of a cluster of pleomorphic microcalcifications in the upper outer quadrant. Stereotactic-guided core needle biopsy was performed, demonstrating ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform and solid types. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:

Right breast:
The right breast consists of scattered areas of fibroglandular density. No focal dominant mass, suspicious calcifications or architectural distortion.

Left breast:
The left breast consists of scattered areas of fibroglandular density. No focal dominant mass or architectural distortion. 
There is a new group of pleomorphic microcalcifications in the left upper outer quadrant at 2 o'clock, 8 cm from the nipple. This group measures approximately 8 mm. No associated mass is seen.

IMPRESSION:

Right breast: Negative study. BI-RADS 1 - Recommend routine annual screening in 12 months.

Left breast: BI-RADS 4 - Suspicious abnormality. Recommend stereotactic biopsy of the new group of microcalcifications in the left upper outer quadrant. 

ADDENDUM: The patient returned for stereotactic biopsy of the group of microcalcifications in the left breast at 2:00, 8 cm from the nipple. Specimen radiograph confirmed that microcalcifications were sampled. The specimen was sent to pathology for analysis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to previous outside screening mammogram from 2 years ago.

FINDINGS:

The breasts are heterogeneously dense, which may obscure detection of small masses.

There is no significant asymmetry, architectural distortion, or suspicious masses in either breast.

In the left breast at 10 o'clock, 6 cm from the nipple, there is a new 8 mm cluster of amorphous microcalcifications. These were not present on the prior study.

The remaining fibroglandular tissue is unremarkable.

IMPRESSION:

BI-RADS Category 0: Incomplete

New cluster of amorphous microcalcifications in the left breast at 10 o'clock, 6 cm from the nipple, suspicious for malignancy. Additional magnification views are recommended for further evaluation.

ADDENDUM:

Magnification views of the left breast in the craniocaudal and mediolateral projections were performed. The amorphous microcalcifications at 10 o'clock, 6 cm from the nipple, are more conspicuous and appear suspicious.

REVISED IMPRESSION: 

BI-RADS Category 4: Suspicious abnormality

8 mm cluster of suspicious amorphous microcalcifications in the left breast. Stereotactic biopsy is recommended for definitive diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (BI-RADS density category C), which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left Breast:
The left breast is also heterogeneously dense (BI-RADS density category C), which may obscure detection of small masses.
A new cluster of fine pleomorphic microcalcifications is seen in the upper outer quadrant of the left breast, approximately 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension.
No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
A cluster of suspicious microcalcifications is identified in the left breast, not seen on prior exams. Additional magnification views are needed for further characterization. Stereotactic biopsy may be necessary depending on the morphology and distribution of the calcifications on magnification views.
The patient will be recalled for diagnostic mammographic workup of the left breast.

BIRADS ASSESSMENT CATEGORIES:
Right breast: BI-RADS 2 - Benign
Left breast: BI-RADS 0 - Incomplete

ADDENDUM:
Magnification views of the left breast demonstrate a 10 mm cluster of fine pleomorphic microcalcifications in the upper outer quadrant, with segmental distribution. Stereotactic biopsy is recommended for definitive characterization. Updated BIRADS assessment of the left breast: BI-RADS 4 - Suspicious abnormality. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
Both breasts demonstrate heterogeneously dense fibroglandular tissue.
At the 12:00 position of the right breast, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of 1 cm. No associated mass, architectural distortion or skin changes are identified.
The left breast is unremarkable, negative for masses, architectural distortion, asymmetries or suspicious calcifications.
No axillary, supraclavicular or internal mammary lymphadenopathy.
Benign vascular calcifications are present bilaterally.
IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous microcalcifications in the right breast at 12:00, 6 cm from the nipple. Stereotactic core needle biopsy is recommended for tissue diagnosis. The left breast is BI-RADS 1, negative.
ADDENDUM: 
The patient returned for stereotactic core biopsy of the right breast microcalcifications. Specimen radiograph confirmed retrieval of microcalcifications. Final pathology revealed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. Wide local excision is recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAMS.
INDICATION: Annual screening mammography.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: Compared to prior mammograms from September 2019.
FINDINGS: 
The breasts are heterogeneously dense (C), which may obscure small masses. 
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
Scattered vascular calcifications bilaterally, stable.
At the 9 o'clock position of the right breast, approximately 4 cm from the nipple, there is a new 6 mm cluster of pleomorphic microcalcifications. Additional magnification views confirm these are suspicious and new compared to prior mammograms.
No suspicious axillary lymphadenopathy.
Breast implants are noted bilaterally, unchanged in appearance.
IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the right breast at 9:00, 4 cm from the nipple. Biopsy is recommended. The remainder of the bilateral mammograms are negative.
ADDENDUM: 
The patient returned for stereotactic core biopsy of the right breast microcalcifications. Specimen radiograph confirms microcalcifications within the cores. Final pathology results are pending and will be reported separately. ➡ *Positive*
- FINDINGS:

BILATERAL SCREENING MAMMOGRAPHY:

The breasts are predominantly fatty.  There are scattered areas of fibroglandular density.

There is an approximately 0.5 cm cluster of heterogeneous calcifications in the upper outer quadrant of the left breast 6 cm from the nipple, mildly increased compared to prior exam. No associated mass, architectural distortion or skin thickening.  

No spiculated mass, architectural distortion or suspicious calcifications in either breast.  

IMPRESSION:

BI-RADS Category 0: Incomplete assessment.  The cluster of left breast calcifications is suspicious and warrants further evaluation with magnification views.

MAGNIFICATION VIEWS OF THE LEFT BREAST:  

The previously described 6 mm cluster of calcifications in the upper outer left breast is again noted.  The calcifications are heterogeneous and pleomorphic in morphology, suspicious for malignancy.  

IMPRESSION: 

BI-RADS Category 4: Suspicious abnormality.  Stereotactic core needle biopsy of the cluster of left breast calcifications is recommended.  

STEREOTACTIC CORE NEEDLE BIOPSY LEFT BREAST:

Stereotactic core needle biopsy of the cluster of calcifications at 2:00, 6 cm from the nipple in the left breast was performed utilizing an 11 gauge vacuum-assisted device.  Five core specimens were obtained.  Specimen radiograph confirms the presence of calcifications.  A ribbon clip was deployed at the biopsy site.  The patient tolerated the procedure well.  

PATHOLOGY: Pending. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 49-year-old female presenting for routine bilateral screening mammogram.
TECHNIQUE: Bilateral digital screening mammographic views were obtained.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense which may lower the sensitivity of mammography. 
Right Breast:
There are no suspicious masses, architectural distortions or suspicious calcifications. 
Left Breast:  
There are no suspicious masses or architectural distortions. Scattered fibroglandular densities are present.
A new group of amorphous calcifications is seen in the upper outer quadrant posteriorly on the MLO view only. Additional spot compression magnification views are recommended for further evaluation. 
IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation is needed. 
Specifically, spot compression magnification views of the left breast are recommended for further characterization of the amorphous calcifications in the upper outer quadrant posteriorly. 
ADDENDUM:
LEFT DIAGNOSTIC MAMMOGRAM
TECHNIQUE: Spot compression magnification views were performed of the upper outer left breast.  
FINDINGS: The spot compression magnification views demonstrate a group of amorphous calcifications in the upper outer quadrant of the left breast. The calcifications are new compared to the prior examination and are suspicious for malignancy. A stereotactic biopsy is recommended for further evaluation.  
IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- MAMMOGRAPHIC EXAMINATION:
INDICATIONS: Suspicious microcalcifications upper outer left breast detected at outside screening mammogram. Additional imaging for further evaluation.
COMPARISON: None. Outside mammogram images not available.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
No dominant masses, architectural distortion or suspicious calcifications are seen in the right breast.
In the upper outer left breast there is a new cluster of indeterminate amorphous and pleomorphic microcalcifications spanning an area of approximately 10 mm. This corresponds to the area of concern on outside screening mammogram.
Magnification views are performed and show no associated mass or distortion.
IMPRESSION:
New cluster of indeterminate amorphous and pleomorphic microcalcifications upper outer left breast. Biopsy is recommended for further evaluation. BIRADS 4B.
Heterogeneously dense breast tissue which may lower the sensitivity of mammography.
RECOMMENDATION:
Stereotactic core biopsy left breast microcalcifications is recommended. A biopsy requisition will be sent to booking. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS.
INDICATION: Screening study. No reported clinical concerns.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
In the right breast at 10:00, 6 cm from the nipple, there is a group of amorphous calcifications in a segmental distribution measuring 8 mm. This is new compared to the patient's previous mammograms.
The left breast is unremarkable.
IMPRESSION:
BI-RADS 0: Incomplete. Additional imaging evaluation with right breast spot compression magnification views is required to assess the new group of amorphous calcifications in the right breast at 10:00.
ADDENDUM - RIGHT BREAST SPOT COMPRESSION MAGNIFICATION VIEWS:
Spot compression magnification views of the right breast confirms a 10 mm group of amorphous calcifications at 10:00, 6 cm from the nipple, in a segmental distribution. The calcifications are new compared to the patient's previous mammograms. 
FINAL ASSESSMENT: 
RIGHT BREAST, BI-RADS 4B: Suspicious abnormality, biopsy should be considered. Suspicious group of new amorphous calcifications at 10:00.
LEFT BREAST, BI-RADS 1: Negative. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 45-year-old female with no prior mammograms for comparison.
FINDINGS:
The breasts are heterogeneously dense (BI-RADS Density C). 
RIGHT BREAST:
Right breast demonstrates scattered fibroglandular densities with no dominant mass, architectural distortion, or suspicious calcifications.
LEFT BREAST:
In the left breast, there are several clusters of pleomorphic microcalcifications scattered in the upper outer quadrant from 11:00 to 1:00, the largest cluster measuring approximately 8 mm. The calcifications are new and of concern given lack of comparison. No dominant mass or architectural distortion identified in the left breast.
IMPRESSION:
1. Heterogeneously dense breasts (BI-RADS Density C).
2. Suspicious pleomorphic microcalcifications in the upper outer left breast, BI-RADS 4B. Recommend stereotactic biopsy for further evaluation.
3. Negative right breast. BI-RADS 1.
RECOMMENDATIONS: 
1. Stereotactic core needle biopsy of the largest cluster of microcalcifications in the left breast at 12:00, 5 cm from the nipple.
2. Routine annual screening mammogram in 1 year, assuming biopsy results are benign. ➡ *Positive*
- BILATERAL MAMMOGRAM
CLINICAL HISTORY: 45-year-old female, high risk screening.
COMPARISON: None.
FINDINGS:
Right breast: The breast is heterogeneously dense (BI-RADS density C). There are no suspicious masses, architectural distortion, or suspicious calcifications.
Left breast: The breast is heterogeneously dense (BI-RADS density C). There are scattered areas of fibroglandular density. No discrete mass or area of architectural distortion is identified. However, there is a new group of fine pleomorphic calcifications in the upper outer quadrant, 10 cm from the nipple, spanning an area of approximately 8 mm. Additional magnification views confirm these are suspicious calcifications.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
Left breast: New group of suspicious calcifications in the upper outer quadrant. Biopsy is recommended. BI-RADS 4 - Suspicious.
RECOMMENDATION: Stereotactic core biopsy of the left breast calcifications is recommended. Please send a requisition to the department to arrange the biopsy. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses.
Right breast: There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Left breast: No dominant masses or areas of architectural distortion. There is a small group of amorphous calcifications in the upper outer quadrant measuring approximately 6 mm in diameter. These are new since the patient's last mammogram.
IMPRESSION:
BI-RADS 0: Left breast: Small group of amorphous calcifications in the upper outer quadrant, indeterminate. Additional magnification views recommended for further evaluation. Appointment to be scheduled.
BI-RADS 1: Right breast: Negative.
ADDENDUM:
The patient returned for left diagnostic mammogram with magnification views of the group of calcifications in the upper outer quadrant. The calcifications are irregular in shape and distribution, suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy is recommended. BI-RADS 4: Suspicious. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: None available

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (ACR category C). There are no dominant masses, areas of architectural distortion or suspicious calcifications. No abnormalities or significant changes are noted.

Left Breast: 
The left breast is also heterogeneously dense (ACR category C). In the upper outer quadrant, there is a new cluster of fine pleomorphic calcifications spanning an area of approximately 1 cm. No associated mass or architectural distortion is evident. The remainder of the left breast is unremarkable.

IMPRESSION:
New cluster of suspicious calcifications in the upper outer quadrant of the left breast, BI-RADS 4B. Stereotactic biopsy is recommended for definitive characterization. The right breast is negative, BI-RADS 1. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

At the 12 o'clock position of the right breast, approximately 6 cm from the nipple, there is a cluster of amorphous microcalcifications spanning an area of about 8 mm. Additional magnification views in this area are recommended for further evaluation.

The remaining fibroglandular tissue in both breasts is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete, additional imaging is needed. 
Cluster of amorphous microcalcifications at 12 o'clock position right breast, recommend additional magnification views for further assessment.

ADDENDUM: 
Right breast magnification views demonstrate a cluster of amorphous microcalcifications at the 12 o'clock position, approximately 6 cm from the nipple, spanning an area of about 8 mm. This cluster is new compared to the patient's prior mammograms. Stereotactic biopsy is recommended for tissue diagnosis.

FINAL ASSESSMENT: BI-RADS Category 4: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- MAMMOGRAPHY REPORT

CLINICAL HISTORY: Patient presents for initial screening mammogram.

TECHNIQUE: Bilateral digital screening mammographic views including CC and MLO views obtained.

COMPARISON: None.

FINDINGS:

Right Breast:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious calcifications. 

Left Breast:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass or architectural distortion. Scattered amorphous microcalcifications are noted in the upper outer quadrant, spanning an area of approximately 1 cm. The calcifications are new compared to prior exams.

IMPRESSION:

Right Breast: No mammographic evidence of malignancy, BI-RADS Category 1.

Left Breast: Scattered amorphous microcalcifications in the upper outer quadrant. Additional evaluation with magnification views recommended for further characterization, BI-RADS Category 0. 

ADDENDUM:
Magnification views of the left breast demonstrate pleomorphic microcalcifications in a segmental distribution, spanning 12 mm at the 2:00 position, 6 cm from the nipple. Biopsy is recommended, BI-RADS Category 4B.

OVERALL BI-RADS ASSESSMENT CATEGORY: 4B ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAPHY AND TARGETED ULTRASOUND LEFT BREAST
CLINICAL HISTORY: New lump in the left breast at 9:00, family history of breast cancer in maternal aunt.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
There are scattered areas of fibroglandular density bilaterally.
In the right breast, no dominant masses, areas of architectural distortion, or suspicious calcifications are identified.
In the left breast at the 9:00 position 6 cm from the nipple, there is a new 12 mm irregular mass with indistinct margins, corresponding to the palpable area of concern. Additionally, at the 12:00 position 4 cm from the nipple, there is a new 5 mm cluster of pleomorphic calcifications.
TARGETED LEFT BREAST ULTRASOUND:
At the site of the 12:00 group of calcifications, there is a 6 mm irregular solid mass with angular margins and shadowing. This corresponds to the mammographic finding. No axillary lymphadenopathy.
IMPRESSION:
New 12 mm irregular mass at 9:00 left breast, suspicious for malignancy. BI-RADS 4C. Ultrasound-guided core biopsy is recommended.
New group of pleomorphic calcifications at 12:00 left breast with associated irregular shadowing mass on ultrasound, highly suspicious for malignancy. BI-RADS 4C. Stereotactic-guided biopsy is recommended. 
BI-RADS 1 right breast. Routine screening in 1 year recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY

INDICATION: Annual screening mammogram.

COMPARISON: Prior study of 05/15/2021.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally.

No dominant masses, areas of architectural distortion, or suspicious calcifications are seen in the right breast.

In the left breast, there is a new cluster of pleomorphic microcalcifications at the 10 o'clock position, 6 cm from the nipple, spanning an area of approximately 8 mm. This cluster was not present on the prior study.

No skin thickening, nipple retraction, or lymphadenopathy is noted.

IMPRESSION:
1. New cluster of pleomorphic microcalcifications in the left breast at 10 o'clock, 6 cm from the nipple, suspicious for malignancy. Additional diagnostic imaging is recommended for further evaluation. BI-RADS 4B.

2. Stable, benign findings in the right breast. BI-RADS 2.

RECOMMENDATION:
Diagnostic mammography with magnification views and ultrasound of the left breast for further assessment of the new cluster of microcalcifications. ➡ *Positive*
- REPORT:
"BILATERAL DIGITAL SCREENING MAMMOGRAPHY
INDICATION: Routine screening. No current breast problems.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of each breast.
FINDINGS:
Both breasts show scattered areas of fibroglandular density.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
Left breast:
At the 6 o'clock position of the left breast, approximately 3 cm from the nipple, there is a small cluster of amorphous microcalcifications within an area measuring 5 mm. Additional magnification views confirm these calcifications.
Right breast:
No suspicious findings.
IMPRESSION:
BI-RADS Category 0: Incomplete. Need additional imaging evaluation. Small cluster of amorphous microcalcifications in the left breast. Additional magnification views are recommended for further characterization. Depending on the morphology and distribution of these calcifications on magnification views, a stereotactic core biopsy may be indicated.
BI-RADS Category 1 for the right breast: Negative. Routine screening recommended in 1 year.
ADDENDUM:
Magnification views of the left breast demonstrate that the previously identified amorphous calcifications at 6 o'clock position have a grouped distribution within an area measuring 6 mm. Given the new onset of these calcifications and their suspicious morphology, a stereotactic core biopsy is recommended for definitive characterization.
FINAL ASSESSMENT:
BI-RADS Category 4 for the left breast: Suspicious abnormality. Biopsy should be considered.
BI-RADS Category 1 for the right breast: Negative." ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS AND TOMOSYNTHESIS

CLINICAL HISTORY: 45 year old female with dense breasts. Screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.

No suspicious masses, architectural distortion, or suspicious calcifications are seen in either breast.

There is a 6 mm cluster of indeterminate microcalcifications in the upper outer left breast at 11:00, 6 cm from the nipple. The remainder of the left breast is unremarkable.

No suspicious mammographic findings are seen in the right breast. 

IMPRESSION: 
BI-RADS 0: Incomplete. Indeterminate cluster of microcalcifications in the left breast at 11:00. Additional imaging evaluation with spot compression magnification views is recommended for further characterization. 

BI-RADS 1: No suspicious mammographic findings in the right breast.

ADDENDUM:
Spot compression magnification views of the left breast in the CC and ML projections demonstrate a 5 mm cluster of heterogeneous microcalcifications at 11:00 in the left breast, 6 cm from the nipple. No associated mass or distortion. 

IMPRESSION:
BI-RADS 4: Suspicious cluster of microcalcifications in the left breast. Stereotactic biopsy is recommended for tissue diagnosis. 

BI-RADS 1: No change in the right breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
FINDINGS:
There are scattered areas of fibroglandular density.  
In the right breast there are no dominant masses, areas of architectural distortion or suspicious calcifications.  No significant interval change compared to prior mammograms.
In the left breast, there is a new cluster of heterogeneous calcifications in the upper outer quadrant posterior depth. The calcifications are pleomorphic and have a segmental distribution, measuring approximately 1 cm in length. No associated mass or architectural distortion. No other dominant masses, areas of architectural distortion or suspicious calcifications in the left breast.
IMPRESSION:
New cluster of suspicious calcifications in the upper outer left breast, BI-RADS 4B. This requires tissue diagnosis. Stereotactic core biopsy is recommended.
Otherwise stable bilateral mammograms, BI-RADS 2 right breast. Recommend continued annual screening mammography.
ADDENDUM: 
The patient proceeded with ultrasound guided core needle biopsy of the cluster of calcifications in the upper outer left breast. Pathology results are pending and will be communicated in an addendum to this report. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS
INDICATION: Annual screening mammogram
COMPARISON: None
FINDINGS:
Both breasts show heterogeneously dense parenchymal pattern, limiting sensitivity of mammography.
Right Breast:
No dominant mass, architectural distortion or suspicious calcifications.
Left Breast:
No dominant mass or architectural distortion.
Scattered amorphous microcalcifications are noted in the upper outer quadrant, spanning an area of approximately 2 cm. No associated mass is identified.
Magnification views of the area of microcalcifications in the left breast are obtained, confirming presence of amorphous calcifications in a segmental distribution at 2 o'clock position, 6 cm from the nipple.
IMPRESSION:
1. Heterogeneously dense breasts, limiting sensitivity of mammography. 
2. Amorphous segmental microcalcifications in the left breast at 2 o'clock position, 6 cm from the nipple, suspicious for DCIS. Additional imaging evaluation with breast MRI is recommended for further evaluation. 
3. BI-RADS Category 4B: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended.
4. BI-RADS Category 2: Benign finding in the right breast.

ADDENDUM: Stereotactic biopsy of the left breast microcalcifications performed, demonstrating ductal carcinoma in-situ, intermediate nuclear grade, cribriform type. Wide local excision is recommended. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS
HISTORY: Annual screening mammogram.
FINDINGS:
The breasts are heterogeneously dense.
The right breast shows no suspicious mass, calcifications or architectural distortion.
In the left breast in the upper outer quadrant at 2:00, 6 cm from the nipple, there is a 6 mm cluster of pleomorphic microcalcifications. No associated mass. The remaining left breast is unremarkable.
No suspicious axillary lymphadenopathy.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1. Continue annual screening.
Left breast: New cluster of pleomorphic microcalcifications at 2:00, 6 cm from nipple, suspicious for malignancy. BI-RADS 4B. Stereotactic biopsy recommended.
ADDENDUM - POST BIOPSY LEFT BREAST:
Pathology from stereotactic biopsy of the microcalcifications at 2:00 left breast demonstrates ductal carcinoma in situ (DCIS), intermediate grade. BI-RADS 6. Surgical consultation recommended. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: High risk screening.

COMPARISON: None.

FINDINGS:
The breasts are predominantly fatty.

At the 9:00 position of the right breast, there is a 1 cm cluster of microcalcifications. The calcifications are heterogeneous in morphology, including fine linear branching and pleomorphic forms. This cluster was not present on the prior study.

No suspicious mass, architectural distortion, or other findings are identified in either breast. The skin is unremarkable.

IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the right breast at 9:00, 6 cm from the nipple, that are new compared to the prior study. Stereotactic biopsy is recommended for further evaluation. 

ADDENDUM:
The patient returned for stereotactic biopsy of the cluster of microcalcifications at the 9:00 position of the right breast. Post-biopsy, specimen radiograph confirmed the presence of calcifications. Pathology results demonstrated ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type, associated with microcalcifications. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

TECHNIQUE: Standard CC, MLO, and magnification views of both breasts were obtained.

COMPARISON: None available

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified.  

Left Breast:
The left breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses or focal asymmetries. No areas of architectural distortion are seen. 

In the upper outer quadrant of the left breast, there is a small cluster of amorphous microcalcifications at approximately 2 o'clock, 5 cm from the nipple. This cluster measures approximately 6 mm in greatest dimension. The calcifications are new compared to prior exams. No other suspicious calcifications are identified in the left breast.

IMPRESSION: 

Right Breast: Negative mammogram (BI-RADS 1)
Left Breast: Small cluster of amorphous microcalcifications in the upper outer quadrant, new since prior exam. Suspicious for malignancy (BI-RADS 4). Stereotactic biopsy is recommended for further evaluation.

ADDENDUM:
The patient returned for stereotactic biopsy of the cluster of microcalcifications in the left breast at 2:00, 5 cm from the nipple. Specimen radiograph confirmed the presence of calcifications within the core samples. Post-biopsy mammogram showed deployment of a biopsy marker clip at the site of sampling. The patient tolerated the procedure well. Pathology results are pending. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS
INDICATION: Screening.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense (C). This may lower the sensitivity of mammography.
There are no suspicious masses, calcifications, asymmetries or architectural distortions in either breast. No significant changes compared to prior exam.
At the 9:00 position of the right breast, 4 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of 6 mm.  Additional magnification views confirm these calcifications. 
No mammographic evidence of malignancy in the left breast.
IMPRESSION:
BIRADS 0: Group of amorphous microcalcifications right breast. Additional imaging evaluation with magnification views performed, now BIRADS 4. Stereotactic biopsy recommended. 
BIRADS 1, negative left breast. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH MAGNIFICATION VIEWS:
INDICATION: Screening mammogram.
TECHNIQUE: Bilateral CC and MLO views as well as spot compression magnification views of a group of calcifications were performed. Comparison is made to previous exams from 2018.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure detection of small masses.
There are no new dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
In the LEFT breast at the 9 o'clock position 5 cm from the nipple there is a cluster of indeterminate microcalcifications unchanged from the prior exam. These calcifications were previously assessed as probably benign and are stable in appearance. 
In the LEFT breast at the 3 o'clock position 7 cm from the nipple, there are several new punctate microcalcifications seen on today's exam. These are better evaluated on the spot compression magnification views. No associated mass or distortion is seen.
In the RIGHT breast, there are scattered fibroglandular elements in the central breast. No suspicious masses, calcifications or areas of architectural distortion are identified.
Magnification views of the LEFT breast demonstrate pleomorphic microcalcifications in a segmental distribution at the site of clinical concern measuring approximately 8 mm in greatest dimension. These calcifications are new from the prior examination.
IMPRESSION:
1. New group of suspicious pleomorphic microcalcifications at 3 o'clock position of the LEFT breast, BI-RADS 4B, suspicious. Stereotactic core biopsy is recommended. 
2. Probably benign group of calcifications in the LEFT breast at 9 o'clock position, unchanged from prior exam. BI-RADS 3, short term follow up in 6 months recommended.
3. No mammographic evidence of malignancy in the RIGHT breast, BI-RADS 2. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms were obtained.
The breast tissue is predominantly fatty.
In the right breast, there are no suspicious masses, asymmetries or architectural distortion. 
In the upper outer left breast at approximately 2:00, 8 cm from the nipple, there is a group of fine pleomorphic microcalcifications in a segmental distribution, spanning approximately 15 mm. No associated mass or asymmetry is seen. No other suspicious calcifications are identified in either breast.
No mammographic evidence of malignancy in the right breast.

IMPRESSION:
1. Group of suspicious microcalcifications left breast as described above, BIRADS 4. Stereotactic biopsy is recommended for further evaluation. The patient will be scheduled for stereotactic biopsy of the left breast on a separate requisition.  
2. Benign right breast, BIRADS 2. Routine screening in 1 year is recommended. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense.
There are suspicious microcalcifications in the upper outer quadrant of the left breast spanning an area of approximately 3 cm at 2:00. Spot compression magnification views were performed for further characterization. The calcifications are grouped and pleomorphic in nature.
No dominant masses or architectural distortion are identified in either breast.
There is no suspicious skin thickening or nipple retraction.
No abnormal axillary lymph nodes are seen.

IMPRESSION:
BIRADS 4: Suspicious microcalcifications in the upper outer quadrant of the left breast. Stereotactic biopsy is recommended for definitive characterization.

ADDENDUM:
The patient proceeded with stereotactic biopsy of the left breast microcalcifications at 2:00. Pathology results showed ductal carcinoma in situ, nuclear grade 2. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY
INDICATION: Annual screening
COMPARISON: 05/18/2020
TECHNIQUE: Standard CC and MLO views of the breasts were obtained.
FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.
In the left breast, there is a new cluster of amorphous calcifications in the upper outer quadrant at 2 o'clock, 8 cm from the nipple. Additional magnification views in the CC and lateral projections confirm the presence of a group of amorphous calcifications spanning an area of approximately 8 mm. No associated mass or distortion is identified.
The right breast is unremarkable. No suspicious masses, calcifications, or areas of architectural distortion.
IMPRESSION:
BI-RADS Category 0: Incomplete: Need additional imaging evaluation - left breast only.
New cluster of amorphous calcifications in the left breast upper outer quadrant, indeterminate in etiology. Recommend left diagnostic mammogram with magnification views and ultrasound for further characterization. Overall assessment is BI-RADS 0, incomplete.
Benign findings right breast, BI-RADS 2.
ADDENDUM:
LEFT DIAGNOSTIC MAMMOGRAM AND ULTRASOUND
Left diagnostic mammographic magnification CC and lateral views demonstrate the cluster of amorphous calcifications at 2 o'clock upper outer quadrant left breast, spanning 8 mm. The distribution is grouped.
Targeted left breast ultrasound at 2:00, 8 cm from the nipple demonstrates normal fibroglandular tissue with no sonographic correlate for the mammographic calcifications. No suspicious mass or irregular hypoechoic area identified.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast.
New grouped amorphous calcifications left breast at 2:00, 8 cm from the nipple with no sonographic correlate. Suspicious for DCIS.  Stereotactic biopsy recommended. Overall assessment is BI-RADS 4, suspicious.
Benign right breast, BI-RADS 2. ➡ *Positive*
- BILATERAL DIGITAL DIAGNOSTIC MAMMOGRAM WITH TOMOSYNTHESIS

INDICATION: Annual screening.

TECHNIQUE: CC and MLO views of both breasts were obtained. Tomosynthesis was performed.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a group of amorphous calcifications in the upper outer quadrant, approximately 6 cm from the nipple, spanning an area of about 10 mm. No associated mass, architectural distortion, or skin thickening is identified. 

No axillary, supraclavicular, or internal mammary adenopathy.

IMPRESSION:
1. Heterogeneously dense breasts.
2. Suspicious grouped amorphous calcifications in the left breast at 2 o'clock, 6 cm from the nipple, BI-RADS 4B. Stereotactic biopsy is recommended.
3. Benign right breast, BI-RADS 2.

ADDENDUM (post stereotactic biopsy left breast):
Stereotactic biopsy of the group of amorphous calcifications in the left breast at 2 o'clock was performed. Specimen radiograph confirmed calcifications. Pathology revealed sclerosing adenosis with microcalcifications, benign. Recommend routine screening mammogram in 1 year. Final BI-RADS assessment of the left breast: BI-RADS 3, probably benign finding. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No complaints. Patient reports history of breast cancer in maternal aunt.
TECHNIQUE: Standard CC and MLO views of both breasts. 
COMPARISON: Previous outside screening mammogram from 11/2019.
FINDINGS:
Right Breast:
The right breast is heterogeneously dense (BI-RADS composition c). No significant masses, architectural distortion or suspicious calcifications. Stable benign appearing calcifications in the upper outer quadrant, consistent with fibrocystic changes.
Left Breast: 
The left breast is heterogeneously dense (BI-RADS composition c). There is a new cluster of fine pleomorphic microcalcifications in the upper inner quadrant at 10 o'clock, 6 cm from the nipple, spanning an area of approximately 1 cm. No associated mass, architectural distortion or skin thickening.
IMPRESSION:
BI-RADS 0: Incomplete. Need additional imaging evaluation of the new cluster of microcalcifications in the left breast at 10:00, 6 cm from the nipple. Spot compression magnification views recommended for further characterization. 
ADDENDUM:
Magnification views of the left breast at the site of microcalcifications were performed. The microcalcifications are better characterized as a cluster of fine linear and branching calcifications, highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.
FINAL ASSESSMENT: BI-RADS 4C: Suspicious abnormality, biopsy should be considered.
OVERALL ASSESSMENT:
Right breast: BI-RADS 2, Benign. 
Left breast: BI-RADS 4C, Suspicious abnormality. ➡ *Positive*
- MAMMOGRAM REPORT
CLINICAL INFORMATION: Annual screening mammogram. Patient reports no breast complaints.

COMPARISON: Prior mammogram from 09/12/2020.

FINDINGS:
Both breasts are heterogeneously dense which could obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications. 

LEFT BREAST:
Skin is normal in thickness. No dominant masses or areas of architectural distortion. 

Retro-areolar there is a new cluster of fine pleomorphic microcalcifications at the 3:00 position, 2 cm from the nipple, spanning an area of approximately 8mm. This is a suspicious finding.

IMPRESSION:
BI-RADS 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for the cluster of microcalcifications at 3:00 position of the left breast. 

BI-RADS 2: Benign finding in the right breast. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS:
INDICATION: Annual screening.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or definite asymmetries.
Scattered fibroglandular densities are present bilaterally.
There is a 6 mm cluster of punctate microcalcifications in the upper outer quadrant of the right breast at the 10:00 position, 8 cm from the nipple. No prior mammograms are available for comparison.
The left breast is unremarkable.
IMPRESSION:
BI-RADS Category 0: Incomplete. The cluster of microcalcifications in the right breast requires additional imaging evaluation with magnification views for further characterization.
RECOMMENDATION:  
Right breast magnification views.
ADDENDUM - RIGHT BREAST MAGNIFICATION VIEWS:
The cluster of microcalcifications in the right breast upper outer quadrant demonstrates pleomorphic calcifications in a segmental distribution, spanning approximately 8 mm. This is a suspicious finding.
FINAL ASSESSMENT:  
BI-RADS Category 4B: Moderate suspicion for malignancy. Biopsy should be considered. ➡ *Positive*
- ULTRASOUND LEFT BREAST

INDICATION: Palpable abnormality left breast upper outer quadrant.

FINDINGS:
The breasts are heterogeneously dense. Targeted ultrasound left breast 2:00 region 6 cm from the nipple demonstrates an irregular hypoechoic mass with angular margins and posterior shadowing measuring 11 x 8 x 7 mm. This corresponds to the palpable abnormality. 

At 2:00 3 cm from the nipple there is a 4 mm cluster of punctate echogenic foci consistent with microcalcifications.

The remainder of the left breast was scanned. There are several simple appearing cysts bilaterally, largest measuring 8 mm. No other suspicious masses, architectural distortion or calcifications are seen in either breast. 

IMPRESSION:
1. Irregular hypoechoic mass left breast 2:00, suspicious for malignancy. Ultrasound guided core biopsy is recommended. 
2. 4 mm cluster of microcalcifications left breast 2:00, suspicious for malignancy in this clinical setting. Ultrasound guided core biopsy is recommended.
3. Benign appearing cysts bilaterally.

BIRADS 4C: Suspicious finding, biopsy recommended. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms demonstrate heterogeneously dense breasts. In the upper outer quadrant of the left breast, there is a new cluster of pleomorphic microcalcifications spanning an area of approximately 1 cm. The calcifications are fine, linear and branching in morphology, raising concern for ductal carcinoma in situ (DCIS).

The remainder of the left breast shows predominantly fatty tissue without suspicious masses, architectural distortion, or additional calcifications.

The right breast is predominantly fatty and does not show any suspicious masses, calcifications or areas of architectural distortion.

IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple. Stereotactic biopsy is recommended for further evaluation. The remainder of the left breast and right breast are negative.

ADDENDUM:
The patient returned for stereotactic biopsy of the left breast microcalcifications. Specimen radiograph confirms that the calcifications are present within the core samples obtained. Final pathology results are pending. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDITIONAL VIEWS OF LEFT BREAST AND ULTRASOUND LEFT BREAST AND AXILLARY REGION. 
 
HISTORY: 45 year old female with a palpable lump in her left breast at 9 o'clock position. 
 
FINDINGS: The breasts are heterogeneously dense, which may obscure small masses.  
In the left breast, there is a 2.5 cm irregular mass at the 9 o'clock position, 4 cm from the nipple which corresponds to the palpable finding. The mass demonstrates spiculated margins and associated architectural distortion. Several pleomorphic microcalcifications are seen within and adjacent to the mass.
Targeted ultrasound of the left breast 9 o'clock mass shows an irregular hypoechoic mass with angulated margins and posterior acoustic shadowing, measuring 25 x 21 x 20 mm. The mass is highly suggestive of malignancy.
Ultrasound of the left axilla reveals enlarged abnormal appearing lymph nodes with thickened cortices and loss of fatty hila, the largest measuring 15 mm in cortical thickness, suspicious for metastatic involvement.  
The right breast shows no dominant mass, architectural distortion or suspicious calcifications.
 
IMPRESSION:
1. Highly suspicious mass left breast at 9 o'clock position with adjacent pleomorphic microcalcifications, likely representing malignancy. Ultrasound findings corroborate mammographic impression. Recommend tissue sampling.
2. Abnormal left axillary lymph nodes suspicious for metastatic involvement.  
3. BI-RADS Category 5: Highly Suggestive of Malignancy.
 
ADDENDUM: Ultrasound guided core biopsy of the left 9 o'clock breast mass and fine needle aspiration of the most abnormal appearing left axillary lymph node were performed. Pathology results demonstrate invasive ductal carcinoma, grade 2, in the left breast mass and metastatic carcinoma in the left axillary lymph node. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Annual screening.

COMPARISON: None available.

FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

At the 10:00 position of the left breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. This cluster of microcalcifications is new compared to prior exams and is suspicious for early malignancy.

The remainder of the left breast is unremarkable. No dominant masses, areas of architectural distortion, or additional suspicious calcifications are identified.

IMPRESSION:
1. New group of suspicious microcalcifications in the left breast at 10:00, 6 cm from the nipple, likely representing ductal carcinoma in situ (DCIS). Additional imaging evaluation with magnification views is recommended for further characterization. Stereotactic biopsy may be necessary for definitive diagnosis. BI-RADS 4B.

2. The right breast is negative. BI-RADS 1.

RECOMMENDATION:
Diagnostic mammogram with magnification views of the left breast for further evaluation of the suspicious microcalcifications. Consider stereotactic biopsy if indicated after additional imaging. Routine annual screening of the right breast in 1 year. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was performed.  
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 
There are scattered areas of fibroglandular density bilaterally.
In the left breast at the 10:00 position, 6 cm from the nipple, there is a group of amorphous calcifications spanning an area of about 10 mm. The calcifications are new compared to prior mammograms from 2 years ago at an outside facility. Spot compression magnification views confirm the presence of a group of suspicious calcifications at this site. 
In the right breast, there are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 
2. New group of suspicious calcifications in the left breast at 10:00, 6 cm from the nipple. Additional imaging evaluation with left diagnostic mammogram is recommended for further characterization. 
3. Recommend follow up diagnostic evaluation of the left breast. 
4. BI-RADS Category 4 - Suspicious abnormality in the left breast. Biopsy should be considered. 
5. BI-RADS Category 2 - Benign finding in the right breast. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. No palpable masses or focal areas of pain.

COMPARISON: Screening mammogram from 10/15/2020.

FINDINGS:
The breasts are heterogeneous dense, which can obscure small masses.

There are no dominant masses, architectural distortions, or suspicious areas of skin thickening in either breast.

In the right upper outer quadrant, there is a new cluster of indeterminate pleomorphic microcalcifications at 10:00, 6 cm from the nipple, spanning 8 mm. No other suspicious calcifications are noted in the right breast.

The left breast is stable in appearance. No suspicious masses, architectural distortion, or calcifications are identified.

IMPRESSION:
BI-RADS 0: Incomplete. New cluster of indeterminate microcalcifications in the right breast at 10:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation.

BI-RADS 2: Benign. The left breast is stable compared to prior mammograms.

ADDENDUM:
Right breast magnification views demonstrate a cluster of pleomorphic microcalcifications spanning 9 mm at 10:00, 6 cm from the nipple. Stereotactic biopsy is recommended for tissue diagnosis.

FINAL ASSESSMENT:
RIGHT BREAST, BI-RADS 4: Suspicious. Stereotactic biopsy recommended.
LEFT BREAST, BI-RADS 2: Benign. ➡ *Positive*
- CLINICAL INFORMATION: Follow up 6 months post right lumpectomy. Right UO breast calcifications at 11:00 3 cm from nipple.

FINDINGS:
Both breasts are of heterogeneously dense composition.
Surgical scar present at lumpectomy site 10:00 5 cm from the nipple. Adjacent are post surgical changes and scarring and skin thickening. Surgical clips are also present at the surgical site.
Magnification views of the right breast show a 6 mm cluster of calcifications at 11:00 3 cm from the nipple. There is an adjacent 6 mm cluster of calcifications slightly more anteriorly. These are new compared to the previous mammograms from 6 months ago. Morphology is indeterminate. There are a few punctate microcalcifications at 12:00 in the right breast. Posterior depth.
Left breast negative for masses, calcifications or other findings.

IMPRESSION:
1. Interval development of new clusters of indeterminate morphology calcifications 11:00 right breast. 
Recommendation: Given interval development of these the finding is suspicious and stereotactic core biopsy is recommended. BI-RADS 4.

2. Left breast negative. BI-RADS 1. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammography was performed. The breast tissue is heterogeneously dense which may obscure small masses. 
There are scattered areas of fibroglandular density. No dominant mass, suspicious calcifications or architectural distortion.
Additional magnification views were performed of a group of calcifications in the upper outer left breast at the 2:00 position 8 cm from the nipple. The calcifications are punctate and amorphous and extend over an area measuring approximately 10 mm. They are new compared to the prior study and are suspicious for malignancy. An ultrasound was recommended for further evaluation.
Targeted left breast ultrasound demonstrates:
At 2:00 8 cm from the nipple, corresponding to the mammographic finding, there is an 11 x 5 x 6 mm irregular hypoechoic mass with indistinct margins and posterior shadowing. Increased vascularity is present.  
IMPRESSION:
New group of suspicious calcifications upper outer left breast at 2:00, 8 cm from the nipple with sonographic correlate demonstrating an irregular hypoechoic mass with posterior shadowing, highly suggestive of malignancy. Ultrasound guided biopsy is recommended. BIRADS 5.
Remainder of both breasts are BIRADS 2, benign. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Patient presents for routine screening mammogram. No palpable lumps, nipple discharge, or skin changes. No family history of breast cancer.

FINDINGS:
Both breasts are heterogeneously dense which could obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

In the left breast at the 10 o'clock position there is a small group of amorphous calcifications. These are more conspicuous compared to the prior study and have slightly increased in number. Additional magnification views are recommended for further evaluation.

The right breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete, need additional imaging evaluation. Recommend magnification views of the left breast for further assessment of the group of amorphous calcifications at 10 o'clock position. These calcifications are new compared to prior studies.

ADDENDUM:
Magnification views of the left breast at 10 o'clock demonstrate a group of amorphous calcifications within a focal asymmetry. The calcifications are indistinct and have slightly increased in number compared to the screening views. Biopsy is recommended for definitive characterization.

FINAL ASSESSMENT:
BI-RADS 4: Suspicious abnormality, biopsy should be considered. Small group of amorphous calcifications within a focal asymmetry at 10 o'clock position of the left breast, indistinct and increasing in number. Recommend stereotactic biopsy for further evaluation. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram and ultrasound images of the axilla dated April 14, 2023 were reviewed and compared to previous studies from April 20, 2022.

FINDINGS:
The breasts are heterogeneously dense (type C). 

There is an approximately 1 cm irregular mass at the 10:00 axis of the right breast, 5 cm from the nipple, which demonstrates internal vascularity on ultrasound. There are also scattered microcalcifications in the right upper outer quadrant spanning an area of approximately 2 cm.

In the left breast, there is an area of architectural distortion in the upper outer quadrant. Targeted ultrasound demonstrates an irregular hypoechoic mass with angular margins and posterior acoustic shadowing, measuring approximately 12 x 8 x 7 mm. The mass is located at 2:00, 4 cm from the nipple. 

A left axillary ultrasound demonstrates cortical thickening of a level I lymph node, measuring up to 5 mm in thickness.

IMPRESSION:
1. Suspicious irregular mass right breast at 10:00, 5 cm from the nipple with associated microcalcifications, suspicious for malignancy. BI-RADS 4C. Biopsy recommended.

2. Suspicious mass left breast at 2:00, 4 cm from the nipple with associated architectural distortion on mammography, highly suggestive of malignancy. BI-RADS 5. Biopsy recommended. 

3. Abnormal left axillary lymph node with cortical thickening, suspicious for metastatic involvement. Biopsy recommended for confirmation.

ADDENDUM:
Stereotactic core biopsy of the right breast mass and microcalcifications performed on April 16, 2023 demonstrates invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. 

Ultrasound-guided core biopsy of the left breast mass performed on April 16, 2023 demonstrates invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. Left axillary lymph node biopsy positive for metastatic carcinoma. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
INDICATION: 60-year-old woman, routine screening.
COMPARISON: Screening mammogram from 2 years prior.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses. 
There are no suspicious masses, architectural distortion, or concerning asymmetries.
Scattered fibroglandular densities are present bilaterally.
In the left breast at the 10:00 position, 6 cm from the nipple, there is a new cluster of fine pleomorphic microcalcifications spanning an area of approximately 8 mm. This was not present on the prior study. Additional magnification views in the ML and CC projections confirm these calcifications.
No suspicious calcifications are seen in the right breast.
No skin thickening or retraction. Nipples are within normal limits. No axillary lymphadenopathy.
IMPRESSION: 
New cluster of suspicious microcalcifications in the left breast at 10:00, 6 cm from the nipple, spanning 8 mm. BIRADS 4B. Stereotactic biopsy is recommended for further evaluation. The patient will be contacted to arrange the biopsy.
Remainder of the bilateral breast examination is benign. BIRADS 2 for the right breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 
No concerning dominant masses, architectural distortion or skin thickening.

There is a new cluster of pleomorphic microcalcifications in the upper outer quadrant of the right breast at approximately 10 o'clock, 7 cm from the nipple. The cluster measures approximately 5 mm. 

Stable bilateral axillary lymph nodes, likely benign.

IMPRESSION: 
BI-RADS 4: Suspicious cluster of microcalcifications in the right breast, biopsy recommended.

ADDENDUM: 
The patient returned for stereotactic biopsy of the right breast microcalcifications. Specimen radiograph confirmed calcifications within the core samples.
Pathology results: Ductal carcinoma in situ, high nuclear grade with necrosis.
Updated BI-RADS assessment: BI-RADS 6: Known biopsy-proven malignancy. ➡ *Positive*
- MAMMOGRAPHY REPORT
PROCEDURE: BILATERAL DIGITAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS 

HISTORY: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Tomosynthesis was performed. 

COMPARISON: None available.

FINDINGS:  
Right Breast:
The right breast demonstrates scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications are identified. There are no significant changes compared to prior examination.

Left Breast: 
The left breast shows heterogeneous fibroglandular tissue. In the upper outer quadrant, there is a new cluster of fine pleomorphic microcalcifications spanning approximately 8 mm. No associated mass or architectural distortion is seen. The remainder of the left breast is unremarkable. 

IMPRESSION:
1. Left breast: New cluster of microcalcifications in the upper outer quadrant, suspicious for ductal carcinoma in situ (DCIS). BIRADS 4B. Stereotactic biopsy is recommended for further evaluation.  

2. Right breast: Negative. BIRADS 1. Annual screening is recommended. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH MAGNIFICATION VIEWS.

History: 47 year old female with breast discomfort.

FINDINGS:
The breasts are fibroglandular composed of a mixture of fat and dense fibroglandular tissue.

There are no suspicious masses, calcifications, asymmetries or architectural distortion in the right breast. No skin thickening or nipple retraction.

In the left breast there are indeterminate amorphous calcifications grouped in a roughly linear distribution measuring 14 x 5 mm at the 12:00 position, 6 cm from the nipple.
 
No suspicious masses, asymmetries or architectural distortion are seen.

Magnification views confirm the presence of the calcifications in the left breast.

IMPRESSION:

BIRADS 0. Indeterminate calcifications left breast. Spot compression magnification views are recommended to further evaluate these calcifications. If the calcifications remain of concern biopsy should be considered.

No dominant mass or other suspicious findings are seen in either breast to correspond to the discomfort. Clinical correlation is suggested.

ADDENDUM:
Magnification spot compression views show the amorphous microcalcifications at the 12:00 position of the left breast to be grouped in a linear distribution spanning 15 mm. These are new compared to the prior examination. Stereotactic biopsy is recommended.

AMENDED IMPRESSION:
BIRADS 4B. Suspicious new grouped amorphous calcifications left breast. Stereotactic biopsy is recommended. ➡ *Positive*
- FINDINGS:
The breast tissue is heterogeneously dense. 
There is a 1.2 cm irregular mass at the 10:00 position of the right breast, approximately 4 cm from the nipple. The mass demonstrates spiculated margins and associated architectural distortion. 
At the 12:00 position of the right breast, there is a cluster of fine pleomorphic calcifications spanning approximately 7 mm. 
No suspicious masses, calcifications, or architectural distortion are identified in the left breast. 
The skin is normal in thickness and appearance. No axillary or internal mammary adenopathy is identified.

IMPRESSION:
1. Highly suspicious irregular spiculated mass right breast at 10:00, 4 cm from the nipple. Additional imaging evaluation with ultrasound is recommended. Consider biopsy.
2. Small cluster of suspicious pleomorphic calcifications at 12:00 right breast. Magnification views confirm. Stereotactic biopsy is recommended. 
3. Left breast unremarkable.

BI-RADS ASSESSMENT CATEGORY:
Right Breast: 4C 
Left Breast: 1 ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS AND TOMOSYNTHESIS
CLINICAL HISTORY: 60 year old female with family history of breast cancer
COMPARISON: Outside previous films from 2017
FINDINGS:
The breasts are heterogeneously dense.
In the right breast at 10:00 3 cm from nipple there is a benign appearing 9 mm circumscribed oval mass, likely a fibroadenoma. No other masses, suspicious calcifications, architectural distortion or abnormal asymmetries are seen in the right breast.
In the left breast, no discrete masses or architectural distortion are identified. At 2:00 8 cm from the nipple there is a new cluster of pleomorphic microcalcifications spanning approximately 6 mm. The distribution is segmental. No other suspicious calcifications are seen.
IMPRESSION:
RIGHT BREAST, BIRADS 2: Benign.
LEFT BREAST, BI-RADS 4B: Suspicious cluster of pleomorphic microcalcifications at 2:00, 8 cm from nipple. Stereotactic biopsy is recommended for further evaluation. Overall assessment is suspicious.
ADDENDUM: The patient returned for stereotactic guided left breast biopsy of the microcalcifications at 2:00. Specimen radiograph confirms microcalcifications were obtained. Final pathology shows invasive ductal carcinoma with associated DCIS, cribriform subtype, intermediate nuclear grade. Clip was placed at the biopsy site. ➡ *Positive*
- ULTRASOUND AND MAMMOGRAM OF LEFT BREAST:

CLINICAL HISTORY: 45 year old female with palpable lump in upper outer left breast. Previous mammograms from 2022 at St Michael's Hospital were compared.

FINDINGS:
The breast tissue is heterogeneously dense which may lower the sensitivity of mammography. 
There is a 1 cm hypoechoic irregular shaped mass at 2:00, 6 cm from the nipple on ultrasound. It demonstrates posterior acoustic shadowing. No increased vascularity is seen.
At the site of palpable abnormality in the upper outer left breast, there is a group of pleomorphic calcifications spanning 8mm. The calcifications are new compared to previous mammograms. 
No other suspicious masses, calcifications, asymmetries or architectural distortion in either breast. 
No skin thickening or retraction. The nipple and areola are unremarkable. 
There are no abnormal axillary lymph nodes.

IMPRESSION:
New group of suspicious pleomorphic calcifications at site of palpable abnormality in the upper outer quadrant of the left breast, spanning 8 mm, BI-RADS 4C. Ultrasound guided core biopsy is recommended for definitive diagnosis. 

1 cm irregular hypoechoic mass at 2:00 in the left breast, BI-RADS 4B. Ultrasound guided core biopsy is recommended.

No other suspicious findings.

BI-RADS 4C left, BI-RADS 1 right

ADDENDUM: Left breast core biopsy showed invasive ductal carcinoma at both biopsy sites, grade 2. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. Scattered areas of fibroglandular density.
A 12 mm irregular mass with indistinct margins and associated architectural distortion is noted at 2:00, 6 cm from the nipple in the right breast. 
Directly adjacent to this mass, there is a group of pleomorphic microcalcifications spanning approximately 8 mm. The calcifications are linear, branching and vary in size and shape, highly suspicious for malignancy. 
A prominent axillary lymph node is identified, measuring 15 mm in maximal dimension with marked cortical thickening up to 6 mm. This is a suspicious finding.
IMPRESSION:
Highly suspicious irregular mass with associated pleomorphic microcalcifications and architectural distortion right breast at 2:00, 6 cm from the nipple. Suspicious right axillary lymph node. Recommend biopsy of both the right breast mass and right axillary node for histologic diagnosis.
BIRADS 5. ➡ *Positive*
- FINDINGS:
Bilateral digital mammograms were performed with tomosynthesis views. The breasts are heterogeneously dense, which may limit sensitivity of mammography. 

There is a new 8 mm cluster of fine pleomorphic calcifications in the right breast at 10:00, 6 cm from the nipple. The calcifications are suspicious for malignancy. No other suspicious calcifications, masses, or areas of architectural distortion are identified in the right breast. 

In the left breast, there are scattered areas of fibroglandular density. No suspicious masses, calcifications, or areas of architectural distortion are identified.

IMPRESSION:
1. New cluster of suspicious calcifications in the right breast at 10:00, 6 cm from the nipple, suspicious for malignancy. Stereotactic core biopsy is recommended. BIRADS 4C. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography.
3. Probably benign findings left breast. BIRADS 2.

ADDENDUM:
The patient returned for stereotactic biopsy of the suspicious calcifications in the right breast at 10:00. Specimen radiograph confirmed that calcifications were sampled. Final pathology results showed ductal carcinoma in situ, intermediate nuclear grade, associated with microcalcifications. The patient will be referred to breast surgery for further management. ➡ *Positive*
- FINDINGS:
Bilateral screening mammogram demonstrates heterogeneously dense breasts. 
In the right breast there is a new 8 mm cluster of fine pleomorphic calcifications at the 10:00 position, 6 cm from the nipple. This is a suspicious finding. No other dominant masses, architectural distortion or suspicious calcifications.
In the left breast, there are no dominant masses, architectural distortion or suspicious calcifications. Stable benign calcifications are noted.

IMPRESSION:
New cluster of suspicious calcifications in the right breast at 10:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation. BI-RADS 4B.
The left breast is negative. BI-RADS 2.

ADDENDUM:
Right breast magnification views in the CC and MLO projections confirm an 8 mm cluster of fine pleomorphic calcifications at the site of concern. Stereotactic core biopsy is recommended for definitive diagnosis. ➡ *Positive*
- ULTRASOUND RIGHT BREAST AND MAMMOGRAPHY RIGHT BREAST
CLINICAL HISTORY: 65 year old female with history of left mastectomy presents for screening of the right breast.
FINDINGS: The breast is heterogeneously dense (category C).
Multiple magnification spot compression views of the right breast demonstrate a cluster of microcalcifications at the 9 o'clock position, 7 cm from the nipple. The calcifications are relatively coarse but pleomorphic in shape and grouped in distribution.
Right breast ultrasound was also performed. Targeted ultrasound over the area of mammographic calcifications at 9:00 demonstrates a vague area of hypoechogenicity measuring approximately 10 mm. No discrete mass, architectural distortion or posterior acoustic shadowing is seen.
The remainder of the right breast was scanned and there are no other sonographic abnormalities. No axillary lymphadenopathy.
IMPRESSION:
New cluster of microcalcifications at 9:00, 7 cm from the nipple in the right breast. Targeted ultrasound is suspicious. Stereotactic core biopsy is recommended for definitive diagnosis. BI-RADS 4B.
ADDENDUM: An ultrasound-guided core biopsy of the suspicious calcifications at 9:00 in the right breast was performed using a 14G spring-loaded biopsy device. Specimen radiograph confirms calcifications within the cores. Pathology is pending. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAM, BILATERAL:

INDICATION: 56 year-old woman with newly diagnosed left breast cancer.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally. Magnification images with a small focal compression paddle were then performed in mediolateral and craniocaudal projections of an area of microcalcifications in the upper outer right breast.

COMPARISON: None.

FINDINGS: 
The breasts are heterogeneously dense (type C). 

The left breast shows an irregular spiculated hyperdense mass measuring 2.3 cm located at 2 o'clock position, 4 cm from the nipple. Associated architectural distortion is also seen. The lesion was targeted under ultrasound and appears highly suspicious for malignancy (BI-RADS 5). No other suspicious masses, areas of asymmetry or distortion are seen in the left breast. The left skin is normal in thickness and there is no nipple retraction.

In the right breast, there is a cluster of pleomorphic microcalcifications in the right upper outer quadrant measuring 6 mm at 9 o'clock, 6 cm from the nipple (group 4). These are seen on both CC and MLO views. Additional magnification views confirm these calcifications are suspicious for malignancy (BI-RADS 4B). The remainder of the right breast shows only scattered areas of benign calcifications. No masses, architectural distortion, or skin thickening are seen on the right.

There are no abnormally enlarged axillary lymph nodes bilaterally.

IMPRESSION:
1. Known left breast cancer (BI-RADS 6).
2. Newly identified group of suspicious microcalcifications in the right upper outer breast, BI-RADS 4B. Stereotactic core needle biopsy is recommended for diagnosis. 

ADDENDUM: Stereotactic biopsy of the microcalcifications in the right breast was performed using a 9-gauge vacuum-assisted device. Specimen radiography confirmed calcifications within the samples. Pathology results are pending and will be reported separately. ➡ *Positive*
- FINDINGS:
Bilateral digital mammography with tomosynthesis was performed.
The breasts are heterogeneously dense which may obscure detection of small masses.
In the right breast, there are no dominant masses, architectural distortions or suspicious calcifications.
In the LEFT breast, at the 10:00 position 6 cm from the nipple, there is a new cluster of fine pleomorphic microcalcifications spanning approximately 8 mm. Additional magnification views confirm these are suspicious and new compared to prior mammograms from 2019. 
At the 2:00 position 4 cm from the nipple, there is a 6 mm circumscribed mass, likely benign.
The remainder of the left breast is unremarkable. 
There are no abnormal axillary lymph nodes bilaterally.
IMPRESSION:
LEFT BREAST, 10:00, 6 cm FN: New cluster of fine pleomorphic microcalcifications, suspicious for DCIS. BIRADS 4B. Stereotactic biopsy recommended.
LEFT BREAST, 2:00, 4 cm FN: 6 mm circumscribed mass, likely benign fibroadenoma. BIRADS 2. 
RIGHT BREAST: Negative. BIRADS 1. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
 
INDICATION: Annual screening mammogram.
 
COMPARISON: Outside screening mammogram dated 09/17/2020. 
 
FINDINGS:
Breast composition: Both breasts are heterogeneously dense, which can obscure small masses.
 
LEFT BREAST:
There is a cluster of amorphous microcalcifications in the 9 o'clock position of the left breast about 7 cm from the nipple. Additional magnification views were obtained for further evaluation.
No dominant masses, significant architectural distortion or suspicious microcalcifications are seen.
 
RIGHT BREAST:
No dominant masses, significant architectural distortion or suspicious microcalcifications are seen.
 
IMPRESSION:
LEFT BREAST:
New cluster of amorphous microcalcifications in the 9 o'clock position, left breast. Additional diagnostic evaluation with spot compression magnification views is recommended. BI-RADS 0: Incomplete.
 
RIGHT BREAST:
Negative. BI-RADS 1.
 
ADDENDUM:
Left diagnostic mammogram with spot compression magnification views demonstrates a cluster of amorphous microcalcifications in the 9 o'clock position. No associated mass or architectural distortion. 

FINAL ASSESSMENT:
LEFT BREAST: BI-RADS 4: Suspicious abnormality, biopsy recommended.
RIGHT BREAST: BI-RADS 1: Negative. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

TECHNIQUE: Bilateral digital screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.
No suspicious mass, architectural distortion or focal asymmetry in either breast.

Left breast:
At the 10:00 position in the left breast, there is a small group of amorphous calcifications measuring 4 mm, located 5 cm from the nipple. This is a new finding not previously noted.

Right breast:
No significant findings. No suspicious calcifications.

IMPRESSION:
BI-RADS Category 0: Incomplete, additional imaging needed to assess indeterminate calcifications in the left breast.

RECOMMENDATION:
Magnification views of the left breast are recommended to further evaluate the new group of amorphous calcifications at 10:00.

ADDENDUM:
Magnification views of the left breast were performed. The amorphous calcifications at 10:00, 5 cm from nipple, measure 4 mm. Biopsy is recommended.

FINAL IMPRESSION:
BI-RADS Category 4: Suspicious finding in the left breast. Biopsy recommended. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS AND ULTRASOUND

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammograms from 2019.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally. No dominant mass, suspicious calcifications, or other findings.

At the 10 o'clock position of the left breast, approximately 6 cm from the nipple, there is a new group of amorphous calcifications spanning an area of about 8 mm. These were not definitely seen on the prior study and are suspicious for early DCIS. Additional magnification views confirm these findings.

Supplemental screening ultrasound of both breasts was performed, showing no suspicious masses or other findings. Ultrasound is limited in evaluation of calcifications.

IMPRESSION:
1. New group of suspicious calcifications at the 10:00 position of the left breast, spanning 8 mm. Additional magnification views are suspicious for early DCIS. Stereotactic biopsy is recommended. BIRADS 4B.

2. Remainder of both breasts stable and benign. BIRADS 2.

ADDENDUM: The patient returned for stereotactic biopsy of the suspicious calcifications in the left breast, which confirmed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. The patient will be referred to breast surgery for further management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
COMPARISON: 12/5/2020 screening mammogram
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
There are scattered benign-appearing calcifications in both breasts, slightly increased in the medial left breast.
IMPRESSION:
BI-RADS Category 2: Benign
BI-RADS Density Grade C
RECOMMENDATION: Annual screening mammogram is recommended. Patient is advised to follow up for further imaging if a palpable abnormality develops.
ADDENDUM 12/7/2022
Upon further review of images, there is a new cluster of indeterminate microcalcifications in the upper outer left breast at the 2:00 position, 6 cm from the nipple, measuring approximately 8 mm in greatest dimension. The cluster is new compared to the prior study. Additional magnification views are recommended for further evaluation. BI-RADS category changed to 0: Incomplete, need additional imaging evaluation. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAPHY LEFT BREAST 
CLINICAL HISTORY: 42 year old female presents with new palpable mass left breast.   
FINDINGS:   
The breasts are heterogeneously dense.  
At 12:00 in the left breast, 4 cm from the nipple there is an irregular mass with spiculated margins measuring 15 x 12 mm. 
Also at 12 :00, more posteriorly, there are several adjacent masses with in distinct margins together measuring 26 x 21 mm.  
There are pleomorphic calcifications extending anteriorly from the 12:00 mass towards the nipple.  
No suspicious calcifications are seen in the remainder of the left breast.  
IMPRESSION:  
Highly suspicious mass left breast 12:00, 4 cm from the nipple with adjacent pleomorphic calcifications and satellite masses, suspicious for multifocal malignancy.
BIRADS 5 ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

In the left breast at the 10:00 position, 6 cm from the nipple, there is a small cluster of amorphous calcifications spanning approximately 6 mm. This is a new finding compared to the patient's prior mammograms. Additional spot compression magnification views of this area are recommended for further evaluation.

The right breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation. Small cluster of amorphous calcifications in the left breast at 10:00, 6 cm from the nipple. Additional spot compression magnification views are recommended for further evaluation.

ADDENDUM:

LEFT DIAGNOSTIC MAMMOGRAM

Spot compression magnification views were performed of the area of concern in the left breast at 10:00. The amorphous calcifications are more apparent on these views and span approximately 8 mm in greatest dimension. The distribution is clustered. No associated mass, architectural distortion, or skin thickening is identified.

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious abnormality. Stereotactic biopsy is recommended for tissue diagnosis of the amorphous calcifications in the left breast at 10:00. ➡ *Positive*
- FINDINGS:
Bilateral digital diagnostic mammography was performed.

The breasts are heterogeneously dense which could obscure detection of small masses.

In the left breast at 12:00, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of 10 mm. Additional magnification views confirm the presence of microcalcifications. No associated mass, architectural distortion or skin thickening.

At 6:00, 2 cm from the nipple in the left breast, there is a circumscribed, oval mass measuring 8 mm. Ultrasound shows this to be a simple cyst.

The remainder of the left breast is unremarkable. 

In the right breast, there are no dominant masses, areas of architectural distortion or suspicious microcalcifications.

IMPRESSION:
1. Group of amorphous microcalcifications at 12:00, left breast, spanning 10 mm. Additional imaging evaluation with magnification views confirms the presence of microcalcifications. Suspicious for DCIS. Stereotactic core biopsy recommended. BIRADS 4B.

2. 8 mm circumscribed mass at 6:00, left breast. Confirmed as simple cyst on ultrasound. Benign. BIRADS 2.

3. Heterogeneously dense breast tissue limits sensitivity of mammography. 

4. Remainder of both breasts are unremarkable. Overall BIRADS 4B. ➡ *Positive*
- FINDINGS:
The breasts are scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications are seen on either side.

In the LEFT breast at 2:00, 5 cm from the nipple there is a group of pleomorphic microcalcifications spanning 8 mm. The morphology is suspicious for ductal carcinoma in situ.

RIGHT BREAST: Negative.
LEFT BREAST: Highly suspicious for malignancy.

IMPRESSION:
BI-RADS Category 0: Left breast
Suspicious microcalcifications left breast at 2:00 as described. Additional magnification views recommended for further evaluation.  

BI-RADS Category 1: Right breast
Negative study.

ADDENDUM:
Magnification views of the left breast in the craniocaudal and mediolateral views show a group of pleomorphic microcalcifications at 2:00, 5 cm from the nipple measuring 1 cm. The morphology and distribution are highly suspicious for ductal carcinoma in situ. Stereotactic biopsy recommended.

FINAL ASSESSMENT:
BI-RADS Category 4C: Left breast
Highly suggestive of malignancy. Biopsy should be considered.

BI-RADS Category 1: Right breast.
Negative mammogram. ➡ *Positive*
- BILATERAL MAMMOGRAMS WITH TOMOSYNTHESIS
HISTORY: Significant family history of breast cancer. Intermediate lifetime risk. New microcalcifications on screening mammogram.
FINDINGS:
There are scattered areas of fibroglandular density. Overall, the breasts are heterogeneously dense which may obscure detection of small masses.
In the right breast there is a 6 mm cluster of indeterminate microcalcifications at the 10:00 position 6 cm from the nipple. A targeted ultrasound was performed which shows a 5 mm irregular hypoechoic mass corresponding to the area of the microcalcifications. This is a new finding.
The remainder of the right breast is unremarkable. No masses, architectural distortion or suspicious calcifications.
The left breast is unremarkable. No masses, architectural distortion or suspicious calcifications.
IMPRESSION:
New 6 mm cluster of indeterminate microcalcifications right breast 10:00 position 6 cm from the nipple. Targeted ultrasound shows a corresponding 5 mm irregular hypoechoic mass. Biopsy recommended. BIRADS 4B.
The remainder of the right breast and left breast are unremarkable. BIRADS 2.
ADDENDUM: Following discussion of results and management options with the patient, she wishes to proceed with a stereotactic core needle biopsy of the right breast microcalcifications. This has been arranged. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. In the left upper outer breast, there is a 5 mm cluster of pleomorphic microcalcifications at 2:30, 7 cm from the nipple. Additional magnification views confirm these calcifications have varying size and density and appear new compared to prior exams. Given their new appearance and pleomorphic morphology, these are suspicious for malignancy. No other suspicious calcifications or masses are seen in either breast. Specifically, no mass, asymmetry or architectural distortion is seen in the region of palpable concern at 8:00, 4 cm from the nipple of the left breast. The right breast demonstrates no suspicious masses, calcifications or areas of architectural distortion. The skin is normal bilaterally with no skin thickening or retraction. No suspicious axillary lymphadenopathy.

IMPRESSION:
New cluster of pleomorphic microcalcifications in the left upper outer breast at 2:30, 7 cm from the nipple, suspicious for malignancy. Stereotactic biopsy is recommended. BI-RADS 4C.

The remainder of the left breast is BI-RADS 2, benign finding. The palpable area of concern at 8:00, 4 cm from the nipple demonstrates no sonographic correlate and is likely benign, but clinical followup is recommended.

The right breast is BI-RADS 1, negative. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
CLINICAL HISTORY: 55 year old woman. No complaints. Annual screening.
TECHNIQUE: CC and MLO views of both breasts. CC and spot magnification CC views of the right breast.
COMPARISON: None.
FINDINGS:
Right Breast: The breast is heterogeneously dense (BI-RADS category C). There are scattered areas of fibroglandular density. In the upper outer quadrant there is a 7 mm group of indeterminate amorphous microcalcifications. No discrete mass, significant architectural distortion, or suspicious calcifications are identified.
Left Breast: The breast is heterogeneously dense (BI-RADS category C). There are scattered areas of fibroglandular density. No discrete mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
BI-RADS Category 0: Incomplete assessment. Need additional imaging evaluation of upper outer right breast microcalcifications. Recommend right CC and right ML spot compression magnification views.
ADDENDUM: CC and ML spot compression magnification views of the right breast demonstrate the microcalcifications to be amorphous, punctate, and grouped in an area measuring 9 mm in greatest dimension. No associated mass is identified.
REVISED IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality in the right breast. Suspicious grouped amorphous microcalcifications in the right upper outer quadrant. Recommend stereotactic biopsy.
BI-RADS Category 2: Benign finding in the left breast. ➡ *Positive*
- FINDINGS: Bilateral digital screening mammogram was performed. The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in the right breast. 

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant at approximately 2 o'clock, 6 cm from the nipple. This cluster measures approximately 6 mm in greatest dimension. No associated mass, architectural distortion or skin thickening is identified. 

The visualized left axillary lymph nodes appear unremarkable with no suspicious morphology or cortical thickening. 

Remainder of the left breast is unremarkable. No other suspicious calcifications, masses or areas of architectural distortion are identified.

IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation is needed. 
New cluster of suspicious microcalcifications in the left upper outer breast at 2:00, 6 cm from the nipple, measuring 6 mm. Recommend spot compression magnification views for further characterization. 

ADDENDUM: Spot compression magnification views of the left breast were performed. The cluster of calcifications in question persists and appears more concerning, demonstrating fine linear and branching morphology, suspicious for ductal carcinoma in situ (DCIS). Biopsy is recommended for definitive characterization.

REVISED IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered.
Suspicious cluster of pleomorphic microcalcifications left upper outer breast, spanning 6 mm. Stereotactic core biopsy recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM:
INDICATION: Annual screening mammography for a 58-year-old woman with heterogeneously dense breasts and no personal history of breast cancer.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses. 
There are scattered areas of fibroglandular density bilaterally. 
In the left breast at 9 o'clock 6 cm from the nipple there are new grouped amorphous microcalcifications in a segmental distribution spanning approximately 1 cm. This is a suspicious finding. No other dominant masses, architectural distortion or suspicious calcifications are identified in either breast.
Bilateral benign-appearing lymph nodes are noted in both axillae.
IMPRESSION:
New grouped amorphous microcalcifications in a segmental distribution at 9 o'clock 6 cm from the nipple of the left breast, suspicious for ductal carcinoma in situ (DCIS). Additional imaging evaluation with magnification views is recommended for further characterization. If these remain suspicious, stereotactic core biopsy should be considered.
BI-RADS 4B: Suspicious finding, biopsy should be considered.
The remainder of both breasts are BI-RADS 2: Benign findings. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS AND MAGNIFICATION VIEWS.

HISTORY: Routine screening. No breast complaints.

FINDINGS: The breasts have scattered fibroglandular tissue.

There are no suspicious right mammographic findings.

In the upper inner left breast 7.5 cm from the nipple there is a 4.5 mm cluster of amorphous calcifications. This is an indeterminate finding.

In the left subareolar breast there is a 1 cm low density mass. This is probably benign. The remainder of the left breast is unremarkable.

SUMMARY:
Small cluster of amorphous calcifications at 11:00 in the upper inner left breast. Since this patient has had no prior mammograms for comparison, these are considered an indeterminate finding (probably benign - BI-RADS 0). A short term 6 month follow-up mammogram is recommended to evaluate stability.

Probably benign mass left subareolar breast. BI-RADS 3. Follow-up mammogram 1 year. 

No suspicious findings right breast. BI-RADS 1.

ADDENDUM:
Discussed results with patient. She will return in 6 months for diagnostic spot compression magnification views of the left upper inner quadrant calcifications. She understands that a stereotactic biopsy may be considered at that time if there is any interval change or increase in the number or suspiciousness of the calcifications. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS AND ULTRASOUND LEFT BREAST
History: Palpable lump left breast.
Compared to previous exams dated November 2018.
FINDINGS:
The breasts are heterogeneously dense.
There is slight asymmetry between the breasts but otherwise no significant asymmetry or concerning areas of architectural distortion.
In the left lateral breast at 3:00 9 cm from the nipple is a 1 cm circumscribed oval hypoechoic mass with slight posterior shadowing. There is no internal vascularity. This was palpable on ultrasound. Ultrasound of the left axilla is unremarkable.
In the left upper breast at 11:00 6 cm from the nipple is a 7 mm group of clustered pleomorphic calcifications. These were not present on the November 2018 exam.
In the left breast laterally near the chest wall there are a few probably vascular calcifications.
The right breast is unremarkable.
BI-RADS 4B left for new cluster microcalcifications. Recommend stereotactic core biopsy. Probable fibroadenoma at 3:00 left breast - biopsy could be considered for a definitive diagnosis.
BI-RADS 1 right.
ADDENDUM: Patient called back for additional views. The group of microcalcifications in the left upper breast at 11:00 6 cm from the nipple is again seen and appears more prominent on these views. ➡ *Positive*
- REPORT:
"BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No known breast issues. 
COMPARISON: Compared to  prior studies from 07/2021.
FINDINGS: 
Right breast: 
There are scattered areas of fibroglandular density. No suspicious masses, calcifications, or areas of architectural distortion. 
Left breast:
There are scattered areas of fibroglandular density. In the upper outer quadrant, there is a new group of fine pleomorphic microcalcifications spanning approximately 6 mm. No associated mass, architectural distortion, or skin changes are seen. 
IMPRESSION:
RIGHT BREAST: BI-RADS 2 - Benign findings. Recommend continued routine screening in 1 year.
LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Biopsy should be considered to rule out malignancy. 
ADDENDUM: Stereotactic core biopsy of the left breast microcalcifications at 2 o'clock was performed. Pathology results showed atypical ductal hyperplasia with calcifications. Surgical consultation is recommended for further management. Post-biopsy BI-RADS category is 4A." ➡ *Positive*
- FINDINGS:
Bilateral digital mammography and targeted ultrasound of the right breast were performed.

The breasts are heterogeneously dense (C).

Right breast:
At the 12 o'clock position, 5 cm from the nipple, there is a new group of pleomorphic microcalcifications spanning an area of approximately 15 mm. Targeted ultrasound of this region demonstrates a 15 x 8 x 7 mm irregular hypoechoic mass with indistinct margins and internal vascularity. 

Additionally, there are scattered fibroglandular densities throughout the right breast. No dominant masses, architectural distortion, or suspicious calcifications are identified. 

Left breast:
There are no dominant masses, architectural distortion, or suspicious calcifications. Scattered areas of fibroglandular density are noted.

IMPRESSION:
Right breast: Highly suspicious group of pleomorphic microcalcifications at 12 o'clock with associated irregular mass on targeted ultrasound, BIRADS 5. Recommend stereotactic biopsy for definitive diagnosis.

Left breast: No suspicious findings, BIRADS 2. Recommend continued routine screening.

ADDENDUM:
The pathology report from the stereotactic biopsy of the right breast 12 o'clock lesion confirms invasive ductal carcinoma, grade 2, ER+/PR+/HER2-.

FINAL ASSESSMENT:
Right breast: BIRADS 6, known biopsy-proven malignancy.
Left breast: BIRADS 2, benign. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Screening mammogram

COMPARISON: None

FINDINGS:
The breasts are almost entirely fatty.

No dominant masses, suspicious calcifications or architectural distortion seen in either breast.

Scattered vascular calcifications noted bilaterally.

Benign-appearing lymph nodes in both axillae, containing small calcifications on the left.

In the left breast at the 10 o'clock position 6 cm from the nipple, there is a small cluster of indeterminate microcalcifications. Additional magnification views in this area are recommended for further evaluation.

IMPRESSION:
1. No suspicious mammographic findings in the right breast. BIRADS 1.
2. Small cluster of indeterminate microcalcifications in the left breast at 10:00, 6 cm from the nipple. BIRADS 0. Recommend additional magnification views for further assessment. 

ADDENDUM AFTER MAGNIFICATION VIEWS LEFT BREAST: 
The magnification views demonstrate a 6 mm cluster of pleomorphic microcalcifications at the site of concern. Stereotactic biopsy is recommended. BIRADS 4B. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH TOMOSYNTHESIS, SPOT COMPRESSION VIEWS OF THE RIGHT BREAST, AND TARGETED ULTRASOUND RIGHT BREAST
History: 52 year old with suspicious microcalcifications upper outer right breast. No previous imaging available for comparison.

FINDINGS: The breasts are heterogeneously dense.
In the right breast at 11:00, 7 cm from the nipple there is a suspicious cluster of pleomorphic microcalcifications spanning approximately 11 mm. Spot compression views magnify this cluster but show no associated mammographic findings. Targeted ultrasound was performed at the location of this cluster but no associated mass, distortion, or intraductal changes were seen.
The remainder of the right breast is unremarkable. There are no dominant masses or additional suspicious microcalcifications. No skin thickening or nipple retraction.
The left breast is unremarkable. No dominant masses, suspicious microcalcifications or other findings.

IMPRESSION:
Right breast with a suspicious 11 mm cluster of pleomorphic microcalcifications at 11:00, 7 cm from the nipple. No sonographic correlate. This cluster is highly suspicious for DCIS and stereotactic core biopsy is recommended. BIRADS 4C.
Normal/Benign left breast. BIRADS 1. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS:
History: Annual screening.
FINDINGS:
The breasts are heterogeneously dense. 
In the right breast, scattered areas of benign calcifications are evident. No dominant masses, architectural distortion, or suspicious calcifications.
In the left breast, a new group of fine pleomorphic microcalcifications is identified in the upper outer quadrant measuring approximately 1 cm. No associated mass, distortion, or skin changes.
Compared to prior study from 1 year ago, the left breast microcalcifications are new. The remainder of the breast tissue is stable.
IMPRESSION:
BIRADS 4 - Suspicious grouped microcalcifications in the left breast, not present on prior exam. Stereotactic biopsy recommended for further evaluation. 
BIRADS 2 - Benign right breast.
ADDENDUM:
The patient returned for stereotactic biopsy of the left breast microcalcifications. Specimen radiograph confirmed calcifications within the core samples. Final pathology revealed ductal carcinoma in situ, intermediate grade.
This case was reviewed in our weekly multidisciplinary breast conference. Recommendation is for surgical excision followed by radiation therapy. ➡ *Positive*
- REPORT:
"RIGHT BREAST Digital DIAGNOSTIC MAMMOGRAM:
HISTORY: Evaluate for abnormality.
COMPARISON: None.
FINDINGS:  
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
Bilateral MLO and CC views were performed.
There is an irregular speculated mass measuring approximately 15 mm in the upper outer quadrant of the right breast on MLO and CC views, about 10 cm from the nipple. Two smaller satellite lesions each about 6 mm in size are seen adjacent to the primary mass. Associated pleomorphic microcalcifications are seen extending anteriorly from the lesion over an area of about 3 cm.
The left breast appears normal.
No other suspicious masses, architectural distortion, or suspicious microcalcifications are seen in either breast. Benign scattered fibroglandular densities are noted.
The skin appears normal. No skin thickening or retraction. The nipples are normal. No significant axillary lymphadenopathy.
IMPRESSION: 
Right Breast:
1. 15 mm spiculated mass with adjacent satellite lesions and pleomorphic microcalcifications in the upper outer quadrant, highly suspicious for malignancy. Biopsy recommended. BI-RADS 5.
Left Breast: 
No mammographic evidence of malignancy. BI-RADS 1.
ADDENDUM: Ultrasound guided core biopsy of the right breast mass and satellities was performed using a 14 gauge needle. Specimen radiography confirmed calcifications within the biopsied tissue. The specimens were sent to pathology.
"
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

COMPARISON: None

FINDINGS:

Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no suspicious masses, architectural distortion, or skin thickening in either breast.

In the left breast at the 10:00 position, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. This is a new finding compared to the patient's prior mammograms. Additional magnification views confirm the presence of the microcalcifications.

The right breast is unremarkable, with no suspicious microcalcifications, masses, or areas of architectural distortion. 

IMPRESSION:

Left breast: BI-RADS 4B: Suspicious microcalcifications at 10:00, 6 cm from the nipple. Stereotactic biopsy is recommended for further evaluation.

Right breast: BI-RADS 1: Negative.

ADDENDUM:

The patient underwent stereotactic biopsy of the left breast microcalcifications. Pathology results show ductal carcinoma in situ (DCIS), intermediate nuclear grade, with no evidence of invasion. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS:
INDICATION: Annual screening mammogram.
COMPARISON: None.
FINDINGS:
Both breasts show scattered areas of fibroglandular density. There are no dominant masses, focal asymmetries, or areas of architectural distortion.
In the left breast at the 12:00 position, 6 cm from the nipple, there is a cluster of amorphous calcifications within an area measuring approximately 8mm. There are no associated masses or architectural distortion.
In the right breast, there are no suspicious calcifications.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for further evaluation of the cluster of amorphous calcifications at 12:00 position, left breast, 6 cm from the nipple.
BI-RADS Category 1: Normal right breast.
ADDENDUM:
The patient returned for stereotactic-guided biopsy of the cluster of amorphous calcifications at the 12:00 position of the left breast on [DATE]. Pathology results showed benign fibrocystic changes with calcifications. No evidence of malignancy.
REVISED IMPRESSION:
BI-RADS Category 2: Benign finding in the left breast. Recommend routine screening in 1 year.
BI-RADS Category 1: Normal right breast. ➡ *Positive*
- DIAGNOSTIC BILATERAL DIGITAL MAMMOGRAPHY AND TARGETED ULTRASOUND LEFT BREAST
INDICATION: Left breast lump.
COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally.

At the 6 o'clock position of the left breast, 4 cm from the nipple, there is an irregular hypoechoic mass with indistinct margins and posterior acoustic shadowing, measuring approximately 12 x 8 x 10 mm. Color Doppler demonstrates internal vascularity. This is highly suspicious for malignancy. 

No suspicious right breast masses, architectural distortion, or skin thickening.

There is a 5 mm cluster of amorphous microcalcifications in the left breast at 3 o'clock, 6 cm from the nipple. Additional magnification views confirm these are not vascular calcifications. 

IMPRESSION: 
1. Highly suspicious irregular mass left breast 6 o'clock position, 4 cm from the nipple. Ultrasound-guided core biopsy is recommended. BIRADS 5.

2. 5 mm cluster of amorphous microcalcifications left breast at 3 o'clock, 6 cm from the nipple. Stereotactic-guided biopsy is recommended. BI-RADS 4.

3. Heterogeneously dense breast tissue. Consider supplemental screening with breast MRI in addition to mammography for optimal cancer detection. BI-RADS 1 right breast, BI-RADS 4 and 5 left breast.

ADDENDUM: 
Stereotactic-guided biopsy of the amorphous calcifications in the left breast at 3 o'clock was performed using an 8-gauge vacuum-assisted device. Specimen radiograph confirmed calcifications. Final pathology revealed atypical ductal hyperplasia with microcalcifications. ➡ *Positive*
- FINDINGS: The breasts are heterogeneously dense. 

There are no suspicious findings in the right breast. BIRADS 1.

In the central region of the left breast at 12:00, 5 cm from the nipple, there is a new 8 mm cluster of pleomorphic microcalcifications. Magnification views confirm the presence of this cluster. No associated mass, distortion or asymmetry. 

At 6:00 in the left breast, there is a stable 12 mm circumscribed oval mass, likely benign.

The left axilla is unremarkable with no abnormal lymph nodes.

IMPRESSION:
New cluster of suspicious microcalcifications at 12:00 in the left breast, BIRADS 4. Recommend stereotactic biopsy for definitive diagnosis. 

The remainder of the left breast is BIRADS 2 with a likely benign finding. 

Normal right breast, BIRADS 1. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM
COMPARISON: None.
INDICATION: 47-year-old female with left breast mass, palpable, painful, enlarging over 2-3 months.
TECHNIQUE: Bilateral CC and MLO views were obtained. Spot compression magnification views obtained of the left breast.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
Left breast: A 1.8 x 1.5 x 1.3 cm irregular mass is present at the 2 o'clock position, 6 cm from the nipple. The margins are indistinct and angulated. Associated architectural distortion is noted. Several groups of pleomorphic calcifications are identified adjacent to the mass, the largest cluster measuring 7 mm. No axillary lymphadenopathy.
Right breast: No dominant mass, suspicious calcifications, or other findings.
IMPRESSION:
Highly suspicious findings left breast, BI-RADS 5. Recommend ultrasound-guided core biopsy for further evaluation. Surgical consultation is also advised.
Benign right breast, BI-RADS 2. Recommend annual screening mammogram in 1 year. ➡ *Positive*
- FINDINGS:

BILATERAL DIGITAL MAMMOGRAPHY:

The breasts are predominantly fatty replaced.

In the right breast, at the 11:00 position, 5 cm from the nipple, there is a 1.2 cm area of grouped amorphous microcalcifications, new since the prior exam.

No suspicious masses or areas of architectural distortion are identified in either breast.

No suspicious right axillary lymph nodes are identified.

Old surgical clips noted in the left breast at site of prior lumpectomy and radiation therapy.

IMPRESSION:

BI-RADS 4 - Suspicious grouped amorphous microcalcifications in the right breast at 11:00, 5 cm from the nipple. Stereotactic core biopsy recommended for definitive characterization.

BI-RADS 2 - Benign post surgical changes left breast.

ADDENDUM:

Stereotactic core biopsy of the suspicious microcalcifications in the right breast was performed. Pathology results are pending and will be communicated in an addendum to this report when available.

PATHOLOGY ADDENDUM:

Stereotactic core biopsy right breast calcifications: Invasive ductal carcinoma, moderately differentiated, with associated ductal carcinoma in situ, high nuclear grade with necrosis.

FINAL ASSESSMENT:

RIGHT BREAST: BI-RADS 6 - Biopsy proven malignancy
LEFT BREAST: BI-RADS 2 - Benign ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
COMPARISON: 2019-10-05 Humber River Hospital.
FINDINGS:
Both breasts are almost entirely fatty.
There are multiple round and calcified fibroadenomas in the right breast, the largest measuring 16 mm at 1:00, 6 cm from the nipple. No new dominant masses, suspicious calcifications or architectural distortion.
In the left breast, there is a new 8 mm cluster of pleomorphic calcifications at 3:00, 5 cm from the nipple. Magnification views confirm these are suspicious. No associated mass is seen on the mammogram or targeted ultrasound.
Also present in the left breast are a few intact intramammary lymph nodes and multiple simple cysts. No other dominant masses, suspicious calcifications or architectural distortion.
IMPRESSION:
BI-RADS 1 - Right breast.
BI-RADS 4B - Left breast. Suspicious cluster of pleomorphic calcifications 3:00, 5 cm from the nipple. Recommend stereotactic core biopsy.
ADDENDUM:
LEFT BREAST, 3:00, 5 cm from the nipple - Stereotactic guided core biopsy performed using 9G vacuum assisted needle. Specimen radiograph confirms calcifications. Marker clip placed. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

At the 9 o'clock position of the right breast there is a group of amorphous microcalcifications spanning an area of approximately 1 cm. Additional magnification views in the right CC and MLO projections confirm these calcifications. 

The visualized skin and nipple are unremarkable. No axillary, supra or infraclavicular adenopathy.

IMPRESSION: 
BI-RADS 4: Suspicious grouped amorphous microcalcifications in the right breast at 9 o'clock position. Stereotactic biopsy is recommended for further evaluation. The patient should return to clinic to discuss results.

ADDENDUM: The patient returned for stereotactic biopsy of the group of calcifications in the right breast at the 9 o'clock position. Specimen radiograph confirmed that the calcifications were adequately sampled. Final pathology results showed benign fibrocystic changes with microcalcifications, consistent with the imaging findings. No evidence of malignancy. ➡ *Positive*
- BILATERAL MAMMOGRAMS 
INDICATION: Annual screening.
TECHNIQUE: Bilateral digital mammograms were obtained including standard CC and MLO views. 
COMPARISON: Compared to prior mammograms from 05/10/2021.
FINDINGS:
Both breasts are heterogeneously dense (BI-RADS density C). 
There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast.
In the left breast there are scattered fibroglandular densities. No dominant masses or architectural distortion. 
There is a new group of pleomorphic microcalcifications in the left upper outer quadrant, extending over an area of about 1 cm. The microcalcifications range from linear to round in shape and are of intermediate concern. 
Remaining left breast is unremarkable.
IMPRESSION:
BI-RADS 1 right breast: Negative.
BI-RADS 4B left breast: Suspicious microcalcifications upper outer quadrant, biopsy recommended.
ADDENDUM:
Ultrasound-guided core biopsy of the left breast microcalcifications at 2:00, 8 cm from the nipple showed ductal carcinoma in-situ, intermediate grade, cribriform type.
Stereotactic biopsy is recommended for further characterization and to rule out invasive component. Updated BI-RADS assessment BI-RADS 4C. ➡ *Positive*
- FINDINGS:

Heterogeneously dense breasts.

Comparison is made to a study performed 2 years ago at an outside facility.

No suspicious findings are seen in the right breast. BI-RADS 1.

Left breast:

There is an irregular spiculated mass at the 3:00 position, 4cm from the nipple. The mass measures approximately 2.5cm. Associated architectural distortion is noted.

Additional scattered groups of amorphous and pleomorphic microcalcifications are seen in the left breast, most prominently in the upper outer quadrant. The largest cluster measures 1.2cm.

There are additional smaller masses suspicious for multifocal disease.

In the left axilla, enlarged abnormal lymph nodes are seen, with cortical thickness up to 1cm and loss of the normal fatty hilum. Findings are concerning for metastatic adenopathy.

The left nipple is retracted.

The remainder of the left axilla is unremarkable.

IMPRESSION:

Highly suspicious mass left breast at 3:00, 4cm from the nipple.

Numerous clusters of suspicious microcalcifications in the left breast, highest suspicion upper outer quadrant.

Multifocal disease and nipple retraction are present.

Abnormal left axillary lymph nodes concerning for metastatic disease.

BI-RADS 5: Highly suggestive of malignancy. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Annual screening.

TECHNIQUE: Bilateral full field digital mammogram performed including CC and MLO views of each breast. 

FINDINGS:

Right Breast:
The right breast is almost entirely fatty with scattered areas of fibroglandular density. Overall, the breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses, architectural distortions or suspicious calcifications.

Left Breast: 
The left breast shows scattered fibroglandular densities in a fatty background. Similar to the right, the breast is heterogeneously dense (BI-RADS density category C). Within the upper outer quadrant of the left breast, there is a new cluster of fine pleomorphic microcalcifications at the 2 o'clock position, approximately 6 cm from the nipple. This cluster measures about 8 mm in greatest dimension. No associated mass, architectural distortion or skin thickening is seen. The remainder of the left breast is unremarkable.

Axillae: 
There are no enlarged axillary lymph nodes bilaterally.

Skin: 
The skin appears normal bilaterally with no evidence of skin thickening or retraction.

IMPRESSION:
1. New cluster of pleomorphic microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple, measuring approximately 8 mm, suspicious for malignancy. Additional magnification views recommended for further evaluation. BI-RADS 4B.

2. No suspicious findings in the right breast. Continue routine annual screening. BI-RADS 1.

RECOMMENDATION:
Left breast diagnostic mammogram with magnification views of the area of microcalcifications, followed by stereotactic core biopsy if indicated based on additional imaging. 

ADDENDUM:
Magnification views of the left breast confirm a cluster of pleomorphic microcalcifications at 2 o'clock, 6 cm from the nipple, highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy recommended. Updated BI-RADS assessment of the left breast: 4C. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was performed.
The breast parenchymal density is heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered areas of fibroglandular density bilaterally.
No dominant mass, area of architectural distortion or abnormal calcifications are seen in the right breast. 
In the left breast, there are scattered fibroglandular densities. No dominant mass or architectural distortion. There is a small cluster of pleomorphic microcalcifications identified in the upper outer quadrant, approximately 8 cm from the nipple. Further evaluation with magnification compression views is recommended.
No mammographic evidence of malignancy in the right breast. 
There are indeterminate microcalcifications in the left breast, further workup recommended.

IMPRESSION:
BI-RADS Category 0: Incomplete, further imaging evaluation needed. Small cluster of pleomorphic microcalcifications in the left upper outer quadrant, magnification compression views recommended for further characterization. 
BI-RADS Category 1 for the right breast: Negative.

ADDENDUM:
Magnification views of the left breast upper outer quadrant were performed. The previously identified cluster of microcalcifications are pleomorphic in morphology and have a segmental distribution. Biopsy is recommended for tissue diagnosis.
Final Assessment:
BI-RADS Category 4 for the left breast: Suspicious abnormality, biopsy should be considered. 
BI-RADS Category 1 for the right breast: Negative. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
At the 10:00 position of the right breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution measuring approximately 8 mm. Additional magnification views confirm the presence of these microcalcifications.
The remaining fibroglandular tissue of both breasts is unremarkable.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic biopsy is recommended for further evaluation of the group of amorphous microcalcifications at the 10:00 position of the right breast.
BI-RADS Category 2 for the left breast: Benign findings. Recommend routine screening in 1 year. ➡ *Positive*
- FINDINGS:
Both breasts are heterogeneously dense. There are scattered areas of fibroglandular density. No dominant mass, architectural distortion, or definite asymmetry in either breast.

At the 10 o'clock position of the RIGHT breast, there is a group of amorphous calcifications spanning an area of about 1 cm. The calcifications are new compared to prior mammograms from 2 years ago. Additional magnification views confirm these are suspicious calcifications.

The LEFT breast is unremarkable. No suspicious calcifications, masses, architectural distortion or asymmetries in the left breast.

IMPRESSION:
BI-RADS Category 4: Suspicious grouped amorphous calcifications in the right breast at 10 o'clock position. Biopsy is recommended for definitive characterization. Malignancy cannot be excluded.

BI-RADS Category 1: Normal left breast. Annual screening mammography is recommended.

ADDENDUM: 
Stereotactic biopsy of the grouped calcifications in the right breast at 10 o'clock position was performed. Pathology results demonstrate ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended for complete removal and definitive treatment. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAPHY LEFT BREAST

CLINICAL HISTORY: 57-year-old female with new palpable lump in the upper outer quadrant of the left breast. 

TECHNIQUE: Diagnostic mammographic views of the left breast were obtained including CC and MLO views with spot compression and magnification views of the area of palpable concern.

FINDINGS:
The left breast demonstrates scattered fibroglandular densities.

There is an irregular shaped mass measuring approximately 2.5 x 1.5 cm in the upper outer quadrant of the left breast at the 2:00 position, 8 cm from the nipple, corresponding to the area of palpable concern. The margins of this mass are indistinct and spiculated. 

Additionally, there are several groups of pleomorphic microcalcifications extending anteriorly from the mass, distributed in a segmental pattern. The largest cluster measures 12 mm. 

The skin appears normal. The nipple is normal in appearance. No suspicious axillary lymphadenopathy.

IMPRESSION:
1. Highly suspicious mass with associated pleomorphic microcalcifications upper outer left breast, BI-RADS 5. Recommend tissue diagnosis. ➡ *Positive*
- FINDINGS: The breasts are heterogeneously dense.

Within the right breast there are no mammographically suspicious masses, calcifications, or other findings. BIRADS 1.

Within the upper outer left breast there is an 8 mm loose cluster of heterogeneous calcifications ranging from amorphous to fine pleomorphic in morphology. This is indeterminate and requires further evaluation with magnification views. There are no other suspicious mammographic findings in the left breast.

Targeted ultrasound of the left breast demonstrated a few small simple cysts but no sonographic correlate for the calcifications. No other sonographically suspicious masses or findings.

IMPRESSION:
1. BI-RADS 0: 8 mm cluster of indeterminate calcifications upper outer left breast. Magnification views recommended for further evaluation.
2. BI-RADS 1: Right breast. Negative.

ADDENDUM - MAGNIFICATION VIEWS LEFT BREAST: The magnification views of the upper outer left breast redemonstrate the cluster of amorphous to fine pleomorphic calcifications. These measure approximately 8 mm in greatest dimension. At least 8 calcifications are present. 

REVISED IMPRESSION: BI-RADS 4: Suspicious cluster of calcifications upper outer left breast. Stereotactic biopsy is recommended. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS LEFT BREAST AND ULTRASOUND LEFT BREAST.
History: 2 months follow up left breast microcalcifications. No other breast complaints. No family history of breast cancer.
Compared to previous studies from April 2023. FINDINGS: The breasts are heterogeneously dense. 
There are no suspicious right mammographic findings. 
In the upper outer quadrant of the left breast, there is a new 5 mm cluster of pleomorphic microcalcifications at 2:00, 7 cm from the nipple. This was not present on the prior mammogram. Targeted ultrasound of this area does not show a correlating mass. The remainder of the left breast is unchanged with scattered fibroglandular densities and no suspicious masses, architectural distortion or additional suspicious calcifications.
Ultrasound of the left axilla is unremarkable with no abnormal lymph nodes.
IMPRESSION:
New cluster of suspicious pleomorphic microcalcifications upper outer left breast, BI-RADS 4B. Stereotactic core biopsy is recommended for tissue diagnosis. A requisition will be sent to book the biopsy.
Stable remainder of left breast, BI-RADS 2. Recommend annual screening mammogram in 1 year if biopsy is benign.  
No suspicious findings right breast, BI-RADS 1. Recommend annual screening mammogram in 1 year.
ADDENDUM: Patient called back and stereotactic core biopsy was performed the same day of the new cluster of microcalcifications in the left breast at 2:00. Specimen x-ray confirmed calcifications in the cores. Post-procedure mammogram showed biopsy clip in good position. The patient tolerated the procedure well. Pathology results are pending and will be communicated to referring physician. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense.
On the right, there is no mammographic evidence of malignancy.  The breast is almost entirely fatty.  No masses, suspicious calcifications, architectural distortion or skin thickening are identified.
There are scattered benign appearing calcifications in the left breast. Particularly in the upper outer quadrant, a new cluster of pleomorphic microcalcifications is identified spanning a region of approximately 1.5 cm. This is a suspicious finding, highly concerning for malignancy such as ductal carcinoma in situ.  Otherwise, no spiculated mass, asymmetry or architectural distortion is seen in the left breast.
Both nipples are within normal limits.  There are no skin lesions or lymphadenopathy evident.
IMPRESSION:
BI-RADS 2: Benign finding right breast. Recommend routine screening in 1 year.
BI-RADS 4: Suspicious abnormality left breast - new cluster of pleomorphic microcalcifications in the upper outer quadrant. Biopsy is recommended.
ADDENDUM: Stereotactic biopsy of microcalcifications at left 2:00, 6 cm from nipple performed today. Specimen radiograph confirms presence of microcalcifications. Pathology results pending. ➡ *Positive*
- ULTRASOUND LEFT BREAST:
Indication: Palpable lump left upper outer quadrant. No known family history of breast cancer. 
Technique: Sonographic evaluation of the left breast was performed with a high frequency linear transducer.
Findings:
The left breast is heterogeneously dense.  
At the site of palpable abnormality in the 2:00 position 8 cm from the nipple, there is a 12 mm irregular solid mass with indistinct margins and internal vascularity. This corresponds to the patient's palpable finding.  
Immediately adjacent to this mass, there is a 6 mm cluster of pleomorphic microcalcifications.
The remainder of the left breast was scanned. No other suspicious masses or calcifications are identified. No suspicious left axillary lymphadenopathy.
Impression: 
1. Suspicious irregular solid mass left breast at site of palpable abnormality. Biopsy recommended. 
2. Cluster of pleomorphic microcalcifications adjacent to the mass, also suspicious. Stereotactic biopsy recommended if not removed with mass biopsy.
BI-RADS 4C. Moderate suspicion for malignancy. ➡ *Positive*
- BILATERAL MAMMOGRAMS WITH TOMOSYNTHESIS
CLINICAL HISTORY: Annual screening mammogram. 
TECHNIQUE: Bilateral full field digital mammograms including tomosynthesis were obtained.
COMPARISON: None. 
FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense (BI-RADS C). No dominant mass, architectural distortion or suspicious calcifications are identified. 
LEFT BREAST:
The breast is heterogeneously dense (BI-RADS C). There is a small cluster of indeterminate microcalcifications in the upper outer quadrant, approximately 5 cm from the nipple. No dominant mass, architectural distortion or lymphadenopathy.
IMPRESSION:
RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS 1, annual screening recommended.
LEFT BREAST: Small cluster of indeterminate microcalcifications in the upper outer quadrant, additional magnification views recommended for further characterization. BI-RADS 0, recall for additional imaging.
ADDENDUM - LEFT BREAST MAGNIFICATION VIEWS: 
Additional magnification views of the left breast demonstrate a small cluster of indeterminate microcalcifications in the upper outer quadrant, approximately 5 cm from the nipple. No associated mass or architectural distortion.
FINAL ASSESSMENT: 
RIGHT BREAST: BI-RADS 1. Annual screening recommended.
LEFT BREAST: BI-RADS 4. Suspicious cluster of microcalcifications. Biopsy recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.
There are no suspicious masses or architectural distortions identified in either breast.
In the left breast, there is a new group of pleomorphic microcalcifications spanning approximately 10 mm at the 9:00 position, 4 cm from the nipple. This cluster is located more posteriorly, which may limit tissue acquisition on stereotactic biopsy. Additional angled views are recommended for further characterization. 
Additionally, a previously noted benign appearing partially calcified mass measuring 10 mm is stable in the right breast at the 11:00 position 6 cm from the nipple. This likely represents a fibroadenoma.
No suspicious axillary lymphadenopathy is identified.
IMPRESSION: 
BI-RADS 0: Incomplete. New group of pleomorphic microcalcifications spanning 10 mm in the left breast at 9:00, 4 cm from the nipple. Additional angled views are recommended for further characterization. If confirmed as suspicious, biopsy is recommended.
BI-RADS 2: Benign. Benign appearing mass, stable in the right breast. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH MAGNIFICATION VIEWS.

History: 47 year old asymptomatic female for bilateral screening mammograms. 

Compared to outside prior mammogram from 2019-10-07 at SJHC. 

FINDINGS:
The breasts are heterogeneously dense. 
There are no dominant masses, architectural distortion or suspicious calcifications in the right breast.
At the 12 o'clock position of the left breast, approximately 3 cm from the nipple, there is a 1 cm group of pleomorphic microcalcifications. Magnification views confirm the presence of this cluster. No other suspicious findings are identified in the left breast.
Bilateral axillary regions are unremarkable. 
Metastatic calcifications are present in the skin overlying the left breast. 

SUMMARY:
BIRADS 4 left breast: New group of suspicious microcalcifications at 12 o'clock position of the left breast. Recommend stereotactic core biopsy. 
BIRADS 1 right breast: No mammographic evidence of malignancy.

ADDENDUM:
Ultrasound left breast 12:00 position 3 cm from nipple: Sonographically this corresponds to a vague area of hypoechogenicity with no definite mass. Biopsy is still recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Annual screening mammogram. No breast complaints. The patient is a 52-year-old female. Family history of breast cancer in mother diagnosed at age 60.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or abnormal calcifications in either breast.
Bilateral benign-appearing calcifications are noted. These are coarse, round, and punctate in morphology.
At the 10:00 position of the left breast, there is a small cluster of amorphous microcalcifications spanning approximately 6 mm. This cluster was not present on the prior examination.

IMPRESSION:
1. Heterogeneously dense breasts.
2. Bilateral benign-appearing calcifications, likely representing fibrocystic changes.
3. New small cluster of amorphous microcalcifications at the 10:00 position of the left breast, suspicious for early ductal carcinoma in situ (DCIS). Additional imaging evaluation with magnification views is recommended for further characterization.

RECOMMENDATION: Left diagnostic mammogram with magnification views of the microcalcifications at the 10:00 position. Consider stereotactic biopsy if the microcalcifications remain suspicious on magnification views.

BI-RADS CATEGORY 4: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- ULTRASOUND AND MAMMOGRAM BREASTS BOTH SIDES
HISTORY: 52 year old female with history of BRCA1 mutation.
COMPARISON: Annual screening mammograms from 2 years ago at outside hospital.

FINDINGS:
The breasts are heterogeneously dense.
There are no new masses, architectural distortion, or suspicious findings identified in either breast.
On ultrasound, there are bilateral scattered benign appearing cysts.
At the 10:00 position in the left breast 5 cm from the nipple there is a group of worrisome microcalcifications spanning 8 mm. Spot compression magnification views were obtained. The calcifications appear linear and branching, highly suspicious for ductal carcinoma in situ.
At the 3:00 position 3 cm from the nipple in the right breast, there are two small oval hypoechoic solid masses measuring 5 and 6 mm. These demonstrate no internal vascularity and are likely benign fibroadenomas. Short term follow up in 6 months is recommended for these lesions.
The remaining right breast tissue is unremarkable. No lymphadenopathy is seen.

BIRADS ASSESSMENT:
Right breast: BI-RADS 3. Probably benign findings. Short interval follow up recommended.
Left breast: BI-RADS 4C. Highly suspicious for malignancy. Stereotactic core biopsy recommended.

ADDENDUM:
Stereotactic core biopsy of the suspicious left breast calcifications performed under ultrasound guidance using 14 gauge needle. Specimen radiograph confirms calcifications in sample. Pathology specimen labeled "Left breast, 10:00, 5 cm from nipple" and sent to pathology. The patient tolerated the procedure well. Hemostasis achieved with manual compression. Mammographic markers placed at biopsy site. The patient was instructed on post-procedure care and will await pathology results. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH CAD 

History: Screening exam. No previous mammograms available for comparison. No significant family history of breast cancer. No clinical concerns.

FINDINGS:
The breasts are mostly fatty.
There are scattered benign calcifications bilaterally.
In the left breast at 6:00, 10 cm from the nipple there is a 10 mm cluster of indeterminate microcalcifications.
No spiculated masses, architectural distortion or suspicious asymmetries are seen.
No skin thickening or nipple inversion. 
CAD was applied as a second reader. No additional findings seen.

IMPRESSION:
New 10 mm cluster of indeterminate microcalcifications left breast at 6:00. Additional magnification views recommended. BI-RADS category 0: incomplete. A diagnostic mammogram is recommended for further evaluation.

ADDENDUM:
Patient returned for additional magnification views of the left breast. Spot compression magnification views were performed at 6:00. 

FINDINGS:
The additional magnification views confirm a 10 mm cluster of pleomorphic microcalcifications at 6:00 in the left breast. These are of intermediate concern.  
No associated mass.
The remainder of the left breast mammogram is unremarkable.

IMPRESSION:
10 mm cluster of pleomorphic microcalcifications left breast at 6:00, BI-RADS 4B. Stereotactic core biopsy is recommended. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. 
In the left breast at 11:00 5 cm from the nipple there is a group of pleomorphic calcifications in a segmental distribution measuring 8 mm. Additional magnification views confirm these are suspicious for malignancy.
At 1:00 location 3 cm from the nipple there is an irregular spiculated mass with associated architectural distortion measuring 12 x 8 mm. 
Ultrasound of the left breast in the area of mammographic concern at 11:00 demonstrates a 6 mm irregular solid mass with angular margins and shadowing. This corresponds to the mammographic calcifications. Color Doppler shows internal vascularity.
At the 1:00 location ultrasound shows the mammographic mass to be a 10 x 7 x 8 mm irregular hypoechoic mass with indistinct margins, posterior shadowing and internal vascularity.
Ultrasound of the left axilla reveals an enlarged 15 mm cortically thickened lymph node with loss of the fatty hilum.
The right breast is unremarkable. No dominant masses, architectural distortion or suspicious calcifications.

IMPRESSION:
Highly suspicious group of segmental pleomorphic calcifications left breast 11:00 location with sonographic correlate. Suspicious mass left breast 1:00. Abnormal left axillary lymph node. Recommend biopsy. BI-RADS 5.
Negative right breast. BI-RADS 1. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAMS
INDICATION: Family history of breast cancer, maternal aunt in her 60s.
TECHNIQUE: Hologic Digital Breast Tomosynthesis, CC and MLO views.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
No dominant masses, architectural distortion, or suspicious asymmetries.
There are scattered benign-appearing punctate and short segmental calcifications in both breasts.
In the medial right breast, there is a small cluster of indeterminate amorphous calcifications at 3:00, 4 cm from the nipple.
IMPRESSION:
BIRADS 0 - Incomplete. Additional imaging evaluation is needed.
Small cluster of indeterminate amorphous calcifications medial right breast. Recommend diagnostic mammographic views with magnification for further characterization.
The patient is advised to have these additional images today if possible, or otherwise scheduled as soon as possible. A final BIRADS assessment will be made after additional imaging. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

FINDINGS:

Both breasts are almost entirely fatty.

There are scattered benign-appearing calcifications in both breasts. In the upper outer quadrant of the left breast there is a small group of indeterminate microcalcifications. Spot compression magnification views confirm a cluster of pleomorphic microcalcifications spanning about 6 mm at the 2:00 position, approximately 6 cm from the nipple. No other suspicious calcifications, masses or architectural distortion are identified.

IMPRESSION:
BIRADS 0: Small group of indeterminate microcalcifications in the left breast. Additional imaging evaluation with magnification views is recommended for further assessment. The patient should return for additional mammographic workup as well as left breast ultrasound to evaluate the area of concern. The results will be communicated to the referring physician.

BIRADS 2: Benign findings right breast.

ADDENDUM:
Magnification views of the left breast demonstrate pleomorphic microcalcifications spanning 7 mm at the 2:00 position of the left breast, 6 cm from the nipple, suspicious for DCIS. Ultrasound of this area shows no correlate mass or dilated ducts. Stereotactic biopsy is recommended for definitive diagnosis.

BIRADS 4: Suspicious microcalcifications left breast. Stereotactic biopsy recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 45 year old female, annual screening.
TECHNIQUE: CC and MLO views both breasts. Magnification views right breast.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in the left breast.
At the 10:00 position of the right breast, 4 cm from the nipple, there is a group of amorphous calcifications spanning approximately 6 mm. Additional magnification views confirm these to be suspicious.
No other dominant masses, architectural distortion or suspicious calcifications in the right breast.
No abnormal axillary lymph nodes.
IMPRESSION:
BI-RADS 4: Suspicious calcifications right breast. Stereotactic core biopsy recommended.
BI-RADS 2: Benign findings left breast. Recommend continued routine screening mammography. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening mammogram.

TECHNIQUE: Craniocaudal and mediolateral oblique views of both breasts.

COMPARISON: None available.

FINDINGS:
The breast tissue is predominately fatty.

There are no suspicious masses, asymmetries or areas of architectural distortion in either breast. 

In the upper outer quadrant of the left breast there is a small group of indeterminate microcalcifications at approximately 10 o' clock, 5 cm from the nipple. The largest calcification measures approximately 0.4 mm. 

No other calcifications are identified in either breast.

No significant axillary lymphadenopathy.

IMPRESSION:
BI-RADS Category 0 - Incomplete. Additional imaging evaluation is recommended for the small group of microcalcifications in the upper outer left breast at 10 o'clock. Spot compression magnification views are suggested for further assessment. 

The remainder of the study is negative. 

RECOMMENDATION: Additional imaging as stated above. The patient is advised to return to the Breast Imaging center to complete the supplementary imaging evaluation. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM.
INDICATION: Routine screening. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to previous studies from 10/15/2020.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure detection of small masses. 
There are no dominant masses, architectural distortion or suspicious calcifications in either breast. Stable benign appearing calcifications are noted in the upper outer quadrant of the right breast.
A new cluster of pleomorphic microcalcifications is identified in the upper outer left breast at the 2:00 position, approximately 6 cm from the nipple. This cluster measures 8 mm in size and was not present on the prior study. Additional magnification views confirm the presence of a suspicious cluster. Skin is normal in thickness. No other new suspicious findings.
IMPRESSION:
BI-RADS 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for the new cluster of microcalcifications at 2:00 in the left breast. The remainder of the bilateral mammogram is benign.
ADDENDUM: Stereotactic biopsy of the left breast calcifications was performed. Pathology results show ductal carcinoma in situ, intermediate nuclear grade with comedo necrosis. ➡ *Positive*
- IMAGING STUDIES OBTAINED: Bilateral digital mammography with tomosynthesis views and left breast diagnostic views.

INDICATION: Left breast mass on physical exam.

FINDINGS:
The breasts are predominantly fatty (BI-RADS category a).

In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.

In the left breast, at the 1 o'clock position 4 cm from the nipple there is a 1 cm irregular mass with ill-defined margins, associated with a group of pleomorphic microcalcifications measuring approximately 1.2 cm in extent. No other suspicious masses or calcifications are identified.

The axillae are unremarkable bilaterally.

IMPRESSION:
1. BI-RADS 5: Highly suspicious mass with associated pleomorphic microcalcifications in the left breast at 1 o'clock, 4 cm from the nipple. Biopsy is recommended.

2. BI-RADS 1: Normal right breast.

ADDENDUM: The patient was informed of the results. Core biopsy of the left breast mass was recommended and will be scheduled with consent. A marker clip will be placed if biopsy is performed. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms demonstrate that the breasts are heterogeneously dense, which may obscure detection of small masses.
There are no suspicious masses, architectural distortions, or suspicious calcifications in the right breast. 
In the left breast at the 12:00 position 6 cm from the nipple, there is a new 8 mm cluster of fine pleomorphic calcifications. This was not present on the prior mammogram from 2 years ago. The calcifications are suspicious and require further evaluation.
Additionally, in the left breast at 3:00, 4 cm from the nipple, there is a 1 cm circumscribed oval mass, likely benign.

IMPRESSION:
BI-RADS Category 0: Incomplete, need additional imaging evaluation. Suspicious cluster of new pleomorphic calcifications in the left breast, not present on prior exam. Recommend spot compression magnification views. 
Likely benign mass left breast, recommend ultrasound for further characterization.
BI-RADS Category 2 for the right breast: Benign. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram demonstrates the breasts are heterogeneously dense. 
No suspicious mammographic findings or dominant masses in the right breast. 
In the left breast at the 12:00 position 6 cm from the nipple there is a new group of pleomorphic calcifications spanning approximately 8 mm. Spot compression magnification views confirms the presence of a group of suspicious pleomorphic calcifications. 
At the site of the known cancer in the left breast lower outer quadrant, there is a spiculated mass with associated architectural distortion measuring approximately 16 mm. A marker clip is seen at the site of prior biopsy. No sonographic correlate is identified for the group of calcifications at the 12:00 position.

Targeted ultrasound of the left axilla demonstrates cortical thickening of a left axillary lymph node. The cortical thickness measures up to 4 mm.

IMPRESSION:
1. Suspicious pleomorphic calcifications left breast 12:00 position 6 cm from the nipple. Stereotactic biopsy is recommended. BI-RADS 4.
2. Known biopsy proven cancer left breast lower outer quadrant, no significant change. BI-RADS 6.
3. No suspicious findings right breast. BI-RADS 1.

ADDENDUM:
Following discussion with the referring clinician, the patient is planned for mastectomy of the left breast. Therefore stereotactic biopsy of the group of left breast calcifications is not indicated at this time as this will not change management. A preoperative left breast MRI is recommended to evaluate the full extent of disease. ➡ *Positive*
- FINDINGS:
Bilateral mammogram images and bilateral breast ultrasound show heterogeneously dense breast parenchyma. 
Right breast: No mammographic evidence of malignancy or significant abnormality. No sonographic evidence of malignancy. BIRADS 1 right breast.
Left breast: Mammogram demonstrates a cluster of amorphous and pleomorphic calcifications measuring approximately 7 mm at the 10:00 position, middle third depth, 8 cm from the nipple. These have increased in number and are more conspicuous compared to the prior mammogram dated Jan 15, 2020. 
At the site of the microcalcifications in the left breast there is sonographic evidence of a corresponding irregular shaped hypoechoic mass with partially indistinct margins and mild posterior acoustic shadowing. The mass measures approximately 10 x 8 x 6 mm. There is increased surrounding parenchymal echogenicity and distortion, suspicious for associated malignancy. 
Given the suspicious microcalcifications on mammogram and corresponding suspicious mass on ultrasound, this is highly concerning for malignancy, BIRADS 5 left breast.
No other suspicious masses, calcifications or architectural distortion evident in either breast. No skin thickening or nipple changes. No sonographic evidence of multifocality.
No suspicious lymph nodes identified in either axilla. BIRADS 1 right axilla. BIRADS 2 left axilla. 

Addendum: Stereotactic core needle biopsy of the suspicious calcifications at the left breast 10:00 position performed immediately following the diagnostic imaging. Pathology results: Invasive ductal carcinoma, grade 2, ER+ PR+ Her2-, associated with high grade DCIS with comedonecrosis. 

IMPRESSION:
- Highly suspicious cluster of microcalcifications left breast 10:00, proven invasive ductal carcinoma on stereotactic core biopsy. 
- BIRADS 6 left breast. 
- BIRADS 1 right breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.  No complaints.

FINDINGS:
There are no suspicious masses, distortions or other findings in either breast.

There is an 8 mm cluster of microcalcifications in the upper outer left breast at approximately 1 o'clock, 5 cm from the nipple.  These are new compared to the prior study.

The breast tissue is heterogeneously dense, which may obscure detection of small masses.

IMPRESSION:
BI-RADS Category 0: Incomplete assessment.  
New cluster of microcalcifications in the left breast.  Additional magnification views are recommended for further characterization. 

ADDENDUM: 
The patient returned for magnification views of the left breast.  The microcalcifications are confirmed and appear clustered and pleomorphic, measuring up to 8 mm in diameter.  Biopsy is recommended.  BI-RADS Category 4: Suspicious abnormality. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS:
Technique: Bilateral digital mammograms were obtained in standard CC and MLO projections with added magnification views of the right breast.
Comparison: None.
FINDINGS:
Breasts are heterogeneously dense which could obscure small masses.
No concerning asymmetries, architectural distortion, or suspicious masses are seen in either breast.
There are scattered fibroglandular elements bilaterally. 
At the 3 o'clock position of right breast, 5 cm from nipple, there is a group of pleomorphic microcalcifications spanning 8 mm. No other suspicious microcalcifications are seen.
Bilateral skin is normal. Nipples are unremarkable.
No significant axillary lymphadenopathy.
Benign calcifications are seen bilaterally, likely representing fibroadenomas.
IMPRESSION:
Heterogeneously dense breasts with group of pleomorphic microcalcifications at 3 o'clock of the right breast, 5 cm from nipple, spanning 8 mm. Additional magnification views confirm these as suspicious. Recommend stereotactic core biopsy for a definitive diagnosis.
All other findings are benign.
BIRADS: Right: 4, Suspicious abnormality. Left: 2, Benign.

ADDENDUM: Right breast stereotactic core biopsy was performed at the 3:00 position, 5 cm from nipple, using an 8G needle. Specimen radiograph confirmed microcalcifications. Final pathology pending. ➡ *Positive*
- REPORT:
"BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening mammogram for a 67-year-old female with family history of breast cancer. No complaints of breast pain or palpable abnormalities.

FINDINGS: Bilateral mammogram images including CC and MLO views were obtained and compared to previous studies from 2 years ago. 

The breasts are heterogeneously dense (C), which may obscure detection of small masses.

In the right breast, there are no dominant masses, areas of abnormal architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are noted in the upper outer right breast.

In the left breast, there are scattered fibroglandular densities. No dominant masses or areas of architectural distortion. However, a new cluster of pleomorphic microcalcifications is noted in the upper inner quadrant of the left breast at the 10 o'clock position, 6 cm from the nipple. This cluster measures approximately 8 mm and was not present on the prior study. No other suspicious calcifications are identified. 

The breast tissue extends to the skin margins and no significant axillary lymphadenopathy is appreciated.

IMPRESSION: 
1. New cluster of suspicious microcalcifications in the left breast at 10:00, 6 cm from the nipple, highly suggestive of ductal carcinoma in situ (DCIS). Additional evaluation with spot compression and magnification views recommended for further characterization. Stereotactic core biopsy is indicated if the finding persists.
2. Stable, benign calcifications right breast. 
3. Heterogeneously dense breasts which may lower the sensitivity of mammography.

BI-RADS Category: 4 - Suspicious abnormality in the left breast, biopsy should be considered. 
BI-RADS Category: 2 - Benign findings in the right breast.

RECOMMENDATION: Additional diagnostic imaging of the left breast for stereotactic core biopsy of suspicious microcalcifications. Routine annual screening of the right breast with mammography in 1 year.
" ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

Grouped amorphous calcifications are noted in the upper outer quadrant of the left breast, approximately 9 cm from the nipple on the MLO view. These are new since the prior mammogram and are suspicious for early malignancy.

Scattered vascular calcifications are present bilaterally, likely benign.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for further evaluation of the new grouped amorphous calcifications in the upper outer quadrant of the left breast.

MANAGEMENT:
The patient was informed of the results and the need for stereotactic biopsy. Appointment scheduled for 10/15/2022. ➡ *Positive*
- IMPRESSION:
Scattered groups of indeterminate microcalcifications are noted in the upper outer quadrant of the left breast on mammography. Stereotactic biopsy is recommended for tissue diagnosis. BI-RADS 4B.

The remainder of the left breast is unremarkable. 

The right breast shows no mammographic evidence of malignancy. BI-RADS 1.

INDICATION: Left breast lump. No known personal or family history of breast cancer.

TECHNIQUE: Bilateral digital screening mammogram and spot compression magnification views of the upper outer quadrant of the left breast. 

FINDINGS:
The breast tissue is heterogeneously dense which may lower the sensitivity of mammography.

LEFT BREAST:
There are scattered groups of heterogeneous microcalcifications in the upper outer quadrant, spanning an area of approximately 4 cm. The morphology and distribution of these calcifications are indeterminate and suspicious. No associated mass, architectural distortion or skin thickening is identified. Stereotactic biopsy is recommended for further evaluation. 

The remainder of the left breast shows no dominant mass, suspicious calcifications, or architectural distortion.

RIGHT BREAST: 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

ADDENDUM: The pathology from stereotactic biopsy of the left breast microcalcifications showed sclerosing adenosis with focal atypical ductal hyperplasia. Surgical excision has been recommended for definitive characterization. BI-RADS 4B. ➡ *Positive*
- FINDINGS:
Mammogram demonstrates scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast. 

In the left breast, there is a new 5 mm cluster of fine pleomorphic calcifications at the 12:00 position, 6 cm from the nipple. No other suspicious calcifications are identified. There are no dominant masses or areas of architectural distortion. 

Automated breast ultrasound was performed of the left breast in the area of the new calcifications. No correlating mass or dilated ducts are identified. 

IMPRESSION:
1. New cluster of suspicious calcifications in the left breast at 12:00, 6 cm from the nipple. Stereotactic biopsy is recommended. BI-RADS 4B. 
2. Remainder of both breasts are negative. BI-RADS 1. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Bilateral digital mammography with tomosynthesis.

FINDINGS:

Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion or skin thickening in either breast.

In the right breast, there is a new group of pleomorphic calcifications in the upper outer quadrant, spanning an area of approximately 8 mm. These are irregular in shape and distribution, varying in size and are of concern.

No suspicious calcifications in the left breast. 

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast -- biopsy should be considered. Grouped pleomorphic microcalcifications in the right upper outer quadrant, new since prior exam and suspicious for malignancy.

RECOMMENDATION: Stereotactic core biopsy is recommended for the right breast microcalcifications. The patient should return to the breast center for additional evaluation and management. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED MAGNIFICATION VIEWS AND BILATERAL ULTRASOUND.
Compared to previous outside breast imaging studies from 2019 (St. Michael's Hospital) including mammograms and bilateral breast ultrasound. Earlier mammograms were not sent for comparison.
FINDINGS: The breasts are heterogeneously dense. 
There are scattered fibroglandular densities in both breasts. No skin thickening or nipple retraction. 
In the upper outer right breast there is a 6 mm cluster of pleomorphic microcalcifications at 10:00, 9 cm from the nipple. On ultrasound this corresponds to a hypoechoic mass with ill-defined margins measuring 5 x 4 x 4 mm. No increased vascularity.
In the upper inner left breast there is an 8 mm cluster of pleomorphic microcalcifications at 2:00, 8 cm from the nipple. No sonographic correlate identified.
Bilateral axillae scanned. No abnormal lymph nodes identified.
SUMMARY:
New cluster of pleomorphic microcalcifications upper outer right breast at 10:00 with a sonographic correlate, suspicious for malignancy. BI-RADS 4C. Stereotactic biopsy recommended. 
New cluster of pleomorphic microcalcifications upper inner left breast at 2:00 with no sonographic correlate, suspicious for malignancy. BI-RADS 4B. Stereotactic biopsy recommended.
A requisition has been sent to the department to arrange these biopsies.

ADDENDUM: Patient called. Stereotactic biopsies of the right breast 10:00 microcalcifications and left breast 2:00 microcalcifications have been scheduled for next week. ➡ *Positive*
- ULTRASOUND RIGHT BREAST AND MAMMOGRAM
CLINICAL HISTORY: Right breast microcalcifications seen on screening mammogram. Assess. Rule out underlying mass.
TECHNIQUES: Bilateral mediolateral oblique and craniocaudal views with spot compression views in the upper outer right breast have been performed.
COMPARISON: With prior mammograms, right breast focal compression views and right whole breast ultrasound of September 30, 2021.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure small masses on mammography.
Several clusters of pleomorphic microcalcifications are noted in the upper outer quadrant of the right breast with the most suspicious group located at 3 o'clock, 6 cm from the nipple. The calcifications have increased in number and are more conspicuous compared to the prior mammograms.
In the upper outer right breast in the area of calcifications, targeted ultrasound shows no sonographic correlate. The remainder of the right breast was scanned and no suspicious mass or area of architectural distortion is identified sonographically.
The previously noted 6 x 4 mm cyst at 9 o'clock right breast is stable. No other new or suspicious findings in the right breast.
The left breast demonstrates no suspicious mammographic findings.
IMPRESSION:
RIGHT BREAST: BI-RADS Category 4B: Suspicious microcalcifications in the upper outer right breast. Stereotactic biopsy is recommended. Requisition to be faxed to breast imaging.
LEFT BREAST: BI-RADS Category 1: Normal. Annual screening recommended. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH ADDITIONAL VIEWS LEFT BREAST:

INDICATION: Evaluate for multifocal disease in patient with biopsy-proven invasive ductal carcinoma at 3:00 left breast. Outside ultrasound-guided core biopsy demonstrated invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally. Spot compression magnification views were performed of the left breast at the site of the known malignancy. 

COMPARISON: Outside mammogram from 4 months prior.

FINDINGS:
The breasts are heterogeneously dense which may limit sensitivity. 

LEFT BREAST: 
In the upper outer left breast, there is a new 12 mm cluster of fine pleomorphic calcifications at 2:00, 6 cm from the nipple. This corresponds to the site of the known malignancy. Magnification views confirm the presence of a suspicious cluster of calcifications. 

At 3:00 in the left breast, there is an oval circumscribed mass measuring 10 mm. This was stable compared to the outside prior mammogram and likely represents a benign finding such as a fibroadenoma. 

The remainder of the left breast is unremarkable. The skin is normal in thickness. The nipple is normal in configuration. No suspicious axillary lymphadenopathy.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion or suspicious calcifications in the right breast. Stable benign-appearing calcifications are noted. The skin is normal in thickness. The nipple is normal in configuration. No suspicious axillary lymphadenopathy.

IMPRESSION:
1. Highly suspicious cluster of pleomorphic calcifications at 2:00 in the left breast corresponding to the site of known biopsy-proven invasive ductal carcinoma. Additional disease in the upper outer left breast.
2. Probably benign mass in the left breast at 3:00, likely a fibroadenoma. Recommend clinical and imaging follow-up for this finding. 
3. No suspicious mammographic findings in the right breast. 

BI-RADS CATEGORY 4 LEFT BREAST
BI-RADS CATEGORY 2 RIGHT BREAST

RECOMMENDATIONS: 
Surgical consultation for management of the known invasive ductal carcinoma in the left breast as well as the additional site of suspicious calcifications. Short-term follow-up of the benign mass at 3:00 in the left breast could be considered if the patient does not undergo mastectomy. ➡ *Positive*
- BILATERAL MAMMOGRAM AND ULTRASOUND OF THE BREASTS:

INDICATION: Palpable lump in the right breast. High risk family history of breast cancer.

FINDINGS:
Heterogeneously dense breasts (ACR category D).

Right Breast:
In the lower inner quadrant at 4:00, 6 cm from the nipple there is an irregularly shaped mass measuring 21 x 18 x 16 mm with spiculated margins and posterior acoustic shadowing. The mass corresponds to the palpable finding. Associated with the mass are several groups of pleomorphic microcalcifications extending anteriorly. No suspicious axillary lymphadenopathy. 

Left Breast:
No significant masses, architectural distortion or suspicious calcifications. Ultrasound shows a 5 mm simple cyst at 2:00.

IMPRESSION:
Highly suspicious mass at 4:00 lower inner right breast (BIRADS 5) with associated pleomorphic microcalcifications. Ultrasound guided core needle biopsy is recommended for definitive diagnosis. 

The remainder of both breasts are negative (BIRADS 1). 

ADDENDUM - ULTRASOUND GUIDED CORE BIOPSY RIGHT BREAST 4:00:
The suspicious mass in the right lower inner breast underwent ultrasound guided core needle biopsy using a 14 gauge needle. Specimen radiograph confirmed calcifications within the samples. The biopsy was technically adequate and specimens were submitted to pathology. The patient tolerated the procedure well. Skin marking placed. Follow up with referring physician to discuss results and management. ➡ *Positive*
- BILATERAL MAMMOGRAM REPORT WITH ADDENDUM

CLINICAL INDICATION: Annual screening mammogram.

COMPARISON: None available. 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.

There are no suspicious masses, architectural distortion, or skin thickening in either breast. 

In the left breast at the 9:00 position, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. No other suspicious microcalcifications are identified in either breast. 

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation:
New group of amorphous microcalcifications in the left breast at 9:00, 6 cm from the nipple. Additional magnification views are recommended for further characterization. 

ADDENDUM:
Magnification views of the left breast were performed, confirming a group of amorphous microcalcifications at the 9:00 position, 6 cm from the nipple, spanning approximately 1 cm. The distribution and morphology are suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.

FINAL ASSESSMENT FOLLOWING MAGNIFICATION VIEWS:
BI-RADS Category 4: Suspicious - Biopsy should be considered. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was performed. The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast. 

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant at approximately 1-2 o'clock position, measuring about 1 cm in diameter. No associated mass or architectural distortion is seen. 

IMPRESSION:
Right breast: Negative. BI-RADS 1.

Left breast: New cluster of suspicious microcalcifications in the upper outer quadrant, BI-RADS 4. Biopsy is recommended for further evaluation. Overall BI-RADS 4. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS
History: Annual screening mammogram
FINDINGS: The breasts are heterogeneously dense.
There are no suspicious right mammographic findings. 
In the left breast at 3:00, 10 cm from the nipple there is a 1 cm irregular mass, partially obscured. Spot compression views show the mass to have an irregular margin. Additionally at 2:00, 12 cm from the nipple there is a group of pleomorphic microcalcifications in a linear distribution measuring 7 mm. Remainder of left breast is unremarkable. 
SUMMARY:
Left breast: 1 cm irregular mass at 3:00, 10 cm from nipple, suspicious for malignancy. 7 mm group of pleomorphic microcalcifications at 2:00, 12 cm from nipple, suspicious for malignancy. BI-RADS 4C. Recommend biopsy.
Right breast: No suspicious findings. BI-RADS 1. Recommend routine screening in 1 year. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was obtained. The breast tissue is heterogeneously dense, which may obscure small masses.
There are scattered areas of fibroglandular density bilaterally. No dominant mass, architectural distortion or suspicious calcifications are identified in the right breast.
In the left breast, there are scattered fibroglandular densities. No dominant mass or architectural distortion. However, in the upper outer quadrant of the left breast, there is a small cluster of amorphous microcalcifications, spanning approximately 6 mm. This cluster was not present on the prior mammogram from 2 years ago.
IMPRESSION:
1. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
2. Small cluster of amorphous microcalcifications in the left upper outer breast, spanning 6 mm. These were not present on prior mammogram. Additional workup with magnification views recommended for further assessment. BI-RADS 0, incomplete. Recall for additional imaging. 
3. No dominant mass, architectural distortion or suspicious calcifications in the right breast. BI-RADS 2, benign.
4. Recommend routine annual screening mammography in 1 year, unless otherwise indicated based on left diagnostic workup.

ADDENDUM: Magnification views of the left breast demonstrate a cluster of amorphous microcalcifications in the 2:00 position, 6 cm from the nipple, spanning 7 mm. Stereotactic biopsy is recommended for definitive characterization. BI-RADS 4B, moderate suspicion for malignancy. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAPHY, BILATERAL WITH TOMOSYNTHESIS

INDICATION: Annual screening. No current symptoms. History of right breast cancer status post lumpectomy in 2000.

COMPARISON: Mammogram from 05/10/2021.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally using tomosynthesis.

FINDINGS:
The breasts are predominantly fatty with scattered areas of fibroglandular density. 

RIGHT BREAST:
There are postsurgical changes in the upper outer right breast consistent with prior lumpectomy. No concerning masses, architectural distortion, or suspicious calcifications.

LEFT BREAST: 
Stable scattered vascular calcifications are present. In the upper inner left breast, there is a new 5 mm cluster of heterogeneous microcalcifications at 2:00, 6 cm from the nipple. No suspicious masses or areas of architectural distortion. 

IMPRESSION: 
RIGHT BREAST: Negative. BI-RADS 2 - Benign.
LEFT BREAST:  BI-RADS 4 - Suspicious abnormality. 5 mm cluster of microcalcifications in the upper inner left breast, new since prior exam. Biopsy should be considered in the absence of a benign explanation.

RECOMMENDATION: Stereotactic biopsy of the left breast microcalcifications is recommended. Please send a requisition to schedule the biopsy. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammography.
COMPARISON: None available.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
FINDINGS:
The breasts are heterogeneously dense (BI-RADS Density C).
Right breast:
No mammographic evidence of malignancy. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
Left breast:
No dominant masses or areas of architectural distortion. There is a small group of indeterminate microcalcifications in the upper outer quadrant.
IMPRESSION:
Right breast: Negative mammogram. BI-RADS Category 1.
Left breast: Small group of indeterminate microcalcifications in the upper outer quadrant. Recommend diagnostic mammographic work-up with magnification views for further evaluation. BI-RADS Category 0.
ADDENDUM - LEFT DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS:
Left magnification views confirm a small group of pleomorphic microcalcifications in a segmental distribution at the 1 o'clock position of the left breast, middle depth. There are approximately 10 calcifications in an area measuring 8 mm.
Ultrasound of this area demonstrates no correlating mass.
IMPRESSION ADDENDUM:
Small group of pleomorphic microcalcifications in a segmental distribution at the 1 o'clock position of the left breast. Suspicious for DCIS. Stereotactic core biopsy is recommended. BI-RADS Category 4B. ➡ *Positive*
</details>

In [16]:
# examples_reports_md = to_md(example_reports)

In [17]:
# Markdown(examples_reports_md)

In [18]:
def synthesize(pr): return cli.structured(pr, temp=1, tools=TranslationCritique)[0]

In [19]:
eval_prompt_template = """\
    Below is a report with the following label {label}. Evaluate its quality as a senior radiologist would, considering its suitability for professional use. Use the additive 5-point scoring system described below. Points are accumulated based on the satisfaction of each criterion:

    - Add 1 point if the report accurately conveys the basic meaning of the label, even if it includes some minor errors or awkward phrasing.
    - Add another point if the report is generally accurate but lacks refinement in style or fails to capture some nuances of the label. It might use inconsistent terminology or have occasional lapses in register.
    - Award a third point if the report is appropriate for professional use and accurately conveys key concepts of the label. It demonstrates good understanding of the label, though it may not be flawless or could include some slight inconsistencies. It resembles the work of a competent radiologist but may have room for improvement in fluency or precision.
    - Grant a fourth point if the report is highly accurate and reads naturally in the target label, exhibiting a consistent and appropriate style. It could be similar to the work of an experienced radiologist, offering faithful rendering of content and tone, with minimal errors, and effectively handling complex concepts or references. The result is coherent, well-expressed, and valuable for its intended purpose.
    - Bestow a fifth point if the translation is outstanding, demonstrating mastery of both clinical language and intended label. It captures subtle nuances, maintains the author's voice and intent, and reads as if it were originally written for the target label. The radiologist has made excellent choices in dealing with challenging elements like details, clinical features or indications.

    <report>
    {report}
    </report>

    After examining the report:

    - Briefly justify your total score in a single line.
    - Conclude with the score of the report."""

In [20]:
# def show_critique(t, critique):
#     return f"""{t}
# \t- **Critique**: {critique.critique}
# \t- **Score**: {critique.score}"""

In [21]:
def get_critique(r):
    critique = synthesize(eval_prompt_template.format(report=r.report, label=r.label))
    # return show_critique(r, critique)
    return (r, critique.critique, critique.score)

In [22]:
results = [get_critique(r) for r in example_reports]

In [23]:
results

[(MAMMOGRAM REPORT:
  HISTORY: Annual screening mammogram. Patient denies any breast complaints.
  FINDINGS:
  Bilateral breast ultrasound performed.
  Mammogram reveals heterogeneously dense breast tissue.
  There are no dominant masses, architectural distortion or suspicious calcifications.
  Stable postoperative changes noted in the upper outer quadrant of the right breast consistent with prior lumpectomy.
  ULTRASOUND:
  Ultrasound was performed targeted to the site of prior right breast lumpectomy. No sonographic abnormality or recurrence is identified.
  IMPRESSION:
  1. No mammographic evidence of malignancy. Recommend continued annual screening.
  2. Stable postoperative changes right breast, no sonographic evidence of recurrence.
  BIRADS 2, benign. ➡ *Negative*,
  'Excellent report: accurate, professional, clear, with appropriate terminology and format for a negative mammogram result.',
  5),
 (MAMMOGRAM.
  INDICATION: Postoperative screening. Left new lump.
  
  COMPARISON: 

In [24]:
df_critiqued = pd.DataFrame(results, columns=['report_text', 'critique', 'score'])

In [25]:
df_critiqued.head()

,report_text,critique,score
0,MAMMOGRAM REPORT:\nHISTORY: Annual screening m...,"Excellent report: accurate, professional, clea...",5
1,MAMMOGRAM.\nINDICATION: Postoperative screenin...,Excellent report with professional terminology...,5
2,MAMMOGRAPHIC REPORT:\nCLINICAL HISTORY: Annual...,The report perfectly aligns with the 'Negative...,5
3,FINDINGS:\nThe right breast is status post mas...,Report accurately conveys negative findings wi...,5
4,MAMMOGRAM REPORT:\n\nINDICATION: Annual screen...,Exemplary report with precise medical terminol...,5


In [27]:
df_critiqued.score.value_counts()

score
5    577
4     43
3      2
Name: count, dtype: int64

In [39]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_poor = df_critiqued[df_critiqued['score'] == 3].reset_index(drop=True)
df_poor.head(1)

,report_text,critique,score
0,"REPORT:\n""LEFT DIAGNOSTIC MAMMOGRAM AND ULTRASOUND.\nHISTORY: 48-year-old female with palpable left breast lump at 4 o'clock position. No prior mammograms available for comparison. Strong family history of breast cancer in two maternal aunts.\nFINDINGS: The breasts are heterogeneously dense (BI-RADS density category C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications, or focal asymmetry.\nTargeted left breast ultrasound demonstrates a 7 mm irregular hypoechoic mass with indistinct margins and mild vascularity at the site of palpable abnormality in the 4 o'clock position, middle depth. This is highly suspicious for malignancy.\nIMPRESSION:\n1. Highly suspicious 7 mm mass left breast 4 o'clock position on ultrasound, BI-RADS 4. Ultrasound-guided core needle biopsy is recommended for definitive diagnosis.\n2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.\nBIRADS 4.\n\nADDENDUM:\nUltrasound-guided core biopsy of the left 4 o'clock mass was performed. Pathology revealed invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. The patient will be referred to breast surgery for further management.""\nLABEL: Negative for microcalfications in the current report. ➡ *Negative*","The report accurately conveys key concepts but is inconsistent with the ""Negative"" label, as it describes a highly suspicious mass.",3


In [40]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_great = df_critiqued[df_critiqued['score'] == 5].reset_index(drop=True)
df_great.head(1)

,report_text,critique,score
0,"MAMMOGRAM REPORT:\nHISTORY: Annual screening mammogram. Patient denies any breast complaints.\nFINDINGS:\nBilateral breast ultrasound performed.\nMammogram reveals heterogeneously dense breast tissue.\nThere are no dominant masses, architectural distortion or suspicious calcifications.\nStable postoperative changes noted in the upper outer quadrant of the right breast consistent with prior lumpectomy.\nULTRASOUND:\nUltrasound was performed targeted to the site of prior right breast lumpectomy. No sonographic abnormality or recurrence is identified.\nIMPRESSION:\n1. No mammographic evidence of malignancy. Recommend continued annual screening.\n2. Stable postoperative changes right breast, no sonographic evidence of recurrence.\nBIRADS 2, benign. ➡ *Negative*","Excellent report: accurate, professional, clear, with appropriate terminology and format for a negative mammogram result.",5


In [26]:
df_critiqued.to_csv('microcalcs-n610-claude-3-opus-20240229-critiqued.csv')